# 1. Libraries

In [1]:
import os

# Maximum CPU utilization
for env_var in ['OMP_NUM_THREADS', 'MKL_NUM_THREADS', 'OPENBLAS_NUM_THREADS', 
                'NUMEXPR_NUM_THREADS', 'VECLIB_MAXIMUM_THREADS']:
    os.environ[env_var] = '8'

os.environ['MKL_DYNAMIC'] = 'FALSE'
os.environ['OMP_DYNAMIC'] = 'FALSE'
os.environ['OMP_PROC_BIND'] = 'TRUE'
os.environ['OMP_PLACES'] = 'threads'

def set_all_seeds(seed=42):
    """
    Set random seeds for all libraries in your notebook for reproducibility.
    Place this at the very beginning of your notebook, right after imports.
    """
    import random
    import numpy as np
    import os
    
    # Python's built-in random module
    random.seed(seed)
    
    # NumPy (critical for pandas, sklearn, scipy operations)
    np.random.seed(seed)
    
    # Set Python hash seed for reproducibility in sets/dicts
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    # TensorFlow settings
    import tensorflow as tf
    tf.random.set_seed(seed)
    # Additional TF settings for deterministic behavior
    #os.environ['TF_DETERMINISTIC_OPS'] = '1'
    #os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    #tf.config.threading.set_inter_op_parallelism_threads(1)
    #tf.config.threading.set_intra_op_parallelism_threads(1)
    
    # PennyLane quantum computing library
    import pennylane as qml
    import pennylane.numpy as qnp
    # PennyLane uses numpy's random state, but we can ensure it's set
    qnp.random.seed(seed)
    
    # For any multiprocessing/threading reproducibility
    #os.environ['OMP_NUM_THREADS'] = '1'
    #os.environ['MKL_NUM_THREADS'] = '1'
    
    print(f"All random seeds set to {seed}")
    print("Deterministic mode enabled for TensorFlow")

set_all_seeds(42)

import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
import pickle
from tqdm import tqdm
import seaborn as sns
from scipy import stats
from matplotlib.dates import DateFormatter
import tensorflow as tf
import re
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import copy

2025-06-27 02:14:54.005353: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-27 02:14:54.005425: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-27 02:14:54.007098: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-27 02:14:54.015219: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-27 02:14:54.985126: W tensorflow/compiler/tf2

All random seeds set to 42
Deterministic mode enabled for TensorFlow


# 1.5 Technical Analysis

In [2]:
"""
Comprehensive Technical Indicator Fixes: Mathematical Stability & Financial Meaning Preservation

This module provides numerically stable implementations of financial technical indicators while
carefully preserving their financial meaning, temporal continuity, and mathematical properties.
Each implementation follows financial research standards and includes proper handling of
edge cases, initialization periods, and numerical stability issues.
"""

import numpy as np
import pandas as pd

class TechnicalIndicators:
    """
    A class for calculating various technical indicators from financial time series data
    with enhanced numerical stability and financial meaning preservation.
    """

    @staticmethod
    def _adaptive_tolerance(price_level, base_tolerance=1e-10, factor=1e-6):
        """
        Calculate an adaptive tolerance based on price magnitude.
        
        Following Daumas et al. (2005) "Validated Roundings of Dot Products by Sticky Accumulation"
        and Muller et al. (2018) "Handbook of Floating-Point Arithmetic" recommending
        relative tolerances for values that scale with magnitude.
        
        Parameters:
        -----------
        price_level: float
            Reference price or magnitude
        base_tolerance: float
            Minimum tolerance threshold
        factor: float
            Scaling factor for price-based component
            
        Returns:
        --------
        float
            Adaptive tolerance value
        """
        return max(base_tolerance, abs(price_level) * factor)
    
    @staticmethod
    def _calculate_trend_direction(series, periods=1):
        """
        Calculate trend direction for a series with proper handling of zeros and NaNs.
        
        Parameters:
        -----------
        series: pandas.Series
            Series to calculate trend direction for
        periods: int
            Number of periods to look back
            
        Returns:
        --------
        pandas.Series
            Series containing trend direction values:
            1 for rising, -1 for falling, 0 for no change
        """
        # Calculate direction safely
        diff = series.diff(periods)
        
        # Initialize direction series
        direction = pd.Series(0, index=series.index)
        
        # Positive direction
        direction[diff > 0] = 1
        
        # Negative direction
        direction[diff < 0] = -1
        
        # For zero-diff values, carry forward previous direction to avoid flicker
        # but only where series values are valid
        # For near-zero diff values, carry forward previous direction to avoid flicker
        # Following Goldberg (1991) "What Every Computer Scientist Should Know About Floating-Point Arithmetic"
        # IEEE 754 standard requires using absolute thresholds for floating-point comparisons
        zero_mask = (diff.abs() < 1e-10) & series.notna()
        if zero_mask.any():
            # Forward-fill only zero-diff positions
            direction_filled = direction.copy()
            direction_filled[zero_mask] = np.nan
            direction_filled = direction_filled.ffill()
            
            # Update direction where diff was zero
            direction[zero_mask] = direction_filled[zero_mask]
            
        return direction

    @staticmethod
    def _extract_window_from_ma_name(ma_name):
        """
        Extract window size from MA column name.
        
        Parameters:
        -----------
        ma_name: str
            MA column name (e.g., 'SMA_20_D', 'EMA_50_D')
            
        Returns:
        --------
        int
            Window size extracted from the name
        """
        import re
        # Pattern to match SMA_20_D, EMA_50_D, etc.
        match = re.search(r'(?:SMA|EMA)_(\d+)', ma_name)
        return int(match.group(1)) if match else 999  # Default to large number if no match
        
    @staticmethod
    def calculate_current_drawdown(df, price_col='Close', include_trend=True, include_metrics=True):
        """
        Calculate the current drawdown from the all-time high with proper handling
        of edge cases and numerical stability.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing price data
        price_col: str
            Column name for price data
        include_trend: bool
            Whether to include trend direction of the drawdown
        include_metrics: bool
            Whether to include z-score metrics
                
        Returns:
        --------
        dict
            Dictionary containing drawdown values and metrics
        """
        # Ensure the DataFrame is sorted by date
        df = df.sort_index()
        
        # Handle missing or invalid values in the price column
        price_series = df[price_col].copy()
        
        # Cannot calculate meaningful drawdown if first value is NaN
        # For subsequent NaNs, forward fill to maintain continuity
        if not pd.isna(price_series.iloc[0]):
            price_series = price_series.ffill()
        
        # Calculate running maximum (all-time high) with proper handling
        running_max = price_series.expanding().max()
        
        # Initialize drawdown series
        drawdown = pd.Series(index=df.index, dtype=float)
        
        # Improved vectorized implementation for percentage drawdown
        valid_mask = (running_max > 0) & running_max.notna() & price_series.notna()
        drawdown.loc[valid_mask] = ((price_series.loc[valid_mask] - running_max.loc[valid_mask]) / 
                                  running_max.loc[valid_mask]) * 100
        
        # Prepare result dictionary
        result = {'Current_Drawdown': drawdown}
        
        # Add trend direction
        if include_trend:
            trend = TechnicalIndicators._calculate_trend_direction(drawdown)
            result['Current_Drawdown_trend'] = trend
        
        # Add z-score with proper statistical handling
        if include_metrics:
            # Calculate historical mean and std of drawdown
            # Use expanding window initially, then transition to rolling
            # This provides stability in early periods and adaptivity later
            lookback = min(250, len(df) // 2) if len(df) > 10 else len(df)
            
            # First calculate expanding statistics for early periods
            exp_mean = drawdown.expanding(min_periods=1).mean()
            exp_std = drawdown.expanding(min_periods=1).std()
            
            # Then calculate rolling statistics for later periods
            roll_mean = drawdown.rolling(window=lookback, min_periods=max(1, lookback//4)).mean()
            roll_std = drawdown.rolling(window=lookback, min_periods=max(1, lookback//4)).std()
            
            # Blend from expanding to rolling as we get more data
            blend_idx = min(lookback*2, len(df))
            drawdown_mean = exp_mean.copy()
            drawdown_std = exp_std.copy()
            
            if len(df) > blend_idx:
                drawdown_mean.iloc[blend_idx:] = roll_mean.iloc[blend_idx:]
                drawdown_std.iloc[blend_idx:] = roll_std.iloc[blend_idx:]
            
            # Calculate z-score with protection against zero std
            min_std = 0.001  # Minimum meaningful standard deviation
            robust_std = np.maximum(drawdown_std, min_std)
            z_score = (drawdown - drawdown_mean) / robust_std
            
            result['Current_Drawdown_z_score'] = z_score
        
        return result

    @staticmethod
    def calculate_moving_average(df, price_col='Close', window=50, ma_type='simple', 
                        timeframe='D', include_stddev=True, include_trend=True):
        """
        Calculate Moving Averages with proper handling of timeframes, initialization,
        and statistical properties.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing price data
        price_col: str
            Column name for price data
        window: int
            Window size for the moving average
        ma_type: str
            Type of moving average ('simple' or 'exponential')
        timeframe: str
            Timeframe for resampling ('D' for daily, 'W' for weekly, 'M' for monthly)
        include_stddev: bool
            Whether to include standardized distance from MA
        include_trend: bool
            Whether to include trend direction of the MA
                
        Returns:
        --------
        dict
            Dictionary containing MA values and related metrics
        """
        # Ensure the DataFrame is sorted by date
        df = df.sort_index()
        
        # Handle resampling with proper financial time series techniques
        if timeframe != 'D':
            # For OHLC type data, use appropriate aggregation
            if all(col in df.columns for col in ['Open', 'High', 'Low', 'Close']):
                # Standard OHLC resampling
                ohlc_dict = {
                    'Open': 'first', 
                    'High': 'max', 
                    'Low': 'min', 
                    'Close': 'last'
                }
                resampled = df.resample(timeframe).agg(ohlc_dict)
            else:
                # For price-only data, use last value
                resampled = df.resample(timeframe)[price_col].last().to_frame()
            
            # Forward fill NaN values (market closed periods)
            resampled = resampled.ffill()
            
            # Convert back to daily frequency to match original data
            price_series = resampled[price_col].reindex(df.index, method='ffill')
        else:
            price_series = df[price_col].copy()
        
        # Handle missing values in price series
        price_series = price_series.ffill().bfill()
        
        # Calculate the moving average with proper min_periods
        min_periods = 1  # Allow calculation from first valid value
        
        if ma_type.lower() == 'simple':
            ma = price_series.rolling(window=window, min_periods=min_periods).mean()
            ma_name = f'SMA_{window}_{timeframe}'
        elif ma_type.lower() == 'exponential':
            # Improved vectorized calculation for EMA
            if len(price_series) >= window:
                initial_ma = price_series.iloc[:window].mean()
                alpha = 2 / (window + 1)
                ma = pd.Series(initial_ma, index=price_series.index[:window])
                ma = pd.concat([ma, price_series.iloc[window:].ewm(alpha=alpha, adjust=False).mean()])
            else:
                # If we don't have enough data, use standard EMA
                ma = price_series.ewm(span=window, adjust=False, min_periods=min_periods).mean()
                
            ma_name = f'EMA_{window}_{timeframe}'
        else:
            raise ValueError("ma_type must be 'simple' or 'exponential'")
        
        # More efficient zero/NaN handling for percentage difference
        valid_mask = (ma != 0) & ma.notna() & price_series.notna()
        pct_diff = pd.Series(index=price_series.index, dtype=float)
        pct_diff.loc[valid_mask] = ((price_series.loc[valid_mask] - ma.loc[valid_mask]) / 
                           ma.loc[valid_mask]) * 100
        pct_diff_name = f'{ma_name}_pct_diff'
        
        result = {
            ma_name: ma,
            pct_diff_name: pct_diff
        }
        
        # Calculate standardized distance with proper statistical handling
        if include_stddev:
            # Use appropriate standard deviation calculation based on MA type
            if ma_type.lower() == 'simple':
                # For SMA, use rolling standard deviation
                std = price_series.rolling(window=window, min_periods=min_periods).std()
            else:
                # For EMA, use exponentially weighted standard deviation
                std = np.sqrt(
                    price_series.ewm(span=window, adjust=False, min_periods=min_periods)
                    .var(bias=False)
                )
            
            # Ensure std is never too small to avoid explosion
            min_std_threshold = price_series.abs().mean() * 0.0001  # Adaptive minimum threshold
            robust_std = pd.Series(
                np.maximum(std.values, min_std_threshold), 
                index=std.index
            )
            
            # Calculate Z-score with protection against invalid values
            z_score = pd.Series(index=price_series.index, dtype=float)
            valid_z_mask = robust_std.notna() & (robust_std > 0) & (price_series - ma).notna()
            z_score[valid_z_mask] = (price_series[valid_z_mask] - ma[valid_z_mask]) / robust_std[valid_z_mask]
            
            # Winsorize extreme z-scores to prevent unreasonable values
            # Use 4 std deviations as cutoff, preserving directionality
            extreme_mask = z_score.abs() > 4
            if extreme_mask.any():
                z_score[extreme_mask] = np.sign(z_score[extreme_mask]) * 4
                
            z_score_name = f'{ma_name}_z_score'
            result[z_score_name] = z_score
        
        # Add trend direction if requested
        if include_trend:
            trend = TechnicalIndicators._calculate_trend_direction(ma)
            trend_name = f'{ma_name}_trend'
            result[trend_name] = trend
        
        return result

    @staticmethod
    def calculate_ma_relationships(df, ma_results):
        """
        Calculate relationships between different moving averages with proper handling
        of cross-MA statistics and numerical stability.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing price data
        ma_results: dict
            Dictionary containing moving average values from calculate_moving_average
        
        Returns:
        --------
        dict
            Dictionary containing MA relationship metrics
        """
        result = {}
        
        # Get all MA keys (exclude metrics like _pct_diff, _z_score, etc.)
        ma_keys = [key for key in ma_results.keys() if ('_pct_diff' not in key and 
                                                      '_z_score' not in key and
                                                      '_trend' not in key)]
        
        # Calculate relationships between all pairs of MAs
        for i, ma1_key in enumerate(ma_keys):
            ma1 = ma_results[ma1_key]
            
            for ma2_key in ma_keys[i+1:]:  # Only calculate each pair once
                ma2 = ma_results[ma2_key]
                
                # Calculate the difference safely
                diff = ma1 - ma2
                
                # Calculate percentage difference relative to slower MA (standard technical analysis approach)
                pct_diff = pd.Series(index=df.index, dtype=float)
                
                # Extract window sizes from MA names to identify slower MA
                ma1_window = TechnicalIndicators._extract_window_from_ma_name(ma1_key)
                ma2_window = TechnicalIndicators._extract_window_from_ma_name(ma2_key)
                
                if ma1_window <= ma2_window:
                    fast_ma, slow_ma = ma1, ma2
                else:
                    fast_ma, slow_ma = ma2, ma1
                
                # Standard technical analysis calculation: (Fast - Slow) / Slow * 100
                # With protection against near-zero denominators
                min_threshold = slow_ma.abs().median() * 0.001  # Adaptive minimum threshold
                robust_denominator = pd.Series(np.maximum(slow_ma.abs(), min_threshold), index=slow_ma.index)
                
                valid_mask = robust_denominator.notna() & (fast_ma - slow_ma).notna()
                pct_diff[valid_mask] = ((fast_ma[valid_mask] - slow_ma[valid_mask]) / 
                                       robust_denominator[valid_mask]) * 100
                
                pct_diff_name = f'{ma1_key}_vs_{ma2_key}_pct_diff'
                result[pct_diff_name] = pct_diff
                
                # Calculate z-score of the difference with proper statistical handling
                # Use expanding window initially to handle early periods, then transition to rolling
                min_window = min(50, len(df) // 4) if len(df) > 10 else len(df)
                
                # First use expanding statistics for early periods
                expanding_std = diff.expanding(min_periods=min_window).std()
                
                # Then calculate rolling statistics with protection against zero std
                rolling_std = diff.rolling(window=50, min_periods=min_window).std()
                
                # Blend from expanding to rolling as we get more data
                blend_idx = min(100, len(df))
                robust_std = expanding_std.copy()
                
                if len(df) > blend_idx:
                    robust_std.iloc[blend_idx:] = rolling_std.iloc[blend_idx:]
                
                # Minimum std threshold based on typical magnitudes
                min_std_threshold = (ma1.abs().mean() + ma2.abs().mean()) / 2 * 0.001
                robust_std = np.maximum(robust_std, min_std_threshold)
                
                # Calculate z-score with protection
                z_score = pd.Series(index=df.index, dtype=float)
                valid_z_mask = robust_std.notna() & (robust_std > 0) & diff.notna()
                z_score[valid_z_mask] = diff[valid_z_mask] / robust_std[valid_z_mask]
                
                # Winsorize extreme values while preserving direction
                extreme_mask = z_score.abs() > 4
                if extreme_mask.any():
                    z_score[extreme_mask] = np.sign(z_score[extreme_mask]) * 4
                
                z_score_name = f'{ma1_key}_vs_{ma2_key}_z_score'
                result[z_score_name] = z_score
                
                # Calculate crossover signal (1 when ma1 crosses above ma2, -1 for below, 0 otherwise)
                ma_diff_sign = np.sign(ma1 - ma2)
                crossover = pd.Series(0, index=df.index)
                
                # Find where sign changes from one day to the next
                for j in range(1, len(df)):
                    if ma_diff_sign.iloc[j-1] < 0 and ma_diff_sign.iloc[j] > 0:
                        crossover.iloc[j] = 1  # Bullish crossover
                    elif ma_diff_sign.iloc[j-1] > 0 and ma_diff_sign.iloc[j] < 0:
                        crossover.iloc[j] = -1  # Bearish crossover
                
                crossover_name = f'{ma1_key}_vs_{ma2_key}_crossover'
                result[crossover_name] = crossover
        
        return result
    
    @staticmethod
    def calculate_rsi(df, price_col='Close', window=14, include_trend=True, include_metrics=True):
        """
        Calculate Relative Strength Index (RSI) with proper handling of edge cases,
        initialization, and Wilder's original methodology.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing price data
        price_col: str
            Column name for price data
        window: int
            Window size for RSI calculation (Wilder's standard is 14)
        include_trend: bool
            Whether to include trend direction of RSI
        include_metrics: bool
            Whether to include z-score metrics
                
        Returns:
        --------
        dict
            Dictionary containing RSI values and metrics
        """
        # Ensure the DataFrame is sorted by date
        df = df.sort_index()
        
        # Calculate price changes with forward fill for missing values
        price_series = df[price_col].ffill()
        delta = price_series.diff()
        
        # Create separate gain and loss series
        gain = pd.Series(0, index=delta.index)
        loss = pd.Series(0, index=delta.index)
        
        # Set values for gain and loss series correctly
        gain[delta > 0] = delta[delta > 0]
        loss[delta < 0] = -delta[delta < 0]  # Make losses positive
        
        # Initialize RSI series
        rsi = pd.Series(index=df.index, dtype=float)
        
        # Vectorized implementation of Wilder's smoothing method
        def wilders_smoothing(data, window):
            """Apply Wilder's smoothing method to a series."""
            # First value is a simple average
            result = pd.Series(index=data.index)
            result.iloc[window] = data.iloc[1:window+1].mean()
            
            # Apply Wilder's smoothing formula for subsequent values
            for i in range(window+1, len(data)):
                result.iloc[i] = ((window - 1) * result.iloc[i-1] + data.iloc[i]) / window
            return result
        
        # Apply Wilder's method if we have enough data
        if len(gain) >= window+1:
            # Calculate avg_gain and avg_loss using Wilder's smoothing
            avg_gain = wilders_smoothing(gain, window)
            avg_loss = wilders_smoothing(loss, window)
            
            # Calculate RS and RSI
            rs = pd.Series(index=avg_gain.index)
            
            # Handle division by zero carefully (no loss case)
            valid_rs_mask = (avg_loss > 0) & avg_gain.notna() & avg_loss.notna()
            rs[valid_rs_mask] = avg_gain[valid_rs_mask] / avg_loss[valid_rs_mask]

            # Special case: no losses in window (RSI = 100)
            # Use absolute threshold following Kahan (1997) "How Futile are Mindless Assessments of
            # Roundoff in Floating-Point Computation?"
            rs[(avg_loss.abs() < 1e-10) & (avg_gain > 0)] = float('inf') # Special case: no losses in window (RSI = 100)
            
            # Special case: no gains in window (RSI = 0)
            rs[(avg_gain.abs() < 1e-10) & (avg_loss > 0)] = 0 # Special case: no gains in window (RSI = 0)
            
            # Calculate RSI based on RS values - handling special cases
            rsi[rs == float('inf')] = 100  # When RS is infinity (no losses), RSI = 100
            rsi[rs == 0] = 0               # When RS is 0 (no gains), RSI = 0
            
            # Standard calculation for normal cases
            normal_mask = rs.notna() & (rs != float('inf')) & (rs != 0)
            rsi[normal_mask] = 100 - (100 / (1 + rs[normal_mask]))
            
            # Handle initial periods (0 to window-1) - extend the first valid RSI backwards
            if rsi.iloc[window:].notna().any():
                first_valid_rsi = rsi.iloc[window:].dropna().iloc[0]
                rsi.iloc[:window] = first_valid_rsi
        else:
            # Not enough data points - use basic calculation
            # This is a simplified approach for very short series
            avg_gain = gain.rolling(window=window, min_periods=1).mean()
            avg_loss = loss.rolling(window=window, min_periods=1).mean()
            
            # Calculate RS (Relative Strength) with protection
            rs = pd.Series(0, index=avg_gain.index)
            valid_mask = (avg_loss > 0) & avg_gain.notna() & avg_loss.notna()
            rs[valid_mask] = avg_gain[valid_mask] / avg_loss[valid_mask]
            
            # Handle special cases
            rs[(avg_loss == 0) & (avg_gain > 0)] = 100  # Very high but not infinite
            
            # Calculate RSI
            rsi = 100 - (100 / (1 + rs))
            rsi[(avg_gain == 0) & (avg_loss > 0)] = 0  # Explicit zero case
        
        # Ensure RSI is within [0, 100] bounds
        rsi = np.clip(rsi, 0, 100)
        
        # Prepare result dictionary
        base_name = f'RSI_{window}'
        result = {base_name: rsi}
        
        # Add trend direction
        if include_trend:
            trend = TechnicalIndicators._calculate_trend_direction(rsi)
            result[f'{base_name}_trend'] = trend
        
        # Add z-score with proper statistical handling
        if include_metrics:
            # Calculate z-score using adaptive lookback to handle varying time series lengths
            lookback = min(250, len(df) // 2) if len(df) > 20 else len(df)
            
            # Calculate historical mean and std of RSI
            rsi_mean = rsi.rolling(window=lookback, min_periods=max(5, lookback//5)).mean()
            rsi_std = rsi.rolling(window=lookback, min_periods=max(5, lookback//5)).std()
            
            # Ensure std is never too small (RSI typically has std > 1 in practice)
            robust_std = np.maximum(rsi_std, 1.0)
            
            # Calculate z-score with protection
            z_score = pd.Series(index=rsi.index, dtype=float)
            valid_mask = robust_std.notna() & (robust_std > 0) & rsi.notna() & rsi_mean.notna()
            z_score[valid_mask] = (rsi[valid_mask] - rsi_mean[valid_mask]) / robust_std[valid_mask]
            
            # Cap extreme z-scores
            z_score = np.clip(z_score, -4, 4)
            
            result[f'{base_name}_z_score'] = z_score
        
        return result
    
    @staticmethod
    def calculate_stochastic_oscillator(df, high_col='High', low_col='Low', 
                                       close_col='Close', k_window=14, d_window=3,
                                       include_raw=True, include_metrics=True, include_trend=True):
        """
        Calculate Stochastic Oscillator with enhanced metrics and robust handling of edge cases.
        Implementation preserves the numerical integrity and financial meaning of the indicator.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing price data
        high_col: str
            Column name for high price
        low_col: str
            Column name for low price
        close_col: str
            Column name for close price
        k_window: int
            Window size for %K calculation
        d_window: int
            Window size for %D calculation (moving average of %K)
        include_raw: bool
            Whether to include raw %K and %D values in the output
        include_metrics: bool
            Whether to include derived metrics in the output
        include_trend: bool
            Whether to include trend direction for %K and %D
            
        Returns:
        --------
        dict
            Dictionary containing Stochastic Oscillator values and related metrics
        """
        # Ensure the DataFrame is sorted by date
        df = df.sort_index()
        
        # Handle missing values in price data
        high_series = df[high_col].ffill().bfill()
        low_series = df[low_col].ffill().bfill()
        close_series = df[close_col].ffill().bfill()
        
        # Calculate components needed for %K
        lowest_low = low_series.rolling(window=k_window, min_periods=1).min()
        highest_high = high_series.rolling(window=k_window, min_periods=1).max()
        
        # Calculate the denominator with validation
        denominator = highest_high - lowest_low
        
        # Create %K series with proper initialization
        k = pd.Series(index=df.index, dtype=float)
        
        # Handle cases where highest_high equals lowest_low using scientifically appropriate threshold
        # Ref: Goldberg (1991) "What Every Computer Scientist Should Know About Floating-Point Arithmetic"
        # IEEE 754 standard recommends using absolute thresholds rather than exact equality tests
        zero_range_mask = denominator.abs() < 1e-10
        
        # For normal cases, calculate using standard formula
        normal_mask = ~zero_range_mask
        k[normal_mask] = 100 * ((close_series[normal_mask] - lowest_low[normal_mask]) / 
                              denominator[normal_mask])
        
        # Check relationship between close and the high/low with consistent threshold
        # Ref: Higham (2002) "Accuracy and Stability of Numerical Algorithms" §1.5
        # recommending consistent threshold application across related comparisons
        if zero_range_mask.any():
            # Create a mathematically consistent comparison for the flat range scenario
            close_equals_high_low = (close_series[zero_range_mask] - highest_high[zero_range_mask]).abs() < 1e-10
            k[zero_range_mask & close_equals_high_low] = 100.0  # Close equals the flat range
            
            # Handle other zero range cases explicitly to maintain numerical consistency
            # This avoids creating signal inconsistencies across near-identical price scenarios
            # Ref: Lo et al. (2000) "Foundations of Technical Analysis" on signal continuity
            other_zero_range = zero_range_mask & ~close_equals_high_low
            if other_zero_range.any():
                if k.notna().any():
                    last_k = k[k.notna()].iloc[-1]
                    k[other_zero_range] = last_k
                else:
                    k[other_zero_range] = 50.0  # Neutral value if no prior K
        
        # Ensure %K is within [0, 100] range (critical for this indicator)
        k = np.clip(k, 0, 100)
        
        # Calculate %D (moving average of %K) with proper handling of early periods
        # Use SMA with minimum periods to allow calculation from first day
        d = k.rolling(window=d_window, min_periods=1).mean()
        
        # Create base names for indicators
        k_name = f'Stochastic_%K_{k_window}'
        d_name = f'Stochastic_%D_{k_window}_{d_window}'
        
        # Prepare result dictionary
        result = {}
        
        # Include raw values if requested
        if include_raw:
            result[k_name] = k
            result[d_name] = d
        
        # Include derived metrics if requested
        if include_metrics:
            # K-D difference
            diff = k - d
            
            # Percentage difference (relative to D) with protection
            # Use absolute value of D in denominator to avoid near-zero division issues
            pct_diff = pd.Series(index=df.index, dtype=float)
            valid_mask = (d.abs() > 1e-10) & d.notna() & diff.notna()
            pct_diff[valid_mask] = diff[valid_mask] / d.abs()[valid_mask] * 100
            
            # For invalid cases, use a normalized version of raw diff
            invalid_mask = ~valid_mask & diff.notna()
            if invalid_mask.any():
                pct_diff[invalid_mask] = diff[invalid_mask] * 2  # Scale to similar magnitude
                
            result[f'{k_name}_pct_diff'] = pct_diff
            
            # Z-score of K-D difference with protection against zero std
            # Use adaptive lookback to handle varying time series lengths
            lookback = min(50, len(df) // 4) if len(df) > 10 else len(df)
            
            # Calculate rolling statistics with minimum periods for early stability
            rolling_std = diff.rolling(window=lookback, min_periods=max(2, lookback//5)).std()
            
            # Ensure std is never too small
            min_std = 0.1  # Minimum std - typical K-D differences are at least this magnitude
            robust_std = np.maximum(rolling_std, min_std)
            
            # Calculate z-score with proper protection
            z_score_diff = pd.Series(index=df.index, dtype=float)
            valid_mask = robust_std.notna() & (robust_std > 0) & diff.notna()
            z_score_diff[valid_mask] = diff[valid_mask] / robust_std[valid_mask]
            
            # Cap extreme z-scores while preserving directionality
            z_score_diff = np.clip(z_score_diff, -4, 4)
            
            result[f'{k_name}_diff_z_score'] = z_score_diff
            
            # Z-score of %K extension from its typical value
            # Calculate historical mean and std of %K itself
            k_mean = k.rolling(window=250, min_periods=max(5, min(250, len(df)//10))).mean()
            k_std = k.rolling(window=250, min_periods=max(5, min(250, len(df)//10))).std()
            
            # Ensure std is reasonable (stochastic typically has std > 5)
            robust_k_std = np.maximum(k_std, 5.0)
            
            # Calculate z-score with protection
            z_score_k_extension = pd.Series(index=df.index, dtype=float)
            valid_mask = robust_k_std.notna() & (robust_k_std > 0) & k.notna() & k_mean.notna()
            z_score_k_extension[valid_mask] = (k[valid_mask] - k_mean[valid_mask]) / robust_k_std[valid_mask]
            
            # Cap extreme values
            z_score_k_extension = np.clip(z_score_k_extension, -4, 4)
            
            result[f'{k_name}_extension_z_score'] = z_score_k_extension
            
            # Z-score of %D extension from its typical value
            d_mean = d.rolling(window=250, min_periods=max(5, min(250, len(df)//10))).mean()
            d_std = d.rolling(window=250, min_periods=max(5, min(250, len(df)//10))).std()
            
            # Ensure std is reasonable
            robust_d_std = np.maximum(d_std, 5.0)
            
            # Calculate z-score with protection
            z_score_d_extension = pd.Series(index=df.index, dtype=float)
            valid_mask = robust_d_std.notna() & (robust_d_std > 0) & d.notna() & d_mean.notna()
            z_score_d_extension[valid_mask] = (d[valid_mask] - d_mean[valid_mask]) / robust_d_std[valid_mask]
            
            # Cap extreme values
            z_score_d_extension = np.clip(z_score_d_extension, -4, 4)
            
            result[f'{d_name}_extension_z_score'] = z_score_d_extension
        
        # Add trend direction if requested
        if include_trend:
            k_trend = TechnicalIndicators._calculate_trend_direction(k)
            d_trend = TechnicalIndicators._calculate_trend_direction(d)
            result[f'{k_name}_trend'] = k_trend
            result[f'{d_name}_trend'] = d_trend
        
        return result
    
    @staticmethod
    def calculate_roc(df, price_col='Close', window=10, include_trend=True, include_metrics=True):
        """
        Calculate Rate of Change (ROC) with proper handling of edge cases, missing values,
        and numerical stability.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing price data
        price_col: str
            Column name for price data
        window: int
            Window size for ROC calculation
        include_trend: bool
            Whether to include trend direction of ROC
        include_metrics: bool
            Whether to include z-score metrics
            
        Returns:
        --------
        dict
            Dictionary containing ROC values and metrics
        """
        # Ensure the DataFrame is sorted by date
        df = df.sort_index()
        
        # Handle missing values in price series
        price_series = df[price_col].ffill().bfill()
        
        # Initialize ROC series
        roc = pd.Series(index=df.index, dtype=float)
        
        # Calculate ROC as percentage change over the specified period
        # Handle division by zero/close to zero carefully
        current_price = price_series
        past_price = price_series.shift(window)
        
        # Standard ROC formula with protection against division by zero/small numbers
        valid_mask = (past_price.abs() > 1e-10) & past_price.notna() & current_price.notna()
        roc[valid_mask] = ((current_price[valid_mask] - past_price[valid_mask]) / 
                         past_price[valid_mask]) * 100
        
        # For first 'window' periods, ROC is undefined - handle this based on financial meaning
        # For stock price indicators, early period ROC often assumed to be 0 (no change yet)
        # or calculated using shortest available lookback
        if window > 1:
            for i in range(1, min(window, len(roc))):
                if i < len(price_series) and pd.isna(roc.iloc[i]) and price_series.iloc[0:i+1].notna().all():
                    # Calculate ROC using available data points
                    if price_series.iloc[0] != 0:
                        roc.iloc[i] = ((price_series.iloc[i] - price_series.iloc[0]) / 
                                     price_series.iloc[0]) * 100
        
        # Handle extreme values based on historical patterns
        # For most financial time series, daily ROC rarely exceeds ±30%
        # Monthly ROC (20-day) rarely exceeds ±50%
        cap_threshold = 50 * (window / 20)  # Scale threshold based on window size
        extreme_mask = roc.abs() > cap_threshold
        if extreme_mask.any():
            # Instead of hard clipping, use soft capping that preserves directionality
            # and relative magnitude between extreme values
            roc[extreme_mask] = np.sign(roc[extreme_mask]) * (
                cap_threshold + np.log1p(roc[extreme_mask].abs() - cap_threshold)
            )
        
        # Prepare result dictionary
        base_name = f'ROC_{window}'
        result = {base_name: roc}
        
        # Add trend direction
        if include_trend:
            trend = TechnicalIndicators._calculate_trend_direction(roc)
            result[f'{base_name}_trend'] = trend
        
        # Add z-score with proper statistical handling
        if include_metrics:
            # Use adaptive lookback to handle varying time series lengths
            lookback = min(250, len(df) // 2) if len(df) > 20 else len(df)
            
            # Calculate historical mean and std of ROC
            roc_mean = roc.rolling(window=lookback, min_periods=max(5, lookback//5)).mean()
            roc_std = roc.rolling(window=lookback, min_periods=max(5, lookback//5)).std()
            
            # Ensure std is never too small
            # ROC standard deviation typically scales with window size
            min_std_threshold = 0.5 * np.sqrt(window)  # Heuristic based on typical financial volatility
            robust_std = np.maximum(roc_std, min_std_threshold)
            
            # Calculate z-score with protection
            z_score = pd.Series(index=roc.index, dtype=float)
            valid_mask = robust_std.notna() & (robust_std > 0) & roc.notna() & roc_mean.notna()
            z_score[valid_mask] = (roc[valid_mask] - roc_mean[valid_mask]) / robust_std[valid_mask]
            
            # Cap extreme z-scores
            z_score = np.clip(z_score, -4, 4)
            
            result[f'{base_name}_z_score'] = z_score
        
        return result
    
    @staticmethod
    def calculate_atr(df, high_col='High', low_col='Low', close_col='Close', window=14, 
                     include_trend=True, include_metrics=True):
        """
        Calculate Average True Range (ATR) with Wilder's methodology and proper handling
        of edge cases, missing values, and numerical stability.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing price data
        high_col: str
            Column name for high price
        low_col: str
            Column name for low price
        close_col: str
            Column name for close price
        window: int
            Window size for ATR calculation
        include_trend: bool
            Whether to include trend direction of ATR
        include_metrics: bool
            Whether to include z-score metrics
                
        Returns:
        --------
        dict
            Dictionary containing ATR values and metrics
        """
        # Ensure the DataFrame is sorted by date
        df = df.sort_index()
        
        # Handle missing values in price data
        high_series = df[high_col].ffill().bfill()
        low_series = df[low_col].ffill().bfill()
        close_series = df[close_col].ffill().bfill()
        
        # Vectorized True Range calculation
        high_low = high_series - low_series
        high_close_prev = (high_series - close_series.shift(1)).abs()
        low_close_prev = (low_series - close_series.shift(1)).abs()
        
        # Combine the three components to get True Range
        tr = pd.concat([high_low, high_close_prev, low_close_prev], axis=1).max(axis=1)
        
        # Handle first period (where close_prev doesn't exist)
        tr.iloc[0] = high_low.iloc[0]
        
        # Calculate ATR using Wilder's smoothing method
        atr = pd.Series(index=df.index, dtype=float)
        
        # First value is simple average of TR
        if len(df) >= window:
            atr.iloc[window-1] = tr.iloc[:window].mean()
            
            # Apply Wilder's smoothing for subsequent periods
            for i in range(window, len(df)):
                atr.iloc[i] = (atr.iloc[i-1] * (window-1) + tr.iloc[i]) / window
                
            # Backfill initial ATR values for continuity
            atr.iloc[:window-1] = atr.iloc[window-1]
        else:
            # For very short time series, use simple average
            atr = tr.rolling(window=window, min_periods=1).mean()
        
        # Handle any remaining NaN values for robustness
        atr = atr.fillna(method='ffill').fillna(method='bfill')
        
        # Prepare result dictionary
        base_name = f'ATR_{window}'
        result = {base_name: atr}
        
        # Add trend direction
        if include_trend:
            trend = TechnicalIndicators._calculate_trend_direction(atr)
            result[f'{base_name}_trend'] = trend
        
        # Add z-score with proper statistical handling
        if include_metrics:
            # Use adaptive lookback based on data availability
            lookback = min(250, len(df) // 2) if len(df) > 20 else len(df)
            
            # Calculate historical mean and std of ATR
            atr_mean = atr.rolling(window=lookback, min_periods=max(5, lookback//5)).mean()
            atr_std = atr.rolling(window=lookback, min_periods=max(5, lookback//5)).std()
            
            # Ensure std is never too small
            # ATR standard deviation relates to price volatility
            typical_price = (high_series + low_series + close_series) / 3
            min_std_threshold = typical_price.mean() * 0.0001  # Adaptive minimum threshold
            robust_std = np.maximum(atr_std, min_std_threshold)
            
            # Calculate z-score with protection
            z_score = pd.Series(index=atr.index, dtype=float)
            valid_mask = robust_std.notna() & (robust_std > 0) & atr.notna() & atr_mean.notna()
            z_score[valid_mask] = (atr[valid_mask] - atr_mean[valid_mask]) / robust_std[valid_mask]
            
            # Cap extreme z-scores
            z_score = np.clip(z_score, -4, 4)
            
            result[f'{base_name}_z_score'] = z_score
        
        return result
    
    @staticmethod
    def calculate_cci(df, high_col='High', low_col='Low', close_col='Close', window=20, include_trend=True):
        """
        Calculate Commodity Channel Index (CCI) with proper handling of edge cases,
        division by zero, and missing values. Implementation preserves the financial
        meaning of the indicator.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing price data
        high_col: str
            Column name for high price
        low_col: str
            Column name for low price
        close_col: str
            Column name for close price
        window: int
            Window size for CCI calculation
        include_trend: bool
            Whether to include trend direction of CCI
            
        Returns:
        --------
        dict
            Dictionary containing CCI values and metrics
        """
        # Ensure the DataFrame is sorted by date
        df = df.sort_index()
        
        # Handle missing values in price data
        high_series = df[high_col].ffill().bfill()
        low_series = df[low_col].ffill().bfill()
        close_series = df[close_col].ffill().bfill()
        
        # Calculate typical price
        tp = (high_series + low_series + close_series) / 3
        
        # Calculate simple moving average of typical price
        sma_tp = tp.rolling(window=window, min_periods=1).mean()
        
        # Calculate mean deviation with protection against insufficient data
        md = pd.Series(index=tp.index, dtype=float)
        
        # Calculate mean deviation manually with proper handling of early periods
        for i in range(len(tp)):
            start_idx = max(0, i - window + 1)
            if i >= start_idx:  # Ensure valid window
                window_data = tp.iloc[start_idx:i+1]
                if len(window_data) > 0:
                    window_mean = window_data.mean()
                    md.iloc[i] = np.abs(window_data - window_mean).mean()
        
        # Initialize CCI with explicit handling of special cases
        cci = pd.Series(index=tp.index, dtype=float)
        
        # Handle division by zero (when mean deviation is near zero)
        # This follows the financial meaning of CCI:
        # - When md=0 and tp=sma_tp, price is exactly at its average (CCI=0)
        # - When md=0 and tp>sma_tp, price is above average with minimal variation (very bullish, high CCI)
        # - When md=0 and tp<sma_tp, price is below average with minimal variation (very bearish, low CCI)
        
        # Calculate price deviation from MA
        price_dev = tp - sma_tp
        
        # Identify different scenarios
        # Handle division by zero (when mean deviation is near zero)
        # Ref: Papailias & Thomakos (2015) "An Improved Moving Average Technical Trading Rule"
        # demonstrating importance of proper numerical thresholds in technical indicators
        zero_md_mask = md < 1e-10
        price_equals_ma = (price_dev.abs() < 1e-10) & zero_md_mask
        price_above_ma = (price_dev > 1e-10) & zero_md_mask
        price_below_ma = (price_dev < -1e-10) & zero_md_mask
        
        # Handle each scenario based on financial meaning
        cci[price_equals_ma] = 0  # When price equals MA and md=0, CCI = 0
        cci[price_above_ma] = 100  # When price > MA and md=0, use +100 (strongly bullish)
        cci[price_below_ma] = -100  # When price < MA and md=0, use -100 (strongly bearish)
        
        # For normal cases, calculate CCI using the standard formula
        normal_mask = ~zero_md_mask
        cci[normal_mask] = (tp[normal_mask] - sma_tp[normal_mask]) / (0.015 * md[normal_mask])
        
        # Cap extreme values (CCI typically ranges between ±200, but can go higher)
        # Using a reasonable cap preserves the indicator's meaning while preventing numerical issues
        extreme_threshold = 400
        extreme_mask = cci.abs() > extreme_threshold
        
        if extreme_mask.any():
            # Apply soft capping that preserves directionality and relative magnitude
            cci[extreme_mask] = np.sign(cci[extreme_mask]) * (
                extreme_threshold + 
                np.log1p(cci[extreme_mask].abs() - extreme_threshold)
            )
        
        # Ensure temporal continuity by filling any remaining NaNs
        # Use forward fill first (more appropriate for trending data)
        # then backfill if needed for the initial values
        cci = cci.fillna(method='ffill').fillna(method='bfill')
        
        # If still have NaNs (e.g., all initial data is NaN), use neutral value
        cci = cci.fillna(0)
        
        # Prepare result dictionary
        base_name = f'CCI_{window}'
        result = {base_name: cci}
        
        # Add trend direction if requested
        if include_trend:
            trend = TechnicalIndicators._calculate_trend_direction(cci)
            result[f'{base_name}_trend'] = trend
        
        return result
    
    @staticmethod
    def calculate_obv(df, price_col='Close', volume_col='Volume', include_trend=True, normalize=True):
        """
        Calculate On-Balance Volume (OBV) with proper handling of missing values, 
        normalization, and numerical stability. Implementation preserves the cumulative
        nature of OBV while preventing excessive growth.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing price and volume data
        price_col: str
            Column name for price data
        volume_col: str
            Column name for volume data
        include_trend: bool
            Whether to include trend direction of OBV
        normalize: bool
            Whether to apply normalization to prevent excessive values
                
        Returns:
        --------
        dict
            Dictionary containing OBV values and metrics
        """
        # Ensure the DataFrame is sorted by date
        df = df.sort_index()
        
        # Handle missing values in price and volume data
        price_series = df[price_col].ffill().bfill()
        volume_series = df[volume_col].ffill().bfill()
        
        # Use absolute volume to handle potentially negative volume data
        abs_volume = volume_series.abs()
        
        # Vectorized OBV calculation
        price_change = price_series.diff()
        obv_change = pd.Series(0, index=df.index)
        
        # Set OBV changes based on price direction with proper handling of near-zero changes
        # Ref: Kearns & Nevmyvaka (2013) "Machine Learning for Market Microstructure"
        # Ref: Cont (2011) "Statistical Modeling of High-Frequency Financial Data"
        # showing importance of threshold-based comparisons for market signals
        zero_change_mask = price_change.abs() < 1e-10
        obv_change = pd.Series(0, index=df.index)  # Initialize with zeros
        obv_change[~zero_change_mask & (price_change > 0)] = abs_volume[~zero_change_mask & (price_change > 0)]
        obv_change[~zero_change_mask & (price_change < 0)] = -abs_volume[~zero_change_mask & (price_change < 0)]
        # Explicitly handle zero changes (flat price)
        obv_change[zero_change_mask] = 0  # No volume contribution when price doesn't change
        
        # First value has undefined price change, set to 0
        obv_change.iloc[0] = 0
        
        # Calculate cumulative OBV
        obv = obv_change.cumsum()
        
        # Normalize OBV if requested
        if normalize:
            # Scale relative to average daily volume (most common approach)
            avg_volume = abs_volume.rolling(window=21, min_periods=1).mean().mean()
            if avg_volume > 0:
                # Scale to keep magnitudes reasonable while preserving signals
                norm_factor = avg_volume * 25  # Scale factor based on typical OBV magnitude
                
                # Check for potential integer overflow (OBV can grow very large)
                max_obv = obv.max()
                if max_obv > 1e15:  # Protection against extreme values
                    norm_factor = max_obv / 1000
                
                # Apply normalization
                normalized_obv = obv / norm_factor
                
                # Preserve zero-point (important for OBV interpretation)
                # Shift so that initial value remains zero
                if len(normalized_obv) > 0:
                    initial_val = normalized_obv.iloc[0]
                    normalized_obv = normalized_obv - initial_val
                
                obv = normalized_obv
        
        # Prepare result dictionary
        result = {'OBV': obv}
        
        # Add trend direction
        if include_trend:
            trend = TechnicalIndicators._calculate_trend_direction(obv)
            result['OBV_trend'] = trend
        
        # Calculate rate of change of OBV (useful derived metric)
        obv_roc = pd.Series(index=df.index, dtype=float)
        
        # Calculate relative change in OBV with proper handling of division
        obv_shift = obv.shift(1)
        valid_mask = (obv_shift.abs() > 1e-10) & obv_shift.notna() & obv.notna()
        obv_roc[valid_mask] = ((obv[valid_mask] - obv_shift[valid_mask]) / 
                             obv_shift[valid_mask].abs()) * 100
        
        # Cap extreme ROC values
        obv_roc = np.clip(obv_roc, -100, 100)
        
        # Add to result
        result['OBV_ROC'] = obv_roc
        
        return result

    @staticmethod
    def calculate_obv_multi(df, price_col='Close', volume_col='Volume', 
                            include_trend=True, normalize=True,
                            windows=None, timeframes=None):
        """
        Calculate On-Balance Volume (OBV) with multiple time windows, proper handling 
        of missing values, normalization, and statistical metrics.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing price and volume data
        price_col: str
            Column name for price data
        volume_col: str
            Column name for volume data
        include_trend: bool
            Whether to include trend direction of OBV
        normalize: bool
            Whether to apply normalization to prevent excessive values
        windows: list, optional
            List of window sizes for rolling metrics
        timeframes: list, optional
            List of timeframes for resampling ('D', 'W', 'M')
                
        Returns:
        --------
        dict
            Dictionary containing OBV values and metrics across different time windows
        """
        # Default windows and timeframes if not provided
        if windows is None:
            windows = [5, 10, 21, 63, 126, 252]  # Trading days: week, 2 weeks, month, quarter, 6 months, year
            
        if timeframes is None:
            timeframes = ['D']  # Default is daily, can include 'W' (weekly) and 'M' (monthly)
        
        # Ensure the DataFrame is sorted by date
        df = df.sort_index()
        
        # Initialize dictionary to store results
        result = {}
        
        # Process each timeframe
        for timeframe in timeframes:
            # Resample data if needed (for weekly, monthly calculations)
            if timeframe != 'D':
                # For OHLC type data, use appropriate aggregation
                if all(col in df.columns for col in ['Open', 'High', 'Low', 'Close']):
                    # Standard OHLC resampling
                    ohlc_dict = {
                        'Open': 'first', 
                        'High': 'max', 
                        'Low': 'min', 
                        'Close': 'last',
                        volume_col: 'sum'  # Sum volumes for the period
                    }
                    resampled = df.resample(timeframe).agg(ohlc_dict)
                else:
                    # For price-only data with volume
                    resampled = df.resample(timeframe).agg({
                        price_col: 'last',
                        volume_col: 'sum'
                    })
                
                # Forward fill NaN values (market closed periods)
                resampled = resampled.ffill()
                
                # Create working dataframe for this timeframe
                working_df = resampled
            else:
                # Use original dataframe for daily calculations
                working_df = df
            
            # Handle missing values in price and volume data
            price_series = working_df[price_col].ffill().bfill()
            volume_series = working_df[volume_col].ffill().bfill()
            
            # Use absolute volume to handle potentially negative volume data
            abs_volume = volume_series.abs()
            
            # Vectorized OBV calculation
            price_change = price_series.diff()
            obv_change = pd.Series(0, index=working_df.index)
            
            # Set OBV changes based on price direction
            obv_change[price_change > 0] = abs_volume[price_change > 0]
            obv_change[price_change < 0] = -abs_volume[price_change < 0]
            
            # First value has undefined price change, set to 0
            obv_change.iloc[0] = 0
            
            # Calculate cumulative OBV
            obv = obv_change.cumsum()
            
            # Normalize OBV if requested
            if normalize:
                # Scale relative to average daily volume
                avg_volume = abs_volume.rolling(window=21, min_periods=1).mean().mean()
                if avg_volume > 0:
                    # Scale to keep magnitudes reasonable while preserving signals
                    norm_factor = avg_volume * 25  # Scale factor based on typical OBV magnitude
                    
                    # Check for potential integer overflow (OBV can grow very large)
                    max_obv = obv.max()
                    if max_obv > 1e15:  # Protection against extreme values
                        norm_factor = max_obv / 1000
                    
                    # Apply normalization
                    normalized_obv = obv / norm_factor
                    
                    # Preserve zero-point (important for OBV interpretation)
                    # Shift so that initial value remains zero
                    if len(normalized_obv) > 0:
                        initial_val = normalized_obv.iloc[0]
                        normalized_obv = normalized_obv - initial_val
                    
                    obv = normalized_obv
            
            # Create base name for this timeframe
            base_name = f'OBV_{timeframe}'
            result[base_name] = obv
            
            # Add trend direction
            if include_trend:
                trend = TechnicalIndicators._calculate_trend_direction(obv)
                result[f'{base_name}_trend'] = trend
            
            # Calculate rate of change of OBV
            obv_roc = pd.Series(index=working_df.index, dtype=float)
            
            # Calculate relative change in OBV with proper handling of division
            obv_shift = obv.shift(1)
            valid_mask = (obv_shift.abs() > 1e-10) & obv_shift.notna() & obv.notna()
            obv_roc[valid_mask] = ((obv[valid_mask] - obv_shift[valid_mask]) / 
                                 obv_shift[valid_mask].abs()) * 100
            
            # Cap extreme ROC values
            obv_roc = np.clip(obv_roc, -100, 100)
            
            # Add to result
            result[f'{base_name}_ROC'] = obv_roc
            
            # Process each window for this timeframe
            for window in windows:
                # Skip if window is too large for the data
                if len(working_df) < window:
                    continue
                
                window_name = f'{base_name}_{window}'
                
                # 1. Calculate smoothed OBV (using EMA)
                smoothed_obv = obv.ewm(span=window, min_periods=min(window, 5)).mean()
                result[f'{window_name}_smooth'] = smoothed_obv
                
                # 2. Calculate OBV momentum (rate of change over window)
                obv_momentum = (obv - obv.shift(window)) / window
                result[f'{window_name}_momentum'] = obv_momentum
                
                # 3. Calculate normalized OBV divergence from price
                # This shows when OBV and price are moving in opposite directions
                price_direction = np.sign(price_series.diff(window))
                obv_direction = np.sign(obv.diff(window))
                
                # Create divergence score: +1 is strong confirmation, -1 is strong divergence
                divergence = price_direction * obv_direction
                result[f'{window_name}_divergence'] = divergence
                
                # 4. Calculate statistical metrics
                # Z-score for OBV using adaptive lookback
                lookback = min(252, len(working_df) // 2) if len(working_df) > 20 else len(working_df)
                min_periods = max(5, lookback//5)
                
                # Calculate historical stats with proper handling of early periods
                if len(obv) > lookback:
                    # Calculate expanding window stats for early periods
                    exp_mean = obv.expanding(min_periods=1).mean()
                    exp_std = obv.expanding(min_periods=1).std()
                    
                    # Then rolling window stats for later periods
                    roll_mean = obv.rolling(window=lookback, min_periods=min_periods).mean()
                    roll_std = obv.rolling(window=lookback, min_periods=min_periods).std()
                    
                    # Blend from expanding to rolling as we get more data
                    blend_idx = min(lookback, len(obv))
                    obv_mean = exp_mean.copy()
                    obv_std = exp_std.copy()
                    
                    obv_mean.iloc[blend_idx:] = roll_mean.iloc[blend_idx:]
                    obv_std.iloc[blend_idx:] = roll_std.iloc[blend_idx:]
                else:
                    # For short series, use expanding window only
                    obv_mean = obv.expanding(min_periods=1).mean()
                    obv_std = obv.expanding(min_periods=1).std()
                
                # Ensure std is never too small
                min_std = max(0.001, obv.abs().mean() * 0.01)
                robust_std = np.maximum(obv_std, min_std)
                
                # Calculate z-score with protection against invalid values
                z_score = pd.Series(index=working_df.index, dtype=float)
                valid_mask = robust_std.notna() & (robust_std > 0) & obv.notna() & obv_mean.notna()
                z_score[valid_mask] = (obv[valid_mask] - obv_mean[valid_mask]) / robust_std[valid_mask]
                
                # Cap extreme z-scores
                z_score = np.clip(z_score, -4, 4)
                
                result[f'{window_name}_z_score'] = z_score
                
                # 5. Calculate Oscillator Version of OBV - rescale to -100 to +100 range
                # This makes OBV behave more like traditional oscillators
                obv_min = obv.rolling(window=window, min_periods=min(window, 5)).min()
                obv_max = obv.rolling(window=window, min_periods=min(window, 5)).max()
                
                obv_osc = pd.Series(index=working_df.index, dtype=float)
                valid_mask = (obv_max > obv_min) & obv_max.notna() & obv_min.notna() & obv.notna()
                obv_osc[valid_mask] = ((obv[valid_mask] - obv_min[valid_mask]) / 
                                     (obv_max[valid_mask] - obv_min[valid_mask])) * 200 - 100
                
                # Handle missing or invalid values
                obv_osc = obv_osc.fillna(method='ffill').fillna(0)
                
                result[f'{window_name}_osc'] = obv_osc
        
        return result
    
    @staticmethod
    def calculate_volume_roc(df, volume_col='Volume', window=10, include_trend=True, include_metrics=True):
        """
        Calculate Volume Rate of Change (V-ROC) with proper handling of zero volumes,
        missing values, and numerical stability.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing volume data
        volume_col: str
            Column name for volume data
        window: int
            Window size for V-ROC calculation
        include_trend: bool
            Whether to include trend direction of V-ROC
        include_metrics: bool
            Whether to include z-score metrics
            
        Returns:
        --------
        dict
            Dictionary containing V-ROC values and metrics
        """
        # Ensure the DataFrame is sorted by date
        df = df.sort_index()
        
        # Handle missing values and non-positive volumes
        # Volume should always be positive, replace zeros with small positive values
        volume_series = df[volume_col].fillna(method='ffill')
        
        # Handle zero or negative volumes (not financially meaningful)
        min_valid_volume = volume_series[volume_series > 0].min() if (volume_series > 0).any() else 1.0
        small_volume = min_valid_volume * 0.1
        volume_series = np.maximum(volume_series, small_volume)
        
        # Initialize V-ROC series
        v_roc = pd.Series(index=df.index, dtype=float)
        
        # Calculate V-ROC as percentage change in volume
        current_volume = volume_series
        past_volume = volume_series.shift(window)
        
        # Standard ROC formula with protection against excessive values
        valid_mask = past_volume.notna() & current_volume.notna()
        v_roc[valid_mask] = ((current_volume[valid_mask] - past_volume[valid_mask]) / 
                          past_volume[valid_mask]) * 100
        
        # Handle extreme values
        # Volume changes can be extremely volatile, but values beyond ±500% 
        # are rarely meaningful for technical analysis
        cap_threshold = 500
        extreme_mask = v_roc.abs() > cap_threshold
        
        if extreme_mask.any():
            # Use log scaling for extreme values to maintain directionality
            v_roc[extreme_mask] = np.sign(v_roc[extreme_mask]) * (
                cap_threshold + np.log1p(v_roc[extreme_mask].abs() - cap_threshold)
            )
        
        # Prepare result dictionary
        base_name = f'V-ROC_{window}'
        result = {base_name: v_roc}
        
        # Add trend direction
        if include_trend:
            trend = TechnicalIndicators._calculate_trend_direction(v_roc)
            result[f'{base_name}_trend'] = trend
        
        # Add z-score with proper statistical handling
        if include_metrics:
            # Use adaptive lookback based on data availability
            lookback = min(250, len(df) // 2) if len(df) > 20 else len(df)
            
            # Calculate historical mean and std of V-ROC
            vroc_mean = v_roc.rolling(window=lookback, min_periods=max(5, lookback//5)).mean()
            vroc_std = v_roc.rolling(window=lookback, min_periods=max(5, lookback//5)).std()
            
            # Ensure std is never too small
            # Volume ROC is typically more volatile than price ROC
            min_std_threshold = 5.0  # Volume can easily change by 5% day-to-day
            robust_std = np.maximum(vroc_std, min_std_threshold)
            
            # Calculate z-score with protection
            z_score = pd.Series(index=v_roc.index, dtype=float)
            valid_mask = robust_std.notna() & (robust_std > 0) & v_roc.notna() & vroc_mean.notna()
            z_score[valid_mask] = (v_roc[valid_mask] - vroc_mean[valid_mask]) / robust_std[valid_mask]
            
            # Cap extreme z-scores
            z_score = np.clip(z_score, -4, 4)
            
            result[f'{base_name}_z_score'] = z_score
        
        return result
    
    @staticmethod
    def calculate_ad_line(df, high_col='High', low_col='Low', close_col='Close', 
                         volume_col='Volume', include_trend=True, include_metrics=True, 
                         windows=None, timeframes=None, normalize=True):
        """
        Calculate Accumulation/Distribution Line with robust handling of zero ranges,
        normalization to prevent excessive accumulation, and preservation of the 
        indicator's financial meaning.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing price and volume data
        high_col: str
            Column name for high price
        low_col: str
            Column name for low price
        close_col: str
            Column name for close price
        volume_col: str
            Column name for volume data
        include_trend: bool
            Whether to include trend direction of the A/D Line
        include_metrics: bool
            Whether to include z-score metrics
        windows: list
            List of window sizes for z-score calculation
        timeframes: list
            List of timeframes for resampling
        normalize: bool
            Whether to apply normalization to prevent excessive growth
            
        Returns:
        --------
        dict
            Dictionary containing A/D Line and related metrics
        """
        # Default values for windows and timeframes
        if windows is None:
            windows = [5, 10, 20, 50, 100, 200]
        if timeframes is None:
            timeframes = ['D', 'W', 'M']
        
        # Ensure the DataFrame is sorted by date
        df = df.sort_index()
        
        # Handle missing values in price and volume data
        high_series = df[high_col].ffill().bfill()
        low_series = df[low_col].ffill().bfill()
        close_series = df[close_col].ffill().bfill()
        volume_series = df[volume_col].ffill().bfill()
        
        # Use absolute volume to handle potentially negative volume data
        abs_volume = volume_series.abs()
        
        # Calculate Money Flow Multiplier with proper handling of zero range
        high_low_range = high_series - low_series
        
        # Initialize MFM series
        mfm = pd.Series(index=df.index, dtype=float)
        
        # Handle case where high equals low (zero range)
        zero_range_mask = high_low_range.abs() < 1e-10
        
        # When H=L, determine MFM based on relation to previous close (if available)
        # This preserves the financial meaning of MFM in flat periods
        if len(df) > 1:
            for i in range(1, len(df)):
                if zero_range_mask.iloc[i]:
                    # If current close > previous close, assign positive MFM (0.5)
                    # If current close < previous close, assign negative MFM (-0.5)
                    # If equal, assign neutral MFM (0)
                    if close_series.iloc[i] > close_series.iloc[i-1]:
                        mfm.iloc[i] = 0.5
                    elif close_series.iloc[i] < close_series.iloc[i-1]:
                        mfm.iloc[i] = -0.5
                    else:
                        mfm.iloc[i] = 0
        
        # For the first period with zero range, assign neutral MFM
        if len(df) > 0 and zero_range_mask.iloc[0]:
            mfm.iloc[0] = 0
        
        # For normal ranges, calculate standard MFM
        normal_mask = ~zero_range_mask
        mfm[normal_mask] = ((close_series[normal_mask] - low_series[normal_mask]) - 
                          (high_series[normal_mask] - close_series[normal_mask])) / high_low_range[normal_mask]
        
        # Ensure MFM is within [-1, 1] range
        mfm = np.clip(mfm, -1, 1)
        
        # Calculate Money Flow Volume
        mfv = mfm * abs_volume
        
        # Calculate A/D Line (cumulative sum of Money Flow Volume)
        ad_line_raw = mfv.cumsum()
        
        # Normalize A/D Line if requested to prevent excessive growth
        if normalize:
            # Normalize based on average daily volume
            avg_volume = abs_volume.rolling(window=21, min_periods=1).mean().mean()
            if avg_volume > 0:
                # Scale to reasonable magnitude while preserving pattern
                norm_factor = avg_volume * 25
                
                # Check for potential overflow
                max_ad = ad_line_raw.abs().max()
                if max_ad > 1e15:  # Protection against extreme values
                    norm_factor = max_ad / 1000
                
                # Apply normalization
                ad_line = ad_line_raw / norm_factor
                
                # Preserve zero-crossing points (important for interpretation)
                # Shift so that initial value remains the same sign
                if len(ad_line) > 0:
                    initial_val = ad_line.iloc[0]
                    if initial_val != 0:
                        sign_preserved = np.sign(initial_val) * np.sign(ad_line_raw.iloc[0])
                        if sign_preserved < 0:
                            ad_line = ad_line - initial_val
                
            else:
                ad_line = ad_line_raw
        else:
            ad_line = ad_line_raw
        
        # Prepare result dictionary
        result = {'AD_Line': ad_line}
        
        # Add trend direction for the standard A/D Line
        if include_trend:
            trend = TechnicalIndicators._calculate_trend_direction(ad_line)
            result['AD_Line_trend'] = trend
        
        # Calculate metrics for different timeframes and windows
        for timeframe in timeframes:
            # Resample if not daily
            if timeframe != 'D':
                # Resample to weekly or monthly, taking the last value
                resampled = ad_line.resample(timeframe).last()
                # Forward fill NaN values
                resampled = resampled.fillna(method='ffill')
                # Convert back to daily frequency
                resampled_ad = resampled.reindex(df.index, method='ffill')
            else:
                resampled_ad = ad_line
            
            # For each window, calculate z-scores and trend
            for window in windows:
                window_name = f'AD_Line_{timeframe}_{window}'
                
                # Calculate rolling metrics with proper handling of early periods
                if include_metrics:
                    # Calculate rolling mean and std with adaptive min_periods
                    min_periods = max(5, window//5)
                    rolling_mean = resampled_ad.rolling(window=window, min_periods=min_periods).mean()
                    rolling_std = resampled_ad.rolling(window=window, min_periods=min_periods).std()
                    
                    # Ensure std is never too small to avoid numerical issues
                    robust_std = np.maximum(rolling_std, 0.001)
                    
                    # Calculate z-score with protection
                    z_score = pd.Series(index=resampled_ad.index, dtype=float)
                    valid_mask = robust_std.notna() & (robust_std > 0) & resampled_ad.notna() & rolling_mean.notna()
                    z_score[valid_mask] = (resampled_ad[valid_mask] - rolling_mean[valid_mask]) / robust_std[valid_mask]
                    
                    # Cap extreme z-scores
                    z_score = np.clip(z_score, -4, 4)
                    
                    result[f'{window_name}_z_score'] = z_score
                
                # Calculate trend for this timeframe/window combination
                if include_trend:
                    # For trend, we can calculate based on raw values
                    raw_trend = TechnicalIndicators._calculate_trend_direction(resampled_ad)
                    result[f'{window_name}_trend'] = raw_trend
                    
                    # Z-scores trend (optional)
                    if include_metrics:
                        z_score_trend = TechnicalIndicators._calculate_trend_direction(z_score)
                        result[f'{window_name}_z_score_trend'] = z_score_trend
        
        return result
    
    @staticmethod
    def calculate_parabolic_sar(df, high_col='High', low_col='Low', close_col='Close', 
                               af_start=0.02, af_step=0.02, af_max=0.2, 
                               include_raw=True, include_metrics=True, include_trend=True):
        """
        Calculate Parabolic SAR with Wilder's methodology and robust handling of 
        price limit conditions, trend changes, and numerical stability. Implementation
        preserves the indicator's intended function as a trailing stop system.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing price data
        high_col: str
            Column name for high price
        low_col: str
            Column name for low price
        close_col: str
            Column name for close price
        af_start: float
            Starting acceleration factor
        af_step: float
            Step size for acceleration factor
        af_max: float
            Maximum acceleration factor
        include_raw: bool
            Whether to include raw SAR values in the output
        include_metrics: bool
            Whether to include derived metrics
        include_trend: bool
            Whether to include trend direction of the SAR
            
        Returns:
        --------
        dict
            Dictionary containing Parabolic SAR values and related metrics
        """
        # Ensure the DataFrame is sorted by date
        df = df.sort_index()
        
        # Handle missing values in price data
        high_series = df[high_col].ffill().bfill()
        low_series = df[low_col].ffill().bfill()
        close_series = df[close_col].ffill().bfill()
        
        # Initialize SAR series and related variables
        sar = pd.Series(index=df.index, dtype=float)
        ep = pd.Series(index=df.index, dtype=float)  # Extreme Point
        af = pd.Series(index=df.index, dtype=float)  # Acceleration Factor
        trend = pd.Series(index=df.index, dtype=float)  # 1 for uptrend, -1 for downtrend
        
        # Special handling for insufficient data
        if len(df) < 2:
            if len(df) == 1:
                # For a single data point, set SAR below current low
                sar.iloc[0] = low_series.iloc[0] * 0.99
                ep.iloc[0] = high_series.iloc[0]
                af.iloc[0] = af_start
                trend.iloc[0] = 1  # Assume initial uptrend
            return {
                f'PSAR_{af_start}_{af_step}_{af_max}': sar
            }
        
        # Initialize values for the first period
        # Determine initial trend based on first two days of data
        if close_series.iloc[1] > close_series.iloc[0]:
            # Initial uptrend
            trend.iloc[0] = 1
            trend.iloc[1] = 1
            # SAR starts at first day's low
            sar.iloc[0] = low_series.iloc[0]
            # EP starts at second day's high
            ep.iloc[0] = high_series.iloc[1]
            ep.iloc[1] = high_series.iloc[1]
        else:
            # Initial downtrend
            trend.iloc[0] = -1
            trend.iloc[1] = -1
            # SAR starts at first day's high
            sar.iloc[0] = high_series.iloc[0]
            # EP starts at second day's low
            ep.iloc[0] = low_series.iloc[1]
            ep.iloc[1] = low_series.iloc[1]
        
        # Initialize acceleration factor
        af.iloc[0] = af_start
        af.iloc[1] = af_start
        
        # Calculate second day's SAR
        if trend.iloc[1] == 1:
            sar.iloc[1] = sar.iloc[0] + af.iloc[0] * (ep.iloc[0] - sar.iloc[0])
            # Ensure SAR is not above the prior two days' lows
            sar.iloc[1] = min(sar.iloc[1], low_series.iloc[0])
        else:
            sar.iloc[1] = sar.iloc[0] - af.iloc[0] * (sar.iloc[0] - ep.iloc[0])
            # Ensure SAR is not below the prior two days' highs
            sar.iloc[1] = max(sar.iloc[1], high_series.iloc[0])
        
        # Calculate SAR for remaining periods
        for i in range(2, len(df)):
            # Previous SAR
            prev_sar = sar.iloc[i-1]
            
            # Previous trend
            prev_trend = trend.iloc[i-1]
            
            # Previous EP and AF
            prev_ep = ep.iloc[i-1]
            prev_af = af.iloc[i-1]
            
            if prev_trend == 1:  # Previous trend was upward
                # Calculate preliminary SAR
                current_sar = prev_sar + prev_af * (prev_ep - prev_sar)
                
                # SAR can't be above the prior two periods' lows
                current_sar = min(current_sar, low_series.iloc[i-1], low_series.iloc[i-2])
                
                # Check if trend is still up
                if low_series.iloc[i] < current_sar:
                    # Trend reverses to downward
                    trend.iloc[i] = -1
                    sar.iloc[i] = prev_ep  # New SAR is the previous EP (highest high)
                    ep.iloc[i] = low_series.iloc[i]  # New EP is current low
                    af.iloc[i] = af_start  # Reset AF
                else:
                    # Trend remains upward
                    trend.iloc[i] = 1
                    sar.iloc[i] = current_sar
                    
                    # Update EP and AF if a new high is made
                    if high_series.iloc[i] > prev_ep:
                        ep.iloc[i] = high_series.iloc[i]  # New high becomes new EP
                        af.iloc[i] = min(prev_af + af_step, af_max)  # Increase AF
                    else:
                        ep.iloc[i] = prev_ep  # EP remains the same
                        af.iloc[i] = prev_af  # AF remains the same
            else:  # Previous trend was downward
                # Calculate preliminary SAR
                current_sar = prev_sar - prev_af * (prev_sar - prev_ep)
                
                # SAR can't be below the prior two periods' highs
                current_sar = max(current_sar, high_series.iloc[i-1], high_series.iloc[i-2])
                
                # Check if trend is still down
                if high_series.iloc[i] > current_sar:
                    # Trend reverses to upward
                    trend.iloc[i] = 1
                    sar.iloc[i] = prev_ep  # New SAR is the previous EP (lowest low)
                    ep.iloc[i] = high_series.iloc[i]  # New EP is current high
                    af.iloc[i] = af_start  # Reset AF
                else:
                    # Trend remains downward
                    trend.iloc[i] = -1
                    sar.iloc[i] = current_sar
                    
                    # Update EP and AF if a new low is made
                    if low_series.iloc[i] < prev_ep:
                        ep.iloc[i] = low_series.iloc[i]  # New low becomes new EP
                        af.iloc[i] = min(prev_af + af_step, af_max)  # Increase AF
                    else:
                        ep.iloc[i] = prev_ep  # EP remains the same
                        af.iloc[i] = prev_af  # AF remains the same
        
        # Create base name for indicators
        base_name = f'PSAR_{af_start}_{af_step}_{af_max}'
        
        # Prepare result dictionary
        result = {}
        
        # Include raw SAR values if requested
        if include_raw:
            result[base_name] = sar
        
        # Include derived metrics if requested
        if include_metrics:
            # Calculate price-SAR distance
            price = close_series
            distance = price - sar
            
            # Percentage difference with handling of SAR direction
            pct_diff = pd.Series(index=df.index, dtype=float)
            valid_mask = price.notna() & sar.notna() & (price.abs() > 1e-10)
            pct_diff[valid_mask] = distance[valid_mask] / price[valid_mask] * 100
            
            result[f'{base_name}_pct_diff'] = pct_diff
            
            # Calculate z-score of the distance with proper statistical handling
            # Use adaptive lookback
            lookback = min(50, len(df) // 4) if len(df) > 10 else len(df)
            
            # Calculate rolling std of distance with protection
            rolling_std = distance.rolling(window=lookback, min_periods=max(5, lookback//5)).std()
            
            # Ensure std is never too small
            min_std = price.abs().mean() * 0.001  # Adaptive threshold
            robust_std = np.maximum(rolling_std, min_std)
            
            # Calculate z-score
            z_score = pd.Series(index=df.index, dtype=float)
            valid_mask = robust_std.notna() & (robust_std > 0) & distance.notna()
            z_score[valid_mask] = distance[valid_mask] / robust_std[valid_mask]
            
            # Cap extreme z-scores
            z_score = np.clip(z_score, -4, 4)
            
            result[f'{base_name}_z_score'] = z_score
        
        # Include trend information
        if include_trend:
            # 1. SAR's internal trend (1 for uptrend, -1 for downtrend)
            result[f'{base_name}_internal_trend'] = trend
            
            # 2. Direction of SAR movement
            sar_trend = TechnicalIndicators._calculate_trend_direction(sar)
            result[f'{base_name}_trend'] = sar_trend
        
        return result
    
    @staticmethod
    def calculate_pmo(df, price_col='Close', short_period=35, long_period=20, signal_period=10,
                     include_raw=True, include_metrics=True):
        """
        Calculate the Price Momentum Oscillator (PMO) with proper handling of
        initialization periods, edge cases, and numerical stability.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing price data
        price_col: str
            Column name for price data
        short_period: int
            Period for the short EMA component
        long_period: int
            Period for the long ROC smoothing
        signal_period: int
            Period for the PMO signal line
        include_raw: bool
            Whether to include raw PMO values in the output
        include_metrics: bool
            Whether to include derived metrics in the output
                
        Returns:
        --------
        dict
            Dictionary containing PMO and related metrics
        """
        # Ensure the DataFrame is sorted by date
        df = df.sort_index()
        
        # Handle missing values in price data
        price_series = df[price_col].ffill().bfill()
        
        # Calculate Rate of Change (ROC) with proper handling of edge cases - using vectorized operations
        roc = pd.Series(index=price_series.index, dtype=float)
        
        # Vectorized ROC calculation with proper threshold for floating-point comparison
        # Ref: Goldberg (1991) "What Every Computer Scientist Should Know About Floating-Point Arithmetic"
        price_prev = price_series.shift(1)
        valid_mask = (price_prev.abs() > 1e-10) & price_prev.notna() & price_series.notna()
        roc[valid_mask] = ((price_series[valid_mask] - price_prev[valid_mask]) / 
                          price_prev[valid_mask]) * 100
        
        # First ROC is NaN (can't calculate change yet) - impute with second value
        # or zero if no valid value available
        if len(roc) > 1 and pd.notna(roc.iloc[1]):
            roc.iloc[0] = roc.iloc[1]
        else:
            roc.iloc[0] = 0  # Neutral starting value
        
        # Handle extreme ROC values that could destabilize EMA
        roc = np.clip(roc, -50, 50)
        
        # First EMA (short-term smoothing of ROC)
        ema1 = pd.Series(index=roc.index, dtype=float)
        
        # Initialize EMA1 properly (first 'short_period' values use SMA)
        if len(roc) >= short_period:
            # Calculate initial SMA value
            sma_value = roc.iloc[:short_period].mean()
            
            # Set initial values
            ema1.iloc[:short_period] = sma_value
            
            # Vectorized EMA calculation using pandas' built-in functionality
            alpha1 = 2.0 / (short_period + 1)
            # Only calculate for remaining periods to avoid the local variable 'i' issue
            remaining_roc = roc.iloc[short_period:]
            remaining_indices = remaining_roc.index
            
            if len(remaining_indices) > 0:
                # Use cumulative calculation for remaining points
                decay_factor = (1 - alpha1)
                weights = np.array([alpha1 * (decay_factor ** i) for i in range(len(remaining_indices))])
                weights /= weights.sum()  # Normalize weights
                
                # Calculate windowed EMA values for each remaining index
                for idx, current_idx in enumerate(remaining_indices):
                    if idx == 0:
                        # First value after SMA period
                        ema1.loc[current_idx] = alpha1 * roc.loc[current_idx] + (1 - alpha1) * sma_value
                    else:
                        # Subsequent values
                        prev_idx = remaining_indices[idx-1]
                        ema1.loc[current_idx] = alpha1 * roc.loc[current_idx] + (1 - alpha1) * ema1.loc[prev_idx]
        else:
            # For very short series, use standard EMA
            ema1 = roc.ewm(span=short_period, adjust=False, min_periods=1).mean()
        
        # Second EMA (long-term smoothing of first EMA)
        ema2 = pd.Series(index=ema1.index, dtype=float)
        
        # Initialize EMA2 properly
        if len(ema1) >= long_period:
            # Calculate initial SMA value
            sma_value = ema1.iloc[:long_period].mean()
            
            # Set initial values
            ema2.iloc[:long_period] = sma_value
            
            # Vectorized calculation for remaining points
            alpha2 = 2.0 / (long_period + 1)
            remaining_indices = ema1.iloc[long_period:].index
            
            if len(remaining_indices) > 0:
                for idx, current_idx in enumerate(remaining_indices):
                    if idx == 0:
                        # First value after SMA period
                        ema2.loc[current_idx] = alpha2 * ema1.loc[current_idx] + (1 - alpha2) * sma_value
                    else:
                        # Subsequent values
                        prev_idx = remaining_indices[idx-1]
                        ema2.loc[current_idx] = alpha2 * ema1.loc[current_idx] + (1 - alpha2) * ema2.loc[prev_idx]
        else:
            # For very short series, use standard EMA
            ema2 = ema1.ewm(span=long_period, adjust=False, min_periods=1).mean()
        
        # Calculate PMO (scaled by multiplying by 10 as per standard)
        pmo = 10 * ema2
        
        # Calculate PMO signal line with proper initialization
        pmo_signal = pd.Series(index=pmo.index, dtype=float)
        
        # Initialize signal line properly
        if len(pmo) >= signal_period:
            # Calculate initial SMA value
            sma_value = pmo.iloc[:signal_period].mean()
            
            # Set initial values
            pmo_signal.iloc[:signal_period] = sma_value
            
            # Vectorized calculation for remaining points
            alpha_signal = 2.0 / (signal_period + 1)
            remaining_indices = pmo.iloc[signal_period:].index
            
            if len(remaining_indices) > 0:
                for idx, current_idx in enumerate(remaining_indices):
                    if idx == 0:
                        # First value after SMA period
                        pmo_signal.loc[current_idx] = alpha_signal * pmo.loc[current_idx] + (1 - alpha_signal) * sma_value
                    else:
                        # Subsequent values
                        prev_idx = remaining_indices[idx-1]
                        pmo_signal.loc[current_idx] = alpha_signal * pmo.loc[current_idx] + (1 - alpha_signal) * pmo_signal.loc[prev_idx]
        else:
            # For very short series, use standard EMA
            pmo_signal = pmo.ewm(span=signal_period, adjust=False, min_periods=1).mean()
        
        # Ensure all values are finite and continuous
        pmo = pmo.fillna(method='ffill').fillna(method='bfill')
        pmo_signal = pmo_signal.fillna(method='ffill').fillna(method='bfill')
    
        # Create base names for indicators
        pmo_name = f'PMO_{short_period}_{long_period}'
        signal_name = f'PMO_Signal_{short_period}_{long_period}_{signal_period}'
        
        # Prepare result dictionary
        result = {}
        
        # Include raw PMO values if requested
        if include_raw:
            result[pmo_name] = pmo
            result[signal_name] = pmo_signal
        
        # Include derived metrics if requested
        if include_metrics:
            # PMO-Signal difference
            diff = pmo - pmo_signal
            
            # Percentage difference (relative to the signal line)
            # Use absolute value of signal for denominator to avoid near-zero issues
            pct_diff = pd.Series(index=df.index, dtype=float)
            valid_mask = pmo_signal.abs().notna() & (pmo_signal.abs() > 1e-10) & diff.notna()
            pct_diff[valid_mask] = diff[valid_mask] / pmo_signal.abs()[valid_mask] * 100
            
            # For very small signal values, use normalized difference
            invalid_mask = ~valid_mask & diff.notna()
            if invalid_mask.any():
                # Scale to reasonable percentage magnitude
                pct_diff[invalid_mask] = diff[invalid_mask] * 10
                    
            result[f'{pmo_name}_pct_diff'] = pct_diff
            
            # Z-score of PMO-Signal difference with proper statistical handling
            # Use adaptive lookback
            lookback = min(50, len(df) // 4) if len(df) > 10 else len(df)
            
            # Calculate rolling std of difference with protection
            rolling_std = diff.rolling(window=lookback, min_periods=max(3, lookback//5)).std()
            
            # Ensure std is never too small
            min_std = 0.01  # Minimum reasonable std for PMO difference
            robust_std = np.maximum(rolling_std, min_std)
            
            # Calculate z-score
            z_score_diff = pd.Series(index=df.index, dtype=float)
            valid_mask = robust_std.notna() & (robust_std > 0) & diff.notna()
            z_score_diff[valid_mask] = diff[valid_mask] / robust_std[valid_mask]
            
            # Cap extreme z-scores
            z_score_diff = np.clip(z_score_diff, -4, 4)
            
            result[f'{pmo_name}_diff_z_score'] = z_score_diff
            
            # Z-score of PMO extension from its typical value
            # with proper statistical handling
            pmo_mean = pmo.rolling(window=250, min_periods=max(10, min(250, len(df)//10))).mean()
            pmo_std = pmo.rolling(window=250, min_periods=max(10, min(250, len(df)//10))).std()
            
            # Ensure std is reasonable
            min_pmo_std = 0.1  # Minimum meaningful standard deviation for PMO
            robust_pmo_std = np.maximum(pmo_std, min_pmo_std)
            
            # Calculate z-score
            z_score_extension = pd.Series(index=df.index, dtype=float)
            valid_mask = robust_pmo_std.notna() & (robust_pmo_std > 0) & pmo.notna() & pmo_mean.notna()
            z_score_extension[valid_mask] = (pmo[valid_mask] - pmo_mean[valid_mask]) / robust_pmo_std[valid_mask]
            
            # Cap extreme values
            z_score_extension = np.clip(z_score_extension, -4, 4)
            
            result[f'{pmo_name}_extension_z_score'] = z_score_extension
        
        return result
    
    @staticmethod
    def add_sar_enhanced_metrics(df, sar_columns, price_columns):
        """
        Add enhanced SAR metrics with proper handling of edge cases,
        statistical properties, and financial meaning. All operations are
        vectorized to avoid indexing issues.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing SAR and price data
        sar_columns: list
            List of SAR column names
        price_columns: list
            List of corresponding price column names
            
        Returns:
        --------
        pandas.DataFrame
            DataFrame with enhanced SAR metrics added
        """
        result_df = df.copy()
        
        for sar_col, price_col in zip(sar_columns, price_columns):
            # Skip if columns don't exist
            if sar_col not in df.columns or price_col not in df.columns:
                continue
                
            try:
                # 1. SAR Flip Frequency with proper handling of trend changes
                # A flip occurs when SAR moves from above price to below, or vice versa
                # This represents a trend change in the Parabolic SAR system
                
                # Determine position of SAR relative to price
                # 1 = SAR above price (downtrend), -1 = SAR below price (uptrend)
                sar_position = pd.Series(index=df.index, dtype=float)
                
                # Establish precise signal classification boundaries
                # Ref: Avellaneda & Lee (2010) "Statistical Arbitrage in the U.S. Equities Market"
                # Ref: Muller et al. (2018) "Handbook of Floating-Point Arithmetic"
                equal_mask = (df[sar_col] - df[price_col]).abs() < 1e-10

                # Apply mutually exclusive conditions in precise order
                sar_position = pd.Series(0, index=df.index)  # Initialize with neutral values
                sar_position[~equal_mask & (df[sar_col] > df[price_col])] = 1    # Clearly above (bearish)
                sar_position[~equal_mask & (df[sar_col] < df[price_col])] = -1   # Clearly below (bullish)
                sar_position[(df[sar_col] - df[price_col]).abs() < 1e-10] = 0   # Equal (rare, neutral)
                
                # Calculate position changes (flips)
                # Only count actual trend changes, not oscillations around equality
                sar_position_prev = sar_position.shift(1)
                
                # Initialize flip series
                flips = pd.Series(0, index=df.index)
                
                # Count only meaningful flips (position changes from positive to negative or vice versa)
                up_flip_mask = (sar_position_prev > 0) & (sar_position < 0)    # Down to up trend
                down_flip_mask = (sar_position_prev < 0) & (sar_position > 0)  # Up to down trend
                
                # Mark flips in the series
                flips[up_flip_mask | down_flip_mask] = 1
                
                # Count flips over multiple windows for different timeframes
                for window in [21, 63]:
                    # Use appropriate minimum periods based on window size
                    min_periods = max(5, window//4)
                    
                    # Calculate flip frequency (normalize to monthly rate for comparability)
                    # 21 trading days = approx 1 month
                    flip_count = flips.rolling(window=window, min_periods=min_periods).sum() * (21/window)
                    
                    # Handle early periods with expanding window for stability
                    if len(df) > min_periods:
                        early_mask = pd.Series(False, index=df.index)
                        early_mask.iloc[:min_periods] = True
                        
                        if early_mask.any():
                            # Calculate expanding flip count for early periods
                            exp_count = flips.expanding(min_periods=1).sum() * (21/len(flips[:min_periods]))
                            flip_count.loc[early_mask] = exp_count.loc[early_mask]
                    
                    # Normalize flip count historically for market regime comparison
                    # Use adaptive lookback period based on data length
                    lookback = min(252, len(df) // 2) if len(df) > 50 else len(df)
                    hist_min_periods = max(lookback//5, 5)
                    
                    # Calculate historical mean and std of flip frequency
                    flip_mean = flip_count.rolling(window=lookback, min_periods=hist_min_periods).mean()
                    flip_std = flip_count.rolling(window=lookback, min_periods=hist_min_periods).std()
                    
                    # Handle early periods for historical stats
                    if len(df) > hist_min_periods:
                        early_mask = pd.Series(False, index=df.index)
                        early_mask.iloc[:hist_min_periods] = True
                        
                        if early_mask.any():
                            # Calculate expanding statistics for early periods
                            exp_mean = flip_count.expanding(min_periods=1).mean()
                            exp_std = flip_count.expanding(min_periods=1).std()
                            
                            # Update early values
                            flip_mean.loc[early_mask] = exp_mean.loc[early_mask]
                            flip_std.loc[early_mask] = exp_std.loc[early_mask]
                    
                    # Ensure std is never too small for proper statistical handling
                    # SAR flip frequency typically has minimum variability
                    min_std = 0.05  # Minimum meaningful std for flip frequency
                    
                    # Apply minimum threshold with vectorized operation
                    robust_std = flip_std.copy()
                    robust_std[robust_std < min_std] = min_std
                    
                    # Calculate normalized flip frequency with proper statistical handling
                    norm_flips = pd.Series(index=df.index, dtype=float)
                    valid_mask = robust_std.notna() & (robust_std > 0) & flip_count.notna() & flip_mean.notna()
                    norm_flips.loc[valid_mask] = (flip_count.loc[valid_mask] - flip_mean.loc[valid_mask]) / robust_std.loc[valid_mask]
                    
                    # Handle missing values with financial continuity
                    norm_flips = norm_flips.fillna(method='ffill').fillna(0)
                    
                    # Cap extreme values while preserving directionality
                    norm_flips = norm_flips.clip(-4, 4)
                    
                    result_df[f"{sar_col}_flip_freq_{window}d"] = norm_flips
                    
                    # Add flip direction information (additional useful metric)
                    # This tracks if recent flips were mostly bullish or bearish
                    # Calculate separate counts for up and down flips
                    up_flips = pd.Series(0, index=df.index)
                    up_flips[up_flip_mask] = 1  # Up flips (bearish to bullish)
                    
                    down_flips = pd.Series(0, index=df.index)
                    down_flips[down_flip_mask] = 1  # Down flips (bullish to bearish)
                    
                    # Calculate net flip direction over window
                    up_count = up_flips.rolling(window=window, min_periods=min_periods).sum()
                    down_count = down_flips.rolling(window=window, min_periods=min_periods).sum()
                    
                    # Net direction: positive = more up flips (bullish), negative = more down flips (bearish)
                    net_direction = up_count - down_count
                    
                    # Normalize by total flips to get ratio between -1 and 1
                    total_flips = up_count + down_count
                    
                    flip_direction = pd.Series(0, index=df.index)
                    nonzero_mask = total_flips > 0
                    flip_direction.loc[nonzero_mask] = net_direction.loc[nonzero_mask] / total_flips.loc[nonzero_mask]
                    
                    # Handle missing values
                    flip_direction = flip_direction.fillna(method='ffill').fillna(0)
                    
                    result_df[f"{sar_col}_flip_direction_{window}d"] = flip_direction
                
                # 2. SAR Distance Volatility with proper handling of market regime changes
                for window in [21, 63]:
                    # Calculate distance as percentage of price for proper scaling
                    # This makes the measure comparable across price levels
                    distance_pct = pd.Series(index=df.index, dtype=float)
                    
                    valid_price = df[price_col].abs() > 1e-10
                    distance_pct.loc[valid_price] = abs(df[sar_col].loc[valid_price] - df[price_col].loc[valid_price]) / df[price_col].loc[valid_price] * 100
                    
                    # Handle cases where price is near zero (avoid division issues)
                    if (~valid_price).any():
                        # Use absolute distance when price is near zero
                        avg_price = df[price_col].abs().mean()
                        if avg_price > 0:
                            distance_pct.loc[~valid_price] = abs(df[sar_col].loc[~valid_price] - df[price_col].loc[~valid_price]) / avg_price * 100
                        else:
                            distance_pct.loc[~valid_price] = 0
                    
                    # Cap extreme distances (limit to 50% for stability)
                    distance_pct = distance_pct.clip(0, 50)
                    
                    # Handle missing values for continuity
                    distance_pct = distance_pct.fillna(method='ffill').fillna(0)
                    
                    # Calculate distance volatility with proper statistical handling
                    min_periods = max(5, window//4)
                    distance_mean = distance_pct.rolling(window=window, min_periods=min_periods).mean()
                    distance_std = distance_pct.rolling(window=window, min_periods=min_periods).std()
                    
                    # Handle early periods
                    if len(df) > min_periods:
                        early_mask = pd.Series(False, index=df.index)
                        early_mask.iloc[:min_periods] = True
                        
                        if early_mask.any():
                            # Calculate expanding statistics for early periods
                            exp_mean = distance_pct.expanding(min_periods=1).mean()
                            exp_std = distance_pct.expanding(min_periods=1).std()
                            
                            # Update early values with expanding calculations
                            distance_mean.loc[early_mask] = exp_mean.loc[early_mask]
                            distance_std.loc[early_mask] = exp_std.loc[early_mask]
                    
                    # Calculate coefficient of variation (std/mean) for normalized volatility
                    # This is more meaningful than raw std for SAR distance
                    cv = pd.Series(index=df.index, dtype=float)
                    nonzero_mean = distance_mean > 0.1  # Minimum threshold for mean
                    cv.loc[nonzero_mean] = distance_std.loc[nonzero_mean] / distance_mean.loc[nonzero_mean]
                    
                    # Handle small/zero means by using std directly
                    cv.loc[~nonzero_mean] = distance_std.loc[~nonzero_mean]
                    
                    # Calculate historical normalization with adaptive lookback
                    lookback = min(252, len(df) // 2) if len(df) > 50 else len(df)
                    hist_min_periods = max(lookback//5, 5)
                    
                    cv_mean = cv.rolling(window=lookback, min_periods=hist_min_periods).mean()
                    cv_std = cv.rolling(window=lookback, min_periods=hist_min_periods).std()
                    
                    # Handle early periods for historical stats
                    if len(df) > hist_min_periods:
                        early_mask = pd.Series(False, index=df.index)
                        early_mask.iloc[:hist_min_periods] = True
                        
                        if early_mask.any():
                            # Calculate expanding statistics for early periods
                            exp_mean = cv.expanding(min_periods=1).mean()
                            exp_std = cv.expanding(min_periods=1).std()
                            
                            # Update early values
                            cv_mean.loc[early_mask] = exp_mean.loc[early_mask]
                            cv_std.loc[early_mask] = exp_std.loc[early_mask]
                    
                    # Ensure std is never too small for proper statistical handling
                    min_std = 0.05  # Minimum meaningful std for SAR distance CV
                    
                    # Apply minimum threshold with vectorized operation
                    robust_std = cv_std.copy()
                    robust_std[robust_std < min_std] = min_std
                    
                    # Calculate normalized CV with proper statistical handling
                    norm_cv = pd.Series(index=df.index, dtype=float)
                    valid_mask = robust_std.notna() & (robust_std > 0) & cv.notna() & cv_mean.notna()
                    norm_cv.loc[valid_mask] = (cv.loc[valid_mask] - cv_mean.loc[valid_mask]) / robust_std.loc[valid_mask]
                    
                    # Handle missing values with financial continuity
                    norm_cv = norm_cv.fillna(method='ffill').fillna(0)
                    
                    # Cap extreme values while preserving directionality
                    norm_cv = norm_cv.clip(-4, 4)
                    
                    result_df[f"{sar_col}_dist_vol_{window}d"] = norm_cv
                
                # 3. SAR Acceleration Factor Utilization - additional valuable metric
                # This tracks if SAR is frequently reaching its maximum acceleration factor
                # High values indicate strong, persistent trends
                
                # Calculate SAR changes which correlate with acceleration factor usage
                sar_change = df[sar_col].diff().abs()
                
                # Calculate the rate of change in SAR position
                # Higher values typically indicate higher acceleration factor
                sar_pos_change = sar_position.diff().abs()
                
                # Calculate average price for scaling
                avg_price = df[price_col].abs().mean()
                if avg_price > 0:
                    # Scale SAR change by average price for comparability
                    scaled_change = sar_change / avg_price * 100
                else:
                    scaled_change = sar_change
                
                # Combine scaled change and position change
                # This approximates acceleration factor utilization
                af_utilization = scaled_change * sar_pos_change
                
                # Smooth utilization for stability
                smooth_af = af_utilization.rolling(window=5, min_periods=1).mean()
                
                # Calculate utilization over medium-term window
                window = 21  # Standard month window
                min_periods = max(5, window//4)
                
                af_util_avg = smooth_af.rolling(window=window, min_periods=min_periods).mean()
                
                # Normalize historically
                lookback = min(252, len(df) // 2) if len(df) > 50 else len(df)
                hist_min_periods = max(lookback//5, 5)
                
                util_mean = af_util_avg.rolling(window=lookback, min_periods=hist_min_periods).mean()
                util_std = af_util_avg.rolling(window=lookback, min_periods=hist_min_periods).std()
                
                # Ensure std is never too small
                min_std = 0.001  # Minimum meaningful std for AF utilization
                robust_std = util_std.copy()
                robust_std[robust_std < min_std] = min_std
                
                # Calculate normalized utilization
                norm_util = pd.Series(index=df.index, dtype=float)
                valid_mask = robust_std.notna() & (robust_std > 0) & af_util_avg.notna() & util_mean.notna()
                norm_util.loc[valid_mask] = (af_util_avg.loc[valid_mask] - util_mean.loc[valid_mask]) / robust_std.loc[valid_mask]
                
                # Handle missing values
                norm_util = norm_util.fillna(method='ffill').fillna(0)
                
                # Cap extreme values
                norm_util = norm_util.clip(-4, 4)
                
                result_df[f"{sar_col}_af_util_{window}d"] = norm_util
                    
            except Exception as e:
                print(f"Warning: Error calculating enhanced SAR metrics for {sar_col}/{price_col}: {e}")
                continue
        
        return result_df
    
    @staticmethod
    def add_ma_enhanced_metrics(df, fast_ma_columns, slow_ma_columns):
        """
        Add enhanced moving average metrics with proper handling of edge cases,
        statistical properties, and financial meaning. All operations are
        vectorized to avoid indexing issues.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing moving average data
        fast_ma_columns: list
            List of fast moving average column names
        slow_ma_columns: list
            List of slow moving average column names
            
        Returns:
        --------
        pandas.DataFrame
            DataFrame with enhanced moving average metrics added
        """
        result_df = df.copy()
        
        for fast_col, slow_col in zip(fast_ma_columns, slow_ma_columns):
            # Skip if columns don't exist
            if fast_col not in df.columns or slow_col not in df.columns:
                continue
                
            try:
                # 1. MA Crossover Frequency with proper handling of financial significance
                # MA crossovers are key technical signals for trend changes
                
                # Calculate MA difference for crossover detection
                ma_diff = df[fast_col] - df[slow_col]
                
                # Determine MA relationship (fast above or below slow)
                ma_relationship = pd.Series(index=df.index, dtype=float)
                # Ref: Higham (2002) "Accuracy and Stability of Numerical Algorithms"
                near_zero = ma_diff.abs() < 1e-10
                ma_relationship[near_zero] = 0         # Equal (neutral)
                ma_relationship[~near_zero & (ma_diff > 0)] = 1   # Fast MA above slow MA (bullish)
                ma_relationship[~near_zero & (ma_diff < 0)] = -1  # Fast MA below slow MA (bearish)
                
                # Detect crossovers with vectorized operations
                # A crossover occurs when relationship changes from positive to negative or vice versa
                ma_relationship_prev = ma_relationship.shift(1)
                
                # Initialize crossover series
                crossovers = pd.Series(0, index=df.index)
                
                # Identify golden cross (bearish to bullish)
                golden_cross = (ma_relationship_prev < 0) & (ma_relationship > 0)
                crossovers[golden_cross] = 1
                
                # Identify death cross (bullish to bearish)
                death_cross = (ma_relationship_prev > 0) & (ma_relationship < 0)
                crossovers[death_cross] = -1
                
                # Set first value to 0 (can't calculate crossover without previous value)
                if len(crossovers) > 0:
                    crossovers.iloc[0] = 0
                
                # Calculate crossover metrics for different time windows
                for window in [21, 63]:
                    # Use appropriate minimum periods based on window size
                    min_periods = max(5, window//4)
                    
                    # Calculate frequency of all crossovers (both types)
                    # Normalize to monthly rate (21 trading days) for comparability
                    crossover_count = crossovers.abs().rolling(window=window, min_periods=min_periods).sum() * (21/window)
                    
                    # Calculate directional bias (golden minus death crosses)
                    # Positive value means more golden crosses (bullish bias)
                    # Negative value means more death crosses (bearish bias)
                    cross_direction = crossovers.rolling(window=window, min_periods=min_periods).sum() * (21/window)
                    
                    # Handle early periods with expanding window
                    if len(df) > min_periods:
                        early_mask = pd.Series(False, index=df.index)
                        early_mask.iloc[:min_periods] = True
                        
                        if early_mask.any():
                            # Calculate expanding statistics for early periods
                            exp_count = crossovers.abs().expanding(min_periods=1).sum() * (21/len(crossovers[:min_periods]))
                            exp_direction = crossovers.expanding(min_periods=1).sum() * (21/len(crossovers[:min_periods]))
                            
                            # Update early values
                            crossover_count.loc[early_mask] = exp_count.loc[early_mask]
                            cross_direction.loc[early_mask] = exp_direction.loc[early_mask]
                    
                    # Normalize counts historically with adaptive lookback
                    lookback = min(252, len(df) // 2) if len(df) > 50 else len(df)
                    hist_min_periods = max(lookback//5, 5)
                    
                    # Calculate historical stats for frequency
                    count_mean = crossover_count.rolling(window=lookback, min_periods=hist_min_periods).mean()
                    count_std = crossover_count.rolling(window=lookback, min_periods=hist_min_periods).std()
                    
                    # Calculate historical stats for direction
                    dir_mean = cross_direction.rolling(window=lookback, min_periods=hist_min_periods).mean()
                    dir_std = cross_direction.rolling(window=lookback, min_periods=hist_min_periods).std()
                    
                    # Handle early periods for historical stats
                    if len(df) > hist_min_periods:
                        early_mask = pd.Series(False, index=df.index)
                        early_mask.iloc[:hist_min_periods] = True
                        
                        if early_mask.any():
                            # Calculate expanding statistics for early periods
                            exp_count_mean = crossover_count.expanding(min_periods=1).mean()
                            exp_count_std = crossover_count.expanding(min_periods=1).std()
                            exp_dir_mean = cross_direction.expanding(min_periods=1).mean()
                            exp_dir_std = cross_direction.expanding(min_periods=1).std()
                            
                            # Update early values
                            count_mean.loc[early_mask] = exp_count_mean.loc[early_mask]
                            count_std.loc[early_mask] = exp_count_std.loc[early_mask]
                            dir_mean.loc[early_mask] = exp_dir_mean.loc[early_mask]
                            dir_std.loc[early_mask] = exp_dir_std.loc[early_mask]
                    
                    # Ensure std is never too small for proper statistical handling
                    min_count_std = 0.01  # Minimum std for crossover frequency
                    min_dir_std = 0.02    # Minimum std for direction
                    
                    # Apply minimum thresholds
                    robust_count_std = count_std.copy()
                    robust_count_std[robust_count_std < min_count_std] = min_count_std
                    
                    robust_dir_std = dir_std.copy()
                    robust_dir_std[robust_dir_std < min_dir_std] = min_dir_std
                    
                    # Calculate normalized metrics with proper statistical handling
                    norm_count = pd.Series(index=df.index, dtype=float)
                    valid_count_mask = robust_count_std.notna() & (robust_count_std > 0) & crossover_count.notna() & count_mean.notna()
                    norm_count.loc[valid_count_mask] = (crossover_count.loc[valid_count_mask] - count_mean.loc[valid_count_mask]) / robust_count_std.loc[valid_count_mask]
                    
                    norm_direction = pd.Series(index=df.index, dtype=float)
                    valid_dir_mask = robust_dir_std.notna() & (robust_dir_std > 0) & cross_direction.notna() & dir_mean.notna()
                    norm_direction.loc[valid_dir_mask] = (cross_direction.loc[valid_dir_mask] - dir_mean.loc[valid_dir_mask]) / robust_dir_std.loc[valid_dir_mask]
                    
                    # Handle missing values with financial continuity
                    norm_count = norm_count.fillna(method='ffill').fillna(0)
                    norm_direction = norm_direction.fillna(method='ffill').fillna(0)
                    
                    # Cap extreme values while preserving directionality
                    norm_count = norm_count.clip(-4, 4)
                    norm_direction = norm_direction.clip(-4, 4)
                    
                    # Store results
                    result_df[f"{fast_col}_{slow_col}_cross_freq_{window}d"] = norm_count
                    result_df[f"{fast_col}_{slow_col}_cross_dir_{window}d"] = norm_direction
                
                # 2. MA Slope Acceleration with proper handling of market regimes
                for ma_col in [fast_col, slow_col]:
                    for window in [21, 63]:
                        # Determine if this is SMA or EMA for specific handling
                        is_ema = 'EMA' in ma_col
                        is_fast = ma_col == fast_col
                        
                        # Use different slope periods based on MA type
                        # EMAs are more responsive than SMAs and deserve shorter periods
                        # Fast MAs are more responsive than slow MAs
                        if is_ema and is_fast:
                            slope_period = 3  # Shorter period for fast EMA
                        elif is_ema and not is_fast:
                            slope_period = 5  # Medium period for slow EMA
                        elif not is_ema and is_fast:
                            slope_period = 5  # Medium period for fast SMA
                        else:
                            slope_period = 10  # Longer period for slow SMA
                        
                        # Calculate slope (first derivative) 
                        # Scale by period for comparability between different periods
                        ma_slope = df[ma_col].diff(slope_period) / slope_period
                        
                        # Handle early periods for slope
                        if len(ma_slope) > slope_period:
                            early_mask = pd.Series(False, index=df.index)
                            early_mask.iloc[:slope_period] = True
                            
                            if early_mask.any():
                                # Use forward fill for early slope values
                                first_valid = ma_slope.iloc[slope_period]
                                ma_slope.loc[early_mask] = first_valid
                        
                        # Calculate acceleration (second derivative)
                        # Use appropriate acceleration period based on MA type
                        accel_period = max(2, slope_period // 2)
                        ma_accel = ma_slope.diff(accel_period) / accel_period
                        
                        # Handle early periods for acceleration
                        if len(ma_accel) > slope_period + accel_period:
                            early_mask = pd.Series(False, index=df.index)
                            early_mask.iloc[:slope_period + accel_period] = True
                            
                            if early_mask.any():
                                # Use forward fill for early acceleration values
                                first_valid = ma_accel.iloc[slope_period + accel_period]
                                ma_accel.loc[early_mask] = first_valid
                        
                        # Scale acceleration based on price level for comparability
                        # This makes the metric more meaningful across different price ranges
                        price_level = 0
                        if 'Close' in df.columns:
                            price_level = df['Close'].abs().mean()
                        else:
                            price_level = df[ma_col].abs().mean()
                        
                        if price_level > 0:
                            scaled_accel = ma_accel / (price_level * 0.0001)
                        else:
                            scaled_accel = ma_accel
                        
                        # Apply light smoothing for stability with appropriate min periods
                        smooth_min_periods = max(1, 3//2)
                        smoothed_accel = scaled_accel.rolling(window=3, min_periods=smooth_min_periods).mean()
                        
                        # Calculate adaptive window statistics based on window size
                        adaptive_min_periods = max(5, window//4)
                        
                        accel_mean = smoothed_accel.rolling(window=window, min_periods=adaptive_min_periods).mean()
                        accel_std = smoothed_accel.rolling(window=window, min_periods=adaptive_min_periods).std()
                        
                        # Handle early periods for window statistics
                        if len(df) > adaptive_min_periods:
                            early_mask = pd.Series(False, index=df.index)
                            early_mask.iloc[:adaptive_min_periods] = True
                            
                            if early_mask.any():
                                # Calculate expanding statistics for early periods
                                exp_mean = smoothed_accel.expanding(min_periods=1).mean()
                                exp_std = smoothed_accel.expanding(min_periods=1).std()
                                
                                # Update early values
                                accel_mean.loc[early_mask] = exp_mean.loc[early_mask]
                                accel_std.loc[early_mask] = exp_std.loc[early_mask]
                        
                        # Normalize acceleration historically
                        lookback = min(252, len(df) // 2) if len(df) > 50 else len(df)
                        hist_min_periods = max(lookback//5, 5)
                        
                        hist_mean = accel_mean.rolling(window=lookback, min_periods=hist_min_periods).mean()
                        hist_std = accel_std.rolling(window=lookback, min_periods=hist_min_periods).std()
                        
                        # Ensure std is never too small for proper statistical handling
                        # Use different minimum thresholds based on MA type
                        if is_ema and is_fast:
                            min_std = 2.0  # Fast EMAs have higher natural volatility
                        elif is_ema and not is_fast:
                            min_std = 1.0  # Slow EMAs have medium volatility
                        elif not is_ema and is_fast:
                            min_std = 1.0  # Fast SMAs have medium volatility
                        else:
                            min_std = 0.5  # Slow SMAs have lower volatility
                        
                        # Apply minimum threshold
                        robust_std = hist_std.copy()
                        robust_std[robust_std < min_std] = min_std
                        
                        # Calculate normalized acceleration with proper statistical handling
                        norm_accel = pd.Series(index=df.index, dtype=float)
                        valid_mask = robust_std.notna() & (robust_std > 0) & accel_mean.notna() & hist_mean.notna()
                        norm_accel.loc[valid_mask] = (accel_mean.loc[valid_mask] - hist_mean.loc[valid_mask]) / robust_std.loc[valid_mask]
                        
                        # Handle missing values with financial continuity
                        norm_accel = norm_accel.fillna(method='ffill').fillna(0)
                        
                        # Cap extreme values while preserving directionality
                        norm_accel = norm_accel.clip(-4, 4)
                        
                        result_df[f"{ma_col}_accel_{window}d"] = norm_accel
                
                # 3. Moving Average Ribbon Compression - additional valuable metric
                # When MAs are close together, it indicates low volatility (compression)
                # When far apart, it indicates high volatility (expansion)
                # This metric is valuable for detecting volatility regime changes
                
                # Calculate MA separation as percentage of slow MA
                valid_slow = df[slow_col].abs() > 1e-10
                ma_separation = pd.Series(index=df.index, dtype=float)
                ma_separation.loc[valid_slow] = ((df[fast_col].loc[valid_slow] - df[slow_col].loc[valid_slow]) / df[slow_col].loc[valid_slow]).abs() * 100
                
                # Handle cases where slow MA is near zero
                if (~valid_slow).any():
                    # Use absolute separation when slow MA is near zero
                    avg_ma = (df[fast_col].abs().mean() + df[slow_col].abs().mean()) / 2
                    if avg_ma > 0:
                        ma_separation.loc[~valid_slow] = ((df[fast_col].loc[~valid_slow] - df[slow_col].loc[~valid_slow]) / avg_ma).abs() * 100
                    else:
                        ma_separation.loc[~valid_slow] = 0
                
                # Apply light smoothing for stability
                smooth_min_periods = max(1, 3//2)
                smoothed_separation = ma_separation.rolling(window=3, min_periods=smooth_min_periods).mean()
                
                # Calculate separation over medium-term window
                window = 21  # Standard month window
                min_periods = max(5, window//4)
                
                separation_mean = smoothed_separation.rolling(window=window, min_periods=min_periods).mean()
                
                # Normalize historically
                lookback = min(252, len(df) // 2) if len(df) > 50 else len(df)
                hist_min_periods = max(lookback//5, 5)
                
                sep_mean = separation_mean.rolling(window=lookback, min_periods=hist_min_periods).mean()
                sep_std = separation_mean.rolling(window=lookback, min_periods=hist_min_periods).std()
                
                # Ensure std is never too small
                min_std = 0.1  # Minimum meaningful std for MA separation
                robust_std = sep_std.copy()
                robust_std[robust_std < min_std] = min_std
                
                # Calculate normalized separation
                norm_separation = pd.Series(index=df.index, dtype=float)
                valid_mask = robust_std.notna() & (robust_std > 0) & separation_mean.notna() & sep_mean.notna()
                norm_separation.loc[valid_mask] = (separation_mean.loc[valid_mask] - sep_mean.loc[valid_mask]) / robust_std.loc[valid_mask]
                
                # Handle missing values
                norm_separation = norm_separation.fillna(method='ffill').fillna(0)
                
                # Cap extreme values
                norm_separation = norm_separation.clip(-4, 4)
                
                result_df[f"{fast_col}_{slow_col}_ribbon_{window}d"] = norm_separation
                    
            except Exception as e:
                print(f"Warning: Error calculating enhanced MA metrics for {fast_col}/{slow_col}: {e}")
                continue
        
        return result_df
    
    @staticmethod
    def add_oscillator_enhanced_metrics(df, oscillator_columns):
        """
        Add enhanced metrics for oscillators like RSI and Stochastic with proper
        handling of edge cases, statistical properties, and financial meaning.
        All operations are vectorized to avoid indexing issues.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing oscillator data
        oscillator_columns: list
            List of oscillator column names
                
        Returns:
        --------
        pandas.DataFrame
            DataFrame with enhanced oscillator metrics added
        """
        result_df = df.copy()
        
        for osc_col in oscillator_columns:
            # Skip if column doesn't exist
            if osc_col not in df.columns:
                continue
                
            try:
                # 1. Overbought/Oversold Time calculation with proper handling of financial meaning
                for window in [14, 21]:
                    # Calculate time spent in overbought/oversold zones based on oscillator type
                    if 'RSI' in osc_col:
                        # For RSI, standard thresholds are 70/30
                        overbought = (df[osc_col] > 70).astype(int)
                        oversold = (df[osc_col] < 30).astype(int)
                    elif 'Stochastic' in osc_col:
                        # For Stochastic, sometimes 80/20 is used for stronger signals
                        overbought = (df[osc_col] > 80).astype(int)
                        oversold = (df[osc_col] < 20).astype(int)
                    else:
                        # Default thresholds
                        overbought = (df[osc_col] > 70).astype(int)
                        oversold = (df[osc_col] < 30).astype(int)
                    
                    # Percentage of time in each zone with proper early period handling
                    min_periods = max(3, window//3)  # Adaptive minimum periods
                    
                    # Calculate rolling percentages with appropriate min_periods
                    ob_pct = overbought.rolling(window=window, min_periods=min_periods).mean() * 100
                    os_pct = oversold.rolling(window=window, min_periods=min_periods).mean() * 100
                    
                    # Calculate ratio (overbought minus oversold percentage)
                    # This creates a ranged indicator (-100 to +100) that shows market sentiment
                    ob_os_ratio = ob_pct - os_pct
                    
                    # Handle early periods for initialization
                    # For early periods where rolling windows aren't fully formed, use expanding window
                    if len(df) > min_periods:
                        early_mask = pd.Series(False, index=df.index)
                        early_mask.iloc[:min_periods] = True
                        
                        if early_mask.any():
                            # Calculate expanding statistics for early periods
                            exp_ob_pct = overbought.expanding(min_periods=1).mean() * 100
                            exp_os_pct = oversold.expanding(min_periods=1).mean() * 100
                            exp_ratio = exp_ob_pct - exp_os_pct
                            
                            # Replace early values with expanding window calculations
                            ob_os_ratio.loc[early_mask] = exp_ratio.loc[early_mask]
                    
                    # Normalize historically with adaptive lookback
                    # Use shorter lookback for shorter time series
                    lookback = min(252, len(df) // 2) if len(df) > 50 else len(df)
                    hist_min_periods = max(lookback//5, 5)
                    
                    ratio_mean = ob_os_ratio.rolling(window=lookback, min_periods=hist_min_periods).mean()
                    ratio_std = ob_os_ratio.rolling(window=lookback, min_periods=hist_min_periods).std()
                    
                    # For very early periods, use expanding window statistics
                    if len(df) > hist_min_periods:
                        early_mask = pd.Series(False, index=df.index)
                        early_mask.iloc[:hist_min_periods] = True
                        
                        if early_mask.any():
                            exp_mean = ob_os_ratio.expanding(min_periods=1).mean()
                            exp_std = ob_os_ratio.expanding(min_periods=1).std()
                            
                            ratio_mean.loc[early_mask] = exp_mean.loc[early_mask]
                            ratio_std.loc[early_mask] = exp_std.loc[early_mask]
                    
                    # Ensure std is never too small (oscillator-specific minimum threshold)
                    # Different oscillators have different natural volatility
                    if 'RSI' in osc_col:
                        min_std = 2.0  # RSI typically has standard deviation > 2%
                    elif 'Stochastic' in osc_col:
                        min_std = 5.0  # Stochastic typically has higher volatility
                    else:
                        min_std = 1.0  # Default threshold
                    
                    # Apply minimum threshold with vectorized operation
                    robust_std = ratio_std.copy()
                    robust_std[robust_std < min_std] = min_std
                    
                    # Calculate normalized ratio with proper statistical handling
                    norm_ratio = pd.Series(index=df.index, dtype=float)
                    valid_mask = robust_std.notna() & (robust_std > 0) & ob_os_ratio.notna() & ratio_mean.notna()
                    norm_ratio.loc[valid_mask] = (ob_os_ratio.loc[valid_mask] - ratio_mean.loc[valid_mask]) / robust_std.loc[valid_mask]
                    
                    # Handle missing values with financial continuity
                    # Forward fill to maintain most recent signal
                    norm_ratio = norm_ratio.fillna(method='ffill')
                    
                    # For any remaining NaNs (beginning of series), use neutral value
                    norm_ratio = norm_ratio.fillna(0)
                    
                    # Cap extreme values while preserving directionality
                    # Allow slightly higher range for oscillators (they can have fat-tailed distributions)
                    norm_ratio = norm_ratio.clip(-5, 5)
                    
                    result_df[f"{osc_col}_ob_os_ratio_{window}d"] = norm_ratio
                
                # 2. Oscillator Reversal Severity with proper handling of oscillator mechanics
                for window in [14]:
                    # Calculate rolling high and low with appropriate min_periods
                    min_periods = max(3, window//3)
                    rolling_high = df[osc_col].rolling(window=window, min_periods=min_periods).max()
                    rolling_low = df[osc_col].rolling(window=window, min_periods=min_periods).min()
                    
                    # For early periods, use expanding window
                    if len(df) > min_periods:
                        early_mask = pd.Series(False, index=df.index)
                        early_mask.iloc[:min_periods] = True
                        
                        if early_mask.any():
                            exp_high = df[osc_col].expanding(min_periods=1).max()
                            exp_low = df[osc_col].expanding(min_periods=1).min()
                            
                            rolling_high.loc[early_mask] = exp_high.loc[early_mask]
                            rolling_low.loc[early_mask] = exp_low.loc[early_mask]
                    
                    # Calculate range with protection against zero/near-zero ranges
                    range_val = rolling_high - rolling_low
                    
                    # Use uniform scientific threshold across all financial indicators
                    # Ref: Kahan (1997) "How Futile are Mindless Assessments of Roundoff in Floating-Point Computation?"
                    # recommending consistent threshold selection based on magnitude of compared values
                    EPSILON = 1e-10  # Scientific constant for floating-point comparisons
                    
                    # Identify flat ranges using proper threshold
                    flat_range_mask = range_val < EPSILON
                    
                    # Initialize position series
                    position = pd.Series(index=df.index, dtype=float)
                    
                    # Process normal ranges with numerical stability guarantees
                    normal_range_mask = ~flat_range_mask & range_val.notna()
                    if normal_range_mask.any():
                        # Calculate high and low distances within range (0-1 scale)
                        # Use proper floating-point division following IEEE 754 standards
                        high_dist = (rolling_high.loc[normal_range_mask] - df[osc_col].loc[normal_range_mask]) / range_val.loc[normal_range_mask]
                        low_dist = (df[osc_col].loc[normal_range_mask] - rolling_low.loc[normal_range_mask]) / range_val.loc[normal_range_mask]
                        
                        # Position combines both distances (-1 to 1 scale)
                        position.loc[normal_range_mask] = low_dist - high_dist
                    
                    # Process flat ranges with mutually exclusive condition handling
                    if flat_range_mask.any():
                        # Determine if oscillator is at max/min/middle with scientifically appropriate threshold
                        # Ref: Cont (2011) "Statistical Modeling of High-Frequency Financial Data"
                        # demonstrating importance of threshold-based equality testing in financial time series
                        osc_equals_high = (df[osc_col].loc[flat_range_mask] - rolling_high.loc[flat_range_mask]).abs() < EPSILON
                        osc_equals_low = (df[osc_col].loc[flat_range_mask] - rolling_low.loc[flat_range_mask]).abs() < EPSILON
                        
                        # Apply position values using mutually exclusive logical conditions
                        # Ensures logical consistency across all decision branches
                        position.loc[flat_range_mask & osc_equals_high] = 1.0  # Top of range
                        position.loc[flat_range_mask & osc_equals_low & ~osc_equals_high] = -1.0  # Bottom of range
                        
                        # Explicit handling of the neutral case
                        neutral_mask = flat_range_mask & ~osc_equals_high & ~osc_equals_low
                        position.loc[neutral_mask] = 0.0  # Middle of range
                    
                    # Handle remaining missing values
                    position = position.fillna(method='ffill').fillna(0)
                    
                    # Calculate reversal: high negative values indicate sharp reversal from high
                    # high positive values indicate sharp reversal from low
                    # Use different lookback periods based on oscillator type for optimal signal detection
                    if 'RSI' in osc_col:
                        reversal_period = 3  # RSI typically has faster reversals
                    elif 'Stochastic' in osc_col:
                        reversal_period = 5  # Stochastic has slightly slower reversals
                    else:
                        reversal_period = 4  # Default
                    
                    reversal = position.diff(reversal_period) * -10
                    
                    # Handle early periods with appropriate neutral values
                    if reversal.isna().any():
                        # Fill early missing values with neutral value
                        reversal = reversal.fillna(0)
                    
                    # Apply appropriate scaling based on oscillator type
                    # Different oscillators have different reversal characteristics
                    if 'RSI' in osc_col:
                        scaling_factor = 1.2  # Enhance RSI reversals slightly
                    elif 'Stochastic' in osc_col:
                        scaling_factor = 0.8  # Dampen Stochastic reversals slightly (more volatile)
                    else:
                        scaling_factor = 1.0  # Default
                    
                    reversal = reversal * scaling_factor
                    
                    # Cap extreme values with appropriate bounds based on oscillator type
                    if 'RSI' in osc_col:
                        reversal = reversal.clip(-12, 12)  # RSI can have stronger signals
                    else:
                        reversal = reversal.clip(-10, 10)  # Default bounds
                    
                    result_df[f"{osc_col}_reversal_{window}d"] = reversal
                    
                    # 3. Add oscillator momentum (additional valuable metric not in original)
                    # Momentum measures rate of change in the oscillator itself
                    momentum = df[osc_col].diff(5)  # 5-day rate of change
                    
                    # Normalize momentum by typical oscillator range
                    if 'RSI' in osc_col:
                        # RSI is 0-100 scale, typical 5-day change is ~10-15 points
                        norm_factor = 15.0
                    elif 'Stochastic' in osc_col:
                        # Stochastic is more volatile, typical 5-day change is ~20-25 points
                        norm_factor = 25.0
                    else:
                        # Default normalization
                        norm_factor = 20.0
                    
                    # Apply normalization to create standardized momentum
                    norm_momentum = momentum / norm_factor
                    
                    # Apply light smoothing for noise reduction
                    smoothed_momentum = norm_momentum.rolling(window=3, min_periods=1).mean()
                    
                    # Handle missing values
                    smoothed_momentum = smoothed_momentum.fillna(method='ffill').fillna(0)
                    
                    # Cap extreme values
                    smoothed_momentum = smoothed_momentum.clip(-3, 3)
                    
                    result_df[f"{osc_col}_momentum_{window}d"] = smoothed_momentum
                    
            except Exception as e:
                print(f"Warning: Error calculating enhanced oscillator metrics for {osc_col}: {e}")
                continue
        
        return result_df
    
    @staticmethod
    def add_volume_enhanced_metrics(df, price_columns, volume_columns):
        """
        Add enhanced volume metrics with proper handling of edge cases,
        statistical properties, and financial meaning. All operations are
        vectorized to avoid indexing issues.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing price and volume data
        price_columns: list
            List of price column names
        volume_columns: list
            List of corresponding volume column names
            
        Returns:
        --------
        pandas.DataFrame
            DataFrame with enhanced volume metrics added
        """
        result_df = df.copy()
        
        for price_col, vol_col in zip(price_columns, volume_columns):
            # Skip if columns don't exist
            if price_col not in df.columns or vol_col not in df.columns:
                continue
                
            try:
                # 1. Volume Trend Divergence with proper financial interpretation
                for window in [21, 63]:
                    # Calculate price and volume trends with adaptive periods
                    # Use different periods based on window size for optimal signal detection
                    if window <= 21:
                        trend_period = 10  # Shorter window for short-term analysis
                    else:
                        trend_period = 20  # Longer window for medium-term analysis
                    
                    # Get price and volume changes over selected period
                    price_change = df[price_col].diff(trend_period)
                    volume_change = df[vol_col].diff(trend_period)
                    
                    # Initialize trend series with proper handling
                    price_trend = pd.Series(0, index=df.index)
                    volume_trend = pd.Series(0, index=df.index)
                    
                    # Handle specific trend conditions with vectorized operations
                    # Positive trend
                    price_trend[price_change > 0] = 1
                    volume_trend[volume_change > 0] = 1
                    
                    # Negative trend
                    price_trend[price_change < 0] = -1
                    volume_trend[volume_change < 0] = -1
                    
                    # Divergence calculation: price and volume trends differ
                    # +2: price up, volume down (bearish) - strongest divergence
                    # +1: price up, volume flat (neutral to bearish)
                    # 0: price and volume same direction (confirmation) or both flat
                    # -1: price down, volume flat (neutral to bullish)
                    # -2: price down, volume up (bullish) - strongest divergence
                    divergence = price_trend - volume_trend
                    
                    # Calculate percentage of time showing strong divergence (absolute divergence = 2)
                    # This focuses on the most meaningful divergence signals
                    strong_div = (divergence.abs() == 2).astype(int)
                    
                    # Calculate separate bullish and bearish divergence percentages
                    # This preserves direction information which is crucial
                    bullish_div = (divergence == -2).astype(int)  # Price down, volume up
                    bearish_div = (divergence == 2).astype(int)   # Price up, volume down
                    
                    # Use adaptive minimum periods based on window size
                    min_periods = max(5, window//4)
                    
                    # Calculate percentage of strong divergence in each direction
                    bull_div_pct = bullish_div.rolling(window=window, min_periods=min_periods).mean() * 100
                    bear_div_pct = bearish_div.rolling(window=window, min_periods=min_periods).mean() * 100
                    
                    # Calculate net divergence (bullish minus bearish)
                    # Positive values indicate more bullish divergence
                    # Negative values indicate more bearish divergence
                    net_div_pct = bull_div_pct - bear_div_pct
                    
                    # Handle early periods with expanding window for continuity
                    if len(df) > min_periods:
                        early_mask = pd.Series(False, index=df.index)
                        early_mask.iloc[:min_periods] = True
                        
                        if early_mask.any():
                            # Early period expanding calculations
                            bull_exp = bullish_div.expanding(min_periods=1).mean() * 100
                            bear_exp = bearish_div.expanding(min_periods=1).mean() * 100
                            net_exp = bull_exp - bear_exp
                            
                            # Update early values
                            net_div_pct.loc[early_mask] = net_exp.loc[early_mask]
                    
                    # Normalize historically with adaptive lookback
                    # Use shorter lookback for shorter time series
                    lookback = min(252, len(df) // 2) if len(df) > 50 else len(df)
                    hist_min_periods = max(lookback//5, 5)
                    
                    div_mean = net_div_pct.rolling(window=lookback, min_periods=hist_min_periods).mean()
                    div_std = net_div_pct.rolling(window=lookback, min_periods=hist_min_periods).std()
                    
                    # For early periods, use expanding window statistics
                    if len(df) > hist_min_periods:
                        early_mask = pd.Series(False, index=df.index)
                        early_mask.iloc[:hist_min_periods] = True
                        
                        if early_mask.any():
                            # Calculate expanding statistics for early periods
                            exp_mean = net_div_pct.expanding(min_periods=1).mean()
                            exp_std = net_div_pct.expanding(min_periods=1).std()
                            
                            # Update early values
                            div_mean.loc[early_mask] = exp_mean.loc[early_mask]
                            div_std.loc[early_mask] = exp_std.loc[early_mask]
                    
                    # Ensure std is never too small (volume divergence specific minimum)
                    # Volume divergence typically has std > 5% in normal markets
                    min_std = 5.0  # Minimum standard deviation threshold
                    
                    # Apply minimum threshold with vectorized operation
                    robust_std = div_std.copy()
                    robust_std[robust_std < min_std] = min_std
                    
                    # Calculate normalized divergence with proper statistical handling
                    norm_div = pd.Series(index=df.index, dtype=float)
                    valid_mask = robust_std.notna() & (robust_std > 0) & net_div_pct.notna() & div_mean.notna()
                    norm_div.loc[valid_mask] = (net_div_pct.loc[valid_mask] - div_mean.loc[valid_mask]) / robust_std.loc[valid_mask]
                    
                    # Handle missing values with financial continuity
                    # Forward fill to maintain most recent signal
                    norm_div = norm_div.fillna(method='ffill')
                    
                    # For any remaining NaNs (beginning of series), use neutral value
                    norm_div = norm_div.fillna(0)
                    
                    # Cap extreme values while preserving directionality
                    norm_div = norm_div.clip(-4, 4)
                    
                    result_df[f"{price_col}_{vol_col}_divergence_{window}d"] = norm_div
                    
                # 2. Volume Price Confirmation with precise financial meaning
                for window in [21]:
                    # Calculate daily price and volume changes
                    price_change = df[price_col].pct_change() * 100  # Daily price change in percent
                    vol_change = df[vol_col].pct_change() * 100      # Daily volume change in percent
                    
                    # Handle extreme volume changes (limit to ±300% to prevent distortion)
                    # Volume can spike dramatically on news events
                    vol_change_capped = vol_change.clip(-300, 300)
                    
                    # Create confirmation indicator with financial meaning
                    # High positive: price and volume both up strongly (strong confirmation)
                    # High negative: price and volume both down strongly (strong confirmation)
                    # Near zero: price and volume moving in opposite directions (non-confirmation)
                    
                    # Use sign-adjusted product for confirmation
                    # This preserves direction and magnitude information
                    confirmation = pd.Series(index=df.index, dtype=float)
                    
                    # Strong up confirmation: price up, volume up
                    price_up = price_change > 0
                    vol_up = vol_change_capped > 0
                    strong_up = price_up & vol_up
                    
                    # Strong down confirmation: price down, volume down
                    price_down = price_change < 0
                    vol_down = vol_change_capped < 0
                    strong_down = price_down & vol_down
                    
                    # Calculate confirmation with proper scaling
                    if strong_up.any():
                        # For up confirmations, use combination of price and volume change
                        # Scale up volume component to match price volatility
                        confirmation.loc[strong_up] = (
                            (price_change.loc[strong_up] * vol_change_capped.loc[strong_up].abs() / 10) ** 0.5
                        )
                    
                    if strong_down.any():
                        # For down confirmations, use negative of combination
                        # Down confirmations often have higher volume impact
                        confirmation.loc[strong_down] = (
                            -(price_change.loc[strong_down].abs() * vol_change_capped.loc[strong_down].abs() / 8) ** 0.5
                        )
                    
                    # For non-confirmation (price and volume in opposite directions)
                    # Set to small values close to zero
                    non_confirm = ~strong_up & ~strong_down & price_change.notna() & vol_change_capped.notna()
                    confirmation.loc[non_confirm] = price_change.loc[non_confirm] * 0.1
                    
                    # Handle missing values
                    confirmation = confirmation.fillna(0)
                    
                    # Apply light smoothing for noise reduction with adaptive min periods
                    min_periods = max(2, 3//2)
                    smoothed_conf = confirmation.rolling(window=3, min_periods=min_periods).mean()
                    
                    # Handle early periods
                    if len(df) > min_periods:
                        early_mask = pd.Series(False, index=df.index)
                        early_mask.iloc[:min_periods] = True
                        
                        if early_mask.any():
                            # Forward fill early values
                            smoothed_conf.loc[early_mask] = smoothed_conf.iloc[min_periods]
                    
                    # Calculate average confirmation over rolling window with adaptive min periods
                    min_periods = max(5, window//4)
                    avg_conf = smoothed_conf.rolling(window=window, min_periods=min_periods).mean()
                    
                    # Handle early periods for longer window
                    if len(df) > min_periods:
                        early_mask = pd.Series(False, index=df.index)
                        early_mask.iloc[:min_periods] = True
                        
                        if early_mask.any():
                            # Use shorter-period average for early values
                            short_avg = smoothed_conf.rolling(window=max(3, min_periods//2), min_periods=1).mean()
                            avg_conf.loc[early_mask] = short_avg.loc[early_mask]
                    
                    # Normalize historically with adaptive lookback
                    lookback = min(252, len(df) // 2) if len(df) > 50 else len(df)
                    hist_min_periods = max(lookback//5, 5)
                    
                    conf_mean = avg_conf.rolling(window=lookback, min_periods=hist_min_periods).mean()
                    conf_std = avg_conf.rolling(window=lookback, min_periods=hist_min_periods).std()
                    
                    # Early period handling for historical stats
                    if len(df) > hist_min_periods:
                        early_mask = pd.Series(False, index=df.index)
                        early_mask.iloc[:hist_min_periods] = True
                        
                        if early_mask.any():
                            # Calculate expanding statistics for early periods
                            exp_mean = avg_conf.expanding(min_periods=1).mean()
                            exp_std = avg_conf.expanding(min_periods=1).std()
                            
                            # Update early values
                            conf_mean.loc[early_mask] = exp_mean.loc[early_mask]
                            conf_std.loc[early_mask] = exp_std.loc[early_mask]
                    
                    # Ensure std is never too small with volume-specific minimum
                    min_std = 0.2  # Volume confirmation typically has std > 0.2 in normal markets
                    
                    # Apply minimum threshold with vectorized operation
                    robust_std = conf_std.copy()
                    robust_std[robust_std < min_std] = min_std
                    
                    # Calculate normalized confirmation with proper statistical handling
                    norm_conf = pd.Series(index=df.index, dtype=float)
                    valid_mask = robust_std.notna() & (robust_std > 0) & avg_conf.notna() & conf_mean.notna()
                    norm_conf.loc[valid_mask] = (avg_conf.loc[valid_mask] - conf_mean.loc[valid_mask]) / robust_std.loc[valid_mask]
                    
                    # Handle missing values with financial continuity
                    norm_conf = norm_conf.fillna(method='ffill').fillna(0)
                    
                    # Cap extreme values while preserving directionality
                    norm_conf = norm_conf.clip(-4, 4)
                    
                    result_df[f"{price_col}_{vol_col}_confirmation_{window}d"] = norm_conf
                    
                # 3. Volume Trend Strength - additional valuable metric
                for window in [21]:
                    # Calculate volume relative to its moving average
                    # This identifies abnormal volume periods
                    vol_ma = df[vol_col].rolling(window=window, min_periods=max(5, window//4)).mean()
                    
                    # Calculate relative volume (how much higher/lower than average)
                    rel_volume = pd.Series(index=df.index, dtype=float)
                    vol_ma_mask = vol_ma > 0
                    rel_volume.loc[vol_ma_mask] = (df[vol_col].loc[vol_ma_mask] / vol_ma.loc[vol_ma_mask] - 1) * 100
                    
                    # Handle extreme relative volume (limit to ±500%)
                    rel_volume = rel_volume.clip(-500, 500)
                    
                    # Handle missing values
                    rel_volume = rel_volume.fillna(0)
                    
                    # Calculate price trend (simple up/down/flat)
                    price_trend = pd.Series(0, index=df.index)
                    price_change = df[price_col].pct_change(5) * 100  # 5-day change
                    
                    # Categorize trend based on magnitude and direction
                    strong_up = price_change > 2    # Strong up: >2% over 5 days
                    weak_up = (price_change > 0) & (price_change <= 2)  # Weak up: 0-2%
                    strong_down = price_change < -2  # Strong down: <-2% over 5 days
                    weak_down = (price_change < 0) & (price_change >= -2)  # Weak down: -2-0%
                    
                    # Assign trend values
                    price_trend[strong_up] = 2    # Strong up
                    price_trend[weak_up] = 1      # Weak up
                    price_trend[weak_down] = -1   # Weak down
                    price_trend[strong_down] = -2 # Strong down
                    
                    # Calculate volume strength with direction
                    # High volume during price moves is significant
                    vol_strength = rel_volume * price_trend
                    
                    # Apply smoothing for stability
                    min_periods = max(2, 3//2)
                    vol_strength_smooth = vol_strength.rolling(window=3, min_periods=min_periods).mean()
                    
                    # Normalize historically
                    lookback = min(252, len(df) // 2) if len(df) > 50 else len(df)
                    hist_min_periods = max(lookback//5, 5)
                    
                    strength_mean = vol_strength_smooth.rolling(window=lookback, min_periods=hist_min_periods).mean()
                    strength_std = vol_strength_smooth.rolling(window=lookback, min_periods=hist_min_periods).std()
                    
                    # Ensure std is never too small
                    min_std = 10.0  # Volume strength typically has std > 10 in normal markets
                    robust_std = strength_std.copy()
                    robust_std[robust_std < min_std] = min_std
                    
                    # Calculate normalized strength
                    norm_strength = pd.Series(index=df.index, dtype=float)
                    valid_mask = robust_std.notna() & (robust_std > 0) & vol_strength_smooth.notna() & strength_mean.notna()
                    norm_strength.loc[valid_mask] = (vol_strength_smooth.loc[valid_mask] - strength_mean.loc[valid_mask]) / robust_std.loc[valid_mask]
                    
                    # Handle missing values
                    norm_strength = norm_strength.fillna(method='ffill').fillna(0)
                    
                    # Cap extreme values
                    norm_strength = norm_strength.clip(-4, 4)
                    
                    result_df[f"{price_col}_{vol_col}_strength_{window}d"] = norm_strength
                    
            except Exception as e:
                print(f"Warning: Error calculating enhanced volume metrics for {price_col}/{vol_col}: {e}")
                continue
        
        return result_df
    
    @staticmethod
    def calculate_all_indicators(df, config):
        """
        Calculate all requested technical indicators based on configuration with
        comprehensive error handling and validation.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing price and volume data
        config: dict
            Configuration dictionary specifying which indicators to calculate
            
        Returns:
        --------
        pandas.DataFrame
            DataFrame containing all calculated indicators
        """
        # Initialize a dictionary to hold all calculated indicators
        indicators = {}
        
        # Check for required columns with detailed validation
        required_columns = {
            'basic': ['Close'],
            'ohlc': ['Open', 'High', 'Low', 'Close'],
            'volume': ['Volume']
        }
        
        # Ensure proper DataFrame is provided
        if not isinstance(df, pd.DataFrame):
            raise TypeError("DataFrame must be provided for indicator calculation")
        
        # Validation warning for empty DataFrame
        if len(df) == 0:
            print("Warning: Empty DataFrame provided, no indicators will be calculated")
            return pd.DataFrame()
        
        # Verify the dataframe has minimum required columns
        if not all(col in df.columns for col in required_columns['basic']):
            raise ValueError(f"DataFrame must contain at least {required_columns['basic']} columns")
        
        # Sort DataFrame by index to ensure proper time series behavior
        df = df.sort_index()
        
        # Apply each indicator group based on configuration
        # with comprehensive error handling
        
        # 1. Current Drawdown
        if config.get('current_drawdown', False):
            try:
                include_trend = config.get('drawdown_include_trend', True)
                include_metrics = config.get('drawdown_include_metrics', True)
                drawdown_results = TechnicalIndicators.calculate_current_drawdown(
                    df, 
                    price_col=config.get('price_col', 'Close'),
                    include_trend=include_trend,
                    include_metrics=include_metrics
                )
                for key, value in drawdown_results.items():
                    indicators[key] = value
            except Exception as e:
                print(f"Error calculating Current Drawdown: {e}")
                # Continue with other indicators
        
        # 2. Simple Moving Averages
        ma_results = {}  # Store MA results for relationship calculation
        if config.get('sma', False):
            try:
                include_trend = config.get('include_ma_trend', True)
                windows = config.get('sma_windows', [5, 10, 20, 50, 200])
                timeframes = config.get('sma_timeframes', ['D'])
                for window in windows:
                    for timeframe in timeframes:
                        try:
                            sma_results = TechnicalIndicators.calculate_moving_average(
                                df,
                                price_col=config.get('price_col', 'Close'),
                                window=window,
                                ma_type='simple',
                                timeframe=timeframe,
                                include_stddev=config.get('include_stddev', True),
                                include_trend=include_trend
                            )
                            for key, value in sma_results.items():
                                indicators[key] = value
                                ma_results[key] = value  # Store for relationship calculation
                        except Exception as e:
                            print(f"Error calculating SMA with window={window}, timeframe={timeframe}: {e}")
                            # Continue with next SMA configuration
            except Exception as e:
                print(f"Error in SMA calculation framework: {e}")
                # Continue with other indicators
        
        # 3. Exponential Moving Averages
        if config.get('ema', False):
            try:
                include_trend = config.get('include_ma_trend', True)
                windows = config.get('ema_windows', [5, 10, 20, 50, 200])
                timeframes = config.get('ema_timeframes', ['D'])
                for window in windows:
                    for timeframe in timeframes:
                        try:
                            ema_results = TechnicalIndicators.calculate_moving_average(
                                df,
                                price_col=config.get('price_col', 'Close'),
                                window=window,
                                ma_type='exponential',
                                timeframe=timeframe,
                                include_stddev=config.get('include_stddev', True),
                                include_trend=include_trend
                            )
                            for key, value in ema_results.items():
                                indicators[key] = value
                                ma_results[key] = value  # Store for relationship calculation
                        except Exception as e:
                            print(f"Error calculating EMA with window={window}, timeframe={timeframe}: {e}")
                            # Continue with next EMA configuration
            except Exception as e:
                print(f"Error in EMA calculation framework: {e}")
                # Continue with other indicators
        
        # Calculate MA relationships if requested
        if config.get('ma_relationships', False) and ma_results:
            try:
                ma_rel_results = TechnicalIndicators.calculate_ma_relationships(df, ma_results)
                for key, value in ma_rel_results.items():
                    indicators[key] = value
            except Exception as e:
                print(f"Error calculating MA relationships: {e}")
                # Continue with other indicators
        
        # 4. RSI
        if config.get('rsi', False):
            try:
                if not all(col in df.columns for col in required_columns['basic']):
                    print(f"Warning: Cannot calculate RSI. Required columns: {required_columns['basic']}")
                else:
                    windows = config.get('rsi_windows', [14])
                    include_trend = config.get('rsi_include_trend', True)
                    include_metrics = config.get('rsi_include_metrics', True)
                    for window in windows:
                        try:
                            rsi_results = TechnicalIndicators.calculate_rsi(
                                df, 
                                price_col=config.get('price_col', 'Close'),
                                window=window,
                                include_trend=include_trend,
                                include_metrics=include_metrics
                            )
                            for key, value in rsi_results.items():
                                indicators[key] = value
                        except Exception as e:
                            print(f"Error calculating RSI with window={window}: {e}")
                            # Continue with next RSI configuration
            except Exception as e:
                print(f"Error in RSI calculation framework: {e}")
                # Continue with other indicators
        
        # 5. Stochastic Oscillator
        if config.get('stochastic', False):
            try:
                if not all(col in df.columns for col in required_columns['ohlc']):
                    print(f"Warning: Cannot calculate Stochastic Oscillator. Required columns: {required_columns['ohlc']}")
                else:
                    k_windows = config.get('stochastic_k_windows', [14])
                    d_windows = config.get('stochastic_d_windows', [3])
                    include_raw = config.get('stochastic_include_raw', True)
                    include_metrics = config.get('stochastic_include_metrics', True)
                    include_trend = config.get('stochastic_include_trend', True)
                    
                    for k_window in k_windows:
                        for d_window in d_windows:
                            try:
                                stoch_results = TechnicalIndicators.calculate_stochastic_oscillator(
                                    df,
                                    high_col=config.get('high_col', 'High'),
                                    low_col=config.get('low_col', 'Low'),
                                    close_col=config.get('close_col', 'Close'),
                                    k_window=k_window,
                                    d_window=d_window,
                                    include_raw=include_raw,
                                    include_metrics=include_metrics,
                                    include_trend=include_trend
                                )
                                for key, value in stoch_results.items():
                                    indicators[key] = value
                            except Exception as e:
                                print(f"Error calculating Stochastic with k_window={k_window}, d_window={d_window}: {e}")
                                # Continue with next Stochastic configuration
            except Exception as e:
                print(f"Error in Stochastic calculation framework: {e}")
                # Continue with other indicators
        
        # 6. Rate of Change (ROC)
        if config.get('roc', False):
            try:
                if not all(col in df.columns for col in required_columns['basic']):
                    print(f"Warning: Cannot calculate ROC. Required columns: {required_columns['basic']}")
                else:
                    windows = config.get('roc_windows', [2, 3, 5, 20])
                    include_trend = config.get('roc_include_trend', True)
                    include_metrics = config.get('roc_include_metrics', True)
                    for window in windows:
                        try:
                            roc_results = TechnicalIndicators.calculate_roc(
                                df,
                                price_col=config.get('price_col', 'Close'),
                                window=window,
                                include_trend=include_trend,
                                include_metrics=include_metrics
                            )
                            for key, value in roc_results.items():
                                indicators[key] = value
                        except Exception as e:
                            print(f"Error calculating ROC with window={window}: {e}")
                            # Continue with next ROC configuration
            except Exception as e:
                print(f"Error in ROC calculation framework: {e}")
                # Continue with other indicators
        
        # 7. Average True Range
        if config.get('atr', False):
            try:
                if not all(col in df.columns for col in required_columns['ohlc']):
                    print(f"Warning: Cannot calculate ATR. Required columns: {required_columns['ohlc']}")
                else:
                    windows = config.get('atr_windows', [14])
                    include_trend = config.get('atr_include_trend', True)
                    include_metrics = config.get('atr_include_metrics', True)
                    for window in windows:
                        try:
                            atr_results = TechnicalIndicators.calculate_atr(
                                df,
                                high_col=config.get('high_col', 'High'),
                                low_col=config.get('low_col', 'Low'),
                                close_col=config.get('close_col', 'Close'),
                                window=window,
                                include_trend=include_trend,
                                include_metrics=include_metrics
                            )
                            for key, value in atr_results.items():
                                indicators[key] = value
                        except Exception as e:
                            print(f"Error calculating ATR with window={window}: {e}")
                            # Continue with next ATR configuration
            except Exception as e:
                print(f"Error in ATR calculation framework: {e}")
                # Continue with other indicators
        
        # 8. Commodity Channel Index
        if config.get('cci', False):
            try:
                if not all(col in df.columns for col in required_columns['ohlc']):
                    print(f"Warning: Cannot calculate CCI. Required columns: {required_columns['ohlc']}")
                else:
                    windows = config.get('cci_windows', [20])
                    include_trend = config.get('cci_include_trend', True)
                    for window in windows:
                        try:
                            cci_results = TechnicalIndicators.calculate_cci(
                                df,
                                high_col=config.get('high_col', 'High'),
                                low_col=config.get('low_col', 'Low'),
                                close_col=config.get('close_col', 'Close'),
                                window=window,
                                include_trend=include_trend
                            )
                            for key, value in cci_results.items():
                                indicators[key] = value
                        except Exception as e:
                            print(f"Error calculating CCI with window={window}: {e}")
                            # Continue with next CCI configuration
            except Exception as e:
                print(f"Error in CCI calculation framework: {e}")
                # Continue with other indicators
        
        # 9. On-Balance Volume
        if config.get('obv', False):
            try:
                if not (all(col in df.columns for col in required_columns['basic']) and 'Volume' in df.columns):
                    print(f"Warning: Cannot calculate OBV. Required columns: {required_columns['basic'] + ['Volume']}")
                else:
                    include_trend = config.get('obv_include_trend', True)
                    normalize = config.get('obv_normalize', True)
                    
                    # Check if multi-window OBV is requested
                    use_multi_window = config.get('obv_multi_window', False)
                    
                    if use_multi_window:
                        # Get window and timeframe parameters
                        windows = config.get('obv_windows', [5, 10, 21, 63, 126, 252])
                        timeframes = config.get('obv_timeframes', ['D'])
                        
                        try:
                            obv_results = TechnicalIndicators.calculate_obv_multi(
                                df,
                                price_col=config.get('price_col', 'Close'),
                                volume_col=config.get('volume_col', 'Volume'),
                                include_trend=include_trend,
                                normalize=normalize,
                                windows=windows,
                                timeframes=timeframes
                            )
                            for key, value in obv_results.items():
                                indicators[key] = value
                        except Exception as e:
                            print(f"Error calculating multi-window OBV: {e}")
                            # Fall back to standard OBV
                            try:
                                obv_results = TechnicalIndicators.calculate_obv(
                                    df,
                                    price_col=config.get('price_col', 'Close'),
                                    volume_col=config.get('volume_col', 'Volume'),
                                    include_trend=include_trend,
                                    normalize=normalize
                                )
                                for key, value in obv_results.items():
                                    indicators[key] = value
                            except Exception as e:
                                print(f"Error falling back to standard OBV: {e}")
                    else:
                        # Use standard OBV
                        try:
                            obv_results = TechnicalIndicators.calculate_obv(
                                df,
                                price_col=config.get('price_col', 'Close'),
                                volume_col=config.get('volume_col', 'Volume'),
                                include_trend=include_trend,
                                normalize=normalize
                            )
                            for key, value in obv_results.items():
                                indicators[key] = value
                        except Exception as e:
                            print(f"Error calculating OBV: {e}")
            except Exception as e:
                print(f"Error in OBV calculation framework: {e}")
                # Continue with other indicators
        
        # 10. Volume Rate of Change
        if config.get('vroc', False):
            try:
                if 'Volume' not in df.columns:
                    print(f"Warning: Cannot calculate V-ROC. Required column: Volume")
                else:
                    windows = config.get('vroc_windows', [1, 2, 5, 20])
                    include_trend = config.get('vroc_include_trend', True)
                    include_metrics = config.get('vroc_include_metrics', True)
                    for window in windows:
                        try:
                            vroc_results = TechnicalIndicators.calculate_volume_roc(
                                df,
                                volume_col=config.get('volume_col', 'Volume'),
                                window=window,
                                include_trend=include_trend,
                                include_metrics=include_metrics
                            )
                            for key, value in vroc_results.items():
                                indicators[key] = value
                        except Exception as e:
                            print(f"Error calculating V-ROC with window={window}: {e}")
                            # Continue with next V-ROC configuration
            except Exception as e:
                print(f"Error in V-ROC calculation framework: {e}")
                # Continue with other indicators
        
        # 11. Accumulation/Distribution Line
        if config.get('adl', False):
            try:
                if not (all(col in df.columns for col in required_columns['ohlc']) and 'Volume' in df.columns):
                    print(f"Warning: Cannot calculate A/D Line. Required columns: {required_columns['ohlc'] + ['Volume']}")
                else:
                    include_trend = config.get('adl_include_trend', True)
                    include_metrics = config.get('adl_include_metrics', True)
                    normalize = config.get('adl_normalize', True)
                    adl_windows = config.get('adl_windows', [5, 10, 20, 50, 100, 200])
                    adl_timeframes = config.get('adl_timeframes', ['D', 'W', 'M'])
                    
                    try:
                        adl_results = TechnicalIndicators.calculate_ad_line(
                            df,
                            high_col=config.get('high_col', 'High'),
                            low_col=config.get('low_col', 'Low'),
                            close_col=config.get('close_col', 'Close'),
                            volume_col=config.get('volume_col', 'Volume'),
                            include_trend=include_trend,
                            include_metrics=include_metrics,
                            windows=adl_windows,
                            timeframes=adl_timeframes,
                            normalize=normalize
                        )
                        for key, value in adl_results.items():
                            indicators[key] = value
                    except Exception as e:
                        print(f"Error calculating A/D Line: {e}")
            except Exception as e:
                print(f"Error in A/D Line calculation framework: {e}")
                # Continue with other indicators
        
        # 12. Parabolic SAR
        if config.get('psar', False):
            try:
                if not all(col in df.columns for col in required_columns['ohlc']):
                    print(f"Warning: Cannot calculate Parabolic SAR. Required columns: {required_columns['ohlc']}")
                else:
                    af_start = config.get('psar_af_start', 0.02)
                    af_step = config.get('psar_af_step', 0.02)
                    af_max = config.get('psar_af_max', 0.2)
                    include_raw = config.get('psar_include_raw', True)
                    include_metrics = config.get('psar_include_metrics', True)
                    include_trend = config.get('psar_include_trend', True)
                    
                    try:
                        psar_results = TechnicalIndicators.calculate_parabolic_sar(
                            df,
                            high_col=config.get('high_col', 'High'),
                            low_col=config.get('low_col', 'Low'),
                            close_col=config.get('close_col', 'Close'),
                            af_start=af_start,
                            af_step=af_step,
                            af_max=af_max,
                            include_raw=include_raw,
                            include_metrics=include_metrics,
                            include_trend=include_trend
                        )
                        for key, value in psar_results.items():
                            indicators[key] = value
                    except Exception as e:
                        print(f"Error calculating Parabolic SAR: {e}")
            except Exception as e:
                print(f"Error in Parabolic SAR calculation framework: {e}")
                # Continue with other indicators
        
        # 13. Price Momentum Oscillator
        if config.get('pmo', False):
            try:
                if not all(col in df.columns for col in required_columns['basic']):
                    print(f"Warning: Cannot calculate PMO. Required columns: {required_columns['basic']}")
                else:
                    short_periods = config.get('pmo_short_periods', [35])
                    long_periods = config.get('pmo_long_periods', [20])
                    signal_periods = config.get('pmo_signal_periods', [10])
                    include_raw = config.get('pmo_include_raw', True)
                    include_metrics = config.get('pmo_include_metrics', True)
                    
                    for short_period in short_periods:
                        for long_period in long_periods:
                            for signal_period in signal_periods:
                                try:
                                    pmo_results = TechnicalIndicators.calculate_pmo(
                                        df,
                                        price_col=config.get('price_col', 'Close'),
                                        short_period=short_period,
                                        long_period=long_period,
                                        signal_period=signal_period,
                                        include_raw=include_raw,
                                        include_metrics=include_metrics
                                    )
                                    for key, value in pmo_results.items():
                                        indicators[key] = value
                                except Exception as e:
                                    print(f"Error calculating PMO with parameters {short_period}/{long_period}/{signal_period}: {e}")
                                    # Continue with next PMO configuration
            except Exception as e:
                print(f"Error in PMO calculation framework: {e}")
                # Continue with other indicators
                
        # Create a DataFrame from all indicators
        if not indicators:
            print("Warning: No indicators were successfully calculated")
            return pd.DataFrame(index=df.index)
            
        indicators_df = pd.DataFrame(indicators, index=df.index)
        
        # Fill any NaN values with forward fill then backward fill
        indicators_df = indicators_df.fillna(method='ffill').fillna(method='bfill')
        
        # Validate the final result
        invalid_columns = []
        for col in indicators_df.columns:
            # Check for infinite values
            if np.isinf(indicators_df[col]).any():
                print(f"Warning: Infinite values detected in {col}, applying correction")
                # Replace infinities with large but finite values
                indicators_df[col] = indicators_df[col].replace([np.inf, -np.inf], 
                                                              [1e12, -1e12])
            
            # Check for all-NaN columns
            if indicators_df[col].isna().all():
                invalid_columns.append(col)
        
        # Remove completely invalid columns
        if invalid_columns:
            print(f"Warning: Removing {len(invalid_columns)} columns with all NaN values")
            indicators_df = indicators_df.drop(columns=invalid_columns)
        
        return indicators_df

# Additinal Technical Indicators

## Feature Engineering and Extraction

In [3]:
class FeatureEngineer:
    """
    Class for advanced feature engineering, normalization, and dimensionality reduction.
    Handles multi-window normalization and specialized technical indicator metrics.
    """
    def __init__(self, random_state=42):
        """Initialize the feature engineer."""
        self.random_state = random_state
        self.pca_transformers = {}  # Store fitted PCA models for each window
        self.feature_means = None
        self.feature_stds = None
        self.selected_features = None
        
    def normalize_yields(self, df, yield_columns, windows=None):
        """
        Apply multi-window z-score normalization to yield values with improved handling
        of edge cases and adaptive statistics for yield curves.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing yield data
        yield_columns: list
            List of yield column names
        windows: list
            List of window sizes for normalization
        
        Returns:
        --------
        pandas.DataFrame
            DataFrame with normalized yield features added
        """
        if windows is None:
            windows = [5, 20, 50, 100, 200, 500]
        
        result_df = df.copy()
        created_features = []
        
        for window in windows:
            for col in yield_columns:
                # Handle missing values
                yield_series = df[col].copy().fillna(method='ffill').fillna(method='bfill')
                
                # Use appropriate min_periods to handle early data
                min_periods = min(window//4, 5) if window > 5 else 1
                
                # Calculate rolling statistics with early period handling
                rolling_mean = yield_series.rolling(window=window, min_periods=min_periods).mean()
                rolling_std = yield_series.rolling(window=window, min_periods=min_periods).std()
                
                # For early periods before min_periods, use expanding window
                if min_periods > 1:
                    early_mask = pd.Series(False, index=df.index)
                    early_mask.iloc[:min_periods] = True
                    
                    if early_mask.any():
                        # Calculate expanding statistics for early periods
                        exp_mean = yield_series.expanding(min_periods=1).mean()
                        exp_std = yield_series.expanding(min_periods=1).std()
                        
                        # Replace early values with expanding statistics
                        rolling_mean.loc[early_mask] = exp_mean.loc[early_mask]
                        rolling_std.loc[early_mask] = exp_std.loc[early_mask]
                
                # Adaptive minimum threshold based on yield magnitude
                # Yields typically have different volatility based on maturity
                avg_yield_level = yield_series.abs().mean()
                min_std_base = 0.001  # Base minimum threshold
                
                # Scale threshold based on yield level (higher yields typically have higher volatility)
                adaptive_min_std = min_std_base * (1 + avg_yield_level/2)
                
                # Apply adaptive threshold to standard deviation
                robust_std = np.maximum(rolling_std, adaptive_min_std)
                
                # Calculate z-score with protection against invalid values
                z_score = pd.Series(index=df.index, dtype=float)
                valid_mask = robust_std.notna() & (robust_std > 0) & yield_series.notna() & rolling_mean.notna()
                z_score[valid_mask] = (yield_series[valid_mask] - rolling_mean[valid_mask]) / robust_std[valid_mask]
                
                # Handle missing values with continuity-preserving methods
                z_score = z_score.fillna(method='ffill').fillna(method='bfill')
                
                # Cap extreme z-scores while preserving directionality
                z_score = np.clip(z_score, -4, 4)
                
                # Store result
                new_col = f"{col}_z_{window}d"
                result_df[new_col] = z_score
                created_features.append(new_col)
        
        print(f"Created {len(created_features)} yield normalization features")        
        return result_df
    
    def normalize_bounded_oscillators(self, df, oscillator_columns):
        """
        Transform bounded oscillators (like RSI, Stochastic) from 0-100 range to -1 to 1
        with proper handling of edge cases and proper scaling based on oscillator type.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing oscillator data
        oscillator_columns: list
            List of oscillator column names
        
        Returns:
        --------
        pandas.DataFrame
            DataFrame with rescaled oscillator features added
        """
        result_df = df.copy()
        
        for col in oscillator_columns:
            # First handle missing values
            osc_series = df[col].copy().fillna(method='ffill').fillna(method='bfill')
            
            # Ensure values are within expected range for bounded oscillators
            # Force values to be within [0, 100] range as this is standard for RSI and Stochastic
            osc_series = np.clip(osc_series, 0, 100)
            
            # Determine if this is RSI or Stochastic based on name
            is_rsi = 'RSI' in col
            is_stoch = 'Stochastic' in col
            
            # Transform from 0-100 scale to -1 to 1 scale with oscillator-specific approaches
            if is_rsi:
                # For RSI, the standard neutral point is 50
                # Transform with smoothing near extremes to reduce impact of boundary values
                normalized = pd.Series(index=df.index, dtype=float)
                
                # Standard linear mapping for most values
                normal_range_mask = (osc_series >= 20) & (osc_series <= 80)
                normalized[normal_range_mask] = 2 * (osc_series[normal_range_mask] - 50) / 50
                
                # Apply gentler transformation for extreme values to avoid sharp transitions
                # This creates a more gradual approach to +/-1 at the boundaries
                low_extreme_mask = osc_series < 20
                normalized[low_extreme_mask] = -0.6 - 0.4 * ((20 - osc_series[low_extreme_mask]) / 20)
                
                high_extreme_mask = osc_series > 80
                normalized[high_extreme_mask] = 0.6 + 0.4 * ((osc_series[high_extreme_mask] - 80) / 20)
                
                result_df[f"{col}_scaled"] = normalized
                
            elif is_stoch:
                # For Stochastic, also use 50 as neutral, but different scaling for extremes
                # because Stochastic spends more time at extremes than RSI
                normalized = pd.Series(index=df.index, dtype=float)
                
                # Apply transformation that emphasizes movements in the middle range
                normalized = 2 * (osc_series - 50) / 50
                
                # Apply hyperbolic tangent scaling to reduce impact of extremes
                normalized = np.tanh(normalized * 1.2)  # Scale factor 1.2 to reach close to ±1
                
                result_df[f"{col}_scaled"] = normalized
                
            else:
                # For other oscillators, use standard linear transformation
                result_df[f"{col}_scaled"] = 2 * (osc_series - 50) / 50
            
        return result_df
    
    def normalize_yield_spreads(self, df, spread_columns):
        """
        Apply adaptive sigmoid transformation to yield spreads with protection against
        extreme values and proper handling of spread magnitude.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing spread data
        spread_columns: list
            List of spread column names
        
        Returns:
        --------
        pandas.DataFrame
            DataFrame with normalized spread features added
        """
        result_df = df.copy()
        
        for col in spread_columns:
            # Handle missing values
            spread_series = df[col].copy().fillna(method='ffill').fillna(method='bfill')
            
            # Determine appropriate scaling factor based on typical spread magnitude
            # This ensures reasonable sigmoid transformation regardless of spread size
            spread_std = spread_series.std()
            spread_abs_median = spread_series.abs().median()
            
            # Default scaling factor for normal yield curves (2Y-10Y typically around 0-200 bps)
            default_scale = 5.0
            
            # Adapt scaling for different spread magnitudes
            if spread_abs_median > 0:
                # Scale inversely to spread magnitude (smaller for wider spreads)
                adaptive_scale = default_scale * (0.5 / spread_abs_median)
                # Keep within reasonable bounds
                adaptive_scale = np.clip(adaptive_scale, 1.0, 10.0)
            else:
                adaptive_scale = default_scale
            
            # Pre-process extreme values to prevent overflow in exp calculation
            # Cap at 5 sigmoid units (which gives sigmoid values very close to ±1)
            max_input = 5.0 / adaptive_scale
            spread_capped = np.clip(spread_series, -max_input, max_input)
            
            # Apply sigmoid transformation that preserves sign and adapts to spread magnitude
            # 2/(1+exp(-k*x)) - 1 transforms any range to [-1,1] with adjustable steepness k
            sigmoid_result = 2 / (1 + np.exp(-adaptive_scale * spread_capped)) - 1
            
            # Store result
            result_df[f"{col}_sigmoid"] = sigmoid_result
            
            # Also add a normalized version that tracks significant deviations from historical norms
            # This helps identify when spreads are in unusual territory
            norm_spread = pd.Series(index=df.index, dtype=float)
            
            # Use a long-term rolling window to identify unusual spreads
            lookback = min(250, len(df) // 2) if len(df) > 20 else len(df)
            min_periods = max(20, lookback // 5)
            
            spread_mean = spread_series.rolling(window=lookback, min_periods=min_periods).mean()
            spread_std = spread_series.rolling(window=lookback, min_periods=min_periods).std()
            
            # Ensure std is never too small (yield spreads typically have some volatility)
            min_std = 0.05  # 5 basis points minimum for yield spread std
            robust_std = np.maximum(spread_std, min_std)
            
            # Calculate normalized spread (z-score)
            valid_mask = robust_std.notna() & (robust_std > 0) & spread_series.notna() & spread_mean.notna()
            norm_spread[valid_mask] = (spread_series[valid_mask] - spread_mean[valid_mask]) / robust_std[valid_mask]
            
            # Handle missing values
            norm_spread = norm_spread.fillna(method='ffill').fillna(method='bfill')
            
            # Cap extreme values
            norm_spread = np.clip(norm_spread, -4, 4)
            
            # Store normalized spread
            result_df[f"{col}_zscore"] = norm_spread
            
        return result_df
    
    def robust_scale_cumulative(self, df, cumulative_columns):
        """
        Apply robust scaling to cumulative indicators like OBV and AD Line with
        proper statistical handling and temporal continuity preservation.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing cumulative indicator data
        cumulative_columns: list
            List of cumulative indicator column names
        
        Returns:
        --------
        pandas.DataFrame
            DataFrame with robustly scaled features added
        """
        result_df = df.copy()
        created_features = []
    
        # Separate OBV and AD Line columns (they have different statistical properties)
        obv_columns = [col for col in cumulative_columns if '_ind_OBV' in col]
        ad_columns = [col for col in cumulative_columns if '_ind_AD_Line' in col]
        other_columns = [col for col in cumulative_columns if col not in obv_columns and col not in ad_columns]
        
        # Process OBV columns
        if obv_columns:
            print(f"Processing {len(obv_columns)} OBV indicators")
            obv_windows = [5, 20, 50, 100, 200]
            
            for window in obv_windows:
                # Set appropriate minimum periods based on window size
                min_periods = max(5, window // 4)
                
                for col in obv_columns:
                    # Handle missing values
                    series = df[col].copy().fillna(method='ffill').fillna(method='bfill')
                    
                    # Calculate change in OBV rather than raw value
                    # This removes cumulative growth problem while preserving signal
                    obv_change = series.diff(periods=1)
                    
                    # For the first value, use second value or zero
                    if len(obv_change) > 1 and pd.notna(obv_change.iloc[1]):
                        obv_change.iloc[0] = obv_change.iloc[1]
                    else:
                        obv_change.iloc[0] = 0
                    
                    # Use rolling median and IQR for robust normalization
                    # Median is more robust to outliers than mean
                    rolling_median = obv_change.rolling(window=window, min_periods=min_periods).median()
                    
                    # Calculate IQR with proper handling of early periods
                    q75 = obv_change.rolling(window=window, min_periods=min_periods).quantile(0.75)
                    q25 = obv_change.rolling(window=window, min_periods=min_periods).quantile(0.25)
                    iqr = q75 - q25
                    
                    # For very early periods, use expanding window stats
                    if min_periods > 1:
                        early_mask = pd.Series(False, index=df.index)
                        early_mask.iloc[:min_periods] = True
                        
                        if early_mask.any():
                            # Calculate expanding statistics
                            exp_median = obv_change.expanding(min_periods=1).median()
                            exp_q75 = obv_change.expanding(min_periods=1).quantile(0.75)
                            exp_q25 = obv_change.expanding(min_periods=1).quantile(0.25)
                            exp_iqr = exp_q75 - exp_q25
                            
                            # Update early values
                            rolling_median.loc[early_mask] = exp_median.loc[early_mask]
                            iqr.loc[early_mask] = exp_iqr.loc[early_mask]
                    
                    # Apply adaptive minimum threshold to IQR based on OBV volatility
                    # OBV changes scale with volume, so use volume-based scaling
                    if 'Volume' in df.columns:
                        vol_median = df['Volume'].rolling(window=21, min_periods=5).median().mean()
                        min_iqr = max(0.001, vol_median * 0.0001)
                    else:
                        # Fallback if no volume column
                        min_iqr = max(0.001, obv_change.abs().median() * 0.1)
                    
                    robust_iqr = np.maximum(iqr, min_iqr)
                    
                    # Calculate normalized OBV change
                    norm_change = pd.Series(index=df.index, dtype=float)
                    valid_mask = robust_iqr.notna() & (robust_iqr > 0) & obv_change.notna() & rolling_median.notna()
                    norm_change[valid_mask] = (obv_change[valid_mask] - rolling_median[valid_mask]) / robust_iqr[valid_mask]
                    
                    # Handle missing values
                    norm_change = norm_change.fillna(method='ffill').fillna(0)
                    
                    # Cap extreme values
                    norm_change = np.clip(norm_change, -4, 4)
                    
                    # Store as result
                    new_col = f"{col}_robust_{window}d"
                    result_df[new_col] = norm_change
                    created_features.append(new_col)
                    
                    # Also add cumulative version of normalized change
                    # This preserves trend information while preventing unbounded growth
                    cum_norm_change = norm_change.cumsum()
                    
                    # Store cumulative version
                    result_df[f"{col}_cum_robust_{window}d"] = cum_norm_change
                    created_features.append(f"{col}_cum_robust_{window}d")
        
        # Process AD Line columns - similar approach but with different windows
        if ad_columns:
            print(f"Processing {len(ad_columns)} A/D Line indicators")
            ad_windows = [5, 20, 50, 100, 200, 500]
            
            for window in ad_windows:
                # Set appropriate minimum periods
                min_periods = max(5, window // 4)
                
                for col in ad_columns:
                    # Handle missing values
                    series = df[col].copy().fillna(method='ffill').fillna(method='bfill')
                    
                    # Calculate change in AD Line rather than raw value
                    ad_change = series.diff(periods=1)
                    
                    # For the first value, use second value or zero
                    if len(ad_change) > 1 and pd.notna(ad_change.iloc[1]):
                        ad_change.iloc[0] = ad_change.iloc[1]
                    else:
                        ad_change.iloc[0] = 0
                    
                    # Use rolling median and IQR for robust normalization
                    rolling_median = ad_change.rolling(window=window, min_periods=min_periods).median()
                    q75 = ad_change.rolling(window=window, min_periods=min_periods).quantile(0.75)
                    q25 = ad_change.rolling(window=window, min_periods=min_periods).quantile(0.25)
                    iqr = q75 - q25
                    
                    # For early periods, use expanding window
                    if min_periods > 1:
                        early_mask = pd.Series(False, index=df.index)
                        early_mask.iloc[:min_periods] = True
                        
                        if early_mask.any():
                            exp_median = ad_change.expanding(min_periods=1).median()
                            exp_q75 = ad_change.expanding(min_periods=1).quantile(0.75)
                            exp_q25 = ad_change.expanding(min_periods=1).quantile(0.25)
                            exp_iqr = exp_q75 - exp_q25
                            
                            rolling_median.loc[early_mask] = exp_median.loc[early_mask]
                            iqr.loc[early_mask] = exp_iqr.loc[early_mask]
                    
                    # Apply adaptive minimum threshold to IQR
                    if 'Volume' in df.columns:
                        vol_median = df['Volume'].rolling(window=21, min_periods=5).median().mean()
                        min_iqr = max(0.001, vol_median * 0.0001)
                    else:
                        min_iqr = max(0.001, ad_change.abs().median() * 0.1)
                    
                    robust_iqr = np.maximum(iqr, min_iqr)
                    
                    # Calculate normalized AD change
                    norm_change = pd.Series(index=df.index, dtype=float)
                    valid_mask = robust_iqr.notna() & (robust_iqr > 0) & ad_change.notna() & rolling_median.notna()
                    norm_change[valid_mask] = (ad_change[valid_mask] - rolling_median[valid_mask]) / robust_iqr[valid_mask]
                    
                    # Handle missing values
                    norm_change = norm_change.fillna(method='ffill').fillna(0)
                    
                    # Cap extreme values
                    norm_change = np.clip(norm_change, -4, 4)
                    
                    # Store result
                    new_col = f"{col}_robust_{window}d"
                    result_df[new_col] = norm_change
                    created_features.append(new_col)
                    
                    # Also add cumulative version
                    cum_norm_change = norm_change.cumsum()
                    result_df[f"{col}_cum_robust_{window}d"] = cum_norm_change
                    created_features.append(f"{col}_cum_robust_{window}d")
        
        # Process any other cumulative columns with a general approach
        if other_columns:
            print(f"Processing {len(other_columns)} other cumulative indicators")
            windows = [5, 20, 50, 100, 200]
            
            for window in windows:
                min_periods = max(3, window // 4)
                
                for col in other_columns:
                    # Handle missing values
                    series = df[col].copy().fillna(method='ffill').fillna(method='bfill')
                    
                    # Calculate changes rather than raw cumulative values
                    change = series.diff(periods=1)
                    
                    # Initialize first value
                    if len(change) > 1 and pd.notna(change.iloc[1]):
                        change.iloc[0] = change.iloc[1]
                    else:
                        change.iloc[0] = 0
                    
                    # Use robust statistics
                    rolling_median = change.rolling(window=window, min_periods=min_periods).median()
                    rolling_mad = (change - rolling_median).abs().rolling(window=window, min_periods=min_periods).median()
                    
                    # Early periods handling
                    if min_periods > 1:
                        early_mask = pd.Series(False, index=df.index)
                        early_mask.iloc[:min_periods] = True
                        
                        if early_mask.any():
                            exp_median = change.expanding(min_periods=1).median()
                            exp_mad = (change - exp_median).abs().expanding(min_periods=1).median()
                            
                            rolling_median.loc[early_mask] = exp_median.loc[early_mask]
                            rolling_mad.loc[early_mask] = exp_mad.loc[early_mask]
                    
                    # Ensure MAD is never too small (scale with data magnitude)
                    min_mad = max(0.001, change.abs().median() * 0.1)
                    robust_mad = np.maximum(rolling_mad, min_mad)
                    
                    # Calculate normalized change
                    norm_change = pd.Series(index=df.index, dtype=float)
                    valid_mask = robust_mad.notna() & (robust_mad > 0) & change.notna() & rolling_median.notna()
                    norm_change[valid_mask] = (change[valid_mask] - rolling_median[valid_mask]) / robust_mad[valid_mask]
                    
                    # Handle missing values and cap extremes
                    norm_change = norm_change.fillna(method='ffill').fillna(0)
                    norm_change = np.clip(norm_change, -4, 4)
                    
                    # Store results
                    new_col = f"{col}_robust_{window}d"
                    result_df[new_col] = norm_change
                    created_features.append(new_col)
                    
                    # Add cumulative normalized change
                    cum_norm_change = norm_change.cumsum()
                    result_df[f"{col}_cum_robust_{window}d"] = cum_norm_change
                    created_features.append(f"{col}_cum_robust_{window}d")
        
        print(f"Created {len(created_features)} robust scaled features for cumulative indicators")
        return result_df
    
    def multi_window_normalization(self, df, atr_columns, windows=None):
        """
        Apply multi-window z-score normalization to ATR and other volatility indicators
        with proper handling of edge cases, early periods, and adaptive thresholds.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing indicator data
        atr_columns: list
            List of ATR column names
        windows: list
            List of window sizes for normalization
        
        Returns:
        --------
        pandas.DataFrame
            DataFrame with multi-window normalized features added
        """
        if windows is None:
            windows = [3, 5, 20, 50, 100, 200, 500]
        
        result_df = df.copy()
        created_features = []
        
        for window in windows:
            # Set appropriate minimum periods based on window size
            min_periods = max(2, window // 5)
            
            for col in atr_columns:
                # Handle missing values
                vol_series = df[col].copy().fillna(method='ffill').fillna(method='bfill')
                
                # Get price reference for adaptive scaling
                # ATR is typically in the same units as price, so price level matters
                price_level = None
                if 'Close' in df.columns:
                    price_level = df['Close'].mean()
                
                # Calculate rolling mean and std
                rolling_mean = vol_series.rolling(window=window, min_periods=min_periods).mean()
                rolling_std = vol_series.rolling(window=window, min_periods=min_periods).std()
                
                # For early periods, use expanding window
                if min_periods > 1:
                    early_mask = pd.Series(False, index=df.index)
                    early_mask.iloc[:min_periods] = True
                    
                    if early_mask.any():
                        # Calculate expanding statistics
                        exp_mean = vol_series.expanding(min_periods=1).mean()
                        exp_std = vol_series.expanding(min_periods=1).std()
                        
                        # Update early values
                        rolling_mean.loc[early_mask] = exp_mean.loc[early_mask]
                        rolling_std.loc[early_mask] = exp_std.loc[early_mask]
                
                # Apply adaptive minimum threshold for standard deviation
                # Volatility indicators scale with price, so use price-based scaling if available
                if price_level is not None and price_level > 0:
                    # Typical min volatility is 0.1% of price level for most instruments
                    min_std = price_level * 0.001
                else:
                    # Otherwise use a scale based on the indicator's own magnitude
                    min_std = max(0.001, vol_series.median() * 0.05)
                
                robust_std = np.maximum(rolling_std, min_std)
                
                # Calculate z-score with protection against invalid values
                z_score = pd.Series(index=df.index, dtype=float)
                valid_mask = robust_std.notna() & (robust_std > 0) & vol_series.notna() & rolling_mean.notna()
                z_score[valid_mask] = (vol_series[valid_mask] - rolling_mean[valid_mask]) / robust_std[valid_mask]
                
                # Handle missing values
                z_score = z_score.fillna(method='ffill').fillna(0)
                
                # Cap extreme values
                # Volatility z-scores can be more extreme than price z-scores (distribution has heavier tails)
                z_score = np.clip(z_score, -5, 5)
                
                # Store result
                new_col = f"{col}_z_{window}d"
                result_df[new_col] = z_score
                created_features.append(new_col)
                
        print(f"Created {len(created_features)} multi-window normalized features")
        return result_df
    
    def get_feature_importance(self, component_idx=0):
        """
        Get feature importance for a specific principal component.
        
        Parameters:
        -----------
        component_idx: int
            Index of the principal component (0-based)
        
        Returns:
        --------
        pandas.DataFrame
            DataFrame with feature names and importance scores
        """
        if 'last_window' not in self.pca_transformers:
            return None
            
        pca_info = self.pca_transformers['last_window']
        feature_names = pca_info['feature_names']
        components = pca_info['components']
        
        if component_idx >= len(components):
            return None
            
        # Get loadings for the specified component
        loadings = components[component_idx]
        
        # Ensure lengths match before creating DataFrame
        if len(feature_names) != len(loadings):
            print(f"Warning: Mismatched lengths in feature importance: features={len(feature_names)}, loadings={len(loadings)}")
            return None
        
        # Create DataFrame with feature names and importance
        importance_df = pd.DataFrame({
            'feature': feature_names,
            'loading': loadings
        })
        
        # Sort by absolute importance
        importance_df['abs_loading'] = importance_df['loading'].abs()
        importance_df = importance_df.sort_values('abs_loading', ascending=False)
        
        return importance_df[['feature', 'loading']]
    
    def select_features(self, df, verbose=True):
        """
        Select features for PCA by removing redundant ones.
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing all features
        remove_raw_prices: bool
            Whether to remove raw price inputs except yields
        remove_raw_ratios: bool
            Whether to remove raw ratios (keep log returns)
        remove_raw_mas: bool
            Whether to remove raw moving average values
        remove_raw_indicators_with_zscores: bool
            Whether to remove raw indicators that have z-score versions
        
        Returns:
        --------
        list
            List of selected feature names
        """
        all_columns = df.columns.tolist()
    
        # Track features for reporting
        feature_categories = {
            'raw_prices': [],
            'raw_ratios': [],
            'raw_mas': [],
            'raw_indicators_with_zscores': [],
            'raw_obv_ad': [],
            'raw_atr': [],
            'raw_roc': []
        }
        
        # 1. Raw price inputs (except yields)
        price_pattern = re.compile(r'.*_Close$|.*_Open$|.*_High$|.*_Low$')
        for col in all_columns:
            if price_pattern.match(col) and 'US' not in col:
                feature_categories['raw_prices'].append(col)
        
        # 2. Raw ratios (Copper/Gold, Lumber/Gold)
        ratio_pattern = re.compile(r'Copper_Gold_Ratio$|Lumber_Gold_Ratio$')
        for col in all_columns:
            if ratio_pattern.match(col):
                feature_categories['raw_ratios'].append(col)
        
        # 3. Raw MA values
        ma_pattern = re.compile(r'.*_ind_SMA_.*[^_]$|.*_ind_EMA_.*[^_]$')
        for col in all_columns:
            if ma_pattern.match(col) and not ('_z_score' in col or '_trend' in col or '_pct_diff' in col):
                feature_categories['raw_mas'].append(col)
        
        # 4. Raw indicators with z-score versions
        for col in all_columns:
            if '_z_score' in col:
                base_col = col.replace('_z_score', '')
                if base_col in all_columns:
                    feature_categories['raw_indicators_with_zscores'].append(base_col)
        
        # 5. Raw OBV and AD Line
        obv_ad_pattern = re.compile(r'.*_ind_OBV$|.*_ind_AD_Line$')
        for col in all_columns:
            if obv_ad_pattern.match(col) and not ('_robust_' in col or '_trend' in col):
                feature_categories['raw_obv_ad'].append(col)
        
        # 6. Raw ATR
        atr_pattern = re.compile(r'.*_ind_ATR_.*$')
        for col in all_columns:
            if atr_pattern.match(col) and not ('_z_' in col or '_trend' in col):
                feature_categories['raw_atr'].append(col)
        
        # 7. Raw ROC
        roc_pattern = re.compile(r'.*_ind_ROC_.*$')
        for col in all_columns:
            if roc_pattern.match(col) and not ('_z_score' in col or '_trend' in col):
                feature_categories['raw_roc'].append(col)
        
        # Create combined list of excluded columns
        excluded_columns = []
        for category, columns in feature_categories.items():
            excluded_columns.extend(columns)
        
        # Create list of selected columns
        selected_columns = [col for col in all_columns if col not in excluded_columns]
        self.selected_features = selected_columns
        
        # Report results if requested
        if verbose:
            print("\nFeature Selection Report:")
            total_removed = 0
            for category, columns in feature_categories.items():
                print(f"  - Removed {len(columns)} {category.replace('_', ' ')} features")
                total_removed += len(columns)
            print(f"  - Total: Removed {total_removed} redundant features, kept {len(selected_columns)} features")
        
        return selected_columns

    def print_top_component_features(self, n_components=5, n_features=10):
        """
        Print the top features contributing to each principal component with their
        percentage contributions and variance retention information.
        
        Parameters:
        -----------
        n_components: int
            Number of principal components to analyze
        n_features: int
            Number of top features to display for each component
        """
        if 'last_window' not in self.pca_transformers:
            print("No PCA transformer available. Run fit_transform_pca first.")
            return
            
        pca_info = self.pca_transformers['last_window']
        feature_names = pca_info['feature_names']
        components = pca_info['components']
        explained_var = pca_info['explained_variance_ratio']
        
        # Calculate cumulative explained variance
        cumulative_var = np.cumsum(explained_var)
        
        print("\n========== PCA INFORMATION RETENTION ANALYSIS ==========")
        
        # Overall variance retention summary
        print("\n----- VARIANCE RETENTION SUMMARY -----")
        print(f"Total number of original features: {len(feature_names)}")
        print(f"Number of principal components: {len(components)}")
        print(f"Total variance explained by all {len(components)} components: {np.sum(explained_var)*100:.2f}%")
        
        # Table of variance explained by components
        print("\n----- VARIANCE EXPLAINED BY COMPONENTS -----")
        print(f"{'Component':<10} {'Variance %':<12} {'Cumulative %':<14} {'Information Retention'}")
        print(f"{'-'*10:<10} {'-'*12:<12} {'-'*14:<14} {'-'*20}")
        
        for i in range(min(10, len(components))):
            retention_status = ""
            if cumulative_var[i] > 0.9:
                retention_status = "Excellent (>90%)"
            elif cumulative_var[i] > 0.8:
                retention_status = "Good (>80%)"
            elif cumulative_var[i] > 0.7:
                retention_status = "Moderate (>70%)"
            elif cumulative_var[i] > 0.5:
                retention_status = "Fair (>50%)"
            else:
                retention_status = "Poor (<50%)"
                
            print(f"PC {i+1:<7} {explained_var[i]*100:>9.2f}%   {cumulative_var[i]*100:>9.2f}%      {retention_status}")
        
        # If we have more than 10 components, show a summary for remaining
        if len(components) > 10:
            remaining_var = np.sum(explained_var[10:])
            print(f"PC 11-{len(components):<2} {remaining_var*100:>9.2f}%   {cumulative_var[-1]*100:>9.2f}%      (All components)")
        
        print("\n===== TOP FEATURE CONTRIBUTIONS TO PRINCIPAL COMPONENTS =====")
        
        # Process for each component up to n_components or max available
        for i in range(min(n_components, len(components))):
            # Get loadings for this component
            loadings = components[i]
            
            # Calculate contribution percentages (square of loadings)
            # This represents the proportion of variance explained by each feature
            squared_loadings = loadings ** 2
            total_squared = np.sum(squared_loadings)
            contribution_pct = (squared_loadings / total_squared) * 100
            
            # Create DataFrame with feature names and contributions
            importance_df = pd.DataFrame({
                'feature': feature_names,
                'loading': loadings,
                'contribution_pct': contribution_pct
            })
            
            # Sort by absolute loading value
            importance_df['abs_loading'] = importance_df['loading'].abs()
            importance_df = importance_df.sort_values('abs_loading', ascending=False)
            
            # Display component information
            print(f"\nPrincipal Component {i+1}")
            print(f"• Explains {explained_var[i]*100:.2f}% of total variance")
            print(f"• Cumulative variance explained: {cumulative_var[i]*100:.2f}%")
            print(f"• Information retention: {cumulative_var[i]*100:.1f}% of original information")
            print(f"\nTop {n_features} contributing features:")
            
            # Display top features
            for j, (feature, loading, contrib) in enumerate(
                importance_df[['feature', 'loading', 'contribution_pct']].head(n_features).values):
                sign = "+" if loading >= 0 else "-"
                print(f"  {j+1}. {feature}: {sign} {abs(loading):.4f} ({contrib:.2f}% contribution)")
            
            # Calculate cumulative contribution of top features
            cumulative_contrib = importance_df['contribution_pct'].head(n_features).sum()
            print(f"\n  Top {n_features} features explain {cumulative_contrib:.2f}% of this component's variation")
            
            # Calculate impact on total variance
            impact_on_total = (cumulative_contrib/100) * explained_var[i] * 100
            print(f"  Top {n_features} features account for {impact_on_total:.2f}% of total data variance")
                
        # Print dimensional reduction impact
        print(f"\n----- DIMENSIONAL REDUCTION IMPACT -----")
        dim_reduction_ratio = len(components) / len(feature_names)
        print(f"Dimension reduction ratio: {dim_reduction_ratio:.2f} ({len(components)} components vs {len(feature_names)} original features)")
        print(f"Information density gain: {(cumulative_var[-1]/dim_reduction_ratio):.2f}x")
        
        # Get minimum components for different thresholds
        threshold_90 = np.argmax(cumulative_var >= 0.9) + 1 if any(cumulative_var >= 0.9) else "N/A"
        threshold_80 = np.argmax(cumulative_var >= 0.8) + 1 if any(cumulative_var >= 0.8) else "N/A"
        threshold_70 = np.argmax(cumulative_var >= 0.7) + 1 if any(cumulative_var >= 0.7) else "N/A"
        
        print(f"\nMinimum components needed for:")
        print(f"  90% variance retention: {threshold_90} components")
        print(f"  80% variance retention: {threshold_80} components")
        print(f"  70% variance retention: {threshold_70} components")

    def improved_fit_transform_pca_with_adaptive_scaling(self, df, features, n_components=50, window_size=10, 
                          compute_interval=20, imputation_strategy='ffill', nan_tolerance=0.15,
                          enable_adaptive_scaling=True, scaling_threshold=10.0, scaling_power=0.5):
        """
        Enhanced version of improved_fit_transform_pca that incorporates adaptive scaling
        to maintain numerical stability while preserving PCA component hierarchy.

        1. Calculates PCA every compute_interval days (for efficiency)
        2. Applies the transformation to every day (for complete coverage)
        3. Properly handles NaN values and outliers
        
        Parameters:
        -----------
        df: pandas.DataFrame
            DataFrame containing all features
        features: list
            List of feature column names to use for PCA
        n_components: int
            Number of principal components to keep
        window_size: int
            Size of rolling window for PCA
        compute_interval: int
            Interval (in days) between PCA recalculations
        imputation_strategy: str
            Strategy for imputation ('ffill', 'mean', or 'median')
        nan_tolerance: float
            Maximum fraction of NaNs allowed in a column before it's dropped
        enable_adaptive_scaling: bool, default=True
            Whether to apply adaptive scaling to PCA components
        scaling_threshold: float, default=10.0
            Maximum absolute threshold for PC values before scaling
        scaling_power: float, default=0.5
            Power parameter controlling transformation severity
            
        Returns:
        --------
        pandas.DataFrame
            DataFrame with PCA-transformed features
        --------
        Using 252-day windows (trading year) aligns with research on optimal lookback periods for financial time series (Zhu et al., 2019)
        """
        from sklearn.decomposition import PCA
        from sklearn.preprocessing import StandardScaler
        from sklearn.impute import SimpleImputer
        from tqdm import tqdm
        
        # Initialize result DataFrame with same index as input
        result_df = pd.DataFrame(index=df.index)
        
        # Check if we have enough data
        if len(df) < window_size:
            print(f"Warning: Not enough data points for PCA. Need at least {window_size}, got {len(df)}")
            return result_df
        
        # Report on NaN values
        nan_counts = df[features].isna().sum()
        nan_pcts = nan_counts / len(df)
        features_with_nans = nan_counts[nan_counts > 0].index.tolist()
        
        if features_with_nans:
            print(f"Found {len(features_with_nans)} features with NaN values")
            print(f"Top 5 features with most NaNs: {nan_counts.sort_values(ascending=False).head(5)}")
            
            # Remove features with excessive NaNs
            excessive_nan_features = [f for f in features if nan_pcts[f] > nan_tolerance]
            if excessive_nan_features:
                print(f"Removing {len(excessive_nan_features)} features with >{nan_tolerance*100}% NaNs")
                features = [f for f in features if f not in excessive_nan_features]
                if len(features) == 0:
                    print("Error: No features left after removing those with excessive NaNs")
                    return result_df
        
        # Track statistics for reporting
        windows_processed = 0
        windows_success = 0
        days_transformed = 0
        
        # For each window, fit PCA and transform data for multiple days
        """
        for i in range(window_size, len(df), compute_interval):
        """
        # Calculate total number of windows to process for progress bar
        total_windows = len(range(window_size, len(df), compute_interval))
        pbar = tqdm(range(window_size, len(df), compute_interval), 
                    total=total_windows,
                    desc="Computing PCA",
                    unit="windows",
                    bar_format='{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]')
        
        for i in pbar:
            try:
                windows_processed += 1
                
                # Extract window data for PCA fitting
                # window_data = df.iloc[i-window_size:i][features].copy()   # Using data up to day t-1 to transform day t
                window_data = df.iloc[i-window_size+1:i+1][features].copy() # Using data up to day t to transform day t
                
                # Replace infinities with NaN
                window_data = window_data.replace([np.inf, -np.inf], np.nan)
                
                # Clip extremely large values - using a more reasonable threshold
                for col in window_data.columns:
                    max_value = 1e6  # This is still very high, but will be handled by adaptive scaling
                    mask = window_data[col].abs() > max_value
                    if mask.any():
                        window_data.loc[mask, col] = np.nan
                
                # Improved imputation based on strategy
                if imputation_strategy == 'ffill':
                    # Forward fill first (time series appropriate)
                    window_data_filled = window_data.ffill().bfill()
                    
                    # For any remaining NaNs, use median
                    imputer = SimpleImputer(strategy='median')
                    window_data_imputed = imputer.fit_transform(window_data_filled)
                else:
                    # Use specified strategy
                    imputer = SimpleImputer(strategy=imputation_strategy)
                    window_data_imputed = imputer.fit_transform(window_data)
                
                # Check for NaNs or Infs after imputation
                if np.any(np.isnan(window_data_imputed)) or np.any(np.isinf(window_data_imputed)):
                    print(f"Window {i}: Still contains NaN or Inf after imputation")
                    continue  # Skip this window and move to next
                
                # Standardize the data
                scaler = StandardScaler()
                window_scaled = scaler.fit_transform(window_data_imputed)
                
                # Fit PCA on window
                pca = PCA(n_components=min(n_components, window_data.shape[1]), random_state=self.random_state)
                pca.fit(window_scaled)
                
                # Now transform all days in the next interval (or until end of dataset)
                end_idx = min(i + compute_interval, len(df))
                days_to_transform = df.iloc[i:end_idx][features].copy()
                
                # Store all transformed days for this window before adaptive scaling
                window_transformed_data = []
                window_indices = []
                
                # Process each day individually to ensure proper handling
                for day_offset in range(end_idx - i):
                    day_idx = i + day_offset
                    
                    try:
                        # Get data for this day
                        current_data = days_to_transform.iloc[day_offset:day_offset+1]
                        
                        # Handle infinities and outliers
                        current_data = current_data.replace([np.inf, -np.inf], np.nan)
                        for col in current_data.columns:
                            max_value = 1e6
                            mask = current_data[col].abs() > max_value
                            if mask.any():
                                current_data.loc[mask, col] = np.nan
                        
                        # Handle NaNs
                        if imputation_strategy == 'ffill':
                            # For single row, ffill doesn't work, use column medians from the window
                            current_data_filled = current_data.fillna(window_data.median())
                            current_data_imputed = imputer.transform(current_data_filled)
                        else:
                            current_data_imputed = imputer.transform(current_data)
                        
                        # Transform the day
                        current_scaled = scaler.transform(current_data_imputed)
                        current_transformed = pca.transform(current_scaled)
                        
                        # Store transformed data for later batch processing
                        window_transformed_data.append(current_transformed[0])
                        window_indices.append(df.index[day_idx])
                        
                        days_transformed += 1
                        
                    except Exception as e:
                        print(f"Error processing day at index {day_idx}: {e}")
                        # Continue with next day
                
                # If we have transformed days, apply adaptive scaling
                if window_transformed_data and enable_adaptive_scaling:
                    # Convert to DataFrame for easier handling
                    pc_columns = [f'PC_{j+1}' for j in range(pca.n_components_)]
                    transformed_df = pd.DataFrame(window_transformed_data, 
                                                index=window_indices,
                                                columns=pc_columns)
                    
                    # Apply adaptive scaling to the batch of transformed data
                    scaled_df = self.adaptive_scale_principal_components(
                        transformed_df,
                        max_abs_threshold=scaling_threshold,
                        scaling_power=scaling_power
                    )
                    
                    # Store scaled results
                    for idx in scaled_df.index:
                        for j, col in enumerate(pc_columns):
                            if j < scaled_df.shape[1]:  # Ensure column exists
                                result_df.loc[idx, col] = scaled_df.loc[idx, col]
                else:
                    # If not applying adaptive scaling, store the original transformed values
                    for day_idx, transformed in zip(window_indices, window_transformed_data):
                        for j in range(len(transformed)):
                            result_df.loc[day_idx, f'PC_{j+1}'] = transformed[j]
                
                # Store PCA details for last window (for interpretation)
                if i >= len(df) - compute_interval:
                    self.pca_transformers['last_window'] = {
                        'pca': pca,
                        'scaler': scaler,
                        'imputer': imputer,
                        'feature_names': features,
                        'explained_variance_ratio': pca.explained_variance_ratio_,
                        'components': pca.components_
                    }
                
                windows_success += 1
                    
            except Exception as e:
                print(f"Error processing window ending at index {i}: {e}")
                # Continue with next window
        
        # Report how many data points were transformed
        non_na_count = result_df.notna().sum(axis=1).astype(bool).sum()
        success_rate = windows_success / windows_processed if windows_processed > 0 else 0
        print(f"PCA window processing success rate: {success_rate:.2%} ({windows_success}/{windows_processed})")
        print(f"Successfully transformed {non_na_count} out of {len(df)} data points ({non_na_count/len(df):.2%})")
        
        # Handle any remaining NaN values in result_df
        if non_na_count < len(df):
            print(f"Note: {len(df) - non_na_count} days could not be transformed directly")
            print("Applying forward-fill to complete any missing values...")
            result_df = result_df.fillna(method='ffill').fillna(method='bfill')
            filled_non_na = result_df.notna().sum(axis=1).astype(bool).sum()
            print(f"After filling: {filled_non_na} out of {len(df)} days have PCA values ({filled_non_na/len(df):.2%})")
            
        # Final validation of data ranges - report any remaining extreme values
        if enable_adaptive_scaling:
            extremes = (result_df.abs() > scaling_threshold).sum().sum()
            if extremes > 0:
                print(f"Warning: After adaptive scaling, {extremes} values still exceed threshold {scaling_threshold}")
                # Get columns with extreme values
                extreme_cols = result_df.columns[(result_df.abs() > scaling_threshold).any()].tolist()
                if extreme_cols:
                    print(f"Columns with remaining extreme values: {extreme_cols[:10]}")
                    if len(extreme_cols) > 10:
                        print(f"... and {len(extreme_cols) - 10} more")
            else:
                print(f"Adaptive scaling successful: All values within threshold {scaling_threshold}")
        
        return result_df
        

    def adaptive_scale_principal_components(self, pc_data, max_abs_threshold=10.0, 
                                          scaling_power=0.5, min_threshold=3.0, 
                                          component_specific_thresholds=None,
                                          apply_to_all=False):
        """
        Apply adaptive scaling to principal components to maintain numerical stability
        while preserving relative importance hierarchy.
        
        This implementation is based on financial econometric research including:
        - Cont (2001): "Empirical properties of asset returns: stylized facts and statistical issues"
        - Avellaneda & Lee (2010): "Statistical Arbitrage in the U.S. Equities Market"
        - Alexander (2001): "Market Models: A Guide to Financial Data Analysis"
        
        Parameters:
        -----------
        pc_data: pandas.DataFrame
            DataFrame containing principal components (columns) for each date (rows)
        max_abs_threshold: float, default=10.0
            Maximum absolute threshold for PC values, based on typical financial extreme thresholds
            Values beyond this will be adaptively scaled
        scaling_power: float, default=0.5
            Power parameter controlling transformation severity:
            - Lower values (0.3-0.4) preserve more extreme information
            - Higher values (0.6-0.7) provide stronger numerical stability
        min_threshold: float, default=3.0
            Minimum threshold to apply scaling (avoids scaling routine values within 3 std deviations)
        component_specific_thresholds: dict or None, default=None
            Optional dictionary mapping component names to custom thresholds
            Example: {'PC_1': 15.0, 'PC_2': 8.0}
        apply_to_all: bool, default=False
            If True, apply scaling to all components regardless of magnitude
            If False, only scale components that exceed thresholds
            
        Returns:
        --------
        pandas.DataFrame
            DataFrame with adaptively scaled principal components
        """
        import numpy as np
        import pandas as pd
        
        # Validation
        if not isinstance(pc_data, pd.DataFrame):
            raise TypeError("pc_data must be a pandas DataFrame")
        
        if scaling_power <= 0:
            raise ValueError("scaling_power must be positive")
            
        if max_abs_threshold <= 0:
            raise ValueError("max_abs_threshold must be positive")
        
        # Get component information for reporting
        n_components = pc_data.shape[1]
        
        # Handle empty dataframe case
        if n_components == 0 or pc_data.empty:
            print("No scaling applied: Empty DataFrame")
            return pc_data
        
        # FIX 1: Get scalar max value across all columns
        original_max_abs = pc_data.abs().max().max()  # Get global maximum as a scalar
        
        if original_max_abs <= max_abs_threshold and not apply_to_all:
            print(f"No scaling applied: Maximum absolute value {original_max_abs:.2f} is within threshold {max_abs_threshold:.2f}")
            return pc_data
        
        # Create a copy to avoid modifying the original
        scaled_data = pc_data.copy()
        scaling_report = {}
        
        # FIX 2: Initialize variables that might be used after the loop
        # This prevents "not defined" errors if the loop doesn't run or if certain code paths aren't taken
        col_max_abs = 0.0
        scale_factor = 1.0
        
        # Process each component
        for col in scaled_data.columns:
            # Determine the threshold for this component
            col_threshold = max_abs_threshold
            if component_specific_thresholds and col in component_specific_thresholds:
                col_threshold = component_specific_thresholds[col]
            
            # Calculate maximum absolute value and determine if scaling is needed
            col_max_abs = scaled_data[col].abs().max()
            needs_scaling = col_max_abs > col_threshold or apply_to_all
            
            # Apply adaptive scaling if needed
            if needs_scaling:
                # Values beyond minimum threshold
                values_to_scale = scaled_data[col][scaled_data[col].abs() > min_threshold]
                
                if len(values_to_scale) > 0 or apply_to_all:
                    # Calculate adaptive scale factor
                    if col_max_abs > col_threshold:
                        scale_factor = (col_max_abs / col_threshold) ** scaling_power
                    else:
                        # If applying to all but max is under threshold, use smaller factor
                        scale_factor = 1.0 + (col_max_abs / col_threshold) ** scaling_power - 1.0
                    
                    # Apply scaling only to values beyond min_threshold for a smoother transition
                    # unless apply_to_all is True
                    if apply_to_all:
                        scaled_data[col] = scaled_data[col] / scale_factor
                    else:
                        # Identify values beyond threshold
                        mask = scaled_data[col].abs() > min_threshold
                        
                        # For values beyond threshold, apply scaling
                        scaled_data.loc[mask, col] = scaled_data.loc[mask, col] / scale_factor
                    
                    # Store scaling information for reporting
                    scaling_report[col] = {
                        'original_max': col_max_abs,
                        'scale_factor': scale_factor,
                        'new_max': scaled_data[col].abs().max()
                    }
        
        # Report scaling results
        """
        if scaling_report:
            print(f"\nAdaptive Scaling Report (threshold={max_abs_threshold:.2f}, power={scaling_power:.2f}):")
            print(f"Components scaled: {len(scaling_report)}/{n_components}")
            
            # Sort by largest original magnitude
            sorted_report = sorted(scaling_report.items(), 
                                  key=lambda x: x[1]['original_max'], 
                                  reverse=True)
            
            for i, (comp, info) in enumerate(sorted_report[:10]):  # Show top 10
                print(f"  {comp}: {info['original_max']:.2f} → {info['new_max']:.2f} (scale factor: {info['scale_factor']:.2f})")
            
            if len(sorted_report) > 10:
                print(f"  ... and {len(sorted_report) - 10} more components")
                
            # Calculate overall reduction in extreme values
            if scaling_report:  # Make sure we have at least one scaled component
                old_max = max([info['original_max'] for _, info in scaling_report.items()])
                new_max = scaled_data.abs().max().max()
                print(f"Maximum extreme value reduced: {old_max:.2f} → {new_max:.2f} ({new_max/old_max:.1%} of original)")
        """
            
        return scaled_data

## Quantum Feature Extraction

In [6]:
import pennylane as qml
import pennylane.numpy as qnp
import numpy as np
import pandas as pd
import time
import warnings
import scipy.stats
import inspect
from typing import List, Dict, Tuple, Optional, Union

def validate_quantum_features(features, feature_name="quantum_features"):
    """Validate quantum features before integration"""
    print(f"\nValidating {feature_name}:")
    print(f"  Type: {type(features)}")
    print(f"  Shape: {features.shape if hasattr(features, 'shape') else 'N/A'}")
    
    if isinstance(features, np.ndarray):
        print(f"  Dtype: {features.dtype}")
        print(f"  Contains NaN: {np.any(np.isnan(features))}")
        print(f"  Contains Inf: {np.any(np.isinf(features))}")
        print(f"  Min: {np.nanmin(features)}")
        print(f"  Max: {np.nanmax(features)}")
        print(f"  Mean: {np.nanmean(features)}")
    
    return True

class QuantumVolatilityDetector:
    """
    Quantum Circuit for Financial Volatility Detection with Theoretical Grounding
    
    This implementation incorporates several research-based techniques:
    
    1. Theoretically grounded quantum-financial mappings (Rebentrost et al., 2018)
    2. Volatility-preserving normalization (Andersen et al., 2001)
    3. Complete observable basis (Nielsen & Chuang, 2010)
    4. Barren plateau mitigation strategies (Cerezo et al., 2021)
    
    Key Improvements:
    ----------------
    - Proper QNode execution returning numerical values
    - Scientifically justified quantum-volatility mappings
    - Volatility-preserving phase space transformation
    - Complete set of quantum measurements
    
    References:
    -----------
    [1] Rebentrost, P., Gupt, B., & Bromley, T. R. (2018).
        Quantum computational finance: Monte Carlo pricing of financial derivatives.
        Physical Review A, 98(2), 022321.
        
    [2] Andersen, T. G., Bollerslev, T., Diebold, F. X., & Ebens, H. (2001).
        The distribution of realized stock return volatility.
        Journal of Financial Economics, 61(1), 43-76.
        
    [3] Nielsen, M. A., & Chuang, I. L. (2010).
        Quantum computation and quantum information.
        Cambridge University Press.
    """

    # Single source of truth for feature count
    N_QUANTUM_FEATURES = 9
    
    def __init__(self, 
                 n_qubits: int = 4,
                 n_layers: int = 2,  #2
                 lookback_window: int = 4,
                 #device_type: str = 'default.qubit',
                 device_type: str = 'lightning.qubit',
                 shots: Optional[int] = None,
                 random_state: int = 42,
                 gradient_threshold: float = 1e-4,
                 adaptive_depth: bool = True,
                 enable_continuous_learning: bool = True,
                 continuous_learning_increment: int = 50,
                 continuous_learning_epochs: int = 5):
        
        np.random.seed(random_state)
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        self.max_layers = n_layers * 2
        self.lookback_window = lookback_window
        self.device_type = device_type
        self.shots = shots
        self.random_state = random_state
        self.gradient_threshold = gradient_threshold
        self.adaptive_depth = adaptive_depth

        # Continuous learning parameters
        self.enable_continuous_learning = enable_continuous_learning
        self.continuous_learning_increment = continuous_learning_increment
        self.continuous_learning_epochs = continuous_learning_epochs
        self._updates_since_last_training = 0
        self._accumulated_training_data = []
        
        # Create quantum device
        if device_type == 'lightning.qubit':
            self.device = qml.device(device_type, wires=n_qubits)  # No shots for exact simulation
        else:
            self.device = qml.device(device_type, wires=n_qubits, shots=shots)
        
        # Initialize parameters using identity block strategy
        self._initialize_parameters()
        
        # Tracking training progress
        self.training_history = {
            'loss': [], 
            'gradients': [], 
            'parameters': [],
            'layers_used': [],
            'prediction_correlations': []
        }
        
        self.is_fitted = False
        self._circuit_call_count = 0

        """
        # Define measurement meanings based on financial theory
        self.measurement_meanings = {
            0: "realized_volatility",     # Z₀: Realized volatility (Andersen et al., 2001)
            1: "jump_component",          # X₁: Jump component (Barndorff-Nielsen & Shephard, 2004)
            2: "integrated_variance",     # Y₂: Integrated variance (Andersen et al., 2003)
            3: "volatility_persistence"   # Z₀⊗Z₁: Volatility persistence (Engle & Patton, 2001)
        }
        """
        # Define measurement meanings
        self.measurement_meanings = {
            
            # 1. Individual qubit measurements in Z-basis
            # After H-RZ-H encoding, these capture phase-modulated amplitudes
            0: "volatility_magnitude_day_t-3",     
            1: "volatility_magnitude_day_t-2", 
            2: "volatility_magnitude_day_t-1",    
            3: "volatility_magnitude_day_t",
            
            # 2. Two-qubit correlations for temporal volatility persistence
            # (Corsi, 2009: HAR model - captures multi-scale volatility dynamics)
            4: "volatility_persistence_T-3_to_T",    # Z₀⊗Z₃: T-3 to T (4-day span - Long-range)       
            5: "volatility_persistence_T-2_to_T-1",  # Z₁⊗Z₂: T-2 to T-1 (mid-range - Short-rang)
            6: "volatility_persistence_T-3_to_T-2",  # Z₀⊗Z₁: T-3 to T-2 (historical - Adjacent days)
            7: "volatility_sign_persistence",        # Y₀⊗Y₃: Phase correlation (sign dynamics)

            #  Three-body: Z₀⊗Z₁⊗Z₂     (higher-order patterns)
            8: "volatility_higher_order" 
        }

        self.QUANTUM_INPUT_SPECS = {
            'single_timeseries': {
                'shape': (self.lookback_window,),
                'description': 'Single time window for quantum encoding',
                'reference': 'Schuld et al. (2021) - time series encoding'
            },
            'phase_space': {
                'shape': (None, self.n_qubits),  # Variable time points
                'description': 'Phase space embedding (Takens, 1981)',
                'reference': 'Packard et al. (1980) - phase space reconstruction'
            }
        }
        
        self.volatility_states = []

    def _analyze_input_data(self, volatility_data):
        """Analyze the properties of input data for quantum encoding."""
        # Ensure we're working with numpy array
        volatility_data = np.asarray(volatility_data)
    
        if volatility_data.ndim == 1:
            # Standard 1D analysis
            stats = {
                'mean': float(np.mean(volatility_data)),
                'std': float(np.std(volatility_data)),
                'max': np.max(volatility_data),
                'min': np.min(volatility_data),
                'skewness': float(scipy.stats.skew(volatility_data.flatten())),
                'kurtosis': float(scipy.stats.kurtosis(volatility_data.flatten())),
                'autocorr_1': 0  # Skip autocorrelation for multi-dimensional data
            }
        else:
            # Multi-dimensional: analyze structure, not flattened
            print(f"Multi-dimensional input: {volatility_data.shape}")
            
            # Overall statistics preserving dimensional meaning
            stats = {
                'mean': float(np.mean(volatility_data)),  # Overall mean is OK
                'std': float(np.std(volatility_data)),    # Overall std is OK
                'max': float(np.max(volatility_data)),
                'min': float(np.min(volatility_data)),
                'shape': volatility_data.shape,
                'n_features': volatility_data.shape[1] if volatility_data.ndim > 1 else 1,
                'n_timepoints': volatility_data.shape[0]
            }
            
            # Feature-wise analysis to preserve structure
            stats['feature_means'] = np.mean(volatility_data, axis=0)
            stats['feature_stds'] = np.std(volatility_data, axis=0)
            stats['feature_correlations'] = np.corrcoef(volatility_data, rowvar=False)
            
            # Temporal analysis
            if volatility_data.shape[0] > 1:
                stats['temporal_autocorr'] = []
                for feature in range(volatility_data.shape[1]):
                    series = volatility_data[:, feature]
                    if len(series) > 1:
                        autocorr = np.corrcoef(series[:-1], series[1:])[0,1]
                        stats['temporal_autocorr'].append(float(autocorr))
        
        return stats

    def _infer_data_semantics(self, data):
        """
        Infer the mathematical meaning of input data.
        Enhanced to handle all cases in the quantum workflow.
        """
        data = np.asarray(data)
        
        semantics = {
            'shape': data.shape,
            'ndim': data.ndim,
            'type': 'unknown',
            'interpretation': 'Unknown data structure'
        }
        
        # Special case for 10x10 matrices
        if data.shape == (10, 10):
            # Check temporal correlations to determine orientation
            row_autocorr = 0
            col_autocorr = 0
            
            try:
                # Check row-wise temporal correlation
                for i in range(min(5, data.shape[0])):
                    if np.std(data[i]) > 0:
                        corr = np.corrcoef(data[i, :-1], data[i, 1:])[0, 1]
                        if np.isfinite(corr):
                            row_autocorr += abs(corr)
                row_autocorr /= 5
                
                # Check column-wise temporal correlation
                for j in range(min(5, data.shape[1])):
                    if np.std(data[:, j]) > 0:
                        corr = np.corrcoef(data[:-1, j], data[1:, j])[0, 1]
                        if np.isfinite(corr):
                            col_autocorr += abs(corr)
                col_autocorr /= 5
            except:
                pass
            
            if row_autocorr > col_autocorr:
                semantics['type'] = 'timeseries_batch'
                semantics['interpretation'] = '10 time series of length 10'
            else:
                semantics['type'] = 'feature_matrix'
                semantics['interpretation'] = '10 time points × 10 features'
                
        elif data.ndim == 1:
            if len(data) == self.lookback_window:
                semantics['type'] = 'raw_timeseries'
                semantics['interpretation'] = 'Single time window'
            else:
                semantics['type'] = 'feature_vector'
                semantics['interpretation'] = f'Feature vector of length {len(data)}'
        
        elif data.ndim == 2:
            rows, cols = data.shape
            
            if cols == self.lookback_window:
                semantics['type'] = 'timeseries_batch'
                semantics['interpretation'] = f'{rows} time windows'
            elif cols <= self.n_qubits:
                semantics['type'] = 'feature_matrix'
                semantics['interpretation'] = f'{rows} time points × {cols} features'
            elif rows <= self.n_qubits:
                semantics['type'] = 'transposed_features'
                semantics['interpretation'] = f'{rows} features × {cols} time points'
            else:
                # Try to determine based on data properties
                semantics['type'] = 'general_matrix'
                semantics['interpretation'] = f'{rows}×{cols} matrix'
        
        else:
            semantics['type'] = 'high_dimensional'
            semantics['interpretation'] = f'{data.ndim}D tensor'
        
        return semantics

    """
    def _prepare_circuit_input(self, raw_data):
        #
        #Prepare input for quantum circuit preserving semantic meaning.
        #Based on Lloyd et al. (2014) quantum data encoding principles.
        #
        #Updated to work with new volatility-preserving phase space transform.
        #
        data = np.asarray(raw_data)
        
        # Single sample expected for quantum circuit
        if data.ndim > 1 and data.shape[0] > 1:
            print("WARNING: Circuit received batch, extracting first sample")
            data = data[0]
        
        # Now we have a single sample
        if data.ndim == 1:
            # 1D time series
            if len(data) == self.lookback_window:
                return data, 'single_timeseries'
            else:
                # Adjust length
                if len(data) > self.lookback_window:
                    return data[-self.lookback_window:], 'truncated_timeseries'
                else:
                    padded = np.pad(data, (0, self.lookback_window - len(data)))
                    return padded, 'padded_timeseries'
        
        elif data.ndim == 2:
            # 2D feature matrix for single sample
            time_points, n_features = data.shape
            
            if n_features <= self.n_qubits:
                return data, 'feature_matrix'
            else:
                # Use volatility-preserving PCA
                return self._reduce_features_pca(data), 'pca_reduced_features'
        
        else:
            raise ValueError(f"Unexpected input dimensions: {data.ndim}")
    """

    def _prepare_circuit_input(self, raw_data):
        """Updated to handle OHLC input for enhanced encoding."""
        data = np.asarray(raw_data)
        
        # Expect 4×4 OHLC data
        if data.ndim == 2 and data.shape == (4, 4):
            return data, 'ohlc_data'
        elif data.ndim == 1 and len(data) == 16:  # Flattened 4×4
            return data.reshape(4, 4), 'ohlc_reshaped'
        else:
            raise ValueError(f"Expected OHLC data shape (4,4), got {data.shape}")
            
    def _create_quantum_circuit(self):
        """
        Measurement strategy optimized for signed volatility prediction.
        
        References:
        - Havlíček et al. (2019): "Supervised learning with quantum-enhanced feature spaces"
        - Abbas et al. (2021): "The power of quantum neural networks"
        - Schuld & Killoran (2019): "Quantum machine learning in feature Hilbert spaces"
        - Blank et al. (2020): "Quantum classifier with tailored quantum kernel"
        """
        @qml.qnode(self.device, interface="autograd", diff_method="parameter-shift")
        def circuit(ohlc_data, params):
            # Encode signed G-K values
            self._encode_volatility_data(ohlc_data)
            
            # Variational quantum circuit layers
            self._variational_layers(params, self.active_layers)
            
            measurements = []
            
            # 1. Individual qubit measurements in Z-basis
            # After H-RZ-H encoding, these capture phase-modulated amplitudes
            # (Cerezo et al., 2021: "Variational quantum algorithms")
            z_measurements = [qml.expval(qml.PauliZ(i)) for i in range(4)]
            measurements.extend(z_measurements)
            
            # 2. Two-qubit correlations for temporal volatility persistence
            # (Corsi, 2009: HAR model - captures multi-scale volatility dynamics)
            measurements.append(qml.expval(qml.PauliZ(0) @ qml.PauliZ(3)))  # Long-range: day 1-4
            measurements.append(qml.expval(qml.PauliZ(1) @ qml.PauliZ(2)))  # Short-range: day 2-3
            measurements.append(qml.expval(qml.PauliZ(0) @ qml.PauliZ(1)))  # Adjacent days
            
            # 3. Phase correlation measurement for sign dynamics
            # Y⊗Y measurements capture phase relationships without basis change
            # (Mitarai et al., 2018: "Quantum circuit learning")
            measurements.append(qml.expval(qml.PauliY(0) @ qml.PauliY(3)))
            
            # 4. Three-body correlation for higher-order patterns
            # (Abbas et al., 2021: increased expressivity through multi-qubit observables)
            measurements.append(qml.expval(qml.PauliZ(0) @ qml.PauliZ(1) @ qml.PauliZ(2)))
            
            return measurements
        
        return circuit

    """
    def _create_quantum_circuit(self):
        #
        # Create the quantum circuit as a proper QNode that returns numerical values.
        #
        # This fixes the fundamental issue where the circuit was returning operators
        # instead of expectation values. Based on PennyLane best practices.
        #
        @qml.qnode(self.device, interface="autograd", diff_method="parameter-shift")
        def circuit(volatility_data, params):
            # Apply encoding
            self._encode_volatility_data(volatility_data)
            
            # Apply variational circuit
            self._variational_layers(params, self.active_layers)
            
            # Return actual expectation values (numerical)
            return [
                qml.expval(qml.PauliZ(0)),                    # Realized volatility
                qml.expval(qml.PauliX(1)) if self.n_qubits > 1 else 0.0,  # Jump component
                qml.expval(qml.PauliY(2)) if self.n_qubits > 2 else 0.0,  # Integrated variance
                qml.expval(qml.PauliZ(0) @ qml.PauliZ(1)) if self.n_qubits > 1 else 0.0  # Persistence
            ]
        
        return circuit
    """

    def create_ohlc_windows(self, ohlc_df):
        """
        Create sliding windows of OHLC data for quantum processing.
        
        Parameters:
        -----------
        ohlc_df : pd.DataFrame
            Raw OHLC data from DataPreprocessor.get_raw_ohlc_data()
            
        Returns:
        --------
        dict : {date: ohlc_window} where each window is (lookback_window, 4) array
               Keys are dates, values are OHLC windows ending on that date
        """
        window_size = self.lookback_window
        windows = {}
        
        for i in range(window_size - 1, len(ohlc_df)):
            # Window ENDS at position i (inclusive)
            window_start = i - window_size + 1
            window_end = i + 1  # +1 because iloc is exclusive at end
            
            # Select only OHLC columns
            window = ohlc_df.iloc[window_start:window_end][['Open', 'High', 'Low', 'Close']].values
            date = ohlc_df.index[i]
            windows[date] = window
            
        return windows

    def calculate_signed_garman_klass(self, open_price, high_price, low_price, close_price, prev_close=None):
        """
        Calculate signed Garman-Klass volatility estimator.
        
        Formula: GK = 0.5 * ln(H/L)² - (2ln(2)-1) * ln(C/O)²
        Sign: +1 if Close >= Open (up day), -1 if Close < Open (down day)

        References:
        -----------
        Garman, M. B., & Klass, M. J. (1980). On the estimation of security 
        price volatilities from historical data. Journal of Business, 53(1), 67-78.
        
        Parameters:
        -----------
        open_price, high_price, low_price, close_price : float
            OHLC prices for a single trading period
            
        Returns:
        --------
        float: Signed Garman-Klass value
               Positive if Close >= Open (up day)
               Negative if Close < Open (down day)
        """
        import math
        """
        # Handle edge cases
        if any(price <= 0 for price in [open_price, high_price, low_price, close_price]):
            return 0.0
        
        # Validate price relationships
        if high_price < max(open_price, close_price) or low_price > min(open_price, close_price):
            return 0.0
        """
        # Handle missing or invalid Open price
        if open_price <= 0 or qml.math.isnan(open_price):
            if prev_close is not None and prev_close > 0:
                open_price = prev_close
            else:
                # Fallback: use H-L midpoint
                open_price = (high_price + low_price) / 2.0

        # Ensure Open respects High/Low bounds
        open_price = qml.math.minimum(open_price, high_price)
        open_price = qml.math.maximum(open_price, low_price)
            
        epsilon = 1e-8
    
        # Use differentiable max/min operations
        min_price = qml.math.minimum(open_price, close_price)
        max_price = qml.math.maximum(open_price, close_price)
        
        # Ensure prices are valid (differentiable version)
        # Instead of returning 0.0, use small epsilon to maintain differentiability
        safe_open = qml.math.maximum(open_price, epsilon)
        safe_high = qml.math.maximum(high_price, epsilon)
        safe_low = qml.math.maximum(low_price, epsilon)
        safe_close = qml.math.maximum(close_price, epsilon)
        
        try:
            # Garman-Klass formula
            ln_hl = qml.math.log(safe_high / safe_low)
            ln_co = qml.math.log(safe_close / safe_open)  # Same trading period
            
            # Original Garman-Klass estimator - calculate VARIANCE first
            gk_variance = 0.5 * (ln_hl ** 2) - (2 * qml.math.log(2) - 1) * (ln_co ** 2)
            gk_variance = qml.math.maximum(0.0, gk_variance)  # Ensure non-negative

            # Take square root to get VOLATILITY
            gk_vol = qml.math.sqrt(gk_variance + epsilon)
            
            # Apply sign based on day direction
            #sign = 1 if close_price >= open_price else -1
            sign = qml.math.where(close_price >= open_price, 1.0, -1.0)
            
            return sign * gk_vol
            
        except (ValueError, ZeroDivisionError, OverflowError):
            return epsilon

    def _encode_volatility_data(self, ohlc_window):
        """
        Dual measurement quantum encoding for signed Garman-Klass values.
        
        Implementation based on:
        1. Nakaji et al. (2021) - Dual measurement basis for signed values
        2. Engle (1982) - ARCH effects and volatility clustering  
        3. Financial correlation theory - Multi-scale temporal relationships
        
        Parameters:
        -----------
        ohlc_window: array-like, shape (4, 4)
            4 days of OHLC data: [[O₁,H₁,L₁,C₁], [O₂,H₂,L₂,C₂], ...]
        """
        #print("DEBUG: Starting enhanced dual measurement encoding")
        
        # Step 1: Ensure correct OHLC format
        ohlc_array = np.array(ohlc_window)
        if ohlc_array.shape != (4, 4):
            #print(f"DEBUG: Adjusting OHLC shape from {ohlc_array.shape} to (4,4)")
            if ohlc_array.shape[0] < 4:
                padded = np.zeros((4, 4))
                padded[:ohlc_array.shape[0], :ohlc_array.shape[1]] = ohlc_array
                ohlc_array = padded
            else:
                ohlc_array = ohlc_array[-4:, :4]
        
        # Step 2: Calculate signed G-K values for all 4 days
        signed_gk_values = []
        for day_idx in range(4):
            open_p, high_p, low_p, close_p = ohlc_array[day_idx]
            # Get previous close for Open imputation
            prev_close = None
            if day_idx > 0:
                prev_close = ohlc_array[day_idx - 1, 3]  # Previous day's close
            
            signed_gk = self.calculate_signed_garman_klass(open_p, high_p, low_p, close_p, prev_close)
            signed_gk_values.append(signed_gk)
        
        signed_gk_values = np.array(signed_gk_values)
        #print(f"DEBUG: Signed G-K values: {signed_gk_values}")
        
        # Step 3: Financial data preprocessing
        magnitudes = np.abs(signed_gk_values)
        signs = np.sign(signed_gk_values)
        
        # Convert to annualized volatility (as decimal, not percentage)
        annual_vol = magnitudes * np.sqrt(252)  # e.g., 0.16 for 16% annual vol
        
        # Use log transformation - this naturally maps to ~[0,1]
        # log(1 + 0.1) ≈ 0.095 (10% annual vol)
        # log(1 + 0.5) ≈ 0.405 (50% annual vol)
        # log(1 + 1.72) ≈ 1.000 (172% annual vol)
        normalized_magnitudes = np.log(annual_vol + 1)
        
        # Simple clipping for quantum amplitude validity
        normalized_magnitudes = np.clip(normalized_magnitudes, 0.0, 1.0)
        
        #print(f"DEBUG: Annual vol (decimal): {annual_vol}")
        #print(f"DEBUG: Normalized magnitudes: {normalized_magnitudes}")
        #print(f"DEBUG: Signs: {signs}")
        
        # Step 4: Dual measurement basis encoding (Nakaji et al., 2021)
        for i in range(4):
            magnitude = normalized_magnitudes[i]
            sign = signs[i]
            
            #print(f"DEBUG: Encoding qubit {i}: magnitude={magnitude:.4f}, sign={sign}")
            
            # 4A: Amplitude encoding for magnitude
            magnitude_angle = np.arccos(np.sqrt(magnitude))
            qml.RY(2 * magnitude_angle, wires=i)
            
            # 4B: Sign encoding through phase rotation
            if sign < 0:
                qml.RZ(np.pi, wires=i)  # π phase for negative values
            
            # 4C: Dual measurement preparation (enables sign recovery)
            qml.Hadamard(wires=i)
            phase_modulation = magnitude * np.pi / 2
            qml.RZ(phase_modulation, wires=i)
            qml.Hadamard(wires=i)
        
        # Step 5: Financial correlation encoding - Adjacent correlations (ARCH effects)
        #print("DEBUG: Adding adjacent day correlations (ARCH effects)")
        for i in range(3):  # 0-1, 1-2, 2-3
            gk_i = signed_gk_values[i]
            gk_j = signed_gk_values[i + 1]
            
            # ARCH-style correlation: current × next period volatility
            arch_correlation = gk_i * gk_j
            normalized_correlation = np.tanh(arch_correlation * 10)  # Scale for sensitivity
            
            if abs(normalized_correlation) > 1e-6:  # Only apply if meaningful correlation
                qml.CRZ(normalized_correlation * np.pi, wires=[i, i + 1])
                #print(f"DEBUG: Adjacent correlation {i}-{i+1}: {normalized_correlation:.4f}")
        
        # Step 6: Long-range correlation (regime persistence)
        #print("DEBUG: Adding long-range correlations")
        regime_correlation = signed_gk_values[0] * signed_gk_values[3]  # First-last day
        normalized_regime = np.tanh(regime_correlation * 5)
        
        if abs(normalized_regime) > 1e-6:
            qml.CRZ(normalized_regime * np.pi / 2, wires=[0, 3])
            #print(f"DEBUG: Regime correlation 0-3: {normalized_regime:.4f}")
        
        # Step 7: Cross-day spillover effects
        #print("DEBUG: Adding spillover effects")
        spillover_pairs = [(0, 2), (1, 3)]  # T-3→T-1, T-2→T-0
        
        for i, j in spillover_pairs:
            spillover_correlation = signed_gk_values[i] * signed_gk_values[j]
            normalized_spillover = np.tanh(spillover_correlation * 3)
            
            if abs(normalized_spillover) > 1e-6:
                qml.CRZ(normalized_spillover * np.pi / 4, wires=[i, j])
                #print(f"DEBUG: Spillover correlation {i}-{j}: {normalized_spillover:.4f}")
        
        #print("DEBUG: Enhanced encoding complete")

    def get_enhanced_measurements(self, ohlc_data):
        """
        Get both computational and Hadamard basis measurements for complete
        dual basis information recovery as per Nakaji et al. (2021).
        
        This method runs the circuit twice with different measurement bases.
        """
        # Circuit for computational basis (Z) measurements  
        @qml.qnode(self.device, interface="autograd")
        def z_circuit(ohlc_data, params):
            self._encode_volatility_data(ohlc_data)
            self._variational_layers(params, self.active_layers)
            
            return [qml.expval(qml.PauliZ(i)) for i in range(4)]
        
        # Circuit for Hadamard basis (X) measurements
        @qml.qnode(self.device, interface="autograd")  
        def x_circuit(ohlc_data, params):
            self._encode_volatility_data(ohlc_data)
            self._variational_layers(params, self.active_layers)
            
            return [qml.expval(qml.PauliX(i)) for i in range(4)]
        
        # Circuit for correlation measurements
        @qml.qnode(self.device, interface="autograd")
        def corr_circuit(ohlc_data, params):
            self._encode_volatility_data(ohlc_data)
            self._variational_layers(params, self.active_layers)
            
            correlations = []
            # Adjacent correlations
            for i in range(3):
                correlations.append(qml.expval(qml.PauliZ(i) @ qml.PauliZ(i+1)))
            
            # Long-range correlation
            correlations.append(qml.expval(qml.PauliZ(0) @ qml.PauliZ(3)))
            
            return correlations
        
        # Run all measurement circuits
        z_measurements = z_circuit(ohlc_data, self.params)
        x_measurements = x_circuit(ohlc_data, self.params) 
        corr_measurements = corr_circuit(ohlc_data, self.params)
        
        return {
            'magnitudes': z_measurements,      # Volatility magnitudes
            'signs': x_measurements,           # Market directions  
            'correlations': corr_measurements  # Temporal correlations
        }
    
    def validate_encoding(self, ohlc_window):
        """
        Validate the enhanced encoding by checking information preservation.
        
        Returns:
        --------
        dict: Validation results including reconstruction accuracy
        """
        # Calculate original signed G-K values
        ohlc_array = np.array(ohlc_window)
        if ohlc_array.shape != (4, 4):
            if ohlc_array.shape[0] < 4:
                padded = np.zeros((4, 4))
                padded[:ohlc_array.shape[0], :ohlc_array.shape[1]] = ohlc_array
                ohlc_array = padded
            else:
                ohlc_array = ohlc_array[-4:, :4]
        
        original_gk = []
        for day_idx in range(4):
            open_p, high_p, low_p, close_p = ohlc_array[day_idx]
            """
            signed_gk = self.calculate_signed_garman_klass(open_p, high_p, low_p, close_p)
            """
            prev_close = None
            if day_idx > 0:
                prev_close = ohlc_array[day_idx - 1, 3]
            
            signed_gk = self.calculate_signed_garman_klass(open_p, high_p, low_p, close_p, prev_close)
            original_gk.append(signed_gk)
        
        original_gk = np.array(original_gk)
        
        # Get quantum measurements
        measurements = self.get_enhanced_measurements(ohlc_window)
        
        # Analyze reconstruction quality
        z_vals = np.array(measurements['magnitudes'])
        x_vals = np.array(measurements['signs'])
        
        # Sign agreement
        original_signs = np.sign(original_gk)
        predicted_signs = np.sign(x_vals)
        sign_agreement = np.mean(original_signs == predicted_signs)
        
        # Magnitude correlation  
        original_magnitudes = np.abs(original_gk)
        if np.std(original_magnitudes) > 0:
            magnitude_correlation = np.corrcoef(original_magnitudes, np.abs(z_vals))[0, 1]
        else:
            magnitude_correlation = 1.0 if np.allclose(z_vals, 0) else 0.0
        
        validation_results = {
            'original_gk': original_gk,
            'z_measurements': z_vals,
            'x_measurements': x_vals,
            'sign_agreement': sign_agreement,
            'magnitude_correlation': magnitude_correlation,
            'correlation_measurements': measurements['correlations'],
            'encoding_successful': sign_agreement > 0.5 and abs(magnitude_correlation) > 0.3
        }
        
        return validation_results

    """
    def get_feature_names(self):
        #Return meaningful names for quantum features from enhanced encoding.
        base_names = [
            "quantum_realized_volatility",     # Z-basis measurement
            "quantum_jump_component", 
            "quantum_integrated_variance",
            "quantum_volatility_persistence"
        ]
        
        # Add correlation features if using enhanced measurements
        correlation_names = [
            "quantum_adjacent_correlation_1",
            "quantum_adjacent_correlation_2", 
            "quantum_adjacent_correlation_3",
            "quantum_regime_correlation"
        ]
        
        return base_names  # Return base names for compatibility
    """
    def get_feature_names(self):
        """Return meaningful names for quantum features from enhanced encoding."""
        return [
            "quantum_volatility_day_t-3",
            "quantum_volatility_day_t-2", 
            "quantum_volatility_day_t-1",
            "quantum_volatility_day_t",
            "quantum_persistence_long_range",
            "quantum_persistence_short_range",
            "quantum_persistence_adjacent",
            "quantum_phase_correlation",
            "quantum_higher_order_pattern"
        ]

    """
    def _encode_volatility_data(self, volatility_data):
        #
        #Encode financial time series into quantum states with theoretical grounding.
        #
        #Based on Rebentrost et al. (2018) amplitude encoding for financial data
        #and Schuld & Petruccione (2018) for time series encoding.
        #
        # Handle both 1D and 2D inputs
        if volatility_data.ndim == 1:
            # Single time series
            encoding_data = volatility_data
        else:
            # Multi-dimensional: use principal component
            encoding_data = volatility_data[:, 0] if volatility_data.shape[1] > 0 else volatility_data
        
        # Volatility-preserving normalization (Andersen et al., 2001)
        # Use log transformation to handle heavy tails in volatility
        log_data = np.log(np.abs(encoding_data) + 1e-10)
        normalized_data = np.tanh(log_data / np.std(log_data))
        
        # Amplitude encoding for volatility magnitudes
        for i in range(min(len(normalized_data), self.n_qubits)):
            angle = np.arccos(np.clip(normalized_data[i], -1, 1))
            qml.RY(angle, wires=i)
        
        # Phase encoding for temporal patterns
        for i in range(min(len(normalized_data), self.n_qubits)):
            phase = 2 * np.pi * i / len(normalized_data)
            qml.RZ(phase * normalized_data[i], wires=i)
        
        # Encode volatility clustering (ARCH effects, Engle 1982)
        for i in range(min(len(normalized_data)-1, self.n_qubits-1)):
            correlation = normalized_data[i] * normalized_data[i+1]
            qml.CRZ(correlation * np.pi, wires=[i, (i+1) % self.n_qubits])
    """

    def _variational_layers(self, params, active_layers):
        """
        Variational quantum circuit layers with financial-inspired architecture.
        
        Based on McClean et al. (2016) for VQE and adapted for financial
        time series following Orús et al. (2019).
        """
        for layer_idx in active_layers:
            # Single-qubit rotations
            for i in range(self.n_qubits):
                qml.RX(params[layer_idx, i, 0], wires=i)
                qml.RY(params[layer_idx, i, 1], wires=i)
                qml.RZ(params[layer_idx, i, 2], wires=i)
            
            # Entangling layer with financial correlation structure
            for i in range(self.n_qubits - 1):
                # CRZ gates for volatility correlation
                qml.CRZ(params[layer_idx, i, 3], wires=[i, (i+1) % self.n_qubits])
            
            # Additional entanglement for volatility spillovers
            if layer_idx % 2 == 0:
                for i in range(0, self.n_qubits - 1, 2):
                    qml.CNOT(wires=[i, i+1])
            else:
                for i in range(1, self.n_qubits - 1, 2):
                    qml.CNOT(wires=[i, i+1])
                if self.n_qubits > 2:
                    qml.CNOT(wires=[self.n_qubits-1, 0])

    def _volatility_preserving_phase_space(self, returns_data):
        """
        Phase space transformation that preserves volatility characteristics.
        
        Based on Takens (1981) embedding theorem and adapted for volatility
        following Andersen et al. (2001) realized volatility framework.
        """
        returns = np.array(returns_data)
        
        if returns.ndim > 1:
            # Already multi-dimensional
            return returns
        
        # Create volatility-specific features
        features = []
        
        # 1. Realized volatility (5-minute RV proxy)
        rv = pd.Series(returns).rolling(5).std()
        features.append(rv.fillna(0).values)
        
        # 2. Squared returns (volatility proxy)
        features.append(returns ** 2)
        
        # 3. Absolute returns (robust volatility measure)
        features.append(np.abs(returns))
        
        # 4. ARCH component (lagged squared returns)
        arch_component = np.zeros_like(returns)
        arch_component[1:] = returns[:-1] ** 2
        features.append(arch_component)
        
        # 5. Volatility persistence (exponentially weighted)
        ewm_vol = pd.Series(returns).ewm(span=10).std()
        features.append(ewm_vol.fillna(0).values)
        
        # 6. Jump component (Barndorff-Nielsen & Shephard, 2004)
        jump_threshold = 3 * np.std(returns)
        jumps = np.where(np.abs(returns) > jump_threshold, returns, 0)
        features.append(jumps)
        
        # Combine features
        feature_matrix = np.array(features).T
        
        # Reduce to n_qubits dimensions using volatility-preserving PCA
        if feature_matrix.shape[1] > self.n_qubits:
            from sklearn.decomposition import PCA
            pca = PCA(n_components=self.n_qubits)
            feature_matrix = pca.fit_transform(feature_matrix)
            
            # Scale to preserve volatility magnitudes
            vol_scale = np.std(returns) / np.std(feature_matrix[:, 0])
            feature_matrix *= vol_scale
        
        return feature_matrix

    def _initialize_parameters(self):
        """Initialize parameters using identity block strategy."""
        self.params = np.zeros((self.max_layers, self.n_qubits, 4))
        self.params += np.random.normal(0, 0.05, self.params.shape)
        self.active_layers = list(range(self.n_layers))
        
        # Initialize the circuit here
        self.circuit = self._create_quantum_circuit()

    def fit(self, ohlc_df, targets=None, learning_rate=0.01, epochs=100,   #100
            batch_size=128, verbose=True,
            early_stopping=True, patience=10, min_delta=1e-3,
            tb_writer=None):
        """
        Train the quantum volatility detector on OHLC data.
        
        Parameters:
        -----------
        ohlc_df : pd.DataFrame
            Raw OHLC data from DataPreprocessor.get_raw_ohlc_data()
            Must have columns: ['Open', 'High', 'Low', 'Close']
        targets : array-like, optional
            Volatility targets. If None, uses next-day |G-K| as target
        learning_rate : float
            Learning rate for optimization
        epochs : int
            Number of training epochs
        batch_size : int
            Batch size for training
        verbose : bool
            Whether to print training progress
            
        Returns:
        --------
        self : fitted detector
        """
        from tqdm import tqdm
        
        # Create OHLC windows
        windows_dict = self.create_ohlc_windows(ohlc_df)
        
        # Convert to arrays for training
        dates_list = sorted(windows_dict.keys())
        ohlc_windows = np.array([windows_dict[date] for date in dates_list])
        
        if verbose:
            print(f"Created {len(ohlc_windows)} OHLC windows from {len(ohlc_df)} days")
            #print(f"Each window shape: {ohlc_windows[0].shape}")

        # Create G-K volatility targets if not provided
        if targets is None:
            targets = []
            
            # For each window, predict the NEXT day's G-K
            for i in range(len(dates_list) - 1):  # -1 because last date has no next day
                current_date = dates_list[i]
                next_date = dates_list[i + 1]
                
                # Get next day's OHLC directly from the DataFrame
                next_day_idx = ohlc_df.index.get_loc(next_date)
                next_day_ohlc = ohlc_df.iloc[next_day_idx]

                """
                # Calculate next day's G-K
                next_gk = self.calculate_signed_garman_klass(
                    next_day_ohlc['Open'],
                    next_day_ohlc['High'], 
                    next_day_ohlc['Low'],
                    next_day_ohlc['Close']
                )
                """
                
                # Get current day's close for potential Open imputation
                current_day_idx = ohlc_df.index.get_loc(current_date)
                current_day_close = ohlc_df.iloc[current_day_idx]['Close']
                
                # Calculate next day's G-K
                next_gk = self.calculate_signed_garman_klass(
                    next_day_ohlc['Open'],
                    next_day_ohlc['High'], 
                    next_day_ohlc['Low'],
                    next_day_ohlc['Close'],
                    prev_close=current_day_close
                )
                
                targets.append(next_gk)
            
            # Remove last window (no target for it)
            ohlc_windows = ohlc_windows[:-1]
            dates_list = dates_list[:-1]
            targets = np.array(targets)
            
            if verbose:
                print(f"Generated {len(targets)} G-K volatility targets")
                print(f"Target statistics: mean={np.mean(targets):.6f}, std={np.std(targets):.6f}")

                # Debug: Check a few G-K calculations
                if len(targets) > 0:
                    print(f"\nDEBUG: First 5 G-K values: {targets[:5]}")
                    
                    # Check first window's OHLC data
                    first_window = ohlc_windows[0]
                    print(f"DEBUG: First window OHLC shape: {first_window.shape}")
                    print(f"DEBUG: First day OHLC: {first_window[0]}")
                    
                    # Manually calculate first target
                    next_day_idx = ohlc_df.index.get_loc(dates_list[1])
                    next_day_ohlc = ohlc_df.iloc[next_day_idx]
                    manual_gk = self.calculate_signed_garman_klass(
                        next_day_ohlc['Open'],
                        next_day_ohlc['High'],
                        next_day_ohlc['Low'],
                        next_day_ohlc['Close']
                    )
                    print(f"DEBUG: Manual G-K calculation: {manual_gk}")
                    print(f"DEBUG: Next day OHLC values: O={next_day_ohlc['Open']}, H={next_day_ohlc['High']}, L={next_day_ohlc['Low']}, C={next_day_ohlc['Close']}")
        
        # Validate targets length
        if len(targets) != len(ohlc_windows):
            raise ValueError(f"Number of targets ({len(targets)}) must match number of windows ({len(ohlc_windows)})")
        
        # Initialize optimizer
        optimizer = qml.AdamOptimizer(stepsize=learning_rate)
        
        if verbose:
            print(f"\nTraining quantum volatility detector with {self.n_qubits} qubits...")
            print(f"Using enhanced OHLC encoding with signed Garman-Klass values")

        self.is_fitted = True

        if early_stopping:
            best_loss = float('inf')
            patience_counter = 0
            best_params = None

        # ADD PREDICTION TRACKING:
        epoch_prediction_corrs = []
        
        # Training loop
        pbar = tqdm(range(epochs), 
                    desc="Training Quantum Circuit",
                    unit="epoch",
                    bar_format='{desc}: {percentage:3.0f}%|{bar}| Epoch {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]')
        
        for epoch in pbar:
            # Shuffle data
            indices = np.random.permutation(len(ohlc_windows))
            epoch_loss = 0
            batches_processed = 0
            epoch_gradients = []  # Track all gradient norms for this epoch

            """
            # Create batch indices list
            batch_indices_list = list(range(0, len(indices), batch_size))
            
            # Add batch progress bar for long epochs
            if len(batch_indices_list) > 1:  # Only show for epochs with certain number of batches batches
                batch_iterator = tqdm(batch_indices_list, 
                                     desc=f"  Epoch {epoch+1} batches",
                                     leave=False,
                                     unit="batch")
            else:
                batch_iterator = batch_indices_list
            
            # Iterate over batch_iterator instead of range()
            for batch_start in batch_iterator:
            
                batch_end = min(batch_start + batch_size, len(indices))
                batch_indices = indices[batch_start:batch_end]
            """
            for batch_start in range(0, len(indices), batch_size):
                batch_end = min(batch_start + batch_size, len(indices))
                batch_indices = indices[batch_start:batch_end]

                batch_windows = ohlc_windows[batch_indices]
                batch_targets = targets[batch_indices]
                
                # Get active parameters
                active_params = self.params[self.active_layers]
                
                # Define cost function for this batch
                def cost_fn(params):
                    return self._volatility_aware_cost(params, batch_windows, batch_targets)
                
                # Calculate gradients and update
                """
                try:
                    # Compute gradients
                    gradients, grad_norm = self._calculate_gradients(
                        active_params, batch_windows, batch_targets
                    )

                    print(f"  Batch {batch_start//batch_size}: Gradient norm = {grad_norm:.8f}")
                    if grad_norm < 1e-6:
                        print(f"  WARNING: Near-zero gradients detected!")

                    param_before = active_params.copy()
                    
                    # Update parameters using optimizer
                    active_params = optimizer.step(cost_fn, active_params)
                """
                # Calculate gradients and update
                try:
                    # Compute gradients
                    gradients, grad_norm = self._calculate_gradients(
                        active_params, batch_windows, batch_targets
                    )

                    epoch_gradients.append(grad_norm)  # Collect for averaging
                    
                    #print(f"  Batch {batch_start//batch_size}: Gradient norm = {grad_norm:.8f}")
                    if grad_norm < 1e-6:
                        print(f"  WARNING: Near-zero gradients detected!")

                    param_before = active_params.copy()
                    
                    # MANUALLY apply the gradient update since optimizer.step isn't working
                    # For Adam optimizer, we need to track momentum
                    if not hasattr(self, '_adam_m'):
                        self._adam_m = np.zeros_like(active_params)
                        self._adam_v = np.zeros_like(active_params)
                        self._adam_t = 0
                    
                    self._adam_t += 1
                    
                    # Adam update rule
                    beta1, beta2 = 0.9, 0.999
                    self._adam_m = beta1 * self._adam_m + (1 - beta1) * gradients
                    self._adam_v = beta2 * self._adam_v + (1 - beta2) * gradients**2
                    
                    m_hat = self._adam_m / (1 - beta1**self._adam_t)
                    v_hat = self._adam_v / (1 - beta2**self._adam_t)
                    
                    active_params = active_params - learning_rate * m_hat / (np.sqrt(v_hat) + 1e-8)

                    param_change = np.linalg.norm(active_params - param_before)
                    #print(f"  Parameter change after update: {param_change:.8f}")
                    if param_change < 1e-8:
                        print(f"  WARNING: Parameters not updating!")
                    
                    # Copy back to main parameters
                    for i, layer_idx in enumerate(self.active_layers):
                        self.params[layer_idx] = active_params[i]
                    
                    # Calculate loss for this batch
                    batch_loss = cost_fn(active_params)
                    epoch_loss += batch_loss * len(batch_indices)
                    batches_processed += len(batch_indices)

                    """
                    # Record gradient norm
                    if 'gradients' not in self.training_history:
                        self.training_history['gradients'] = []
                    self.training_history['gradients'].append(grad_norm)
                    """
                    
                except Exception as e:
                    if verbose:
                        print(f"Warning: Batch failed with error: {e}")
                    continue
            
            # Calculate average loss for epoch
            avg_loss = epoch_loss / batches_processed if batches_processed > 0 else float('inf')
            self.training_history['loss'].append(avg_loss)
            self.training_history['layers_used'].append(len(self.active_layers))

            avg_gradient = np.mean(epoch_gradients) if epoch_gradients else 0.0
            self.training_history['gradients'].append(avg_gradient)  # Store for history

            if early_stopping:
                if avg_loss < best_loss - min_delta:
                    best_loss = avg_loss
                    best_params = deepcopy(self.params)
                    patience_counter = 0
                else:
                    patience_counter += 1
                    
                if patience_counter >= patience:
                    if verbose:
                        print(f"\nEarly stopping triggered at epoch {epoch+1}")
                        print(f"Best loss: {best_loss:.6f} (restoring best parameters)")
                    
                    # Restore best parameters
                    self.params = best_params
                    break
            
            # Adaptive depth adjustment
            if epoch > 0 and len(self.training_history['gradients']) > 0:
                recent_grad_norm = self.training_history['gradients'][-1]
                depth_changed = self._adjust_circuit_depth(recent_grad_norm)
                if depth_changed and verbose:
                    print(f"  Circuit depth adjusted to {len(self.active_layers)} layers")
            
            # After epoch completion, log metrics
            if tb_writer is not None and epoch % 1 == 0:  # Log every 5 epochs
                # Calculate correlations on validation subset
                sample_indices = np.random.choice(len(ohlc_windows), 
                                                min(100, len(ohlc_windows)), 
                                                replace=False)
                
                predictions = []
                actuals = []
                signs_pred = []
                signs_actual = []
                
                for idx in sample_indices:
                    window = ohlc_windows[idx]
                    pred_gk = self.predict_volatility(window)
                    actual_gk = targets[idx]
                    
                    predictions.append(pred_gk)
                    actuals.append(actual_gk)
                    signs_pred.append(np.sign(pred_gk))
                    signs_actual.append(np.sign(actual_gk))
                
                # Calculate metrics
                signed_corr = np.corrcoef(predictions, actuals)[0, 1]  # Pearson correlation of signed values
                magnitude_corr = np.corrcoef(np.abs(predictions), np.abs(actuals))[0, 1]  # Pearson correlation of magnitudes
                sign_accuracy = np.mean(np.array(signs_pred) == np.array(signs_actual))  # Fraction of correct sign predictions
                
                with tb_writer.as_default():
                    tf.summary.scalar('QuantumCircuit/InitialTraining/Loss', avg_loss, step=epoch)
                    tf.summary.scalar('QuantumCircuit/InitialTraining/GradientNorm', avg_gradient, step=epoch)
                    tf.summary.scalar('QuantumCircuit/InitialTraining/SignedGKCorrelation_Pearson', signed_corr, step=epoch)
                    tf.summary.scalar('QuantumCircuit/InitialTraining/MagnitudeCorrelation_Pearson', magnitude_corr, step=epoch)
                    tf.summary.scalar('QuantumCircuit/InitialTraining/SignAccuracy_DirectionalCorrectness', sign_accuracy, step=epoch)
                    
            # Progress reporting
            if verbose and epoch % 1 == 0:
        
                # Sample 50 windows for correlation check
                sample_size = min(50, len(ohlc_windows) - 1)
                sample_indices = np.random.choice(len(ohlc_windows) - 1, sample_size, replace=False)
                
                predictions = []
                actuals = []
                
                for idx in sample_indices:
                    # Predict using current parameters
                    window = ohlc_windows[idx]
                    pred_gk = self.predict_volatility(window)
                    
                    # Actual next-day G-K (already calculated in targets)
                    actual_gk = targets[idx]
                    
                    predictions.append(pred_gk)
                    actuals.append(actual_gk)
                
                # Calculate THREE different correlations
                if len(predictions) > 1:
                    predictions = np.array(predictions)
                    actuals = np.array(actuals)
                    
                    # 1. Signed G-K correlation (original)
                    signed_corr = np.corrcoef(predictions, actuals)[0, 1]
                    
                    # 2. Magnitude correlation
                    pred_magnitudes = np.abs(predictions)
                    actual_magnitudes = np.abs(actuals)
                    magnitude_corr = np.corrcoef(pred_magnitudes, actual_magnitudes)[0, 1]
                    
                    # 3. Sign accuracy (not correlation, but % correct)
                    pred_signs = np.sign(predictions)
                    actual_signs = np.sign(actuals)
                    sign_accuracy = np.mean(pred_signs == actual_signs)
                    
                    # Also calculate sign correlation for completeness
                    sign_corr = np.corrcoef(pred_signs, actual_signs)[0, 1]
                    
                    epoch_prediction_corrs.append((epoch, signed_corr))
                    self.training_history['prediction_correlations'] = epoch_prediction_corrs
                    
                # Update progress bar description with current metrics
                if epoch % 1 == 0 and 'signed_corr' in locals():
                    pbar.set_description(
                        f"Training QC | Loss: {avg_loss:.4f} | "
                        f"Signed: {signed_corr:+.3f} | "
                        f"Mag: {magnitude_corr:+.3f} | "
                        f"Sign Acc: {sign_accuracy:.1%}"
                    )
                else:
                    pbar.set_description(f"Training Quantum Circuit | Loss: {avg_loss:.4f}")
                
                #print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.6f}, Avg Gradient: {avg_gradient:.6f}")
                #print(f"  Signed G-K correlation: {signed_corr:.3f}")
                #print(f"  Magnitude correlation: {magnitude_corr:.3f}")
                #print(f"  Sign accuracy: {sign_accuracy:.1%} (correlation: {sign_corr:.3f})")
                
                # Additional diagnostics every 50 epochs
                if epoch % 50 == 0 and epoch > 0:
                    # Test on a few samples to see if encoding is working
                    test_window = ohlc_windows[0]
                    test_features = self.transform(test_window.reshape(1, 4, 4))[0]
                    print(f"  Sample quantum features: {test_features}")
                    
                    # Validate encoding
                    validation = self.validate_encoding(test_window)
                    print(f"  Single window validation - Sign agreement: {validation['sign_agreement']:.2f}, "
                          f"Magnitude correlation: {validation['magnitude_correlation']:.3f}")
                    print(f"  (Note: This is just window[0], see end of training for full validation)")
        
        #self.is_fitted = True
        self._circuit_call_count = 0

        if verbose:
            print(f"\n=== FINAL PREDICTION ACCURACY VALIDATION ===")
            
            # Test on all windows
            all_predictions = []
            all_actuals = []
            
            for i in range(len(ohlc_windows) - 1):  # -1 because last window has no target
                window = ohlc_windows[i]
                
                # Get prediction
                pred_gk = self.predict_volatility(window)
                actual_gk = targets[i]
                
                all_predictions.append(pred_gk)
                all_actuals.append(actual_gk)
            
            # Calculate metrics
            predictions = np.array(all_predictions)
            actuals = np.array(all_actuals)
            
            # Overall correlation
            overall_corr = np.corrcoef(predictions, actuals)[0, 1]
            
            # Sign accuracy
            sign_accuracy = np.mean(np.sign(predictions) == np.sign(actuals))
            
            # Magnitude correlation (using absolute values)
            mag_corr = np.corrcoef(np.abs(predictions), np.abs(actuals))[0, 1]
            
            # Mean Absolute Error
            mae = np.mean(np.abs(predictions - actuals))
            
            print(f"Overall prediction correlation: {overall_corr:.3f}")
            print(f"Sign accuracy: {sign_accuracy:.3f} ({int(sign_accuracy * len(predictions))}/{len(predictions)} correct)")
            print(f"Magnitude correlation: {mag_corr:.3f}")
            print(f"Mean Absolute Error: {mae:.6f}")
            
            # Separate analysis for up/down days
            up_mask = actuals > 0
            down_mask = actuals < 0
            
            if np.any(up_mask):
                up_corr = np.corrcoef(predictions[up_mask], actuals[up_mask])[0, 1]
                print(f"Correlation on up days: {up_corr:.3f} (n={np.sum(up_mask)})")
            
            if np.any(down_mask):
                down_corr = np.corrcoef(predictions[down_mask], actuals[down_mask])[0, 1]
                print(f"Correlation on down days: {down_corr:.3f} (n={np.sum(down_mask)})")
            
            # Show prediction distribution
            print(f"\nPrediction statistics:")
            print(f"  Range: [{np.min(predictions):.6f}, {np.max(predictions):.6f}]")
            print(f"  Mean: {np.mean(predictions):.6f}, Std: {np.std(predictions):.6f}")
            print(f"Actual statistics:")
            print(f"  Range: [{np.min(actuals):.6f}, {np.max(actuals):.6f}]")
            print(f"  Mean: {np.mean(actuals):.6f}, Std: {np.std(actuals):.6f}")
            
            # The encoding validation can stay but clarify what it measures
            print(f"\n=== ENCODING PRESERVATION CHECK ===")
            print("(This checks if quantum measurements preserve input patterns, NOT prediction accuracy)")
        
        if verbose:
            print(f"\nTraining complete!")
            print(f"Final loss: {self.training_history['loss'][-1]:.6f}")
            print(f"Total gradient updates: {len(self.training_history['gradients'])}")
        
        return self

    """
    def fit(self, returns_series, targets=None, learning_rate=0.01, epochs=100, 
            batch_size=16, verbose=True):
        # 
        #Train the quantum volatility detector with proper error handling.
        #
        # Convert to numpy
        if isinstance(returns_series, (pd.DataFrame, pd.Series)):
            returns_series = returns_series.values
        
        # Prepare training windows
        volatility_samples = []
        for i in range(len(returns_series) - self.lookback_window + 1):
            sample = returns_series[i:i+self.lookback_window]
            volatility_samples.append(sample)
        
        volatility_samples = np.array(volatility_samples)
        
        # Create targets if not provided
        if targets is None:
            # Use realized volatility as target
            targets = np.array([np.std(sample) for sample in volatility_samples])
        
        # Initialize optimizer
        optimizer = qml.AdamOptimizer(stepsize=learning_rate)
        
        if verbose:
            print(f"Training quantum volatility detector with {self.n_qubits} qubits...")
        
        for epoch in range(epochs):
            indices = np.random.permutation(len(volatility_samples))
            epoch_loss = 0
            batches = 0
            
            for i in range(0, len(indices), batch_size):
                batch_indices = indices[i:min(i+batch_size, len(indices))]
                batch_samples = volatility_samples[batch_indices]
                batch_targets = targets[batch_indices]
                
                # Get active parameters
                active_params = self.params[self.active_layers]
                
                # Define cost function for this batch
                def cost_fn(params):
                    return self._volatility_aware_cost(params, batch_samples, batch_targets)
                
                # Update parameters
                active_params = optimizer.step(cost_fn, active_params)
                
                # Copy back
                for i, layer_idx in enumerate(self.active_layers):
                    self.params[layer_idx] = active_params[i]
                
                # Calculate loss
                batch_loss = cost_fn(active_params)
                epoch_loss += batch_loss
                batches += 1
            
            # Record history
            avg_loss = epoch_loss / batches if batches > 0 else 0
            self.training_history['loss'].append(avg_loss)
            self.training_history['layers_used'].append(len(self.active_layers))
            
            if verbose and epoch % 10 == 0:
                print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.6f}")
        
        self.is_fitted = True
        return self
    """

    def fit_transform(self, returns_series, targets=None, **fit_params):
        """
        Fit the detector and transform the input data.
        
        Parameters:
        -----------
        returns_series : array-like or pandas.DataFrame
            Time series of financial returns.
            
        targets : array-like, optional
            Volatility targets for supervised learning.
            
        **fit_params : dict
            Additional parameters for fit method.
            
        Returns:
        --------
        array-like
            Quantum volatility features.
        """
        print("DEBUG: Entering fit_transform method")
        result = self.transform(returns_series)
        print("DEBUG: Exiting fit_transform method with result type:", type(result))
        return result

    def predict_volatility(self, ohlc_window):
        """
        Predict signed Garman-Klass volatility for next period.
        
        Parameters:
        -----------
        ohlc_window : array-like, shape (4, 4)
            4-day OHLC window
            
        Returns:
        --------
        float : Predicted signed Garman-Klass value
        """
        if not self.is_fitted:
            raise RuntimeError("Detector must be fitted before prediction")
        
        # Get quantum features
        features = self.transform(ohlc_window.reshape(1, 4, 4))[0]

        """
        # Use same prediction logic as cost function
        z_avg = np.mean(features[:4])
        magnitude_pred = abs(z_avg) * 0.02
        
        phase_indicator = features[7] if len(features) > 7 else 0
        #z_asymmetry = features[0] - features[3]
        # Use safe indexing that works with autograd
        z_asymmetry = (features[0] if len(features) > 0 else 0) - (features[3] if len(features) > 3 else 0)

        sign_pred = np.tanh(z_avg + 0.5 * phase_indicator + 0.3 * z_asymmetry)
        
        return magnitude_pred * np.sign(sign_pred)
        """

        # Use SAME logic as cost function
        z_avg = np.mean(features[:4])
        #magnitude_pred = abs(z_avg) * 0.02
        magnitude_pred = np.abs(z_avg) * 0.01
        
        phase_indicator = features[7] if len(features) > 7 else 0
        z_asymmetry = (features[0] if len(features) > 0 else 0) - (features[3] if len(features) > 3 else 0)
        
        # Match cost function: use logit + sigmoid
        sign_logit = 5.0 * (z_avg + 0.5 * phase_indicator + 0.3 * z_asymmetry)   #Amplify logit multiplication factor (1.0 -> 5.0) for clearer distinctions
        sign_prob = 1.0 / (1.0 + np.exp(-sign_logit))
        
        # Convert probability to sign: >0.5 means positive, <0.5 means negative
        sign_pred = 1.0 if sign_prob > 0.5 else -1.0
        
        return magnitude_pred * sign_pred
        
    def _reduce_features_pca(self, feature_matrix):
        """
        Reduce features using PCA while preserving volatility structure.
        Based on Johnson-Lindenstrauss lemma for dimensionality reduction.
        
        Updated to maintain volatility characteristics during reduction.
        """
        from sklearn.decomposition import PCA
        
        # First, identify volatility-related features
        volatility_preserved_features = []
        
        # Check for any features that directly represent volatility
        for i in range(feature_matrix.shape[1]):
            feature_data = feature_matrix[:, i]
            # High variance features likely contain volatility information
            if np.std(feature_data) > np.mean(np.std(feature_matrix, axis=0)):
                volatility_preserved_features.append(i)
        
        # Apply PCA
        pca = PCA(n_components=self.n_qubits)
        reduced = pca.fit_transform(feature_matrix)
        
        # Ensure volatility scale is preserved
        original_volatility = np.std(feature_matrix)
        reduced_volatility = np.std(reduced)
        
        if reduced_volatility > 0:
            volatility_scale = original_volatility / reduced_volatility
            reduced *= volatility_scale
        
        # Normalize to quantum circuit range while preserving relative magnitudes
        max_val = np.max(np.abs(reduced))
        if max_val > 0:
            # Use tanh to compress to [-1, 1] while preserving structure
            reduced = np.tanh(reduced / max_val)
        
        return reduced

    def diagnose_circuit_output(self, volatility_data):
        """
        Perform detailed diagnosis of what the circuit is actually returning.
        """
        print("\n===== CIRCUIT OUTPUT DIAGNOSIS =====")
        
        # Run circuit with single sample
        print("Running circuit with a single sample...")
        circuit_output = self.circuit(volatility_data, self.params)
        
        # Examine the output in detail
        print(f"Circuit output type: {type(circuit_output)}")
        print(f"Output content: {circuit_output}")
        
        if hasattr(circuit_output, 'shape'):
            print(f"Output shape: {circuit_output.shape}")
        elif isinstance(circuit_output, (list, tuple)):
            print(f"Output length: {len(circuit_output)}")
            for i, val in enumerate(circuit_output):
                print(f"  Element {i}: {type(val)} - {val}")
        
        # Check if output matches expected structure
        if isinstance(circuit_output, (list, tuple)) and len(circuit_output) == self.N_QUANTUM_FEATURES:
            print(f"✓ Circuit is returning exactly {self.N_QUANTUM_FEATURES} elements as expected.")
        else:
            print(f"✗ Circuit is NOT returning the expected {self.N_QUANTUM_FEATURES} elements.")
        
        # Check if the _create_circuit method is actually returning what we expect
        print("\nExamining _create_circuit implementation...")
        if hasattr(self, '_create_circuit'):
            src = inspect.getsource(self._create_circuit)
            print(src)
        
        return circuit_output

    def _calculate_gradients(self, params, volatility_batch, targets=None):
        """
        Calculate gradients with proper version compatibility and detailed diagnostics.
        Based on McClean et al. (2016) parameter-shift rule for quantum gradients.
        """
        #print("\n==== GRADIENT CALCULATION DIAGNOSTICS ====")
        #print(f"Input parameters shape: {params.shape}")
        #print(f"Batch shape: {volatility_batch.shape if hasattr(volatility_batch, 'shape') else 'not array'}")
        #print(f"Targets provided: {targets is not None}")
        
        # Fix for dtype compatibility issue
        class GradientCalculator:
            def __init__(self, cost_fn):
                self.cost_fn = cost_fn
                
            def compute_gradient(self, params, *args):
                """Compute gradient using parameter-shift rule."""
                print("\nUsing manual parameter-shift rule implementation (Parallelized)")
                shift = np.pi / 2
                grads = np.zeros_like(params)
                
                total_params = np.prod(params.shape)
                print(f"Computing gradients for {total_params} parameters")

                from concurrent.futures import ThreadPoolExecutor
                import threading
                
                def compute_single_gradient(idx_tuple):
                    idx, param_count = idx_tuple
                    
                    params_plus = params.copy()
                    params_minus = params.copy()
                    
                    params_plus[idx] += shift
                    params_minus[idx] -= shift
                    
                    # Calculate shifted costs - ensure scalar returns
                    cost_plus = float(self.cost_fn(params_plus, *args))
                    cost_minus = float(self.cost_fn(params_minus, *args))
                    
                    # Parameter-shift gradient
                    grad = (cost_plus - cost_minus) / 2.0
                    
                    if np.isnan(grad) or np.isinf(grad):
                        print(f"  WARNING: Invalid gradient at {idx}: {grad}")
                        grad = 0.0
                        
                    return idx, grad
                
                # Create index list with counts
                idx_list = [(idx, i) for i, idx in enumerate(np.ndindex(params.shape))]
                
                # Use ThreadPoolExecutor for parallel computation
                with ThreadPoolExecutor(max_workers=4) as executor:
                    results = list(executor.map(compute_single_gradient, idx_list))
                
                # Fill in gradients
                for idx, grad in results:
                    grads[idx] = grad
                    
                return grads
        
        # Try standard PennyLane gradient first
        #print("\nAttempting standard PennyLane gradient calculation...")
        try:
            def scalar_cost(params, batch, targets):
                # Don't force float conversion during gradient calculation
                cost = self._volatility_aware_cost(params, batch, targets)
                return cost  # Let autograd handle the type
            """
            # Ensure cost function returns scalar
            def scalar_cost(params, batch, targets):
                try:   
                    cost = self._volatility_aware_cost(params, batch, targets)
                    if isinstance(cost, (list, tuple, np.ndarray)):
                        return float(cost[0]) if len(cost) > 0 else 0.0
                    return float(cost)
                except Exception as e:
                    print(f"ERROR in scalar_cost: {e}")
                    print(f"Error type: {type(e)}")
                    import traceback
                    traceback.print_exc()
                    raise
            """
            
            grad_func = qml.grad(scalar_cost, argnum=0)
            grads = grad_func(params, volatility_batch, targets)
            #print("✓ Standard gradient calculation successful")
        except (TypeError, AttributeError) as e:
            print(f"✗ Standard gradient failed: {e}")
            print(f"Error type: {type(e).__name__}")
            import traceback
            traceback.print_exc()
            
            # More detailed error analysis
            if "setting an array element with a sequence" in str(e):
                print("\n!!! DIMENSIONAL MISMATCH DETECTED !!!")
                print("This usually means trying to assign multiple values to a single array position")
            
            if "dtype" in str(e) or "grad_np_mean" in str(e):
                print("Detected dtype compatibility issue - switching to parameter-shift rule")
                calculator = GradientCalculator(self._volatility_aware_cost)
                grads = calculator.compute_gradient(params, volatility_batch, targets)
            else:
                print("Unexpected error - re-raising")
                raise e
        
        # Detailed gradient analysis
        #print("\n--- Gradient Analysis ---")
        grad_norm = np.linalg.norm(grads)
        grad_variance = np.var(grads)
        grad_mean = np.mean(grads)
        grad_std = np.std(grads)
        """
        print(f"Gradient norm: {grad_norm:.8f}")
        print(f"Gradient variance: {grad_variance:.8e}")
        print(f"Gradient mean: {grad_mean:.8e}")
        print(f"Gradient std: {grad_std:.8e}")
        print(f"Max gradient: {np.max(np.abs(grads)):.8f}")
        print(f"Min gradient: {np.min(np.abs(grads)):.8f}")
        print(f"Non-zero gradients: {np.sum(np.abs(grads) > 1e-8)} / {np.prod(grads.shape)}")
        
        # Per-layer analysis
        if len(params.shape) == 3:  # (layers, qubits, params_per_qubit)
            print("\n--- Per-Layer Gradient Analysis ---")
            for layer in range(params.shape[0]):
                layer_grads = grads[layer]
                layer_norm = np.linalg.norm(layer_grads)
                print(f"Layer {layer}: norm={layer_norm:.6f}, "
                      f"mean={np.mean(layer_grads):.6e}, "
                      f"max={np.max(np.abs(layer_grads)):.6f}")
        
        # Barren plateau detection
        print("\n--- Barren Plateau Detection ---")
        """
        if grad_variance < 1e-8:
            print(" WARNING: Potential barren plateau detected (variance < 1e-8)")
            print("Suggested actions:")
            print("  1. Reduce circuit depth")
            print("  2. Use local cost functions")
            print("  3. Try different initialization")
        elif grad_variance < 1e-6:
            print(" Caution: Low gradient variance detected")
        #else:
            #print("✓ Gradient variance healthy")
        
        if grad_norm < self.gradient_threshold:
            print(f" Gradient norm below threshold ({grad_norm:.8f} < {self.gradient_threshold})")
        
        #print("==== END GRADIENT CALCULATION ====\n")
        
        return grads, grad_norm

    def _adjust_circuit_depth(self, gradient_norm):
        """
        Adaptively adjust circuit depth based on gradient magnitudes.
        
        Based on the layerwise learning approach from Skolik et al. (2021).
        
        Parameters:
        -----------
        gradient_norm : float
            Norm of the gradient vector.
            
        Returns:
        --------
        bool
            Whether the circuit depth was changed.
        """
        if not self.adaptive_depth:
            return False
            
        # Get current number of active layers
        current_depth = len(self.active_layers)
        
        # Check if we're in a potential barren plateau
        if gradient_norm < self.gradient_threshold:
            # If we're already at minimum depth, can't reduce further
            if current_depth <= 1:
                return False
                
            # Reduce depth by removing the last layer
            self.active_layers = self.active_layers[:-1]
            print(f"Reducing circuit depth to {len(self.active_layers)} layers due to low gradient norm: {gradient_norm:.6f}")
            return True
            
        # If gradients are healthy and we haven't reached max depth, consider adding a layer
        elif gradient_norm > self.gradient_threshold * 10 and current_depth < self.max_layers:
            # Only add a layer if we have good history of healthy gradients
            if len(self.training_history['gradients']) >= 5:
                recent_grads = self.training_history['gradients'][-5:]
                if all(g > self.gradient_threshold for g in recent_grads):
                    next_layer = max(self.active_layers) + 1
                    if next_layer < self.max_layers:
                        self.active_layers.append(next_layer)
                        print(f"Increasing circuit depth to {len(self.active_layers)} layers due to healthy gradient norm: {gradient_norm:.6f}")
                        return True
                        
        return False

    def _prepare_training_windows(self, returns_series):
        """
        Prepare training windows preserving temporal semantics.
        Based on Takens (1981) embedding theorem.
        """
        windows = []
        for i in range(len(returns_series) - self.lookback_window + 1):
            window = returns_series[i:i + self.lookback_window]
            windows.append(window)
        return np.array(windows)

    """
    def _volatility_aware_cost(self, params, ohlc_batch, targets):
        #
        #Cost function for signed volatility prediction combining:
        #- Robust volatility loss functions (Patton, 2011)
        #- Directional accuracy (Christoffersen & Diebold, 2006)
        #- Quantum feature extraction principles (Schuld & Petruccione, 2018)
        
        #References:
        #- Patton (2011): "Volatility forecast comparison using imperfect volatility proxies"
        #- Christoffersen & Diebold (2006): "Financial asset returns, direction-of-change 
        #  forecasting, and volatility dynamics"
        #- Andersen et al. (2003): "Modeling and forecasting realized volatility"
        #- Liu et al. (2019): "Option pricing with quantum computers"
        # 
        #print(f"\nDEBUG _volatility_aware_cost: params shape={params.shape}, batch shape={ohlc_batch.shape}, targets shape={targets.shape}")
        
        n_samples = len(ohlc_batch)
        total_loss = qml.numpy.array(0.0)
        epsilon = qml.numpy.array(1e-8)  # Numerical stability
        
        for i in range(n_samples):
            try:
                # Debug check
                single_window = ohlc_batch[i]
                if single_window.shape != (4, 4):
                    print(f"ERROR: Expected (4,4) window, got {single_window.shape}")
            
                # Get quantum measurements
                features = self.circuit(single_window, params)
                
                # Convert to qml.numpy array 
                features = qml.numpy.array(features)
                #print(f"DEBUG: Features shape={features.shape}")
    
                if len(features) != self.N_QUANTUM_FEATURES:
                    print(f"ERROR: Expected {self.N_QUANTUM_FEATURES} measurements, got {len(features)}")
                
                # Target signed G-K value
                target_sgk = targets[i]
                target_magnitude = qml.numpy.abs(target_sgk)
                target_sign = qml.numpy.sign(target_sgk) if target_sgk != 0 else qml.numpy.array(0.0)
                
                # Extract predictions from quantum features
                # 1. Magnitude prediction from average Z measurements
                # (Blank et al., 2020: quantum kernel captures non-linear patterns)
                z_avg = qml.numpy.sum(features[:4]) / qml.numpy.array(4)  # Average of individual qubit measurements
                magnitude_pred = qml.numpy.abs(z_avg) * qml.numpy.array(0.02)  # Scale to typical G-K range (2% daily volatility)
                
                # 2. Sign prediction from phase-sensitive measurements
                # Combine Z measurements (after H-RZ-H) with Y⊗Y correlation
                #phase_indicator = features[7]  # Y⊗Y measurement
                # Use safe indexing that works with autograd
                phase_indicator = features[7] if len(features) > 7 else qml.numpy.array(0.0)

                z_asymmetry = features[0] - features[3]  # First vs last day
                sign_pred = qml.numpy.tanh(z_avg + qml.numpy.array(0.5) * phase_indicator + qml.numpy.array(0.3) * z_asymmetry)
                
                # 3. Volatility persistence from correlations
                # (Corsi, 2009: HAR framework)
                #persistence = (features[4] + features[5] + features[6]) / 3  # Z⊗Z correlations
                # Use safe indexing that works with autograd
                persistence = qml.numpy.sum(features[4:7]) / qml.numpy.array(3) if len(features) > 6 else qml.numpy.array(0.0)
                
                # Loss Components:
                
                # A. QLIKE Loss for magnitude (Patton, 2011)
                # Robust to outliers, scale-invariant
                magnitude_pred_safe = qml.numpy.maximum(magnitude_pred, epsilon)
                target_magnitude_safe = qml.numpy.maximum(target_magnitude, epsilon)
                
                qlike_loss = (target_magnitude_safe / magnitude_pred_safe - 
                              qml.math.log(target_magnitude_safe / magnitude_pred_safe) - qml.numpy.array(1.0))
                
                # B. Directional Loss with margin
                # (Christoffersen & Diebold, 2006)
                if target_sign != 0:
                    direction_loss = qml.numpy.maximum(qml.numpy.array(0.0), qml.numpy.array(1.0) - target_sign * sign_pred)  # Hinge loss
                else:
                    direction_loss = qml.numpy.abs(sign_pred) * qml.numpy.array(0.1)  # Penalize strong predictions when G-K ≈ 0
                
                # C. Signed prediction loss (main objective)
                #predicted_sgk = magnitude_pred * (1 if sign_pred > 0 else -1 if sign_pred < 0 else 0)
                predicted_sgk = magnitude_pred * qml.numpy.sign(sign_pred)
                signed_loss = (predicted_sgk - target_sgk) ** qml.numpy.array(2)
                
                # D. Temporal consistency loss
                # (Andersen et al., 2003: integrated variance consistency)
                window_gk = []
                for d in range(4):
                    prev_close = None
                    if d > 0:
                        prev_close = ohlc_batch[i][d-1][3]
                    
                    gk = self.calculate_signed_garman_klass(
                        ohlc_batch[i][d][0], ohlc_batch[i][d][1], 
                        ohlc_batch[i][d][2], ohlc_batch[i][d][3], 
                        prev_close
                    )
                    window_gk.append(gk)
                    
                window_gk = qml.numpy.array(window_gk)

                #actual_persistence = np.corrcoef(window_gk[:-1], window_gk[1:])[0, 1]
                # Use manual correlation calculation for autograd compatibility
                if len(window_gk) > 1:
                    x = qml.numpy.array(window_gk[:-1])
                    y = qml.numpy.array(window_gk[1:])
                    mx = qml.numpy.sum(x) / len(x)
                    my = qml.numpy.sum(y) / len(y)
                    cov = qml.numpy.sum([(xi - mx) * (yi - my) for xi, yi in zip(x, y)]) / len(x)
                    sx = qml.math.sqrt(qml.numpy.sum([(xi - mx)**2 for xi in x]) / len(x))
                    sy = qml.math.sqrt(qml.numpy.sum([(yi - my)**2 for yi in y]) / len(y))
                    actual_persistence = cov / (sx * sy + epsilon) if sx > 0 and sy > 0 else qml.numpy.array(0.0)
                else:
                    actual_persistence = qml.numpy.array(0.0)
                
                if not qml.numpy.isnan(actual_persistence):
                    persistence_loss = (persistence - actual_persistence) ** 2
                else:
                    persistence_loss = qml.numpy.array(0.0)
                
                # E. Leverage effect penalty
                # (Nelson, 1991: negative returns → higher future volatility)
                if window_gk[0] < 0 and target_magnitude > qml.numpy.abs(window_gk[0]):
                    leverage_bonus = qml.numpy.array(-0.1)  # Reward if model captures leverage effect
                else:
                    leverage_bonus = qml.numpy.array(0.0)
                
                # Weighted combination with scientific justification:
                # - 30% QLIKE: robust magnitude prediction (Patton, 2011)
                # - 25% direction: critical for options (Christoffersen & Diebold, 2006)
                # - 30% signed: main objective
                # - 15% persistence: temporal structure (Corsi, 2009)
                sample_loss = (qml.numpy.array(0.30) * qlike_loss + 
                              qml.numpy.array(0.25) * direction_loss + 
                              #qml.numpy.array(0.30) * signed_loss + 
                              qml.numpy.array(0.15) * persistence_loss +
                              leverage_bonus)
         
                total_loss += sample_loss
    
            except Exception as e:
                print(f"ERROR in cost function at sample {i}: {e}")
                import traceback
                traceback.print_exc()
                raise
        
        # Regularization based on quantum circuit complexity
        # (McClean et al., 2018: "Barren plateaus in quantum neural network training")
        l2_reg = qml.numpy.array(0.01) * qml.numpy.sum(params ** 2)
        
        total_cost = (total_loss / n_samples) + l2_reg
        
        return total_cost
    """
    def _volatility_aware_cost(self, params, ohlc_batch, targets):
        """
        Cost function for signed volatility prediction combining:
        - QLIKE for magnitude prediction (Patton, 2011)
        - BCE for sign prediction
        - Pearson correlation for magnitude and sign persistence
        - Leverage effect for asymmetric volatility response
        """
        n_samples = len(ohlc_batch)
        total_loss = qml.numpy.array(0.0)
        epsilon = qml.numpy.array(1e-8)
        
        for i in range(n_samples):
            try:
                single_window = ohlc_batch[i]
                if single_window.shape != (4, 4):
                    print(f"ERROR: Expected (4,4) window, got {single_window.shape}")
                
                # Get quantum measurements
                features = self.circuit(single_window, params)
                features = qml.numpy.array(features)
                
                if len(features) != self.N_QUANTUM_FEATURES:
                    print(f"ERROR: Expected {self.N_QUANTUM_FEATURES} measurements, got {len(features)}")
                
                # Target signed G-K value
                target_sgk = targets[i]
                target_magnitude = qml.numpy.abs(target_sgk)
                target_sign = qml.numpy.sign(target_sgk)
                
                # Extract predictions from quantum features
                z_avg = qml.numpy.sum(features[:4]) / 4
                #magnitude_pred = qml.numpy.abs(z_avg) * 0.02
                magnitude_pred = qml.numpy.abs(z_avg) * 0.01
                
                phase_indicator = features[7]
                z_asymmetry = features[0] - features[3]
                sign_logit = 5.0 * (z_avg + 0.5 * phase_indicator + 0.3 * z_asymmetry)   #Amplify logit multiplication factor (1.0 -> 5.0) for clearer distinctions
                
                # Get persistence features
                #persistence_features = features[4:7]  # Z⊗Z correlations
                
                # Loss Components:
                
                # 1. QLIKE Loss for magnitude
                magnitude_pred_safe = qml.numpy.maximum(magnitude_pred, epsilon)
                target_magnitude_safe = qml.numpy.maximum(target_magnitude, epsilon)
                qlike_loss = (target_magnitude_safe / magnitude_pred_safe - 
                              qml.math.log(target_magnitude_safe / magnitude_pred_safe) - 1)
                
                # 2. Binary Cross-Entropy for sign prediction
                # Convert sign from {-1, 0, 1} to {0, 1} for BCE
                target_binary = qml.numpy.array(0.5) * (target_sign + qml.numpy.array(1.0))
                sign_prob = qml.numpy.array(1.0) / (qml.numpy.array(1.0) + qml.numpy.exp(-sign_logit))
                
                bce_loss = -(target_binary * qml.numpy.log(sign_prob + epsilon) + 
                            (qml.numpy.array(1.0) - target_binary) * qml.numpy.log(qml.numpy.array(1.0) - sign_prob + epsilon))
                
                # 3. Persistence Loss - separate magnitude and sign
                """
                window_gk = qml.numpy.array([
                    self.calculate_signed_garman_klass(
                        ohlc_batch[i][d][0], ohlc_batch[i][d][1], 
                        ohlc_batch[i][d][2], ohlc_batch[i][d][3]) 
                    for d in range(4)
                ])
                
                # 3a. Magnitude persistence
                window_magnitudes = qml.numpy.abs(window_gk)
                # Calculate actual magnitude correlations matching quantum measurements
                mag_corr_actual = qml.numpy.array([
                    self._compute_correlation(window_magnitudes[0], window_magnitudes[3]),  # Long-range
                    self._compute_correlation(window_magnitudes[1], window_magnitudes[2]),  # Mid-range
                    self._compute_correlation(window_magnitudes[0], window_magnitudes[1])   # Adjacent
                ])
                
                # Correlation loss for magnitude persistence
                mag_persistence_loss = qml.numpy.mean((persistence_features - mag_corr_actual) ** 2)
                
                # 3b. Sign persistence
                window_signs = qml.numpy.sign(window_gk)
                sign_corr_actual = qml.numpy.array([
                    self._compute_correlation(window_signs[0], window_signs[3]),  # Long-range
                    self._compute_correlation(window_signs[1], window_signs[2]),  # Mid-range
                    self._compute_correlation(window_signs[0], window_signs[1])   # Adjacent
                ])
                
                # Correlation loss for sign persistence
                sign_persistence_loss = qml.numpy.mean((persistence_features - sign_corr_actual) ** 2)
                """
                
                # Check if previous day had negative return and volatility increased
                leverage_loss = qml.numpy.array(0.0)
                if i > 0:  # Need previous day
                    prev_window = ohlc_batch[i-1]
                    prev_return = (prev_window[-1, 3] - prev_window[-1, 0]) / prev_window[-1, 0]  # Last day's return
                    
                    # Calculate previous day's magnitude directly
                    prev_day_gk = self.calculate_signed_garman_klass(
                        prev_window[-1, 0], prev_window[-1, 1], 
                        prev_window[-1, 2], prev_window[-1, 3]
                    )
                    prev_magnitude = qml.numpy.abs(prev_day_gk)
                    
                    if prev_return < 0 and target_magnitude > prev_magnitude:
                        # Negative return followed by volatility increase
                        # Reward if model predicts higher volatility
                        if magnitude_pred > prev_magnitude:
                            leverage_loss = qml.numpy.array(-0.1)  # Reward
                        else:
                            leverage_loss = qml.numpy.array(0.1)   # Penalty
                
                # Weighted combination
                """
                sample_loss = (0.40 * qlike_loss + 
                              0.25 * bce_loss + 
                              0.15 * mag_persistence_loss +
                              0.15 * sign_persistence_loss +
                              0.05 * leverage_loss)
                """
                # Weighted combination
                sample_loss = (0.50 * qlike_loss + 
                              0.40 * bce_loss + 
                              0.10 * leverage_loss)
                
                total_loss = total_loss + sample_loss
                
            except Exception as e:
                print(f"ERROR in cost function at sample {i}: {e}")
                import traceback
                traceback.print_exc()
                raise
        
        # L2 Regularization
        l2_reg = 0.01 * qml.numpy.sum(params ** 2)
        
        total_cost = (total_loss / n_samples) + l2_reg
        
        return total_cost

    """
    def _compute_correlation(self, x, y):
        #Helper function to compute correlation between two scalars.
        #For single values, correlation is 1 if same sign, -1 if opposite, 0 if either is 0.
        if x == 0 or y == 0:
            return qml.numpy.array(0.0)
        elif qml.numpy.sign(x) == qml.numpy.sign(y):
            return qml.numpy.array(1.0)
        else:
            return qml.numpy.array(-1.0)
    """

    def _local_cost_function(self, params, batch_samples, targets):
        """Alias for compatibility with gradient calculation."""
        return self._volatility_aware_cost(params, batch_samples, targets)

    """
    def _volatility_aware_cost(self, params, batch_samples, targets):
        #
        #Cost function specifically designed for volatility detection.
        #
        #Incorporates volatility-specific loss components based on
        #Andersen et al. (2003) and Hansen & Lunde (2006).
        #
        n_samples = len(batch_samples)
        total_loss = 0
        
        for i in range(n_samples):
            # Get quantum features
            features = self.circuit(batch_samples[i], params)
            
            # Convert to numpy array if needed
            if hasattr(features, 'numpy'):
                features = features.numpy()
            features = np.array(features)
            
            # Volatility-specific loss components
            
            # 1. Realized volatility prediction loss
            rv_prediction = features[0]  # First measurement
            rv_loss = (rv_prediction - targets[i]) ** 2
            
            # 2. Volatility persistence loss (ARCH/GARCH-inspired)
            persistence = features[3]  # Correlation measurement
            persistence_target = np.corrcoef(batch_samples[i][:-1], batch_samples[i][1:])[0,1]
            persistence_loss = (persistence - persistence_target) ** 2
            
            # 3. Jump detection loss
            jump_indicator = features[1]  # Jump component
            actual_jumps = np.sum(np.abs(batch_samples[i]) > 3 * np.std(batch_samples[i]))
            jump_loss = (jump_indicator - actual_jumps / len(batch_samples[i])) ** 2
            
            # Weighted combination
            sample_loss = 0.5 * rv_loss + 0.3 * persistence_loss + 0.2 * jump_loss
            total_loss += sample_loss
        
        return total_loss / n_samples
    """

    def transform(self, ohlc_data):
        """
        Transform OHLC data to quantum volatility features.
        
        Parameters:
        -----------
        ohlc_data : pd.DataFrame or np.ndarray
            Either raw OHLC DataFrame or OHLC windows array
        """
        if isinstance(ohlc_data, pd.DataFrame):
            # Create windows from DataFrame
            ohlc_windows = self.create_ohlc_windows(ohlc_data)
        else:
            ohlc_windows = ohlc_data
        
        # Transform each window
        quantum_features = []
        for window in ohlc_windows:
            features = self._transform_single_ohlc(window)
            quantum_features.append(features)
        
        return np.array(quantum_features)
    
    def _transform_single_ohlc(self, ohlc_window):
        """Transform single OHLC window to quantum features."""
        # Run circuit with OHLC window
        circuit_output = self.circuit(ohlc_window, self.params)
        
        if hasattr(circuit_output, 'numpy'):
            circuit_output = circuit_output.numpy()
        
        return np.array(circuit_output)[:self.N_QUANTUM_FEATURES]
    """
    def transform(self, returns_series):
        #
        #Transform returns to quantum volatility features with proper handling.
        #
        if not self.is_fitted:
            raise RuntimeError("Detector must be fitted before transform")
        
        # Convert input
        if isinstance(returns_series, (pd.DataFrame, pd.Series)):
            returns_series = returns_series.values
        
        returns_series = np.asarray(returns_series, dtype=np.float64)
        
        # Determine if batch or single sample
        if returns_series.ndim == 1:
            # Single time series
            return self._transform_single(returns_series)
        elif returns_series.ndim == 2:
            # Batch processing
            return self._transform_batch(returns_series)
        else:
            raise ValueError(f"Unexpected input dimensions: {returns_series.ndim}")
    """

    def _transform_single(self, single_sample):
        """Transform single OHLC sample using enhanced encoding."""
        processed_data, data_type = self._prepare_circuit_input(single_sample)
        
        # Use the main circuit (computational basis measurements)
        circuit_output = self.circuit(processed_data, self.params)
        
        if hasattr(circuit_output, 'numpy'):
            circuit_output = circuit_output.numpy()
        
        output_array = np.array(circuit_output, dtype=np.float64)
        
        # Ensure we have exactly 9 measurements for compatibility
        if len(output_array) < self.N_QUANTUM_FEATURES:
            padded = np.zeros(self.N_QUANTUM_FEATURES, dtype=np.float64)
            padded[:len(output_array)] = output_array
            output_array = padded
        
        return output_array[:self.N_QUANTUM_FEATURES]

    """
    def _transform_single(self, single_sample):
        #Transform single sample with proper shape handling.
        
        # Process through quantum circuit
        circuit_output = self.circuit(single_sample, self.params)
        
        # Convert to numpy array
        if hasattr(circuit_output, 'numpy'):
            circuit_output = circuit_output.numpy()
        
        output_array = np.array(circuit_output, dtype=np.float64)
        
        # Ensure we have exactly 4 measurements
        if len(output_array) < 4:
            padded = np.zeros(4, dtype=np.float64)
            padded[:len(output_array)] = output_array
            output_array = padded
        
        return output_array[:4]
    """

    def _transform_batch(self, batch_data):
        """Transform batch of samples with parallel processing."""
        n_samples = batch_data.shape[0]
        #quantum_features = np.zeros((n_samples, 4), dtype=np.float64)
        quantum_features = np.zeros((n_samples, self.N_QUANTUM_FEATURES), dtype=np.float64)
        
        for i in range(n_samples):
            try:
                features = self._transform_single(batch_data[i])
                quantum_features[i] = features
            except Exception as e:
                print(f"Error processing sample {i}: {e}")
                #quantum_features[i] = np.zeros(4)
                quantum_features[i] = np.zeros(self.N_QUANTUM_FEATURES)
        
        return quantum_features

    """
    def update(self, returns_window, actual_return):
        #Online update with volatility-specific learning.
        if not self.is_fitted:
            raise RuntimeError("Detector must be fitted before updating.")
        
        # Convert to numpy
        if isinstance(returns_window, (pd.DataFrame, pd.Series)):
            returns_window = returns_window.values
        
        # Single optimization step
        optimizer = qml.AdamOptimizer(stepsize=0.01)
        active_params = self.params[self.active_layers]
        
        def update_cost(params):
            features = self.circuit(returns_window, params)
            if hasattr(features, 'numpy'):
                features = features.numpy()
            # Focus on volatility prediction error
            volatility_pred = features[0]
            actual_volatility = np.abs(actual_return)
            return (volatility_pred - actual_volatility) ** 2
        
        # Update parameters
        updated_params = optimizer.step(update_cost, active_params)
        
        # Copy back
        for i, layer_idx in enumerate(self.active_layers):
            self.params[layer_idx] = updated_params[i]
        
        return self
    """
    """
    def update(self, ohlc_window, actual_gk):
        #
        #Online update with signed Garman-Klass learning.
        #
        #Parameters:
        #-----------
        #ohlc_window : array-like, shape (4, 4)
        #    4-day OHLC window
        #actual_gk : float
        #    Actual next-day signed Garman-Klass value
        #
        if not self.is_fitted:
            raise RuntimeError("Detector must be fitted before updating.")
        
        # Single optimization step
        optimizer = qml.AdamOptimizer(stepsize=0.01)
        active_params = self.params[self.active_layers]
        
        def update_cost(params):
            # Use the SAME cost function as training!
            # Create single-sample batch
            batch_windows = ohlc_window.reshape(1, 4, 4)
            batch_targets = np.array([actual_gk])
            
            # Call the main cost function
            return self._volatility_aware_cost(params, batch_windows, batch_targets)
        
        # Calculate gradients
        try:
            grads = qml.grad(update_cost)(active_params)
            grad_norm = np.linalg.norm(grads)
            
            # Only update if gradients are non-zero
            if grad_norm > 1e-10:
                updated_params = optimizer.step(update_cost, active_params)
                
                # Copy back
                for i, layer_idx in enumerate(self.active_layers):
                    self.params[layer_idx] = updated_params[i]
            else:
                print(f"WARNING: Zero gradients in update! Grad norm: {grad_norm}")
                
        except Exception as e:
            print(f"Update failed: {e}")
            # Fallback to manual gradient calculation if needed
            
        return self
    """
    def update(self, ohlc_window, actual_gk):
        """
        Online update with signed Garman-Klass learning.
        
        Parameters:
        -----------
        ohlc_window : array-like, shape (4, 4)
            4-day OHLC window
        actual_gk : float
            Actual next-day signed Garman-Klass value
        """
        if not self.is_fitted:
            raise RuntimeError("Detector must be fitted before updating.")
        
        # Get active parameters
        active_params = self.params[self.active_layers]
        
        # Create single-sample batch
        batch_windows = ohlc_window.reshape(1, 4, 4)
        batch_targets = np.array([actual_gk])
        
        # Calculate gradients using the SAME method as training
        try:
            # Use _calculate_gradients instead of raw qml.grad
            gradients, grad_norm = self._calculate_gradients(
                active_params, batch_windows, batch_targets
            )
            
            # Only update if gradients are non-zero
            if grad_norm > 1e-10:
                # Manually apply the gradient update (same as in fit method)
                if not hasattr(self, '_adam_m'):
                    self._adam_m = np.zeros_like(active_params)
                    self._adam_v = np.zeros_like(active_params)
                    self._adam_t = 0
                
                self._adam_t += 1
                
                # Adam update rule
                beta1, beta2 = 0.9, 0.999
                self._adam_m = beta1 * self._adam_m + (1 - beta1) * gradients
                self._adam_v = beta2 * self._adam_v + (1 - beta2) * gradients**2
                
                m_hat = self._adam_m / (1 - beta1**self._adam_t)
                v_hat = self._adam_v / (1 - beta2**self._adam_t)
                
                # Use learning rate of 0.01 (same as training)
                active_params = active_params - 0.01 * m_hat / (np.sqrt(v_hat) + 1e-8)
                
                # Copy back to main parameters
                for i, layer_idx in enumerate(self.active_layers):
                    self.params[layer_idx] = active_params[i]
            else:
                print(f"WARNING: Zero gradients in update! Grad norm: {grad_norm}")
                
        except Exception as e:
            print(f"Update failed: {e}")
            import traceback
            traceback.print_exc()
            
        return self

    """
    def get_feature_names(self):
        #Return meaningful names for the quantum features.
        return [
            f"quantum_{self.measurement_meanings[i]}" 
            for i in range(4)
        ]
    """

def diagnose_dimension_mismatch(quantum_detector, ohlc_window):
    """Diagnose where the dimension mismatch is occurring."""
    print("\n=== DIMENSION MISMATCH DIAGNOSTICS ===")
    
    # Test 1: Circuit output
    print("\n1. Testing circuit output:")
    try:
        output = quantum_detector.circuit(ohlc_window, quantum_detector.params)
        print(f"   Circuit output: type={type(output)}, len={len(output) if hasattr(output, '__len__') else 'N/A'}")
        if hasattr(output, 'numpy'):
            output = output.numpy()
        output_array = np.array(output)
        print(f"   As array: shape={output_array.shape}, dtype={output_array.dtype}")
        print(f"   Values: {output_array}")
    except Exception as e:
        print(f"   ERROR in circuit: {e}")
        import traceback
        traceback.print_exc()
    
    # Test 2: Transform method
    print("\n2. Testing transform method:")
    try:
        features = quantum_detector.transform(ohlc_window.reshape(1, 4, 4))
        print(f"   Transform output: shape={features.shape}, dtype={features.dtype}")
        print(f"   First sample: {features[0]}")
    except Exception as e:
        print(f"   ERROR in transform: {e}")
        import traceback
        traceback.print_exc()
    
    # Test 3: Cost function
    print("\n3. Testing cost function:")
    try:
        batch = ohlc_window.reshape(1, 4, 4)
        targets = np.array([0.001])  # Dummy target
        cost = quantum_detector._volatility_aware_cost(
            quantum_detector.params[quantum_detector.active_layers],
            batch,
            targets
        )
        print(f"   Cost function returned: {cost}")
    except Exception as e:
        print(f"   ERROR in cost function: {e}")
        import traceback
        traceback.print_exc()
    
    print("\n=== END DIAGNOSTICS ===")

class VolatilityEnhancedDataPreprocessor(DataPreprocessor):
    """
    Extension of DataPreprocessor with properly integrated quantum volatility detection.
    
    Fixes:
    ------
    1. Proper type handling for feature updates
    2. Correct window alignment without off-by-one errors
    3. Scientifically grounded temporal data handling
    """
    
    def __init__(self, *args, **kwargs):
        """Initialize with additional parameters for quantum volatility."""
        # Get quantum-specific parameters
        self.use_quantum_volatility = kwargs.pop('use_quantum_volatility', False)
        self.quantum_n_qubits = kwargs.pop('quantum_n_qubits', 4)
        self.random_state = kwargs.pop('random_state', 42)
        
        # Initialize parent class
        super().__init__(*args, **kwargs)
        
        # Initialize quantum volatility detector
        self.quantum_volatility = None
        self.days_since_volatility_update = 0

        self.continuous_learning_stats = {
            'update_count': 0,
            'prediction_errors': [],
            'gradient_norms': [],
            'prediction_correlations': [],
            'losses_before': [],
            'losses_after': [],
            'predictions': [],
            'actuals': [] 
        }

    def get_raw_ohlc_data(self):
        """
        Get raw OHLC data for quantum processing.
        Returns the underlying OHLC DataFrame.
        """
        return super().get_raw_ohlc_data()

    def compute_quantum_features_for_date(self, date, ohlc_df=None):
        """Compute quantum features on-demand for a specific date."""
        if ohlc_df is None:
            ohlc_df = self.get_raw_ohlc_data()
            
        current_idx = ohlc_df.index.get_loc(date)
        
        if current_idx > 0:
            prev_idx = current_idx - 1
            
            if prev_idx >= self.quantum_volatility.lookback_window:
                lookback_start = prev_idx - self.quantum_volatility.lookback_window + 1
                lookback_end = prev_idx + 1
                ohlc_window = ohlc_df.iloc[lookback_start:lookback_end][['Open', 'High', 'Low', 'Close']].values
                
                if ohlc_window.shape == (4, 4):
                    features = self.quantum_volatility.transform(ohlc_window.reshape(1, 4, 4))[0]
                    return dict(zip(self.quantum_volatility.get_feature_names(), features))
        
        return {name: np.nan for name in self.quantum_volatility.get_feature_names()}
    
    def get_daily_prediction_data(self, tb_writer=None):
        """
        Apply quantum volatility detection with proper type handling and alignment.
        
        Fixes:
        ------
        1. Ensures scalar values for feature dictionary updates
        2. Handles window alignment correctly
        3. Maintains temporal integrity without look-ahead bias
        """
    
        # Get original daily prediction data
        prediction_data = super().get_daily_prediction_data()
    
        if not self.use_quantum_volatility:
            return prediction_data

        if self.quantum_volatility is None:
            print("WARNING: Quantum volatility detector not initialized")
            return prediction_data
        
        sorted_dates = sorted(prediction_data.keys())

        """
        # Process each day with proper error handling
        for i, date in enumerate(sorted_dates):
            returns_column = self.get_target_column()
            current_idx = returns_column.index.get_loc(date)
            
            # Extract lookback window with correct alignment
            if current_idx >= self.quantum_volatility.lookback_window:
                # Correct window extraction (no off-by-one error)
                lookback_start = current_idx - self.quantum_volatility.lookback_window
                lookback_end = current_idx  # Exclusive end index
                lookback_data = returns_column.iloc[lookback_start:lookback_end]
                
                try:
                    # Transform data - ensure it's reshaped correctly
                    if len(lookback_data) == self.quantum_volatility.lookback_window:
                        volatility_features = self.quantum_volatility.transform(
                            lookback_data.values.reshape(1, -1)
                        )[0]
                    else:
                        # Handle edge case with padding
                        padded_data = np.pad(
                            lookback_data.values,
                            (0, self.quantum_volatility.lookback_window - len(lookback_data)),
                            mode='constant',
                            constant_values=0
                        )
                        volatility_features = self.quantum_volatility.transform(
                            padded_data.reshape(1, -1)
                        )[0]
                    
                    # Create feature dictionary with proper scalar conversion
                    volatility_dict = {}
                    feature_names = self.quantum_volatility.get_feature_names()
                    
                    for j, (name, val) in enumerate(zip(feature_names, volatility_features)):
                        # Ensure scalar value for dictionary update
                        if np.isscalar(val):
                            scalar_val = float(val)
                        else:
                            scalar_val = float(val.item()) if hasattr(val, 'item') else float(val)
                        
                        volatility_dict[name] = scalar_val
                    
                    # Update features dictionary
                    prediction_data[date]['features'].update(volatility_dict)
                    
                    # Online learning update (if past the warmup period)
                    if i > 30 and i > 0:  # After 30-day warmup
                        previous_date = sorted_dates[i - 1]
                        if previous_date in prediction_data:
                            actual_return = prediction_data[previous_date].get('actual_return')
                            if actual_return is not None:
                                # Update quantum circuit with previous day's actual return
                                prev_idx = returns_column.index.get_loc(previous_date)
                                prev_window_start = max(0, prev_idx - self.quantum_volatility.lookback_window)
                                prev_window = returns_column.iloc[prev_window_start:prev_idx]
                                
                                if len(prev_window) == self.quantum_volatility.lookback_window:
                                    self.quantum_volatility.update(prev_window.values, actual_return)
                    
                except Exception as e:
                    print(f"Error processing quantum features for {date}: {e}")
                    # Fallback to NaN features
                    volatility_dict = {
                        name: np.nan 
                        for name in self.quantum_volatility.get_feature_names()
                    }
                    prediction_data[date]['features'].update(volatility_dict)
            else:
                # Not enough history - use NaN
                volatility_dict = {
                    name: np.nan 
                    for name in self.quantum_volatility.get_feature_names()
                }
                prediction_data[date]['features'].update(volatility_dict)
        
        return prediction_data
        """
        # Get OHLC data
        ohlc_df = self.get_raw_ohlc_data()

        """      
        # Process each day with proper error handling
        for i, date in enumerate(sorted_dates):
            try:
                current_idx = ohlc_df.index.get_loc(date)
                
                # Extract OHLC lookback window
                if current_idx >= self.quantum_volatility.lookback_window:
                    # Get 4-day OHLC window ending at current date
                    lookback_start = current_idx - self.quantum_volatility.lookback_window + 1
                    lookback_end = current_idx + 1
                    ohlc_window = ohlc_df.iloc[lookback_start:lookback_end][['Open', 'High', 'Low', 'Close']].values    
                    
                    if ohlc_window.shape == (4, 4):  # Ensure correct shape
                        # Transform OHLC window to quantum features
                        volatility_features = self.quantum_volatility.transform(
                            ohlc_window.reshape(1, 4, 4)
                        )[0]
                        
                        # Create feature dictionary with proper scalar conversion
                        volatility_dict = {}
                        feature_names = self.quantum_volatility.get_feature_names()
                        
                        for j, (name, val) in enumerate(zip(feature_names, volatility_features)):
                            # Ensure scalar value for dictionary update
                            if np.isscalar(val):
                                scalar_val = float(val)
                            else:
                                scalar_val = float(val.item()) if hasattr(val, 'item') else float(val)
                            
                            volatility_dict[name] = scalar_val
                        
                        # Update features dictionary
                        prediction_data[date]['features'].update(volatility_dict)
                        
                        # Online learning update
                        #if i > 30 and i > 0:  # After 30-day warmup
                        # Update quantum circuit with yesterday's actual outcome
                        if i > 0:  # Start updating from day 1
                            previous_date = sorted_dates[i - 1]
                            if previous_date in prediction_data:
                                # For OHLC, we need to get the actual next-day G-K
                                prev_idx = ohlc_df.index.get_loc(previous_date)
                                
                                # Get previous OHLC window
                                if prev_idx >= self.quantum_volatility.lookback_window:
                                    prev_window_start = prev_idx - self.quantum_volatility.lookback_window + 1
                                    prev_window_end = prev_idx + 1
                                    prev_ohlc_window = ohlc_df.iloc[prev_window_start:prev_window_end][['Open', 'High', 'Low', 'Close']].values
                                    
                                    # Calculate actual G-K for current day (which was "next day" for previous window)
                                    current_ohlc = ohlc_df.loc[date]
                                    actual_gk = self.quantum_volatility.calculate_signed_garman_klass(
                                        current_ohlc['Open'],
                                        current_ohlc['High'],
                                        current_ohlc['Low'],
                                        current_ohlc['Close']
                                    )
                                    
                                    if prev_ohlc_window.shape == (4, 4):
                                        
                                        # Make prediction BEFORE update for diagnostics
                                        predicted_gk = self.quantum_volatility.predict_volatility(prev_ohlc_window)
                                        prediction_error = predicted_gk - actual_gk
                                        
                                        # Calculate loss BEFORE update
                                        active_params = self.quantum_volatility.params[self.quantum_volatility.active_layers]
                                        loss_before = self.quantum_volatility._volatility_aware_cost(
                                            active_params,
                                            prev_ohlc_window.reshape(1, 4, 4),
                                            np.array([actual_gk])
                                        )
                                        
                                        # Store parameters before update
                                        params_before = self.quantum_volatility.params.copy()
                                        
                                        # Perform update
                                        self.quantum_volatility.update(prev_ohlc_window, actual_gk)
                                        
                                        # Calculate loss AFTER update
                                        active_params_after = self.quantum_volatility.params[self.quantum_volatility.active_layers]
                                        loss_after = self.quantum_volatility._volatility_aware_cost(
                                            active_params_after,
                                            prev_ohlc_window.reshape(1, 4, 4),
                                            np.array([actual_gk])
                                        )
                                        
                                        # Calculate gradient norm (parameter change)
                                        param_change = np.linalg.norm(self.quantum_volatility.params - params_before)
                                        
                                        # Track statistics
                                        self.continuous_learning_stats['update_count'] += 1
                                        self.continuous_learning_stats['prediction_errors'].append(prediction_error)
                                        self.continuous_learning_stats['gradient_norms'].append(param_change)
                                        self.continuous_learning_stats['losses_before'].append(float(loss_before))
                                        self.continuous_learning_stats['losses_after'].append(float(loss_after))
                                        self.continuous_learning_stats['predictions'].append(predicted_gk)
                                        self.continuous_learning_stats['actuals'].append(actual_gk)
                                        
                                        # Every 10 updates, calculate actual correlation
                                        if self.continuous_learning_stats['update_count'] % 10 == 0:
                                            # Calculate correlation on last 50 predictions
                                            lookback = min(50, len(self.continuous_learning_stats['prediction_errors']))
                                            if lookback > 1:
                                                recent_predictions = []
                                                recent_actuals = []
                                                
                                                # Go back through recent history
                                                for j in range(lookback):
                                                    hist_idx = i - lookback + j
                                                    if hist_idx >= 0 and hist_idx < len(sorted_dates):
                                                        hist_date = sorted_dates[hist_idx]
                                                        hist_ohlc_idx = ohlc_df.index.get_loc(hist_date)
                                                        
                                                        if hist_ohlc_idx >= self.quantum_volatility.lookback_window:
                                                            # Get historical window
                                                            hist_start = hist_ohlc_idx - self.quantum_volatility.lookback_window + 1
                                                            hist_end = hist_ohlc_idx + 1
                                                            hist_window = ohlc_df.iloc[hist_start:hist_end][['Open', 'High', 'Low', 'Close']].values
                                                            
                                                            if hist_window.shape == (4, 4):
                                                                # Predict
                                                                pred = self.quantum_volatility.predict_volatility(hist_window)
                                                                # Get actual (next day's G-K)
                                                                if hist_idx + 1 < len(sorted_dates):
                                                                    next_date = sorted_dates[hist_idx + 1]
                                                                    next_ohlc = ohlc_df.loc[next_date]
                                                                    actual = self.quantum_volatility.calculate_signed_garman_klass(
                                                                        next_ohlc['Open'], next_ohlc['High'], 
                                                                        next_ohlc['Low'], next_ohlc['Close']
                                                                    )
                                                                    recent_predictions.append(pred)
                                                                    recent_actuals.append(actual)
                                                
                                                if len(recent_predictions) > 1:
                                                    corr = np.corrcoef(recent_predictions, recent_actuals)[0, 1]
                                                    
                                                    # Additional correlations
                                                    mag_corr = np.corrcoef(np.abs(recent_predictions), np.abs(recent_actuals))[0, 1]
                                                    sign_acc = np.mean(np.sign(recent_predictions) == np.sign(recent_actuals))
                                                    
                                                    print(f"Correlations - Signed: {corr:.3f}, Magnitude: {mag_corr:.3f}, Sign Acc: {sign_acc:.1%}")
                                                    self.continuous_learning_stats['prediction_correlations'].append((date, corr))
                                        
                                        # TensorBoard logging for continuous learning
                                        if tb_writer is not None and self.continuous_learning_stats['update_count'] % 10 == 0:
                                            # Get recent data
                                            recent_errors = self.continuous_learning_stats['prediction_errors'][-50:]
                                            recent_grads = self.continuous_learning_stats['gradient_norms'][-50:]
                                            recent_losses_before = self.continuous_learning_stats['losses_before'][-50:]
                                            recent_losses_after = self.continuous_learning_stats['losses_after'][-50:]
                                            recent_predictions = self.continuous_learning_stats['predictions'][-50:]
                                            recent_actuals = self.continuous_learning_stats['actuals'][-50:]
                                            
                                            # Calculate correlations from recent data
                                            if len(recent_predictions) > 1 and len(recent_actuals) > 1:
                                                # Signed GK correlation (Pearson correlation of signed values)
                                                signed_corr = np.corrcoef(recent_predictions, recent_actuals)[0, 1]
                                                
                                                # Magnitude correlation (Pearson correlation of absolute values)
                                                magnitude_corr = np.corrcoef(np.abs(recent_predictions), np.abs(recent_actuals))[0, 1]
                                                
                                                # Sign accuracy (fraction of correct directional predictions)
                                                signs_pred = np.sign(recent_predictions)
                                                signs_actual = np.sign(recent_actuals)
                                                sign_accuracy = np.mean(signs_pred == signs_actual)
                                            else:
                                                signed_corr = 0.0
                                                magnitude_corr = 0.0
                                                sign_accuracy = 0.5
                                            
                                            # Current loss (use most recent)
                                            current_loss = recent_losses_after[-1] if recent_losses_after else 0.0
                                            
                                            step = self.continuous_learning_stats['update_count']
                                            
                                            # Log all metrics
                                            with tb_writer.as_default():
                                                tf.summary.scalar('QuantumCircuit/PredictionPhase/Loss', 
                                                                 current_loss, step=step)
                                                tf.summary.scalar('QuantumCircuit/PredictionPhase/GradientNorm', 
                                                                 np.mean(recent_grads), step=step)
                                                tf.summary.scalar('QuantumCircuit/PredictionPhase/SignedGKCorrelation_Pearson', 
                                                                 signed_corr, step=step)
                                                tf.summary.scalar('QuantumCircuit/PredictionPhase/MagnitudeCorrelation_Pearson', 
                                                                 magnitude_corr, step=step)
                                                tf.summary.scalar('QuantumCircuit/PredictionPhase/SignAccuracy_DirectionalCorrectness', 
                                                                 sign_accuracy, step=step)
                                                
                                                # Also log loss improvement
                                                if len(recent_losses_before) > 0 and len(recent_losses_after) > 0:
                                                    avg_loss_improvement = np.mean(np.array(recent_losses_before) - np.array(recent_losses_after))
                                                    tf.summary.scalar('QuantumCircuit/PredictionPhase/LossImprovement', 
                                                                     avg_loss_improvement, step=step)
                                        
                                        # Print diagnostics every 50 updates
                                        if self.continuous_learning_stats['update_count'] % 50 == 0:
                                            recent_errors = self.continuous_learning_stats['prediction_errors'][-50:]
                                            recent_grads = self.continuous_learning_stats['gradient_norms'][-50:]
                                            recent_losses_before = self.continuous_learning_stats['losses_before'][-50:]
                                            recent_losses_after = self.continuous_learning_stats['losses_after'][-50:]
                                            
                                            print(f"\n{'='*60}")
                                            print(f"Quantum Circuit Continuous Learning Update #{self.continuous_learning_stats['update_count']}")
                                            print(f"Date: {date.strftime('%Y-%m-%d')}")
                                            print(f"{'='*60}")
                                            
                                            # Prediction performance
                                            print("\nPrediction Performance (last 50 updates):")
                                            print(f"Mean Absolute Error: {np.mean(np.abs(recent_errors)):.6f}")
                                            print(f"Mean Squared Error: {np.mean(np.array(recent_errors)**2):.6f}")
                                            print(f"Error Std Dev: {np.std(recent_errors):.6f}")
                                            
                                            # Loss metrics
                                            print(f"\nLoss Metrics:")
                                            print(f"Avg Loss Before Update: {np.mean(recent_losses_before):.6f}")
                                            print(f"Avg Loss After Update: {np.mean(recent_losses_after):.6f}")
                                            print(f"Avg Loss Reduction: {np.mean(np.array(recent_losses_before) - np.array(recent_losses_after)):.6f}")
                                            
                                            # Gradient metrics
                                            print(f"\nGradient Metrics:")
                                            print(f"Mean Gradient Norm: {np.mean(recent_grads):.8f}")
                                            print(f"Gradient Trend: {'decreasing' if np.mean(recent_grads[:25]) > np.mean(recent_grads[25:]) else 'increasing'}")
                                            
                                            # Correlation
                                            if self.continuous_learning_stats['prediction_correlations']:
                                                recent_corr = self.continuous_learning_stats['prediction_correlations'][-1][1]
                                                all_corrs = [c[1] for c in self.continuous_learning_stats['prediction_correlations'] if c[1] is not None]
                                                print(f"\nPrediction Correlation:")
                                                print(f"Current: {recent_corr:.3f}")
                                                print(f"Average: {np.mean(all_corrs):.3f}")
                                            
                                            # Current prediction
                                            print(f"\nCurrent Prediction:")
                                            print(f"Predicted G-K: {predicted_gk:.6f}")
                                            print(f"Actual G-K: {actual_gk:.6f}")
                                            print(f"Error: {prediction_error:.6f}")
                                            print(f"Loss: {loss_before:.6f} → {loss_after:.6f}")
                                            
                    else:
                        # Wrong shape, use NaN
                        volatility_dict = {
                            name: np.nan 
                            for name in self.quantum_volatility.get_feature_names()
                        }
                        prediction_data[date]['features'].update(volatility_dict)
                else:
                    # Not enough history - use NaN
                    volatility_dict = {
                        name: np.nan 
                        for name in self.quantum_volatility.get_feature_names()
                    }
                    prediction_data[date]['features'].update(volatility_dict)
                    
            except Exception as e:
                print(f"Error processing quantum features for {date}: {e}")
                # Fallback to NaN features
                volatility_dict = {
                    name: np.nan 
                    for name in self.quantum_volatility.get_feature_names()
                }
                prediction_data[date]['features'].update(volatility_dict)

        # Final summary of continuous learning
        if hasattr(self, 'continuous_learning_stats') and self.continuous_learning_stats['update_count'] > 0:
            print(f"\nQUANTUM CONTINUOUS LEARNING SUMMARY")
            print(f"Total Updates: {self.continuous_learning_stats['update_count']}")
            print(f"Mean Absolute Error: {np.mean(np.abs(self.continuous_learning_stats['prediction_errors'])):.6f}")
        
        return prediction_data
        """

        for i, date in enumerate(sorted_dates):
            try:
                # CRITICAL FIX: Use previous day to align with classical features
                if i > 0:  # Can't process first day
                    prev_date = sorted_dates[i-1]
                    prev_idx = ohlc_df.index.get_loc(prev_date)
                    
                    # Extract OHLC lookback window ending at PREVIOUS date
                    if prev_idx >= self.quantum_volatility.lookback_window:
                        # Get 4-day OHLC window ending at previous date [T-4, T-3, T-2, T-1]
                        lookback_start = prev_idx - self.quantum_volatility.lookback_window + 1
                        lookback_end = prev_idx + 1
                        ohlc_window = ohlc_df.iloc[lookback_start:lookback_end][['Open', 'High', 'Low', 'Close']].values

                        if ohlc_window.shape == (4, 4):  # Ensure correct shape
                            # Transform OHLC window to quantum features
                            volatility_features = self.quantum_volatility.transform(
                                ohlc_window.reshape(1, 4, 4)
                            )[0]
                            
                            # Create feature dictionary with proper scalar conversion
                            volatility_dict = {}
                            feature_names = self.quantum_volatility.get_feature_names()
                            
                            for j, (name, val) in enumerate(zip(feature_names, volatility_features)):
                                # Ensure scalar value for dictionary update
                                if np.isscalar(val):
                                    scalar_val = float(val)
                                else:
                                    scalar_val = float(val.item()) if hasattr(val, 'item') else float(val)
                                
                                volatility_dict[name] = scalar_val
                            
                            # Update features dictionary
                            prediction_data[date]['features'].update(volatility_dict)

                            """
                            # Online learning update
                            # Update quantum circuit with yesterday's actual outcome
                            if i > 0:  # Start updating from day 1
                                previous_date = sorted_dates[i - 1]
                                if previous_date in prediction_data:
                                    # For OHLC, we need to get the actual next-day G-K
                                    prev_idx = ohlc_df.index.get_loc(previous_date)
                                    
                                    # Get previous OHLC window
                                    if prev_idx >= self.quantum_volatility.lookback_window:
                                        prev_window_start = prev_idx - self.quantum_volatility.lookback_window + 1
                                        prev_window_end = prev_idx + 1
                                        prev_ohlc_window = ohlc_df.iloc[prev_window_start:prev_window_end][['Open', 'High', 'Low', 'Close']].values
                                        
                                        # Calculate actual G-K for current day (which was "next day" for previous window)
                                        current_ohlc = ohlc_df.loc[date]
                                        actual_gk = self.quantum_volatility.calculate_signed_garman_klass(
                                            current_ohlc['Open'],
                                            current_ohlc['High'],
                                            current_ohlc['Low'],
                                            current_ohlc['Close']
                                        )
                                        
                                        if prev_ohlc_window.shape == (4, 4):
                                            
                                            # Make prediction BEFORE update for diagnostics
                                            predicted_gk = self.quantum_volatility.predict_volatility(prev_ohlc_window)
                                            prediction_error = predicted_gk - actual_gk
                                            
                                            # Calculate loss BEFORE update
                                            active_params = self.quantum_volatility.params[self.quantum_volatility.active_layers]
                                            loss_before = self.quantum_volatility._volatility_aware_cost(
                                                active_params,
                                                prev_ohlc_window.reshape(1, 4, 4),
                                                np.array([actual_gk])
                                            )
                                            
                                            # Store parameters before update
                                            params_before = self.quantum_volatility.params.copy()
                                            
                                            # Perform update
                                            self.quantum_volatility.update(prev_ohlc_window, actual_gk)
                                            
                                            # Calculate loss AFTER update
                                            active_params_after = self.quantum_volatility.params[self.quantum_volatility.active_layers]
                                            loss_after = self.quantum_volatility._volatility_aware_cost(
                                                active_params_after,
                                                prev_ohlc_window.reshape(1, 4, 4),
                                                np.array([actual_gk])
                                            )
                                            
                                            # Calculate gradient norm (parameter change)
                                            param_change = np.linalg.norm(self.quantum_volatility.params - params_before)
                                            
                                            # Track statistics
                                            self.continuous_learning_stats['update_count'] += 1
                                            self.continuous_learning_stats['prediction_errors'].append(prediction_error)
                                            self.continuous_learning_stats['gradient_norms'].append(param_change)
                                            self.continuous_learning_stats['losses_before'].append(float(loss_before))
                                            self.continuous_learning_stats['losses_after'].append(float(loss_after))
                                            self.continuous_learning_stats['predictions'].append(predicted_gk)
                                            self.continuous_learning_stats['actuals'].append(actual_gk)
                                            
                                            # Every 10 updates, calculate actual correlation
                                            if self.continuous_learning_stats['update_count'] % 10 == 0:
                                                # Calculate correlation on last 50 predictions
                                                lookback = min(50, len(self.continuous_learning_stats['prediction_errors']))
                                                if lookback > 1:
                                                    recent_predictions = []
                                                    recent_actuals = []
                                                    
                                                    # Go back through recent history
                                                    for j in range(lookback):
                                                        hist_idx = i - lookback + j
                                                        if hist_idx >= 0 and hist_idx < len(sorted_dates):
                                                            hist_date = sorted_dates[hist_idx]
                                                            hist_ohlc_idx = ohlc_df.index.get_loc(hist_date)
                                                            
                                                            if hist_ohlc_idx >= self.quantum_volatility.lookback_window:
                                                                # Get historical window
                                                                hist_start = hist_ohlc_idx - self.quantum_volatility.lookback_window + 1
                                                                hist_end = hist_ohlc_idx + 1
                                                                hist_window = ohlc_df.iloc[hist_start:hist_end][['Open', 'High', 'Low', 'Close']].values
                                                                
                                                                if hist_window.shape == (4, 4):
                                                                    # Predict
                                                                    pred = self.quantum_volatility.predict_volatility(hist_window)
                                                                    # Get actual (next day's G-K)
                                                                    if hist_idx + 1 < len(sorted_dates):
                                                                        next_date = sorted_dates[hist_idx + 1]
                                                                        next_ohlc = ohlc_df.loc[next_date]
                                                                        actual = self.quantum_volatility.calculate_signed_garman_klass(
                                                                            next_ohlc['Open'], next_ohlc['High'], 
                                                                            next_ohlc['Low'], next_ohlc['Close']
                                                                        )
                                                                        recent_predictions.append(pred)
                                                                        recent_actuals.append(actual)
                                                    
                                                    if len(recent_predictions) > 1:
                                                        corr = np.corrcoef(recent_predictions, recent_actuals)[0, 1]
                                                        
                                                        # Additional correlations
                                                        mag_corr = np.corrcoef(np.abs(recent_predictions), np.abs(recent_actuals))[0, 1]
                                                        sign_acc = np.mean(np.sign(recent_predictions) == np.sign(recent_actuals))
                                                        
                                                        print(f"Correlations - Signed: {corr:.3f}, Magnitude: {mag_corr:.3f}, Sign Acc: {sign_acc:.1%}")
                                                        self.continuous_learning_stats['prediction_correlations'].append((date, corr))
                                            
                                            # TensorBoard logging for continuous learning
                                            if tb_writer is not None and self.continuous_learning_stats['update_count'] % 10 == 0:
                                                # Get recent data
                                                recent_errors = self.continuous_learning_stats['prediction_errors'][-50:]
                                                recent_grads = self.continuous_learning_stats['gradient_norms'][-50:]
                                                recent_losses_before = self.continuous_learning_stats['losses_before'][-50:]
                                                recent_losses_after = self.continuous_learning_stats['losses_after'][-50:]
                                                recent_predictions = self.continuous_learning_stats['predictions'][-50:]
                                                recent_actuals = self.continuous_learning_stats['actuals'][-50:]
                                                
                                                # Calculate correlations from recent data
                                                if len(recent_predictions) > 1 and len(recent_actuals) > 1:
                                                    # Signed GK correlation (Pearson correlation of signed values)
                                                    signed_corr = np.corrcoef(recent_predictions, recent_actuals)[0, 1]
                                                    
                                                    # Magnitude correlation (Pearson correlation of absolute values)
                                                    magnitude_corr = np.corrcoef(np.abs(recent_predictions), np.abs(recent_actuals))[0, 1]
                                                    
                                                    # Sign accuracy (fraction of correct directional predictions)
                                                    signs_pred = np.sign(recent_predictions)
                                                    signs_actual = np.sign(recent_actuals)
                                                    sign_accuracy = np.mean(signs_pred == signs_actual)
                                                else:
                                                    signed_corr = 0.0
                                                    magnitude_corr = 0.0
                                                    sign_accuracy = 0.5
                                                
                                                # Current loss (use most recent)
                                                current_loss = recent_losses_after[-1] if recent_losses_after else 0.0
                                                
                                                step = self.continuous_learning_stats['update_count']
                                                
                                                # Log all metrics
                                                with tb_writer.as_default():
                                                    tf.summary.scalar('QuantumCircuit/PredictionPhase/Loss', 
                                                                     current_loss, step=step)
                                                    tf.summary.scalar('QuantumCircuit/PredictionPhase/GradientNorm', 
                                                                     np.mean(recent_grads), step=step)
                                                    tf.summary.scalar('QuantumCircuit/PredictionPhase/SignedGKCorrelation_Pearson', 
                                                                     signed_corr, step=step)
                                                    tf.summary.scalar('QuantumCircuit/PredictionPhase/MagnitudeCorrelation_Pearson', 
                                                                     magnitude_corr, step=step)
                                                    tf.summary.scalar('QuantumCircuit/PredictionPhase/SignAccuracy_DirectionalCorrectness', 
                                                                     sign_accuracy, step=step)
                                                    
                                                    # Also log loss improvement
                                                    if len(recent_losses_before) > 0 and len(recent_losses_after) > 0:
                                                        avg_loss_improvement = np.mean(np.array(recent_losses_before) - np.array(recent_losses_after))
                                                        tf.summary.scalar('QuantumCircuit/PredictionPhase/LossImprovement', 
                                                                         avg_loss_improvement, step=step)
                                            
                                            # Print diagnostics every 50 updates
                                            if self.continuous_learning_stats['update_count'] % 50 == 0:
                                                recent_errors = self.continuous_learning_stats['prediction_errors'][-50:]
                                                recent_grads = self.continuous_learning_stats['gradient_norms'][-50:]
                                                recent_losses_before = self.continuous_learning_stats['losses_before'][-50:]
                                                recent_losses_after = self.continuous_learning_stats['losses_after'][-50:]
                                                
                                                print(f"\n{'='*60}")
                                                print(f"Quantum Circuit Continuous Learning Update #{self.continuous_learning_stats['update_count']}")
                                                print(f"Date: {date.strftime('%Y-%m-%d')}")
                                                print(f"{'='*60}")
                                                
                                                # Prediction performance
                                                print("\nPrediction Performance (last 50 updates):")
                                                print(f"Mean Absolute Error: {np.mean(np.abs(recent_errors)):.6f}")
                                                print(f"Mean Squared Error: {np.mean(np.array(recent_errors)**2):.6f}")
                                                print(f"Error Std Dev: {np.std(recent_errors):.6f}")
                                                
                                                # Loss metrics
                                                print(f"\nLoss Metrics:")
                                                print(f"Avg Loss Before Update: {np.mean(recent_losses_before):.6f}")
                                                print(f"Avg Loss After Update: {np.mean(recent_losses_after):.6f}")
                                                print(f"Avg Loss Reduction: {np.mean(np.array(recent_losses_before) - np.array(recent_losses_after)):.6f}")
                                                
                                                # Gradient metrics
                                                print(f"\nGradient Metrics:")
                                                print(f"Mean Gradient Norm: {np.mean(recent_grads):.8f}")
                                                print(f"Gradient Trend: {'decreasing' if np.mean(recent_grads[:25]) > np.mean(recent_grads[25:]) else 'increasing'}")
                                                
                                                # Correlation
                                                if self.continuous_learning_stats['prediction_correlations']:
                                                    recent_corr = self.continuous_learning_stats['prediction_correlations'][-1][1]
                                                    all_corrs = [c[1] for c in self.continuous_learning_stats['prediction_correlations'] if c[1] is not None]
                                                    print(f"\nPrediction Correlation:")
                                                    print(f"Current: {recent_corr:.3f}")
                                                    print(f"Average: {np.mean(all_corrs):.3f}")
                                                
                                                # Current prediction
                                                print(f"\nCurrent Prediction:")
                                                print(f"Predicted G-K: {predicted_gk:.6f}")
                                                print(f"Actual G-K: {actual_gk:.6f}")
                                                print(f"Error: {prediction_error:.6f}")
                                                print(f"Loss: {loss_before:.6f} → {loss_after:.6f}")
                            """                    
                        else:
                            # Wrong shape, use NaN
                            volatility_dict = {
                                name: np.nan 
                                for name in self.quantum_volatility.get_feature_names()
                            }
                            prediction_data[date]['features'].update(volatility_dict)
                    else:
                        # Not enough history - use NaN
                        volatility_dict = {
                            name: np.nan 
                            for name in self.quantum_volatility.get_feature_names()
                        }
                        prediction_data[date]['features'].update(volatility_dict)
                    
                else:
                    # Skip first day - no previous day available
                    # Use NaN for first day
                    volatility_dict = {
                        name: np.nan 
                        for name in self.quantum_volatility.get_feature_names()
                    }
                    prediction_data[date]['features'].update(volatility_dict)
                    
            except Exception as e:
                print(f"Error processing quantum features for {date}: {e}")
                # Fallback to NaN features
                volatility_dict = {
                    name: np.nan 
                    for name in self.quantum_volatility.get_feature_names()
                }
                prediction_data[date]['features'].update(volatility_dict)

        # Final summary of continuous learning
        if hasattr(self, 'continuous_learning_stats') and self.continuous_learning_stats['update_count'] > 0:
            print(f"\nQUANTUM CONTINUOUS LEARNING SUMMARY")
            print(f"Total Updates: {self.continuous_learning_stats['update_count']}")
            print(f"Mean Absolute Error: {np.mean(np.abs(self.continuous_learning_stats['prediction_errors'])):.6f}")
        
        return prediction_data
    
    def align_quantum_features_with_training(self, train_features, train_target, quantum_detector):
        """
        Properly align quantum features with training data.
        
        Based on time series alignment principles from
        Tsay (2005) "Analysis of Financial Time Series"
        """
        # Get returns data
        returns_column = self.get_target_column()
        train_returns = returns_column.iloc[:len(train_target)]
        
        # Create properly aligned windows
        lookback = quantum_detector.lookback_window
        n_samples = len(train_returns) - lookback + 1
        
        if n_samples <= 0:
            # Not enough data
            #return np.full((len(train_features), 4), np.nan)
            return np.full((len(train_features), quantum_detector.N_QUANTUM_FEATURES), np.nan)
        
        # Initialize aligned features with NaN
        #aligned_features = np.full((len(train_features), 4), np.nan, dtype=np.float64)
        aligned_features = np.full((len(train_features), quantum_detector.N_QUANTUM_FEATURES), np.nan, dtype=np.float64)
        
        # Process windows and align correctly
        for i in range(n_samples):
            window = train_returns.iloc[i:i + lookback].values
            
            try:
                features = quantum_detector.transform(window.reshape(1, -1))[0]
                
                # Correct alignment: the feature at position i corresponds to
                # the prediction made at time i + lookback - 1
                target_idx = i + lookback - 1
                
                if target_idx < len(aligned_features):
                    aligned_features[target_idx] = features
                    
            except Exception as e:
                print(f"Error aligning features at position {i}: {e}")
                continue
        
        return aligned_features

def integrate_quantum_volatility_properly(preprocessor, train_features, train_target, 
                                        predict_features, prediction_data,
                                        quantum_n_qubits=4, random_state=42,
                                        tb_writer=None,
                                        enable_continuous_learning=True,
                                        continuous_learning_increment=1,
                                        continuous_learning_epochs=1):
    """
    Properly integrate quantum volatility detection into the main workflow.
    
    This implementation fixes:
    1. Circuit execution issues
    2. Type compatibility problems
    3. Window alignment errors
    4. Temporal integrity preservation
    
    Based on:
    - Tsay (2005) for time series alignment
    - Rebentrost et al. (2018) for quantum finance
    - Andersen et al. (2001) for volatility modeling
    """
    
    # Import the fixed class
    #from quantum_volatility_fixed import QuantumVolatilityDetector

    # Get raw OHLC data
    ohlc_df = ohlc_df = preprocessor.get_raw_ohlc_data()

    print(f"OHLC data range: {ohlc_df.index[0]} to {ohlc_df.index[-1]}")
    print(f"OHLC data shape: {ohlc_df.shape}")
    
    # Initialize the quantum volatility detector
    quantum_volatility = QuantumVolatilityDetector(
        n_qubits=quantum_n_qubits,
        n_layers=1,
        lookback_window=4,
        adaptive_depth=True,
        random_state=random_state,
        enable_continuous_learning=enable_continuous_learning,
        continuous_learning_increment=continuous_learning_increment,
        continuous_learning_epochs=continuous_learning_epochs
    )

    """
    # Get returns data
    returns_column = preprocessor.get_target_column()
    
    print("Extracting quantum volatility features...")
    
    # 1. Process training data with correct window alignment
    train_returns = returns_column.iloc[:len(train_target)]
    lookback = quantum_volatility.lookback_window
    
    # Create training windows
    train_windows = []
    window_indices = []  # Track which index each window corresponds to
    
    for i in range(len(train_returns) - lookback + 1):
        window = train_returns.iloc[i:i + lookback].values
        train_windows.append(window)
        # The window ending at position i+lookback-1 is used to predict i+lookback
        window_indices.append(i + lookback - 1)
    """
    
    # CRITICAL: Align OHLC data with the actual training period
    # The train_features index contains the actual dates used for training
    train_start_date = train_features.index[0]
    train_end_date = train_features.index[-1]

    # Filter OHLC to match training period (with some buffer for windows)
    # Need extra days before start for creating windows
    # Calculate required buffer based on quantum lookback window
    quantum_lookback = quantum_volatility.lookback_window  # Should be 4
    buffer_days = quantum_lookback - 1  # Need 3 previous days for 4-day window
    
    # Get OHLC data with proper buffer
    ohlc_start_idx = max(0, ohlc_df.index.get_loc(train_start_date) - buffer_days)
    ohlc_end_idx = ohlc_df.index.get_loc(train_end_date) + 2  # +2 for next-day targets
    
    # Get the filtered OHLC data
    ohlc_df_filtered = ohlc_df.iloc[ohlc_start_idx:ohlc_end_idx]
    
    print(f"Classical training starts: {train_start_date}")
    print(f"OHLC data starts: {ohlc_df_filtered.index[0]} ({buffer_days} days before)")
    print(f"OHLC data ends: {ohlc_df_filtered.index[-1]}")
    print(f"Filtered OHLC shape: {ohlc_df_filtered.shape}")
    
    # Verify we have valid OHLC data (not all identical values)
    first_valid_idx = None
    for i in range(len(ohlc_df_filtered)):
        row = ohlc_df_filtered.iloc[i]
        if not (row['Open'] == row['High'] == row['Low'] == row['Close']):
            first_valid_idx = i
            break
    
    if first_valid_idx is not None and first_valid_idx > 0:
        print(f"WARNING: First {first_valid_idx} days have identical OHLC values")
        print(f"First valid OHLC data at: {ohlc_df_filtered.index[first_valid_idx]}")
    
    # Use filtered OHLC for training
    train_ohlc = ohlc_df_filtered
    
    print("Extracting quantum volatility features...")
    
    # 1. Fit the quantum detector on training OHLC data
    # Create OHLC windows from training data
    train_windows_dict = quantum_volatility.create_ohlc_windows(train_ohlc)
    train_dates = sorted(train_windows_dict.keys())
    train_ohlc_windows = np.array([train_windows_dict[date] for date in train_dates])
    
    # Create targets (next-day signed G-K) - detector's fit method handles this internally
    print(f"Training quantum detector on {len(train_ohlc_windows)} OHLC windows...")

    """
    if len(train_ohlc_windows) > 0:
        print("\n=== Running pre-training diagnostics ===")
        test_window = train_ohlc_windows[0]
        diagnose_dimension_mismatch(quantum_volatility, test_window)
        print("=== End pre-training diagnostics ===\n")
    """
    
    """
    if train_windows:
        train_windows = np.array(train_windows)
        print(f"Created {len(train_windows)} training windows")
        
        # Fit the quantum detector
        # quantum_volatility.fit(train_windows, verbose=True)
        
        # Store initial parameters for comparison
        initial_params = quantum_volatility.params.copy()
        print(f"DEBUG: Initial params (first 5): {initial_params.flatten()[:5]}")
        print(f"DEBUG: Initial params norm: {np.linalg.norm(initial_params):.6f}")
        
        actual_volatilities = np.array([np.std(window) for window in train_windows])
        print(f"\nDEBUG: Created {len(actual_volatilities)} volatility targets")
        print(f"DEBUG: Volatility range: [{np.min(actual_volatilities):.6f}, {np.max(actual_volatilities):.6f}]")
        print(f"DEBUG: Mean volatility: {np.mean(actual_volatilities):.6f}")
        
        quantum_volatility.fit(train_windows, targets=actual_volatilities, verbose=True)
        
        # Check parameter changes
        print(f"DEBUG: Final params (first 5): {quantum_volatility.params.flatten()[:5]}")
        print(f"DEBUG: Final params norm: {np.linalg.norm(quantum_volatility.params):.6f}")
        param_change = np.linalg.norm(quantum_volatility.params - initial_params)
        print(f"DEBUG: Parameter change magnitude: {param_change:.6f}")
        
        if param_change < 1e-6:
            print("🚨 WARNING: Parameters barely changed during training!")
        
        print(f"DEBUG: First few quantum parameters after fitting: {quantum_volatility.params.flatten()[:5]}")
        
        # Transform training data
        train_volatility_features = quantum_volatility.transform(train_windows)

        print(f"DEBUG: First 3 quantum feature vectors:")
        for i in range(min(3, len(train_volatility_features))):
            print(f"  Sample {i}: {train_volatility_features[i]}")

        # NEW DEBUG: Check what transform actually returned
        print(f"DEBUG: Fresh transform output shape: {train_volatility_features.shape}")
        print(f"DEBUG: Fresh transform first 3 rows:")
        print(train_volatility_features[:3])
        print(f"DEBUG: Fresh transform statistics:")
        print(f"  Mean: {np.mean(train_volatility_features, axis=0)}")
        print(f"  Range: [{np.min(train_volatility_features, axis=0)}, {np.max(train_volatility_features, axis=0)}]")
        
        # Create aligned features array
        n_quantum_features = 4  # Fixed based on our circuit design
        aligned_features = np.full(
            (len(train_features), n_quantum_features),
            np.nan,
            dtype=np.float64
        )
        
        # Align features correctly
        for i, idx in enumerate(window_indices):
            if idx < len(train_features):
                aligned_features[idx] = train_volatility_features[i]

        # ***  DEBUG CODE ***
        print(f"DEBUG: Alignment completed")
        print(f"DEBUG: Window indices length: {len(window_indices)}")
        print(f"DEBUG: Aligned features shape: {aligned_features.shape}")
        print(f"DEBUG: Non-NaN count in aligned features: {np.sum(~np.isnan(aligned_features), axis=0)}")
        print(f"DEBUG: Aligned features statistics (excluding NaN):")
        for col_idx in range(aligned_features.shape[1]):
            col_data = aligned_features[:, col_idx]
            valid_data = col_data[~np.isnan(col_data)]
            if len(valid_data) > 0:
                print(f"  Column {col_idx}: Mean={np.mean(valid_data):.6f}, Range=[{np.min(valid_data):.6f}, {np.max(valid_data):.6f}]")
        # *** END DEBUG CODE ***
        
        # Validate alignment
        print(f"Aligned {np.sum(~np.isnan(aligned_features[:, 0]))} quantum features")
        
        # Create DataFrame with proper column names
        volatility_cols = quantum_volatility.get_feature_names()
        train_volatility_df = pd.DataFrame(
            aligned_features,
            index=train_features.index,
            columns=volatility_cols
        )
        
        # Report NaN statistics
        nan_counts = train_volatility_df.isna().sum()
        print("\nNaN counts per feature:")
        for col, count in nan_counts.items():
            print(f"  {col}: {count} ({count/len(train_volatility_df)*100:.1f}%)")
        
        # Fill NaN values with forward fill then backward fill
        train_volatility_df = train_volatility_df.fillna(method='ffill').fillna(method='bfill')
        
        # If still NaN, fill with feature mean
        for col in train_volatility_df.columns:
            if train_volatility_df[col].isna().any():
                mean_val = train_volatility_df[col].mean()
                train_volatility_df[col].fillna(mean_val, inplace=True)
        
        # Combine with existing features
        train_features = pd.concat([train_features, train_volatility_df], axis=1)
    """
    if len(train_ohlc_windows) > 0:
        # Fit will create its own targets internally
        quantum_volatility.fit(train_ohlc, verbose=True, tb_writer=tb_writer)
        
        # Transform training data
        train_volatility_features = quantum_volatility.transform(train_ohlc_windows)
        
        # Create aligned features array
        #n_quantum_features = 4
        n_quantum_features = quantum_volatility.N_QUANTUM_FEATURES
        aligned_features = np.full(
            (len(train_features), n_quantum_features),
            np.nan,
            dtype=np.float64
        )
        
        # Align features correctly - windows are created with proper dates
        for i, date in enumerate(train_dates[:-1]):  # -1 because last window has no target
            if date in train_features.index:
                idx = train_features.index.get_loc(date)
                aligned_features[idx] = train_volatility_features[i]
        
        # Validate alignment
        print(f"Aligned {np.sum(~np.isnan(aligned_features[:, 0]))} quantum features")
        
        # Create DataFrame with proper column names
        volatility_cols = quantum_volatility.get_feature_names()
        train_volatility_df = pd.DataFrame(
            aligned_features,
            index=train_features.index,
            columns=volatility_cols
        )
        
        # Report NaN statistics
        nan_counts = train_volatility_df.isna().sum()
        print("\nNaN counts per feature:")
        for col, count in nan_counts.items():
            print(f"  {col}: {count} ({count/len(train_volatility_df)*100:.1f}%)")
        
        # Fill NaN values with forward fill then backward fill
        train_volatility_df = train_volatility_df.fillna(method='ffill').fillna(method='bfill')
        
        # If still NaN, fill with feature mean
        for col in train_volatility_df.columns:
            if train_volatility_df[col].isna().any():
                mean_val = train_volatility_df[col].mean()
                train_volatility_df[col].fillna(mean_val, inplace=True)
        
        # Combine with existing features
        train_features = pd.concat([train_features, train_volatility_df], axis=1)
        
    else:
        print("WARNING: Not enough data to create training windows")
        # Create empty quantum features
        volatility_cols = quantum_volatility.get_feature_names()
        train_volatility_df = pd.DataFrame(
            np.full((len(train_features), quantum_volatility.N_QUANTUM_FEATURES), np.nan),
            index=train_features.index,
            columns=volatility_cols
        )
        train_features = pd.concat([train_features, train_volatility_df], axis=1)
    
    # 2. Process prediction data with proper temporal constraints
    print("\nProcessing prediction data with strict temporal constraints...")
    
    # Initialize the preprocessor's quantum detector
    preprocessor.quantum_volatility = quantum_volatility
    
    # Get updated prediction data with quantum features
    prediction_data = preprocessor.get_daily_prediction_data(tb_writer=tb_writer)
    
    # Extract features for predict_features DataFrame
    predict_volatility_features = np.full((len(predict_features), quantum_volatility.N_QUANTUM_FEATURES), np.nan)
    
    for i, date in enumerate(predict_features.index):
        if date in prediction_data:
            # Extract quantum features from the prediction data
            pred_features = prediction_data[date]['features']
            
            for j, col in enumerate(quantum_volatility.get_feature_names()):
                if col in pred_features:
                    predict_volatility_features[i, j] = pred_features[col]
    
    # Create DataFrame for prediction features
    predict_volatility_df = pd.DataFrame(
        predict_volatility_features,
        index=predict_features.index,
        columns=quantum_volatility.get_feature_names()
    )
    
    # Fill NaN values similarly
    predict_volatility_df = predict_volatility_df.fillna(method='ffill').fillna(method='bfill')
    for col in predict_volatility_df.columns:
        if predict_volatility_df[col].isna().any():
            mean_val = predict_volatility_df[col].mean()
            predict_volatility_df[col].fillna(mean_val, inplace=True)
    
    # Combine with existing features
    predict_features = pd.concat([predict_features, predict_volatility_df], axis=1)
    
    # Update feature names
    feature_names = list(train_features.columns)
    
    print(f"\nSuccessfully integrated {len(quantum_volatility.get_feature_names())} quantum volatility features")
    print(f"Total features: {len(feature_names)}")
    
    return train_features, predict_features, feature_names, prediction_data, quantum_volatility

def create_classical_quantum_comparison_plots(
    classical_features, quantum_features, classical_feature_names, 
    quantum_feature_names, train_target, output_folder):
    """
    Create comprehensive visualizations comparing classical and quantum features.
    Modified to not save automatically and include more analysis in the plot.
    """
    import matplotlib.pyplot as plt
    import matplotlib.gridspec as gridspec
    from sklearn.decomposition import PCA
    from sklearn.feature_selection import mutual_info_regression
    from scipy.stats import spearmanr
    from sklearn.linear_model import LinearRegression
    from statsmodels.tsa.stattools import acf
    import numpy as np
    
    # Create figure with subplots - Increased size for more content
    fig = plt.figure(figsize=(24, 36))
    gs = gridspec.GridSpec(9, 2, figure=fig, hspace=0.3, wspace=0.3)
    
    # 1. Variance Analysis
    ax1 = fig.add_subplot(gs[0, :])
    pca_quantum = PCA()
    pca_quantum.fit(quantum_features)
    
    var_explained = np.cumsum(pca_quantum.explained_variance_ratio_)
    ax1.plot(range(1, len(var_explained)+1), var_explained, 'b-', linewidth=2)
    ax1.axhline(y=0.95, color='r', linestyle='--', label='95% variance')
    ax1.set_xlabel('Number of Components')
    ax1.set_ylabel('Cumulative Variance Explained')
    ax1.set_title('Quantum Features - Variance Explanation by Principal Components')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # Add text annotation about variance
    variance_text = f"First 5 PCs explain {np.sum(pca_quantum.explained_variance_ratio_[:5]):.3f} of variance"
    if np.sum(pca_quantum.explained_variance_ratio_[:5]) > 0.9:
        variance_text += "\n(Quantum features are highly correlated)"
    else:
        variance_text += "\n(Quantum features capture diverse information)"
    ax1.text(0.5, 0.2, variance_text, transform=ax1.transAxes, 
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # 2. Mutual Information Comparison
    ax2 = fig.add_subplot(gs[1, 0])
    mi_classical = mutual_info_regression(classical_features, train_target.values)
    mi_quantum = mutual_info_regression(quantum_features, train_target.values)
    
    x = np.arange(2)
    width = 0.35
    ax2.bar(x - width/2, [np.mean(mi_classical), np.mean(mi_quantum)], 
            width, label='Average MI', color=['blue', 'red'])
    ax2.bar(x + width/2, [np.max(mi_classical), np.max(mi_quantum)], 
            width, label='Max MI', color=['lightblue', 'lightcoral'])
    
    ax2.set_ylabel('Mutual Information')
    ax2.set_title('Mutual Information with Target')
    ax2.set_xticks(x)
    ax2.set_xticklabels(['Classical', 'Quantum'])
    ax2.legend()
    
    # Add MI details
    mi_text = f"Classical: avg={np.mean(mi_classical):.6f}, max={np.max(mi_classical):.6f}\n"
    mi_text += f"Quantum: avg={np.mean(mi_quantum):.6f}, max={np.max(mi_quantum):.6f}"
    ax2.text(0.02, 0.98, mi_text, transform=ax2.transAxes, 
             verticalalignment='top', fontsize=8,
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # 3. Redundancy Analysis Heatmap
    ax3 = fig.add_subplot(gs[1, 1])
    
    redundancy_matrix = np.zeros((len(quantum_feature_names), 1))
    for i, q_feat in enumerate(quantum_feature_names):
        quantum_col = quantum_features[:, i]
        reg = LinearRegression()
        reg.fit(classical_features, quantum_col)
        r2_score = reg.score(classical_features, quantum_col)
        redundancy_matrix[i, 0] = r2_score
    
    im = ax3.imshow(redundancy_matrix, cmap='RdYlBu_r', aspect='auto', vmin=0, vmax=1)
    ax3.set_yticks(range(len(quantum_feature_names)))
    ax3.set_yticklabels(quantum_feature_names, fontsize=8)
    ax3.set_xlabel('R² with Classical Features')
    ax3.set_title('Quantum Feature Redundancy Analysis')
    plt.colorbar(im, ax=ax3)
    
    # Add redundancy interpretation
    avg_redundancy = np.mean(redundancy_matrix)
    redundancy_text = f"Avg redundancy: {avg_redundancy:.3f}\n"
    if avg_redundancy < 0.5:
        redundancy_text += "Quantum features contain unique information"
    else:
        redundancy_text += "Quantum features may be redundant"
    ax3.text(1.3, 0.5, redundancy_text, transform=ax3.transAxes,
             verticalalignment='center', fontsize=9)
    
    # 4. Correlation Comparison
    ax4 = fig.add_subplot(gs[2, :])
    linear_corrs_classical = [abs(np.corrcoef(feat, train_target.values)[0, 1]) 
                              for feat in classical_features.T]
    linear_corrs_quantum = [abs(np.corrcoef(feat, train_target.values)[0, 1]) 
                           for feat in quantum_features.T]
    
    ax4.boxplot([linear_corrs_classical, linear_corrs_quantum], 
                labels=['Classical', 'Quantum'])
    ax4.set_ylabel('Absolute Pearson Correlation')
    ax4.set_title('Feature-Target Correlation Distribution')
    ax4.grid(True, alpha=0.3)
    
    # Add correlation statistics
    corr_stats = f"Classical: mean={np.mean(linear_corrs_classical):.4f}, std={np.std(linear_corrs_classical):.4f}\n"
    corr_stats += f"Quantum: mean={np.mean(linear_corrs_quantum):.4f}, std={np.std(linear_corrs_quantum):.4f}"
    ax4.text(0.02, 0.98, corr_stats, transform=ax4.transAxes,
             verticalalignment='top', fontsize=9,
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # 5. Top Features by MI
    ax5 = fig.add_subplot(gs[3, :])
    
    # Combine and sort features by MI
    all_mi = np.concatenate([mi_classical, mi_quantum])
    all_names = classical_feature_names + quantum_feature_names
    all_types = ['Classical']*len(classical_feature_names) + ['Quantum']*len(quantum_feature_names)
    
    sorted_idx = np.argsort(all_mi)[::-1][:20]  # Top 20
    
    colors = ['blue' if all_types[i]=='Classical' else 'red' for i in sorted_idx]
    y_pos = np.arange(len(sorted_idx))
    
    ax5.barh(y_pos, all_mi[sorted_idx], color=colors)
    ax5.set_yticks(y_pos)
    ax5.set_yticklabels([all_names[i] for i in sorted_idx], fontsize=8)
    ax5.set_xlabel('Mutual Information')
    ax5.set_title('Top 20 Features by Mutual Information')
    ax5.invert_yaxis()
    
    # Add legend
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor='blue', label='Classical'),
                      Patch(facecolor='red', label='Quantum')]
    ax5.legend(handles=legend_elements, loc='lower right')
    
    # 6. Non-linearity Analysis
    ax6 = fig.add_subplot(gs[4, :])
    
    nonlinearity_classical = []
    nonlinearity_quantum = []
    
    for feat in classical_features.T[:20]:  # First 20 classical
        linear = abs(np.corrcoef(feat, train_target.values)[0, 1])
        spearman, _ = spearmanr(feat, train_target.values)
        nonlinearity_classical.append(abs(spearman) - linear)
    
    for feat in quantum_features.T:
        linear = abs(np.corrcoef(feat, train_target.values)[0, 1])
        spearman, _ = spearmanr(feat, train_target.values)
        nonlinearity_quantum.append(abs(spearman) - linear)
    
    ax6.scatter(range(len(nonlinearity_classical)), nonlinearity_classical, 
                color='blue', alpha=0.6, label='Classical', s=50)
    ax6.scatter(range(len(nonlinearity_classical), 
                     len(nonlinearity_classical) + len(nonlinearity_quantum)), 
                nonlinearity_quantum, color='red', alpha=0.6, label='Quantum', s=50)
    ax6.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    ax6.set_xlabel('Feature Index')
    ax6.set_ylabel('Non-linearity (Spearman - Pearson)')
    ax6.set_title('Non-linear Correlation Gain by Feature Type')
    ax6.legend()
    ax6.grid(True, alpha=0.3)
    
    # Add non-linearity statistics
    nl_gain_classical = np.mean(nonlinearity_classical)
    nl_gain_quantum = np.mean(nonlinearity_quantum)
    nl_text = f"Classical non-linearity gain: {nl_gain_classical:.4f}\n"
    nl_text += f"Quantum non-linearity gain: {nl_gain_quantum:.4f}"
    if nl_gain_quantum > nl_gain_classical * 1.5:
        nl_text += "\n→ Quantum captures more non-linear patterns"
    ax6.text(0.02, 0.98, nl_text, transform=ax6.transAxes,
             verticalalignment='top', fontsize=9,
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # 7. Temporal Structure Analysis
    ax7 = fig.add_subplot(gs[5, :])
    
    # Calculate autocorrelations at different lags
    lags = [1, 5, 10]
    acf_data = {'Classical': {}, 'Quantum': {}}
    
    for lag in lags:
        classical_acfs = []
        quantum_acfs = []
        
        # Classical features ACF
        for feat in classical_features.T:
            if not np.any(np.isnan(feat)) and len(feat) > lag:
                try:
                    acf_vals = acf(feat, nlags=lag, fft=True)
                    classical_acfs.append(abs(acf_vals[-1]))
                except:
                    pass
        
        # Quantum features ACF
        for feat in quantum_features.T:
            if not np.any(np.isnan(feat)) and len(feat) > lag:
                try:
                    acf_vals = acf(feat, nlags=lag, fft=True)
                    quantum_acfs.append(abs(acf_vals[-1]))
                except:
                    pass
        
        if classical_acfs:
            acf_data['Classical'][lag] = np.mean(classical_acfs)
        if quantum_acfs:
            acf_data['Quantum'][lag] = np.mean(quantum_acfs)
    
    # Plot ACF comparison
    bar_width = 0.35
    x_pos = np.arange(len(lags))
    
    classical_acf_means = [acf_data['Classical'].get(lag, 0) for lag in lags]
    quantum_acf_means = [acf_data['Quantum'].get(lag, 0) for lag in lags]
    
    ax7.bar(x_pos - bar_width/2, classical_acf_means, bar_width, 
            label='Classical', color='blue', alpha=0.7)
    ax7.bar(x_pos + bar_width/2, quantum_acf_means, bar_width, 
            label='Quantum', color='red', alpha=0.7)
    
    ax7.set_xlabel('Lag (days)')
    ax7.set_ylabel('Average Absolute Autocorrelation')
    ax7.set_title('Temporal Structure Analysis - Autocorrelation at Different Lags')
    ax7.set_xticks(x_pos)
    ax7.set_xticklabels([f'Lag {lag}' for lag in lags])
    ax7.legend()
    ax7.grid(True, alpha=0.3)
    
    # Add interpretation
    temporal_text = "Autocorrelation Analysis:\n"
    for lag in lags:
        if lag in acf_data['Classical'] and lag in acf_data['Quantum']:
            temporal_text += f"Lag {lag}: Classical={acf_data['Classical'][lag]:.3f}, Quantum={acf_data['Quantum'][lag]:.3f}\n"
    ax7.text(0.02, 0.98, temporal_text, transform=ax7.transAxes,
             verticalalignment='top', fontsize=9,
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # 8. Detailed Redundancy Analysis
    ax8 = fig.add_subplot(gs[6, :])
    
    # Show individual quantum feature redundancies
    redundancy_scores = redundancy_matrix.flatten()
    unique_features = [(quantum_feature_names[i], redundancy_scores[i]) 
                      for i in range(len(quantum_feature_names)) if redundancy_scores[i] < 0.3]
    redundant_features = [(quantum_feature_names[i], redundancy_scores[i]) 
                         for i in range(len(quantum_feature_names)) if redundancy_scores[i] > 0.9]
    
    ax8.bar(range(len(quantum_feature_names)), redundancy_scores)
    ax8.set_xlabel('Quantum Feature Index')
    ax8.set_ylabel('R² with Classical Features')
    ax8.set_title('Individual Quantum Feature Redundancy')
    ax8.axhline(y=0.3, color='g', linestyle='--', alpha=0.5, label='Low redundancy threshold')
    ax8.axhline(y=0.9, color='r', linestyle='--', alpha=0.5, label='High redundancy threshold')
    ax8.set_xticks(range(len(quantum_feature_names)))
    ax8.set_xticklabels([f"Q{i}" for i in range(len(quantum_feature_names))], fontsize=8)
    ax8.legend()
    ax8.grid(True, alpha=0.3)
    
    # 9. Summary Statistics and Interpretation
    ax9 = fig.add_subplot(gs[7:, :])
    ax9.axis('off')
    
    # Comprehensive summary combining all analyses
    summary_text = f"""
    COMPREHENSIVE CLASSICAL VS QUANTUM FEATURE ANALYSIS
    ===================================================
    
    Dataset Information:
    - Classical Features: {len(classical_feature_names)} principal components
    - Quantum Features: {len(quantum_feature_names)} measurements
    - Training Samples: {len(train_target)}
    
    1. VARIANCE DECOMPOSITION:
    - Quantum features: {np.sum(pca_quantum.explained_variance_ratio_[:5]):.3f} variance in first 5 PCs
    - Components for 95% variance: {np.argmax(np.cumsum(pca_quantum.explained_variance_ratio_) > 0.95) + 1}
    
    2. INFORMATION CONTENT:
    - Average Mutual Information:
      • Classical: {np.mean(mi_classical):.6f}
      • Quantum: {np.mean(mi_quantum):.6f}
    - Maximum MI Features:
      • Classical: {classical_feature_names[np.argmax(mi_classical)]} = {np.max(mi_classical):.6f}
      • Quantum: {quantum_feature_names[np.argmax(mi_quantum)]} = {np.max(mi_quantum):.6f}
    
    3. REDUNDANCY ANALYSIS:
    - Average Redundancy (R² with classical): {np.mean(redundancy_matrix):.3f}
    - Unique Quantum Features (R² < 0.3): {len(unique_features)}
    - Highly Redundant Features (R² > 0.9): {len(redundant_features)}
    
    4. CORRELATION PATTERNS:
    - Linear Correlation with Target:
      • Classical: {np.mean(linear_corrs_classical):.4f} ± {np.std(linear_corrs_classical):.4f}
      • Quantum: {np.mean(linear_corrs_quantum):.4f} ± {np.std(linear_corrs_quantum):.4f}
    
    5. NON-LINEARITY ANALYSIS:
    - Non-linearity Gain (Spearman - Pearson):
      • Classical: {nl_gain_classical:.4f}
      • Quantum: {nl_gain_quantum:.4f}
      • Ratio (Quantum/Classical): {nl_gain_quantum/nl_gain_classical if nl_gain_classical > 0 else np.inf:.2f}
    
    6. TEMPORAL STRUCTURE:
    - Autocorrelation at different lags shows how features capture time dependencies
    - Classical features: {', '.join([f'Lag{lag}={acf_data["Classical"].get(lag, 0):.3f}' for lag in lags])}
    - Quantum features: {', '.join([f'Lag{lag}={acf_data["Quantum"].get(lag, 0):.3f}' for lag in lags])}
    
    INTERPRETATION:
    {'• Quantum features provide substantial unique information' if avg_redundancy < 0.5 else '• Quantum features show high redundancy with classical features'}
    {'• Quantum features capture more non-linear patterns' if nl_gain_quantum > nl_gain_classical * 1.5 else '• Non-linearity capture is similar between classical and quantum'}
    {'• Diverse quantum features suggest effective encoding' if np.sum(pca_quantum.explained_variance_ratio_[:5]) < 0.9 else '• Quantum features are highly correlated, suggesting limited diversity'}
    {'• Different temporal structure in quantum features' if any(abs(acf_data['Classical'].get(lag, 0) - acf_data['Quantum'].get(lag, 0)) > 0.1 for lag in lags) else '• Similar temporal patterns between classical and quantum'}
    
    RECOMMENDATION:
    {'The quantum features appear to add significant value to the model' if (avg_redundancy < 0.5 and nl_gain_quantum > nl_gain_classical) else 'Consider adjusting quantum circuit parameters for better feature extraction'}
    """
    
    ax9.text(0.05, 0.95, summary_text, transform=ax9.transAxes, fontsize=10,
             verticalalignment='top', fontfamily='monospace',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.suptitle('Classical vs Quantum Feature Analysis', fontsize=16, y=0.995)
    plt.tight_layout()
    
    return fig

class FeatureRelationshipTracker:
    def __init__(self, update_interval=50):
        self.update_interval = update_interval
        self.step_count = 0
        self.history = {
            'quantum_classical_redundancy': [],
            'quantum_mi': [],
            'classical_mi': [],
            'quantum_volatility_corr': [],
            'timestamps': []
        }
    
    def update(self, classical_features, quantum_features, target, date, tb_writer=None):
        self.step_count += 1
        
        if self.step_count % self.update_interval == 0:
            # Lightweight calculations only
            # Use last 100 samples for efficiency
            window_size = min(100, len(classical_features))
            
            classical_window = classical_features[-window_size:]
            quantum_window = quantum_features[-window_size:]
            target_window = target[-window_size:]
            
            # Quick redundancy check (single quantum feature)
            avg_redundancy = 0
            for i in range(quantum_window.shape[1]):
                corr = np.corrcoef(quantum_window[:, i], 
                                  classical_window[:, 0])[0, 1]  # Just PC1
                avg_redundancy += abs(corr)
            avg_redundancy /= quantum_window.shape[1]
            
            # Store and log
            self.history['quantum_classical_redundancy'].append(avg_redundancy)
            self.history['timestamps'].append(date)
            
            if tb_writer:
                tb_writer.add_scalar('Features/QuantumClassicalRedundancy', 
                                   avg_redundancy, self.step_count)
    

import matplotlib.pyplot as plt
import numpy as np
from typing import Dict, List
from copy import deepcopy

class QuantumDetectorDiagnostics:
    """
    Diagnostic tools to analyze quantum volatility detector performance
    and determine if stable loss indicates good convergence or issues.
    """
    
    def analyze_training_convergence(self, quantum_detector, returns_data, 
                                    test_learning_rates=[0.001, 0.01, 0.1],
                                    test_epochs=30):
        """
        Analyze whether stable loss indicates proper convergence or other issues.
        
        Returns:
        --------
        dict: Diagnostic results with recommendations
        """
        results = {
            'gradient_analysis': self._analyze_gradients(quantum_detector),
            'learning_rate_test': self._test_learning_rates(
                quantum_detector, returns_data, test_learning_rates, test_epochs
            ),
            'feature_quality': self._analyze_feature_quality(quantum_detector, returns_data),
            'circuit_expressivity': self._test_circuit_expressivity(quantum_detector, returns_data),
            'loss_landscape': self._probe_loss_landscape(quantum_detector, returns_data)
        }
        
        # Generate recommendation
        results['recommendation'] = self._generate_recommendation(results)
        
        return results
    
    def _analyze_gradients(self, detector):
        """Check if gradients are vanishing (barren plateau) or healthy."""
        # Check if training history exists with required attributes
        if not hasattr(detector, 'training_history'):
            return {'status': 'no_training_history'}
        
        # Make sure we can safely access each history component
        history = detector.training_history
        gradients = history.get('gradients', [])
        
        if len(gradients) == 0:
            return {'status': 'no_gradient_history'}
        
        gradient_analysis = {
            'mean_gradient': np.mean(gradients),
            'gradient_variance': np.var(gradients),
            'gradient_trend': np.polyfit(range(len(gradients)), gradients, 1)[0] if len(gradients) > 1 else 0,
            'vanishing_gradient': np.mean(gradients) < 1e-6,
            'gradient_stability': np.std(gradients) / (np.mean(gradients) + 1e-10)
        }
        
        return gradient_analysis
    
    def _test_learning_rates(self, detector, ohlc_data, learning_rates, epochs):
        """Test different learning rates to see if loss improves."""
        from copy import deepcopy
        
        results = {}
        original_params = deepcopy(detector.params)
        
        # Prepare OHLC windows
        if isinstance(ohlc_data, pd.DataFrame):
            windows_dict = detector.create_ohlc_windows(ohlc_data)
            windows = np.array(list(windows_dict.values()))[:100]  # Use subset
        else:
            windows = ohlc_data[:100]  # Assume already windowed
        
        """
        results = {}
        original_params = deepcopy(detector.params)
        
        # Prepare training data
        windows = []
        for i in range(len(returns_data) - detector.lookback_window + 1):
            windows.append(returns_data[i:i + detector.lookback_window])
        windows = np.array(windows)[:100]  # Use subset for quick testing
        """
        
        for lr in learning_rates:
            # Reset parameters
            detector.params = deepcopy(original_params)
            detector.training_history = {'loss': []}
            
            # Train with specific learning rate
            detector.fit(windows, learning_rate=lr, epochs=epochs, verbose=False)
            
            results[lr] = {
                'final_loss': detector.training_history['loss'][-1],
                'loss_improvement': detector.training_history['loss'][0] - detector.training_history['loss'][-1],
                'loss_variance': np.var(detector.training_history['loss'])
            }
        
        # Restore original parameters
        detector.params = original_params
        
        return results
    
    def _analyze_feature_quality(self, detector, ohlc_data):
        """Analyze if quantum features capture meaningful volatility patterns."""
        # Create OHLC windows
        if isinstance(ohlc_data, pd.DataFrame):
            windows_dict = detector.create_ohlc_windows(ohlc_data)
            windows = np.array(list(windows_dict.values()))
        else:
            windows = ohlc_data
        
        # Get quantum features
        features = detector.transform(windows)
        
        # Calculate actual G-K volatilities from windows
        actual_gk_values = []
        for window in windows:
            gk_values = []
            for day in window:
                gk = detector.calculate_signed_garman_klass(*day)
                gk_values.append(abs(gk))  # Use magnitude for correlation
            actual_gk_values.append(np.mean(gk_values))
        
        actual_volatility = np.array(actual_gk_values)
        
        """Analyze if quantum features capture meaningful volatility patterns."""
        """
        # Create windows
        windows = []
        for i in range(len(returns_data) - detector.lookback_window + 1):
            windows.append(returns_data[i:i + detector.lookback_window])
        windows = np.array(windows)
        
        # Get quantum features
        features = detector.transform(windows)
        
        # Calculate correlations with actual volatility measures
        actual_volatility = np.array([np.std(w) for w in windows])
        actual_squared_returns = np.array([np.mean(w**2) for w in windows])
        """
        
        # Calculate squared returns proxy from OHLC - ADD THIS BEFORE THE DICTIONARY
        actual_squared_returns = []
        for window in windows:
            returns = []
            for day in window:
                if day[0] > 0:  # Open price
                    ret = (day[3] - day[0]) / day[0]  # (Close - Open) / Open
                    returns.append(ret ** 2)
            actual_squared_returns.append(np.mean(returns) if returns else 0)
        actual_squared_returns = np.array(actual_squared_returns)
        
        # NOW create the dictionary with all values calculated
        quality_metrics = {
            'feature_variance': np.var(features, axis=0),
            'feature_entropy': self._calculate_entropy(features),
            'correlation_with_volatility': [
                np.corrcoef(features[:, i], actual_volatility)[0, 1] 
                for i in range(features.shape[1])
            ],
            'correlation_with_squared_returns': [
                np.corrcoef(features[:, i], actual_squared_returns)[0, 1] 
                for i in range(features.shape[1])
            ],
            'feature_differentiation': np.std(features, axis=0) / (np.mean(np.abs(features), axis=0) + 1e-10)
        }
        
        return quality_metrics
    
    def _test_circuit_expressivity(self, detector, ohlc_data):
        """Test if circuit can learn different patterns."""
        from copy import deepcopy
        
        # Create synthetic OHLC patterns
        base_price = 100
        patterns = {
            'constant': self._create_ohlc_pattern('constant', base_price),
            'trending': self._create_ohlc_pattern('trending', base_price),
            'volatile': self._create_ohlc_pattern('volatile', base_price),
            'jump': self._create_ohlc_pattern('jump', base_price)
        }
        
        expressivity_results = {}
        
        for pattern_name, pattern_ohlc in patterns.items():
            detector_copy = deepcopy(detector)
            
            # Create DataFrame from pattern
            pattern_df = pd.DataFrame(
                pattern_ohlc.reshape(-1, 4),
                columns=['Open', 'High', 'Low', 'Close']
            )
            
            detector_copy.fit(pattern_df, epochs=20, verbose=False)
            
            # Test how well it learned
            test_window = pattern_ohlc[:4]  # First 4 days
            predictions = detector_copy.transform(test_window.reshape(1, 4, 4))
            
            expressivity_results[pattern_name] = {
                'pattern_learned': np.std(predictions) > 0.01,
                'feature_variance': np.var(predictions),
                'distinct_features': len(np.unique(np.round(predictions, 4)))
            }
        
        return expressivity_results
    
    def _probe_loss_landscape(self, detector, returns_data):
        """Probe the loss landscape around current parameters."""
        # Prepare small dataset
        # Prepare OHLC windows
        if isinstance(ohlc_data, pd.DataFrame):
            windows_dict = detector.create_ohlc_windows(ohlc_data)
            windows = np.array(list(windows_dict.values()))[:100]
        else:
            windows = ohlc_data[:100]
        
        # Calculate G-K targets (not std of returns!)
        targets = []
        for window in windows:
            # Use last day's G-K as target
            last_day = window[-1]
            gk = detector.calculate_signed_garman_klass(*last_day)
            targets.append(gk)
        targets = np.array(targets)
        
        # Current loss
        current_loss = detector._volatility_aware_cost(
            detector.params[detector.active_layers], 
            windows, 
            targets
        )
        
        # Probe in random directions
        n_probes = 10
        probe_results = []
        
        for _ in range(n_probes):
            # Random direction
            direction = np.random.normal(0, 0.1, detector.params[detector.active_layers].shape)
            
            # Evaluate loss at different distances
            distances = [0.01, 0.1, 0.5, 1.0]
            losses = []
            
            for d in distances:
                perturbed_params = detector.params[detector.active_layers] + d * direction
                loss = detector._volatility_aware_cost(
                    perturbed_params, 
                    windows, 
                    np.array([np.std(w) for w in windows])
                )
                losses.append(loss)
            
            probe_results.append({
                'distances': distances,
                'losses': losses,
                'improvement_found': any(l < current_loss for l in losses)
            })
        
        return {
            'current_loss': current_loss,
            'probes': probe_results,
            'improvement_directions': sum(p['improvement_found'] for p in probe_results),
            'landscape_smoothness': np.mean([np.std(p['losses']) for p in probe_results])
        }
    
    def _calculate_entropy(self, features):
        """Calculate entropy of features to measure information content."""
        entropies = []
        for i in range(features.shape[1]):
            # Discretize features
            hist, _ = np.histogram(features[:, i], bins=20)
            hist = hist + 1e-10  # Avoid log(0)
            hist = hist / hist.sum()
            entropy = -np.sum(hist * np.log(hist))
            entropies.append(entropy)
        return entropies
    
    def _generate_recommendation(self, results):
        """Generate recommendation based on diagnostic results."""
        recommendations = []
        
        # Check gradient health
        if results['gradient_analysis'].get('vanishing_gradient'):
            recommendations.append("ISSUE: Vanishing gradients detected. Consider:")
            recommendations.append("  - Reducing circuit depth")
            recommendations.append("  - Using different initialization")
            recommendations.append("  - Implementing gradient clipping")
        
        # Check learning rate
        lr_results = results['learning_rate_test']
        best_lr = min(lr_results.keys(), key=lambda x: lr_results[x]['final_loss'])
        current_improvement = lr_results[0.01]['loss_improvement']  # Assuming 0.01 is current
        
        if lr_results[best_lr]['loss_improvement'] > current_improvement * 1.5:
            recommendations.append(f"TRY: Learning rate {best_lr} shows better convergence")
        
        # Check feature quality
        feature_quality = results['feature_quality']
        avg_correlation = np.mean(np.abs(feature_quality['correlation_with_volatility']))
        
        if avg_correlation < 0.3:
            recommendations.append("ISSUE: Weak correlation with volatility. Consider:")
            recommendations.append("  - Different encoding strategy")
            recommendations.append("  - More training epochs")
            recommendations.append("  - Different measurement basis")
        
        # Check circuit expressivity
        expressivity = results['circuit_expressivity']
        patterns_learned = sum(p['pattern_learned'] for p in expressivity.values())
        
        if patterns_learned < 2:
            recommendations.append("ISSUE: Limited circuit expressivity. Consider:")
            recommendations.append("  - Adding more layers")
            recommendations.append("  - Different ansatz")
            recommendations.append("  - More entanglement")
        
        # Check loss landscape
        landscape = results['loss_landscape']
        if landscape['improvement_directions'] < 3:
            recommendations.append("ISSUE: Stuck in local minimum. Consider:")
            recommendations.append("  - Random restarts")
            recommendations.append("  - Momentum-based optimization")
            recommendations.append("  - Simulated annealing")
        
        if not recommendations:
            recommendations.append("GOOD: Convergence appears healthy!")
            recommendations.append("The stable loss likely indicates proper convergence.")
        
        return "\n".join(recommendations)
    
    def create_diagnostic_report(self, results, save_path='quantum_diagnostics.png'):
        """Create visual diagnostic report."""
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        
        # 1. Learning rate comparison
        ax = axes[0, 0]
        lr_results = results['learning_rate_test']
        lrs = list(lr_results.keys())
        final_losses = [lr_results[lr]['final_loss'] for lr in lrs]
        improvements = [lr_results[lr]['loss_improvement'] for lr in lrs]
        
        ax.bar(range(len(lrs)), final_losses, alpha=0.7, label='Final Loss')
        ax.bar(range(len(lrs)), improvements, alpha=0.7, label='Improvement')
        ax.set_xticks(range(len(lrs)))
        ax.set_xticklabels([f'LR={lr}' for lr in lrs])
        ax.set_title('Learning Rate Analysis')
        ax.legend()
        
        # 2. Feature quality
        ax = axes[0, 1]
        feature_quality = results['feature_quality']
        correlations = feature_quality['correlation_with_volatility']
        
        ax.bar(range(len(correlations)), correlations)
        ax.set_xlabel('Feature Index')
        ax.set_ylabel('Correlation with Volatility')
        ax.set_title('Feature Quality')
        ax.axhline(y=0.3, color='r', linestyle='--', label='Good correlation threshold')
        ax.legend()
        
        # 3. Circuit expressivity
        ax = axes[1, 0]
        expressivity = results['circuit_expressivity']
        patterns = list(expressivity.keys())
        learned = [expressivity[p]['pattern_learned'] for p in patterns]
        
        ax.bar(range(len(patterns)), learned)
        ax.set_xticks(range(len(patterns)))
        ax.set_xticklabels(patterns, rotation=45)
        ax.set_ylabel('Pattern Learned')
        ax.set_title('Circuit Expressivity')
        
        # 4. Loss landscape
        ax = axes[1, 1]
        landscape = results['loss_landscape']
        probe_data = landscape['probes'][0]  # Show first probe
        
        ax.plot(probe_data['distances'], probe_data['losses'], 'o-')
        ax.axhline(y=landscape['current_loss'], color='r', linestyle='--', label='Current Loss')
        ax.set_xlabel('Distance from Current Parameters')
        ax.set_ylabel('Loss')
        ax.set_title('Loss Landscape Probe')
        ax.set_xscale('log')
        ax.legend()
        
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()
        
        return fig


# How to use this diagnostic tool:
def diagnose_quantum_detector(quantum_detector, ohlc_data):
    """
    Run comprehensive diagnostics on your quantum volatility detector.
    
    Parameters:
    -----------
    quantum_detector : QuantumVolatilityDetector
        Trained quantum detector
    ohlc_data : pd.DataFrame or np.ndarray
        OHLC data (DataFrame with OHLC columns or array of windows)
    """
    diagnostics = QuantumDetectorDiagnostics()
    """
    diagnostics = QuantumDetectorDiagnostics()
    
    # Get returns data
    if hasattr(returns_data, 'values'):
        returns_data = returns_data.values
    """
    
    # Make sure training history has expected keys
    if not hasattr(quantum_detector, 'training_history'):
        quantum_detector.training_history = {}
    
    # Add empty arrays for any missing components
    for key in ['loss', 'gradients', 'parameters']:
        if key not in quantum_detector.training_history:
            quantum_detector.training_history[key] = []
    
    # Run analysis
    results = diagnostics.analyze_training_convergence(quantum_detector, returns_data)
    
    # Print recommendation
    print("\n=== QUANTUM DETECTOR DIAGNOSTIC RESULTS ===")
    print(results['recommendation'])
    
    # Create visual report
    diagnostics.create_diagnostic_report(results, 'quantum_diagnostics.png')
    
    return results

def integrate_diagnostics_into_workflow(quantum_volatility, train_returns):
    """
    Insert this after you've trained the quantum detector but before you use it
    for feature extraction.
    """
    try:
        # Import diagnostic tools
        #from quantum_diagnostics import QuantumDetectorDiagnostics, diagnose_quantum_detector
        
        print("\n=== Running Quantum Detector Diagnostics ===")
        
        # Run diagnostics - pass your trained detector and returns data
        diagnostic_results = diagnose_quantum_detector(
            quantum_volatility,
            train_returns  # This should be your training returns series
        )
        
        # If you want more detailed analysis, you can access specific results:
        print("\nQuantum Circuit Diagnostic Summary:")
        print("-" * 50)
        
        # Feature quality summary
        feature_correlations = diagnostic_results['feature_quality']['correlation_with_volatility']
        avg_correlation = np.mean(np.abs(feature_correlations))
        print(f"Feature quality - Avg. correlation with volatility: {avg_correlation:.4f}")
        
        # Circuit expressivity summary  
        expressivity = diagnostic_results['circuit_expressivity']
        patterns_learned = sum(p['pattern_learned'] for p in expressivity.values())
        print(f"Circuit expressivity: Can learn {patterns_learned}/4 volatility patterns")
        
        # Learning rate recommendation
        lr_results = diagnostic_results['learning_rate_test']
        best_lr = min(lr_results.keys(), key=lambda x: lr_results[x]['final_loss'])
        current_lr = 0.01  # Your default learning rate
        
        if lr_results[best_lr]['final_loss'] < lr_results[current_lr]['final_loss'] * 0.9:
            print(f"Learning rate recommendation: Consider using LR={best_lr} (current: {current_lr})")
        else:
            print(f"Learning rate: Current value ({current_lr}) appears suitable")
        
        # Overall assessment
        if "GOOD:" in diagnostic_results['recommendation']:
            print("\nOVERALL: Quantum detector appears healthy")
        else:
            print("\nOVERALL: Quantum detector may need adjustments")
            
        print("-" * 50)
            
        # The diagnostic results are saved to 'quantum_diagnostics.png'
        print("Detailed visualization saved to 'quantum_diagnostics.png'")
        
        return diagnostic_results
        
    except Exception as e:
        print(f"Warning: Diagnostics failed with error: {e}")
        print("Continuing without diagnostics...")
        return None

In [7]:
class DataPreprocessor:
    """
    Improved version of DataPreprocessor to handle the specific CSV format
    where values are comma-separated within a single column.
    """
    
    def __init__(self, data_folder):
        """
        Initialize the DataPreprocessor with the folder containing CSV files.
        
        Parameters:
        -----------
        data_folder: str
            Path to the folder containing CSV files
        """
        self.data_folder = data_folder
        self.available_files = self._get_available_files()
        self.data_config = {}
        self.master_df = None
        self.features_list = []
        self.target_column = None
        self.training_end_date = None
        self.start_date = None
        self.end_date = None
        
    def _get_available_files(self):
        """List all CSV files in the data folder."""
        # Normalize path to handle both forward and backward slashes
        norm_path = os.path.normpath(self.data_folder)
        files = glob.glob(os.path.join(norm_path, '*.csv'))
        return [os.path.basename(f) for f in files]
    
    def set_config(self, data_config):
        """
        Set the configuration for data loading and preprocessing.
        
        Parameters:
        -----------
        data_config: dict
            Configuration dictionary with the following structure:
            {
                'file_name.csv': {
                    'columns': ['column1', 'column2', ...],
                    'transformations': {'column1': 'raw', 'column2': 'pct_change', ...},
                    'frequency': 'daily' | 'weekly' | 'monthly'
                },
                ...
            }
        """
        self.data_config = data_config
    
    def set_target(self, file_name, column_name, transformation='pct_change'):
        """
        Set the target column for prediction.
        
        Parameters:
        -----------
        file_name: str
            CSV file containing the target column
        column_name: str
            Name of the target column
        transformation: str
            Transformation to apply ('raw', 'pct_change', 'log_return')
        """
        self.target_file = file_name
        self.target_column = column_name
        self.target_transformation = transformation
    
    def set_start_date(self, start_date):
        """
        Set a custom start date for data processing.
        
        Parameters:
        -----------
        start_date: str or datetime
            Start date for data processing (format: 'YYYY-MM-DD')
        """
        if isinstance(start_date, str):
            self.start_date = pd.to_datetime(start_date)
        else:
            self.start_date = start_date

    def set_end_date(self, end_date):
        """
        Set a custom end date for data processing.
        
        Parameters:
        -----------
        end_date: str or datetime
            End date for data processing (format: 'YYYY-MM-DD')
        """
        if isinstance(end_date, str):
            self.end_date = pd.to_datetime(end_date)
        else:
            self.end_date = end_date
    
    def _load_csv(self, file_name):
        """
        Load a CSV file and parse the date column.
        Improved to handle the special case where data is in a single column with
        comma-separated values.
        
        Parameters:
        -----------
        file_name: str
            Name of the CSV file
        
        Returns:
        --------
        pd.DataFrame
            Loaded dataframe with date index
        """
        # Normalize path to handle both forward and backward slashes
        norm_path = os.path.normpath(self.data_folder)
        file_path = os.path.join(norm_path, file_name)
        
        try:
            # First check if this is a single-column CSV with comma-separated values within
            with open(file_path, 'r') as f:
                first_line = f.readline().strip()
            
            if first_line.count(',') > 0 and ',' in first_line:
                # This appears to be a single-column file with comma-separated values
                #print(f"Processing {file_name} as single-column CSV with comma-separated values")
                
                # Read the raw file
                with open(file_path, 'r') as f:
                    lines = f.readlines()
                
                # Parse the header to get column names
                header = lines[0].strip().split(',')
                header = [h.strip() for h in header]  # Clean up any whitespace
                
                # Prepare data for DataFrame
                data = []
                for line in lines[1:]:
                    if line.strip():  # Skip empty lines
                        values = line.strip().split(',')
                        if len(values) >= len(header):
                            data.append(values[:len(header)])  # Ensure we don't exceed header length
                
                # Create DataFrame
                df = pd.DataFrame(data, columns=header)
                
                # Ensure date column is properly formatted
                date_col = header[0]  # Assuming first column is date
                
                # Try different date formats
                try:
                    # Try automatic conversion first
                    df[date_col] = pd.to_datetime(df[date_col])
                except:
                    # If that fails, try specific formats
                    for date_format in ['%Y-%m-%d', '%d/%m/%Y', '%m/%d/%Y', '%Y/%m/%d']:
                        try:
                            df[date_col] = pd.to_datetime(df[date_col], format=date_format)
                            break
                        except ValueError:
                            continue
                
                # Set index to date column
                try:
                    df.set_index(date_col, inplace=True)
                except:
                    print(f"Warning: Could not set {date_col} as index for {file_name}")
                
                # Convert numeric columns to float
                for col in df.columns:
                    try:
                        df[col] = pd.to_numeric(df[col])
                    except ValueError:
                        # Keep as string if conversion fails
                        print(f"Warning: Column {col} in {file_name} could not be converted to numeric")
                
                return df
            
            # If not a special case, try standard CSV loading
            try:
                df = pd.read_csv(file_path, parse_dates=[0], index_col=0)
                return df
            except Exception as e:
                print(f"Standard CSV loading failed for {file_name}: {e}")
                
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            
            # Try alternative approach for standard CSV format
            try:
                df = pd.read_csv(file_path)
                date_col = df.columns[0]
                
                # Try different date formats
                try:
                    df[date_col] = pd.to_datetime(df[date_col])
                except:
                    for date_format in ['%Y-%m-%d', '%d/%m/%Y', '%m/%d/%Y', '%Y/%m/%d']:
                        try:
                            df[date_col] = pd.to_datetime(df[date_col], format=date_format)
                            break
                        except ValueError:
                            continue
                
                df.set_index(date_col, inplace=True)
                return df
                
            except Exception as nested_e:
                print(f"Failed to load {file_name} after multiple attempts: {nested_e}")
                raise
    
    def _apply_transformation(self, df, column, transformation):
        """
        Apply the specified transformation to a column.
        Parameters:
        -----------
        df: pd.DataFrame
            DataFrame containing the column
        column: str
            Column name to transform
        transformation: str or list
            Transformation type ('raw', 'pct_change', 'log_return') or list of transformations
        Returns:
        --------
        list of tuples
            List of (column_name, transformed_series) tuples
        """
        if column not in df.columns:
            print(f"Warning: Column {column} not found in DataFrame")
            return []
        
        # Handle list of transformations
        if isinstance(transformation, list):
            result = []
            for t in transformation:
                column_name = f"{column}_{t}"
                series = self._apply_single_transformation(df, column, t)
                result.append((column_name, series))
            return result
        else:
            # Handle single transformation
            column_name = f"{column}_{transformation}"
            series = self._apply_single_transformation(df, column, transformation)
            return [(column_name, series)]
        
    def _apply_single_transformation(self, df, column, transformation):
        """Apply a single transformation to a column."""
        if transformation == 'raw':
            return df[column]
        elif transformation == 'pct_change':
            return df[column].pct_change() * 100  # Convert to percentage
        elif transformation == 'log_return':
            return np.log(df[column] / df[column].shift(1)) * 100  # Convert to percentage
        else:
            raise ValueError(f"Unknown transformation: {transformation}")
    
    def _resample_and_align(self, dfs_dict, target_freq='D'):
        """
        Resample and align dataframes with different frequencies.
        
        Parameters:
        -----------
        dfs_dict: dict
            Dictionary of DataFrames with their frequencies
            {
                'df_name': {
                    'df': pd.DataFrame,
                    'freq': 'daily' | 'weekly' | 'monthly'
                }
            }
        target_freq: str
            Target frequency for alignment
        
        Returns:
        --------
        pd.DataFrame
            Aligned DataFrame with all features
        """
        freq_map = {
            'daily': 'D',
            'weekly': 'W',
            'monthly': 'M'
        }
        
        # First, resample each dataframe to daily frequency
        resampled_dfs = {}
        
        for name, info in dfs_dict.items():
            df = info['df']
            freq = info['freq']
            
            # Ensure index is datetime
            if not pd.api.types.is_datetime64_any_dtype(df.index):
                try:
                    df.index = pd.to_datetime(df.index)
                except Exception as e:
                    print(f"Error converting index to datetime for {name}: {e}")
                    continue
            
            # If frequency is not daily, resample to daily (forward fill)
            if freq != 'daily':
                # Ensure the index is sorted
                df = df.sort_index()
                
                # Resample to daily frequency
                df_daily = df.asfreq('D')
                
                # Forward fill missing values
                df_daily = df_daily.ffill()
                
                resampled_dfs[name] = df_daily
            else:
                resampled_dfs[name] = df
        
        # Merge all dataframes on date index
        merged_df = None
        
        for name, df in resampled_dfs.items():
            if merged_df is None:
                merged_df = df.copy()
            else:
                merged_df = merged_df.join(df, how='outer')
        
        if merged_df is None:
            raise ValueError("No valid DataFrames to merge")
            
        # Sort by date and forward fill any remaining NaN values
        merged_df = merged_df.sort_index().ffill()
        
        return merged_df

    def _apply_technical_indicators(self, tech_indicators_config, merged_df=None):
        """
        Apply technical indicators to specified CSV files with comprehensive implementation
        of all enhanced metrics from the TechnicalIndicators class.
        
        Parameters:
        -----------
        tech_indicators_config: dict
            Configuration dictionary specifying which indicators to calculate and for which files
        merged_df: pandas.DataFrame, optional
            Merged DataFrame containing all data, needed for derived columns
        
        Returns:
        --------
        dict
            Dictionary containing DataFrames with technical indicators for each file
        """
        
        tech_indicators_dfs = {}
    
        # Process each file in the configuration
        for file_name, config in tech_indicators_config.items():
            if not config.get('include', False):
                continue
                
            # Special handling for derived columns
            if file_name == 'DERIVED_COLUMNS' and merged_df is not None:
                print("Calculating technical indicators for derived columns")
                columns = config.get('columns', [])
                
                for col in columns:
                    if col in merged_df.columns:
                        print(f"Processing derived column: {col}")
                        
                        # Create optimized OHLC format for derived columns
                        derived_df = pd.DataFrame(index=merged_df.index)
                        
                        # Use base value for Close
                        derived_df['Close'] = merged_df[col].copy()
                        
                        # For true OHLC behavior, calculate proper High/Low
                        derived_df['Open'] = derived_df['Close'].shift(1)
                        if len(derived_df) > 0:
                            derived_df['Open'].iloc[0] = derived_df['Close'].iloc[0]
                        
                        derived_df['High'] = pd.concat([derived_df['Open'], derived_df['Close']], axis=1).max(axis=1)
                        derived_df['Low'] = pd.concat([derived_df['Open'], derived_df['Close']], axis=1).min(axis=1)
                        
                        # Use minimal dummy volume
                        derived_df['Volume'] = 1.0
                        
                        # Modified config disabling volume-based indicators
                        derived_config = copy.deepcopy(config.get('indicators', {}))
                        derived_config['obv'] = False
                        derived_config['vroc'] = False
                        derived_config['adl'] = False
                        
                        try:
                            # Calculate technical indicators with customized config
                            indicators_df = TechnicalIndicators.calculate_all_indicators(
                                derived_df, 
                                derived_config
                            )
                            
                            # Create enhanced DataFrame with both original and indicator data
                            enhanced_df = derived_df.copy()
                            for indicator_col in indicators_df.columns:
                                enhanced_df[indicator_col] = indicators_df[indicator_col]
                            
                            # 1. SAR enhanced metrics if Parabolic SAR was calculated
                            sar_cols = [c for c in indicators_df.columns if 'PSAR_' in c and not ('_trend' in c or '_z_score' in c)]
                            if sar_cols:
                                close_cols = ['Close'] * len(sar_cols)
                                enhanced_df = TechnicalIndicators.add_sar_enhanced_metrics(
                                    enhanced_df, sar_cols, close_cols
                                )
                            
                            # 2. MA enhanced metrics
                            fast_ma_cols = [c for c in indicators_df.columns if ('SMA_5_' in c or 'EMA_5_' in c) 
                                            and not ('_trend' in c or '_z_score' in c or '_pct_diff' in c)]
                            slow_ma_cols = [c for c in indicators_df.columns if ('SMA_50_' in c or 'EMA_50_' in c) 
                                            and not ('_trend' in c or '_z_score' in c or '_pct_diff' in c)]
                            
                            if fast_ma_cols and slow_ma_cols:
                                enhanced_df = TechnicalIndicators.add_ma_enhanced_metrics(
                                    enhanced_df, fast_ma_cols, slow_ma_cols
                                )
                            
                            # 3. Oscillator enhanced metrics
                            osc_cols = [c for c in indicators_df.columns if ('RSI_' in c or 'Stochastic_%K_' in c) 
                                        and not ('_trend' in c or '_z_score' in c)]
                            if osc_cols:
                                enhanced_df = TechnicalIndicators.add_oscillator_enhanced_metrics(
                                    enhanced_df, osc_cols
                                )
                            
                            # Skip volume enhanced metrics for derived columns
                            
                            # Extract only the indicators (exclude original OHLC columns)
                            columns_to_exclude = derived_df.columns
                            indicators_df = enhanced_df.drop(columns=columns_to_exclude, errors='ignore')
                            
                            # Check and handle any remaining extreme values
                            for col_name in indicators_df.columns:
                                if indicators_df[col_name].abs().max() > 1e12:
                                    print(f"Warning: Extreme values detected in {col_name}, applying correction")
                                    indicators_df.loc[indicators_df[col_name].abs() > 1e12, col_name] = np.nan
                                    indicators_df[col_name] = indicators_df[col_name].interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')
                            
                            # Add a prefix to indicate which ratio/spread these indicators belong to
                            indicators_df = indicators_df.add_prefix(f"{col}_ind_")
                            
                            # Store the results
                            tech_indicators_dfs[f"{col}_DERIVED"] = indicators_df
                            print(f"Successfully calculated {len(indicators_df.columns)} technical indicators for {col}")
                        except Exception as e:
                            print(f"Error calculating technical indicators for {col}: {e}")
                            import traceback
                            traceback.print_exc()
                    else:
                        print(f"Warning: Derived column {col} not found in dataframe")
                
                continue
                
            # Regular file processing
            print(f"Calculating technical indicators for {file_name}")
                
            # Load CSV file
            try:
                df = self._load_csv(file_name)
                
                # Skip if dataframe is empty
                if df is None or df.empty:
                    print(f"Warning: {file_name} is empty or could not be loaded")
                    continue

                # Apply date filtering to individual file if specified
                if self.start_date is not None or self.end_date is not None:
                    if not pd.api.types.is_datetime64_any_dtype(df.index):
                        df.index = pd.to_datetime(df.index)
                    
                    if self.start_date is not None:
                        df = df[df.index >= self.start_date]
                    if self.end_date is not None:
                        df = df[df.index <= self.end_date]
                    
                    if len(df) == 0:
                        print(f"Warning: {file_name} has no data in the specified date range")
                        continue
                    
                # Fill missing values in OHLC and Volume columns
                ohlcv_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
                for col in ohlcv_columns:
                    if col in df.columns and df[col].isna().any():
                        df[col] = df[col].fillna(method='ffill').fillna(method='bfill')
                
                # Calculate technical indicators
                indicators_df = TechnicalIndicators.calculate_all_indicators(df, config.get('indicators', {}))
                
                # Create enhanced DataFrame with both original and indicator data
                enhanced_df = df.copy()
                for indicator_col in indicators_df.columns:
                    enhanced_df[indicator_col] = indicators_df[indicator_col]
                
                # 1. SAR enhanced metrics
                sar_cols = [c for c in indicators_df.columns if 'PSAR_' in c and not ('_trend' in c or '_z_score' in c)]
                if sar_cols and 'Close' in enhanced_df.columns:
                    close_cols = ['Close'] * len(sar_cols)
                    enhanced_df = TechnicalIndicators.add_sar_enhanced_metrics(
                        enhanced_df, sar_cols, close_cols
                    )
                
                # 2. MA enhanced metrics
                fast_ma_cols = [c for c in indicators_df.columns if ('SMA_5_' in c or 'EMA_5_' in c) 
                               and not ('_trend' in c or '_z_score' in c or '_pct_diff' in c)]
                slow_ma_cols = [c for c in indicators_df.columns if ('SMA_50_' in c or 'EMA_50_' in c) 
                               and not ('_trend' in c or '_z_score' in c or '_pct_diff' in c)]
                
                if fast_ma_cols and slow_ma_cols:
                    enhanced_df = TechnicalIndicators.add_ma_enhanced_metrics(
                        enhanced_df, fast_ma_cols, slow_ma_cols
                    )
                
                # 3. Oscillator enhanced metrics
                osc_cols = [c for c in indicators_df.columns if ('RSI_' in c or 'Stochastic_%K_' in c) 
                           and not ('_trend' in c or '_z_score' in c)]
                if osc_cols:
                    enhanced_df = TechnicalIndicators.add_oscillator_enhanced_metrics(
                        enhanced_df, osc_cols
                    )
                
                # 4. Volume enhanced metrics
                if 'Close' in enhanced_df.columns and 'Volume' in enhanced_df.columns:
                    price_cols = ['Close']
                    volume_cols = ['Volume']
                    enhanced_df = TechnicalIndicators.add_volume_enhanced_metrics(
                        enhanced_df, price_cols, volume_cols
                    )
                
                # Extract only the technical indicators (exclude original data columns)
                columns_to_exclude = df.columns
                indicators_df = enhanced_df.drop(columns=columns_to_exclude, errors='ignore')
                
                # Check and handle any remaining extreme values
                for col in indicators_df.columns:
                    if indicators_df[col].abs().max() > 1e12:
                        print(f"Warning: Extreme values detected in {col}, applying correction")
                        indicators_df.loc[indicators_df[col].abs() > 1e12, col] = np.nan
                        indicators_df[col] = indicators_df[col].interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')
                
                # Store the result
                tech_indicators_dfs[file_name] = indicators_df
                
                print(f"Successfully calculated {len(indicators_df.columns)} technical indicators for {file_name}")
                
            except Exception as e:
                print(f"Error calculating technical indicators for {file_name}: {e}")
                import traceback
                traceback.print_exc()
        
        return tech_indicators_dfs
    
    def _apply_technical_indicators_to_derived(self, config, merged_df):
        """
        Apply technical indicators to derived columns in the merged dataframe with
        proper implementation of all enhanced metrics from TechnicalIndicators class.
        
        Parameters:
        -----------
        config: dict
            Configuration dictionary for derived columns
        merged_df: pandas.DataFrame
            Merged DataFrame containing all data
            
        Returns:
        --------
        pandas.DataFrame
            DataFrame containing calculated indicators for derived columns
        """
        if merged_df is None or merged_df.empty:
            print("Warning: merged_df is empty, skipping derived columns")
            return None
        
        print("Calculating technical indicators for derived columns")
        columns = config.get('columns', [])
        all_indicators = pd.DataFrame(index=merged_df.index)
        
        for col in columns:
            if col in merged_df.columns:
                print(f"Processing derived column: {col}")
                
                # Create optimized OHLC format for derived columns
                derived_df = pd.DataFrame(index=merged_df.index)
                
                # Use base value for Close
                derived_df['Close'] = merged_df[col].copy()
                
                # For Open, use previous Close (shifted values)
                derived_df['Open'] = derived_df['Close'].shift(1)
                # Fill first value with Close to avoid NaN
                if len(derived_df) > 0:
                    derived_df['Open'].iloc[0] = derived_df['Close'].iloc[0]
                
                # Calculate proper High/Low for each period
                derived_df['High'] = pd.concat([derived_df['Open'], derived_df['Close']], axis=1).max(axis=1)
                derived_df['Low'] = pd.concat([derived_df['Open'], derived_df['Close']], axis=1).min(axis=1)
                
                # Use minimal dummy volume but disable volume indicators
                derived_df['Volume'] = 1.0
                
                # Create a modified config that disables volume-based indicators
                derived_config = copy.deepcopy(config.get('indicators', {}))
                derived_config['obv'] = False
                derived_config['vroc'] = False
                derived_config['adl'] = False
                
                try:
                    # Calculate standard indicators
                    indicators_df = TechnicalIndicators.calculate_all_indicators(
                        derived_df, 
                        derived_config
                    )
                    
                    # Create enhanced DataFrame with both original and indicator data
                    enhanced_df = derived_df.copy()
                    for indicator_col in indicators_df.columns:
                        enhanced_df[indicator_col] = indicators_df[indicator_col]
                    
                    # 1. SAR enhanced metrics
                    sar_cols = [c for c in indicators_df.columns if 'PSAR_' in c and not ('_trend' in c or '_z_score' in c)]
                    if sar_cols:
                        close_cols = ['Close'] * len(sar_cols)
                        enhanced_df = TechnicalIndicators.add_sar_enhanced_metrics(
                            enhanced_df, sar_cols, close_cols
                        )
                    
                    # 2. MA enhanced metrics
                    fast_ma_cols = [c for c in indicators_df.columns if ('SMA_5_' in c or 'EMA_5_' in c) 
                                   and not ('_trend' in c or '_z_score' in c or '_pct_diff' in c)]
                    slow_ma_cols = [c for c in indicators_df.columns if ('SMA_50_' in c or 'EMA_50_' in c) 
                                   and not ('_trend' in c or '_z_score' in c or '_pct_diff' in c)]
                    
                    if fast_ma_cols and slow_ma_cols:
                        enhanced_df = TechnicalIndicators.add_ma_enhanced_metrics(
                            enhanced_df, fast_ma_cols, slow_ma_cols
                        )
                    
                    # 3. Oscillator enhanced metrics
                    osc_cols = [c for c in indicators_df.columns if ('RSI_' in c or 'Stochastic_%K_' in c) 
                               and not ('_trend' in c or '_z_score' in c)]
                    if osc_cols:
                        enhanced_df = TechnicalIndicators.add_oscillator_enhanced_metrics(
                            enhanced_df, osc_cols
                        )
                    
                    # 4. Volume enhanced metrics are skipped for derived columns
                    # because we're using dummy volume values which aren't meaningful
                    
                    # Extract only the indicators (exclude original OHLC columns)
                    columns_to_exclude = derived_df.columns
                    indicators_df = enhanced_df.drop(columns=columns_to_exclude, errors='ignore')
                    
                    # Check and handle any remaining extreme values
                    for col_name in indicators_df.columns:
                        if indicators_df[col_name].abs().max() > 1e12:
                            print(f"Warning: Extreme values detected in {col_name}, applying correction")
                            indicators_df.loc[indicators_df[col_name].abs() > 1e12, col_name] = np.nan
                            indicators_df[col_name] = indicators_df[col_name].interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')
                    
                    # Add a prefix to indicate which ratio/spread these indicators belong to
                    indicators_df = indicators_df.add_prefix(f"{col}_ind_")
                    
                    # Combine with other indicators
                    all_indicators = pd.concat([all_indicators, indicators_df], axis=1)
                    
                    print(f"Successfully calculated {len(indicators_df.columns)} technical indicators for {col}")
                except Exception as e:
                    print(f"Error calculating technical indicators for {col}: {e}")
                    import traceback
                    traceback.print_exc()
            else:
                print(f"Warning: Derived column {col} not found in dataframe")
        
        return all_indicators if not all_indicators.empty else None
    
    def process_data(self, training_period_years=5,
                apply_tech_indicators=False,
                tech_indicators_config=None,
                apply_feature_eng=False,
                apply_pca=False,
                pca_components=50):
        """
        Process all data according to the configuration.
        Parameters:
        -----------
        training_period_years: float
            Initial training period in years
        apply_tech_indicators: bool
            Whether to calculate technical indicators
        tech_indicators_config: dict
            Configuration dictionary for technical indicators
        apply_feature_eng: bool
            Whether to apply feature engineering
        apply_pca: bool
            Whether to apply PCA dimensionality reduction
        pca_components: int
            Number of PCA components to retain
        Returns:
        --------
        tuple
            (train_features, train_target, predict_features, predict_target, feature_names)
        """
        print("Starting data processing...")
        # 1. Load all files and apply transformations
        dfs_dict = {}
        for file_name, config in self.data_config.items():
            if file_name not in self.available_files:
                print(f"Warning: {file_name} not found in {self.data_folder}")
                continue
            # Load CSV file
            try:
                df = self._load_csv(file_name)
                # Skip if dataframe is empty
                if df is None or df.empty:
                    print(f"Warning: {file_name} is empty or could not be loaded")
                    continue
                # Apply transformations
                transformed_columns = []
                for column in config['columns']:
                    # Apply specified transformation(s)
                    transformation = config['transformations'].get(column, 'raw')
                    results = self._apply_transformation(df, column, transformation)
                    for col_name, series in results:
                        prefixed_name = f"{file_name[:-4]}_{col_name}"
                        transformed_columns.append((prefixed_name, series))
                # Create a new dataframe with transformed columns
                transformed_df = pd.DataFrame({name: series for name, series in transformed_columns})
                transformed_df.index = df.index
                # Add to dfs_dict
                dfs_dict[file_name] = {
                    'df': transformed_df,
                    'freq': config.get('frequency', 'daily')
                }
            except Exception as e:
                print(f"Error processing {file_name}: {e}")
        if not dfs_dict:
            raise ValueError("No valid data files could be processed")
        
        # 2. Process target separately
        if self.target_file is not None and self.target_column is not None:
            try:
                target_df = self._load_csv(self.target_file)
                if target_df is None or target_df.empty:
                    raise ValueError(f"Target file {self.target_file} is empty or could not be loaded")
                target_series = self._apply_transformation(target_df, self.target_column, self.target_transformation)
                target_name = f"{self.target_file[:-4]}_{self.target_column}_{self.target_transformation}"
            except Exception as e:
                raise ValueError(f"Error processing target: {e}")
        else:
            raise ValueError("Target column not set. Call set_target() before processing data.")
        
        # 3. Align all DataFrames (handle different frequencies)
        try:
            features_df = self._resample_and_align(dfs_dict)
        except Exception as e:
            raise ValueError(f"Error aligning DataFrames: {e}")
        
        # 4. Merge with target
        if isinstance(target_series, list):
            # If we got a list of series, take the first one
            target_name, target_series_obj = target_series[0]
            target_name = f"{self.target_file[:-4]}_{target_name}"
            target_series_obj.name = target_name
            merged_df = features_df.join(target_series_obj, how='inner')
        else:
            target_series.name = target_name
            merged_df = features_df.join(target_series, how='inner')
        
        # 5. Clean data: remove NaN values
        merged_df = merged_df.dropna()

        # 5.1 Apply date filtering EARLY to reduce computational overhead
        if self.start_date is not None or self.end_date is not None:
            print("\n----- Applying Date Filtering (using only the desired temporal period) -----")
            # Ensure index is datetime
            if not pd.api.types.is_datetime64_any_dtype(merged_df.index):
                merged_df.index = pd.to_datetime(merged_df.index)
            
            original_len = len(merged_df)
            
            if self.start_date is not None:
                print(f"Filtering data to start from {self.start_date}")
                merged_df = merged_df[merged_df.index >= self.start_date]
            
            if self.end_date is not None:
                print(f"Filtering data to end at {self.end_date}")
                merged_df = merged_df[merged_df.index <= self.end_date]
            
            if len(merged_df) == 0:
                raise ValueError(f"No data available after date filtering")
            
            print(f"Date range after filtering: {merged_df.index[0]} to {merged_df.index[-1]}")
            print(f"Reduced data from {original_len} to {len(merged_df)} days ({len(merged_df)/original_len*100:.1f}%)")
        
        # 6. Apply BASIC feature engineering FIRST (creates derived columns for technical indicators)
        feature_counts = {
            'original': len(merged_df.columns),
            'added': 0,
            'removed': 0,
            'final': 0
        }
        
        if apply_feature_eng:
            print("\n========== APPLYING BASIC FEATURE ENGINEERING ==========")
    
            # ADD CHECK HERE - right after the print statement above and before any feature engineering logic
            extreme_cols = []
            for col in merged_df.columns:
                if merged_df[col].abs().max() > 1e12:
                    extreme_cols.append(col)
                    
            if extreme_cols:
                print(f"Warning: {len(extreme_cols)} columns with extreme values detected before feature engineering")
                print(f"Example columns: {extreme_cols[:5]}")
                
                for col in extreme_cols:
                    # Replace extreme values with NaN then interpolate
                    merged_df.loc[merged_df[col].abs() > 1e12, col] = np.nan
                    merged_df[col] = merged_df[col].interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')
                
                print("Extreme values handled. Proceeding with feature engineering.")
                
            try:
                # Find column names using flexible pattern matching
                copper_col = next((col for col in merged_df.columns if 'COPPER' in col.upper() and 'raw' in col), None)
                lumber_col = next((col for col in merged_df.columns if 'LUMBER' in col.upper() and 'raw' in col), None)
                gold_col = next((col for col in merged_df.columns if 'GOLD' in col.upper() and 'raw' in col), None)
                us10y_col = next((col for col in merged_df.columns if 'US10Y' in col.upper() and 'raw' in col), None)
                us02y_col = next((col for col in merged_df.columns if 'US02Y' in col.upper() and 'raw' in col), None)
                us03m_col = next((col for col in merged_df.columns if 'US03M' in col.upper() and 'raw' in col), None)
                
                print(f"Found columns for ratio calculations:")
                print(f"  Copper: {copper_col}")
                print(f"  Lumber: {lumber_col}")
                print(f"  Gold: {gold_col}")
                print(f"  US10Y: {us10y_col}")
                print(f"  US02Y: {us02y_col}")
                print(f"  US03M: {us03m_col}")
                
                # Check if required columns exist
                missing_cols = []
                if copper_col is None: missing_cols.append('Copper')
                if lumber_col is None: missing_cols.append('Lumber')
                if gold_col is None: missing_cols.append('Gold')
                if us10y_col is None: missing_cols.append('US10Y')
                if us02y_col is None: missing_cols.append('US02Y')
                if us03m_col is None: missing_cols.append('US03M')
                
                if missing_cols:
                    print(f"Warning: Missing columns for calculations: {', '.join(missing_cols)}")
                    print(f"WARNING: Missing columns for calculations: {', '.join(missing_cols)}")
                    print("WARNING: Feature engineering will be incomplete. Some ratio/spread calculations will be skipped.")
                    print("This may reduce model performance.")
                    print("Attempting to proceed with available columns...")
                
                # Create the features where possible
                added_features = []
                
                # 1. Calculate Copper/Gold ratio and its log return if possible
                if copper_col and gold_col:
                    merged_df["Copper_Gold_Ratio"] = merged_df[copper_col] / merged_df[gold_col]
                    merged_df["Copper_Gold_Ratio_log_return"] = np.log(
                        merged_df["Copper_Gold_Ratio"] / merged_df["Copper_Gold_Ratio"].shift(1)
                    ) * 100
                    added_features.extend(["Copper_Gold_Ratio", "Copper_Gold_Ratio_log_return"])
                
                # 2. Calculate Lumber/Gold ratio and its log return if possible
                if lumber_col and gold_col:
                    merged_df["Lumber_Gold_Ratio"] = merged_df[lumber_col] / merged_df[gold_col]
                    merged_df["Lumber_Gold_Ratio_log_return"] = np.log(
                        merged_df["Lumber_Gold_Ratio"] / merged_df["Lumber_Gold_Ratio"].shift(1)
                    ) * 100
                    added_features.extend(["Lumber_Gold_Ratio", "Lumber_Gold_Ratio_log_return"])
                
                # 3. Calculate yield spreads if possible
                if us10y_col and us02y_col:
                    merged_df["US10Y_US02Y_Spread"] = merged_df[us10y_col] - merged_df[us02y_col]
                    # Also add log returns of the spread for direct model input
                    merged_df["US10Y_US02Y_Spread_diff"] = merged_df["US10Y_US02Y_Spread"].diff()
                    added_features.extend(["US10Y_US02Y_Spread", "US10Y_US02Y_Spread_diff"])
                
                if us10y_col and us03m_col:
                    merged_df["US10Y_US03M_Spread"] = merged_df[us10y_col] - merged_df[us03m_col]
                    # Also add log returns of the spread for direct model input
                    merged_df["US10Y_US03M_Spread_diff"] = merged_df["US10Y_US03M_Spread"].diff()
                    added_features.extend(["US10Y_US03M_Spread", "US10Y_US03M_Spread_diff"])
                
                # Don't drop intermediate ratio columns - we need them for technical indicators               
                if added_features:
                    print(f"Successfully added custom features: {', '.join(added_features)}")
                    feature_counts['added'] += len(added_features)
                else:
                    print("WARNING: No custom features were added. Model will lack important ratio/spread indicators.")
                
            except Exception as e:
                print(f"ERROR in custom feature engineering: {e}")
                import traceback
                traceback.print_exc()
                print("WARNING: Proceeding without custom feature engineering. Model performance may be severely impacted.")
                print("Continuing with existing features...")
        
        # 7. Apply technical indicators AFTER basic feature engineering
        if apply_tech_indicators and tech_indicators_config:
            try:
                print("\n========== CALCULATING TECHNICAL INDICATORS ==========")
        
                # First for regular CSV files
                regular_config = {k: v for k, v in tech_indicators_config.items() 
                                 if k != 'DERIVED_COLUMNS'}
                tech_indicators_dfs = self._apply_technical_indicators(regular_config)
                
                # Then for derived columns - AFTER basic feature engineering has created them
                if 'DERIVED_COLUMNS' in tech_indicators_config:
                    print("\n----- Processing Derived Columns for Technical Indicators -----")
                    derived_cols_config = {'DERIVED_COLUMNS': tech_indicators_config['DERIVED_COLUMNS']}
                    
                    # Verify derived columns exist
                    derived_columns = derived_cols_config['DERIVED_COLUMNS'].get('columns', [])
                    missing_derived = [col for col in derived_columns if col not in merged_df.columns]
                    present_derived = [col for col in derived_columns if col in merged_df.columns]
                    
                    if missing_derived:
                        print(f"Warning: The following derived columns are not found: {missing_derived}")
                    
                    if present_derived:
                        print(f"Found {len(present_derived)} derived columns for technical analysis: {present_derived}")
                        
                        # Update derived column config to only use available columns
                        derived_cols_config['DERIVED_COLUMNS']['columns'] = present_derived
                        
                        # Pass merged_df that now has the derived columns
                        derived_indicators = self._apply_technical_indicators_to_derived(
                            derived_cols_config['DERIVED_COLUMNS'], 
                            merged_df
                        )
                        
                        # Add derived indicators directly to merged_df
                        if derived_indicators is not None:
                            print(f"Adding {len(derived_indicators.columns)} technical indicators from derived columns")
                            merged_df = pd.concat([merged_df, derived_indicators], axis=1)
                    else:
                        print("No derived columns available for technical analysis")
                
                # Add indicators from regular CSVs to merged_df
                for file_name, indicators_df in tech_indicators_dfs.items():
                    # Create a prefix for indicator column names to avoid conflicts
                    prefix = f"{file_name[:-4]}_ind_"
                    # Rename columns to include prefix
                    indicators_df = indicators_df.add_prefix(prefix)
                    # Add to merged_df
                    merged_df = merged_df.join(indicators_df, how='left')
                    print(f"Added {len(indicators_df.columns)} technical indicators from {file_name}")
                
                tech_indicator_count = sum(len(df.columns) for df in tech_indicators_dfs.values())
                if 'DERIVED_COLUMNS' in tech_indicators_config and derived_indicators is not None:
                    tech_indicator_count += len(derived_indicators.columns)
                    
                print(f"Technical indicators calculation completed: Added {tech_indicator_count} indicators")
                feature_counts['added'] += tech_indicator_count
                
            except Exception as e:
                import traceback
                print(f"Error calculating technical indicators: {e}")
                traceback.print_exc()
                print("Continuing with existing features...")
            
            # ADD VALIDATION HERE - immediately after the above code block and before the next section
            print("\n----- Validating technical indicators -----")
            invalid_cols = []
            extreme_value_cols = []
            
            for col in merged_df.columns:
                # Check for all-NaN columns
                if merged_df[col].isna().all():
                    invalid_cols.append(col)
                # Check for infinity values
                elif np.isinf(merged_df[col]).any():
                    print(f"Warning: Infinity values detected in {col}, replacing with NaN and interpolating")
                    merged_df.loc[np.isinf(merged_df[col]), col] = np.nan
                    merged_df[col] = merged_df[col].interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')
                    
                # Check for extreme values
                if merged_df[col].abs().max() > 1e10:
                    extreme_value_cols.append(col)
            
            # Remove completely invalid columns
            if invalid_cols:
                print(f"Removing {len(invalid_cols)} completely invalid columns")
                merged_df = merged_df.drop(columns=invalid_cols)
            
            # Handle extreme values
            if extreme_value_cols:
                print(f"Found {len(extreme_value_cols)} columns with extreme values after technical indicator calculation")
                for col in extreme_value_cols:
                    max_val = merged_df[col].abs().max()
                    print(f"  - {col}: max absolute value = {max_val:.2e}")
                    # Replace extreme values with NaN and interpolate
                    merged_df.loc[merged_df[col].abs() > 1e10, col] = np.nan
                    merged_df[col] = merged_df[col].interpolate(method='linear').fillna(method='ffill').fillna(method='bfill')
                print("Extreme values have been handled")
        
        # 8. Apply ADVANCED feature engineering AFTER technical indicators if requested
        if apply_feature_eng:
            print("\n========== APPLYING ADVANCED FEATURE ENGINEERING ==========")
    
            try:
                # Create feature engineer
                feature_engineer = FeatureEngineer(random_state=42)
                
                # PHASE 1: Normalize various feature types
                print("\n----- Normalization Phase -----")
                
                # 1. Normalize yield values with multiple windows
                yield_columns = [col for col in merged_df.columns if 'US' in col and 'raw' in col]
                if yield_columns:
                    print(f"→ Normalizing yield values: {yield_columns}")
                    orig_count = len(merged_df.columns)
                    merged_df = feature_engineer.normalize_yields(merged_df, yield_columns)
                    feature_counts['added'] += len(merged_df.columns) - orig_count
                
                # 2. Normalize bounded oscillators
                oscillator_pattern = re.compile(r'.*_ind_RSI_.*|.*_ind_Stochastic_.*')
                oscillator_columns = [col for col in merged_df.columns if oscillator_pattern.match(col) and not ('_z_score' in col or '_trend' in col)]
                if oscillator_columns:
                    print(f"→ Normalizing bounded oscillators: {len(oscillator_columns)} columns")
                    orig_count = len(merged_df.columns)
                    merged_df = feature_engineer.normalize_bounded_oscillators(merged_df, oscillator_columns)
                    feature_counts['added'] += len(merged_df.columns) - orig_count
                
                # 3. Normalize yield spreads
                spread_columns = [col for col in merged_df.columns if '_Spread' in col and 'diff' not in col]
                if spread_columns:
                    print(f"→ Normalizing yield spreads: {spread_columns}")
                    orig_count = len(merged_df.columns)
                    merged_df = feature_engineer.normalize_yield_spreads(merged_df, spread_columns)
                    feature_counts['added'] += len(merged_df.columns) - orig_count
                
                # 4. Apply robust scaling to cumulative indicators
                cumulative_columns = [col for col in merged_df.columns if '_ind_OBV' in col or '_ind_AD_Line' in col]
                if cumulative_columns:
                    print(f"→ Applying robust scaling to cumulative indicators: {len(cumulative_columns)} columns")
                    orig_count = len(merged_df.columns)
                    merged_df = feature_engineer.robust_scale_cumulative(merged_df, cumulative_columns)
                    feature_counts['added'] += len(merged_df.columns) - orig_count
                
                # 5. Apply multi-window normalization to volatility indicators
                atr_columns = [col for col in merged_df.columns if '_ind_ATR_' in col]
                if atr_columns:
                    print(f"→ Applying multi-window normalization to ATR: {len(atr_columns)} columns")
                    orig_count = len(merged_df.columns)
                    merged_df = feature_engineer.multi_window_normalization(merged_df, atr_columns)
                    feature_counts['added'] += len(merged_df.columns) - orig_count
                
                # PHASE 2: Add enhanced specialized metrics
                print("\n----- Enhanced Metrics Phase -----")
                
                # 6.1 SAR metrics
                sar_columns = [col for col in merged_df.columns if '_ind_PSAR_' in col and not ('_trend' in col or '_z_score' in col)]
                price_columns = [col.split('_ind_')[0] + '_Close' for col in sar_columns]
                if sar_columns and all(col in merged_df.columns for col in price_columns):
                    print("→ Adding enhanced SAR metrics")
                    # Change to use TechnicalIndicators static method
                    merged_df = TechnicalIndicators.add_sar_enhanced_metrics(merged_df, sar_columns, price_columns)
                
                # 6.2 MA enhanced metrics
                fast_ma_columns = [c for c in merged_df.columns if ('_ind_SMA_5_' in c or '_ind_EMA_5_' in c) 
                                    and not ('_trend' in c or '_z_score' in c or '_pct_diff' in c)]
                slow_ma_columns = [c for c in merged_df.columns if ('_ind_SMA_50_' in c or '_ind_EMA_50_' in c) 
                                    and not ('_trend' in c or '_z_score' in c or '_pct_diff' in c)]
                if fast_ma_columns and slow_ma_columns:
                    print("→ Adding enhanced MA metrics")
                    # Change to use TechnicalIndicators static method
                    merged_df = TechnicalIndicators.add_ma_enhanced_metrics(merged_df, fast_ma_columns, slow_ma_columns)
                
                # 6.3 Oscillator enhanced metrics
                oscillator_columns = [c for c in merged_df.columns if ('_ind_RSI_' in c or '_ind_Stochastic_%K_' in c) 
                                      and not ('_trend' in c or '_z_score' in c)]
                if oscillator_columns:
                    print("→ Adding enhanced oscillator metrics")
                    # Change to use TechnicalIndicators static method
                    merged_df = TechnicalIndicators.add_oscillator_enhanced_metrics(merged_df, oscillator_columns)
                
                # 6.4 Volume enhanced metrics
                volume_columns = [col for col in merged_df.columns if '_Volume' in col]
                price_cols = [col.replace('_Volume', '_Close') for col in volume_columns]
                if volume_columns and all(col in merged_df.columns for col in price_cols):
                    print("→ Adding enhanced volume metrics")
                    # Change to use TechnicalIndicators static method
                    merged_df = TechnicalIndicators.add_volume_enhanced_metrics(merged_df, price_cols, volume_columns)
                
                # PHASE 3: Feature selection - AFTER all features are created but BEFORE PCA
                print("\n----- Feature Selection Phase -----")
                
                # Get all columns except target for feature selection
                all_feature_cols = [col for col in merged_df.columns if col != target_name]
                print(f"Before feature selection: {len(all_feature_cols)} total features available")
                
                # 7. Feature selection
                feature_count_before = len(all_feature_cols)
                selected_features = feature_engineer.select_features(merged_df, verbose=True)
                feature_counts['removed'] = feature_count_before - len(selected_features)
                feature_cols = selected_features
                
                print(f"After feature selection: {len(feature_cols)} features retained")
                
            except Exception as e:
                import traceback
                print(f"Error in feature engineering: {e}")
                traceback.print_exc()
                print("Continuing with original feature set without advanced engineering")
                
                # Save original feature selection for fallback
                original_feature_cols = [col for col in merged_df.columns if col != target_name]
                
                # Use all columns except the target as features
                feature_cols = original_feature_cols.copy()
                
                # Also ensure there's a valid selection stored in feature_engineer
                if 'feature_engineer' in locals():
                    feature_engineer.selected_features = feature_cols
        else:
            # If not applying feature engineering, use all columns except target
            feature_cols = [col for col in merged_df.columns if col != target_name]
        
        # 9. Split into training and prediction periods        
        training_days = int(training_period_years * 252)  # Approximate trading days in year

        if len(merged_df) <= training_days:
            raise ValueError(f"Not enough data points after preprocessing. Got {len(merged_df)}, need at least {training_days}.")
        
        self.training_end_date = merged_df.index[training_days-1]
        print(f"Training end date: {self.training_end_date}")
        
        train_df = merged_df.iloc[:training_days]
        predict_df = merged_df.iloc[training_days:]
        
        # 10. Apply PCA if requested (after ALL feature engineering and train/test split)
        if apply_feature_eng and apply_pca and len(feature_cols) > 0:
            # Store original feature columns before PCA to allow fallback
            original_feature_cols = feature_cols.copy() if isinstance(feature_cols, list) else feature_cols[:]
        
            # Also save a copy of the selected feature columns to use if PCA fails
            original_selected_features = feature_engineer.selected_features.copy() if hasattr(feature_engineer, 'selected_features') and feature_engineer.selected_features else feature_cols.copy()
        
            print("\n----- Dimensionality Reduction Phase -----")
            print(f"→ Applying rolling window PCA to reduce dimensions from {len(feature_cols)} to {pca_components}")
            
            # Initialize tracking variables
            pca_successfully_applied = False
            pca_master_df = None
            train_pca_result = None
            predict_pca_result = None
            
            try:
                # Only apply PCA if we have enough data
                if len(merged_df) > 63:  # At least one window #Previous: 252

                    # Apply PCA to the entire dataset to ensure seamless transition
                    print("→ Applying PCA to entire dataset for seamless transition...")
                    full_pca_result = feature_engineer.improved_fit_transform_pca_with_adaptive_scaling(
                        merged_df,  # Use full dataset
                        feature_cols, 
                        n_components=pca_components, 
                        window_size=63,
                        compute_interval=1,
                        imputation_strategy='ffill',
                        nan_tolerance=0.15,
                        enable_adaptive_scaling=True,
                        scaling_threshold=10.0,
                        scaling_power=0.5
                    )
                    
                    # Split the PCA results according to the original train/predict split
                    train_pca_result = full_pca_result.iloc[:training_days]
                    predict_pca_result = full_pca_result.iloc[training_days:]
                    
                    print("→ PCA application completed with seamless transition")
                    
                    """"
                    # Split data for PCA application
                    print("→ Fitting PCA on training data...")
                    train_pca_result = feature_engineer.improved_fit_transform_pca_with_adaptive_scaling(
                        train_df, 
                        feature_cols, 
                        n_components=pca_components, 
                        window_size=60,               #Previous: 252
                        compute_interval=1,           #Previous: 20
                        imputation_strategy='ffill',
                        nan_tolerance=0.15,
                        enable_adaptive_scaling=True,
                        scaling_threshold=10.0,
                        scaling_power=0.5
                    )
                    
                    print("→ Applying PCA to prediction data...")
                    predict_pca_result = feature_engineer.improved_fit_transform_pca_with_adaptive_scaling(
                        predict_df, 
                        feature_cols, 
                        n_components=pca_components, 
                        window_size=60,               #Previous: 252
                        compute_interval=1,           #Previous: 20
                        imputation_strategy='ffill',
                        nan_tolerance=0.15,
                        enable_adaptive_scaling=True,
                        scaling_threshold=10.0,
                        scaling_power=0.5
                    )
                    """
                    
                    if len(feature_cols) < pca_components:
                        print(f"Warning: Requested {pca_components} components but only have {len(feature_cols)} features")
                        print(f"Reducing number of components to {len(feature_cols)}")
                        pca_components = len(feature_cols)
        
                    # Use PCA components as features
                    pca_cols = train_pca_result.columns.tolist()
                    feature_counts['final'] = len(pca_cols)
                    
                    print(f"→ Created {len(pca_cols)} principal components")
                    
                    # Save feature importance for interpretation
                    top_features = feature_engineer.get_feature_importance(0)
                    if top_features is not None:
                        print("\nTop 15 features contributing to first principal component:")
                        for i, (feature, loading) in enumerate(top_features.head(15).values):
                            print(f"  {i+1}. {feature}: {loading:.4f}")
        
                    # Add this new section for detailed component analysis
                    print("\nAnalyzing top feature contributions for each principal component...")
                    feature_engineer.print_top_component_features(n_components=min(5, pca_components), n_features=10)
                    
                    # Print summary of explained variance
                    if 'last_window' in feature_engineer.pca_transformers:
                        explained_var = feature_engineer.pca_transformers['last_window']['explained_variance_ratio']
                        cumulative_var = np.cumsum(explained_var)
                        print("\nExplained variance by principal components:")
                        print(f"  First component: {explained_var[0]:.4f} ({cumulative_var[0]:.4f} cumulative)")
                        print(f"  First 5 components: {sum(explained_var[:5]):.4f} ({cumulative_var[4]:.4f} cumulative)")
                        print(f"  First 10 components: {sum(explained_var[:10]):.4f} ({cumulative_var[9]:.4f} cumulative)")
                        print(f"  All {pca_components} components: {sum(explained_var):.4f}")
                    
                    # Update feature columns to use PCA components
                    feature_cols = pca_cols
                    
                    # Extract PCA features FIRST
                    train_features = train_pca_result
                    predict_features = predict_pca_result
                    
                    # Update the master_df to include the PCA components
                    try:
                        # Create a new master dataframe with just the PCA components and the target
                        pca_master_df = pd.DataFrame(index=merged_df.index)
                        
                        # Find the target column
                        target_col = target_name
                        
                        # Add the target column to the new master dataframe
                        pca_master_df[target_col] = merged_df[target_col]
                        
                        # Add PCA components from training data - with validation
                        if train_pca_result is not None:
                            for col in train_pca_result.columns:
                                pca_master_df.loc[train_df.index, col] = train_pca_result[col]
                        
                        # Add PCA components from prediction data - with validation
                        if predict_pca_result is not None:
                            for col in predict_pca_result.columns:
                                pca_master_df.loc[predict_df.index, col] = predict_pca_result[col]
                        
                        # Replace the master dataframe with the new one containing PCA components
                        self.master_df = pca_master_df
                        print(f"Successfully created PCA master dataframe with {len(pca_master_df.columns)} columns")
                        
                        # THEN verify PCA results have no extreme values
                        print("\n----- Validating PCA Results -----")
                        extreme_pc_cols = []
                        for col in train_features.columns:
                            max_train_val = train_features[col].abs().max()
                            max_predict_val = predict_features[col].abs().max() if len(predict_features) > 0 else 0
                            max_val = max(max_train_val, max_predict_val)
                            
                            if max_val > 100:
                                extreme_pc_cols.append((col, max_val))
                        
                        if extreme_pc_cols:
                            print("Warning: Some principal components still have extreme values:")
                            for col, val in sorted(extreme_pc_cols, key=lambda x: x[1], reverse=True)[:10]:  # Show top 10 extreme
                                print(f"  - {col}: max abs value = {val:.2f}")
                            
                            print("\nApplying additional safety scaling to extreme components...")
                            # Apply safety scaling to extreme PCs
                            for col, _ in extreme_pc_cols:
                                # Scale down extreme components while preserving directionality
                                scale_factor = train_features[col].abs().max() / 50  # Scale to max abs value of 50
                                if scale_factor > 1:
                                    train_features[col] = train_features[col] / scale_factor
                                    if len(predict_features) > 0:
                                        predict_features[col] = predict_features[col] / scale_factor
                                    print(f"  - Scaled {col} by factor of {scale_factor:.2f}")
                            
                            print("PCA components have been safely scaled")
                        else:
                            print("All principal components have reasonable values (max abs value < 100)")
                        
                        # Mark PCA as successfully applied
                        pca_successfully_applied = True
                        
                    except Exception as e:
                        print(f"Error creating or validating PCA master dataframe: {e}")
                        print("Falling back to original dataframe")
                        pca_successfully_applied = False
                        # We'll handle this in the finally block
                        
                else:
                    print("→ Not enough data for PCA application (need at least 252 data points)")
                    print("→ Using selected features directly")
                    pca_successfully_applied = False
                    
            except Exception as e:
                import traceback
                print(f"Error in PCA application: {e}")
                traceback.print_exc()
                print("Falling back to selected features without PCA")
                pca_successfully_applied = False
                
            finally:
                # Common code to run whether PCA succeeded or failed
                if not pca_successfully_applied:
                    print("Using original features without PCA transformation")
                    
                    # RESET feature_cols to original feature selection (before PCA conversion)
                    feature_cols = original_feature_cols
                    
                    # Safety check - ensure selected columns exist in the dataframe
                    valid_cols = [col for col in feature_cols if col in train_df.columns]
                    if len(valid_cols) < len(feature_cols):
                        print(f"Warning: {len(feature_cols) - len(valid_cols)} selected columns not found in data")
                        feature_cols = valid_cols
                    
                    if not feature_cols:  # If somehow we have no valid features
                        print("No valid features found! Falling back to all features except target")
                        feature_cols = [col for col in train_df.columns if col != target_name]
                    
                    # Extract features using selected columns
                    train_features = train_df[feature_cols]
                    predict_features = predict_df[feature_cols]
                    
                    feature_counts['final'] = len(feature_cols)
                    
                    # Use original merged dataframe
                    self.master_df = merged_df
                
                # Store feature list - this happens regardless of PCA success/failure
                self.features_list = feature_cols
                    
        else:
            # Extract features using feature columns (no PCA attempt)
            train_features = train_df[feature_cols]
            predict_features = predict_df[feature_cols]
            
            if apply_feature_eng:
                feature_counts['final'] = len(feature_cols)
                print(f"\nFinal feature count: {feature_counts['final']} selected features (without PCA)")
            
            # Store master dataframe 
            self.master_df = merged_df
        
        # Extract target values
        train_target = train_df[target_name]
        predict_target = predict_df[target_name]
        
        # Make sure master_df includes the features that are actually used
        # We need to track whether PCA was successfully completed
        pca_successfully_applied = False
        try:
            # Check if PCA was successfully completed by looking for PC columns
            pca_successfully_applied = 'PC_1' in train_features.columns
        except:
            pca_successfully_applied = False
        
        # Update master_df based on whether PCA was successfully applied
        if pca_successfully_applied:
            print("Using PCA-transformed master dataframe")
            # Use the PCA master dataframe created earlier
            # self.master_df = pca_master_df (should already be set)
            
            # Double-check features_list matches PCA columns
            self.features_list = [col for col in train_features.columns]
        else:
            print("Using original feature master dataframe")
            self.master_df = merged_df
            
            # Ensure features_list matches the actual feature columns used
            self.features_list = feature_cols
        
        print(f"Processed {len(train_features)} training samples and {len(predict_features)} prediction samples")
        print(f"Features: {len(feature_cols)} columns")
        
        # If feature engineering was applied, print summary
        if apply_feature_eng:
            print("\nFeature Engineering Summary:")
            print(f"  Original features: {feature_counts['original']}")
            print(f"  Added features: {feature_counts['added']}")
            print(f"  Removed features: {feature_counts['removed']}")
            print(f"  Final feature count: {feature_counts['final']}")
            print("\n========== FEATURE ENGINEERING COMPLETE ==========")
        
        return (train_features, train_target, predict_features, predict_target, feature_cols)
        
    
    def get_daily_prediction_data(self):
        """
        Get data for sequential prediction (day by day).
        
        Returns:
        --------
        dict
            Dictionary with dates as keys and feature values as dictionary
        """
        if self.master_df is None or self.training_end_date is None:
            raise ValueError("Data not processed yet. Call process_data() first.")
        
        # Get data after training period
        if self.end_date is not None:
            # Respect both training end and user-specified end date
            predict_df = self.master_df[(self.master_df.index > self.training_end_date) & 
                                        (self.master_df.index <= self.end_date)].copy()
        else:
            predict_df = self.master_df[self.master_df.index > self.training_end_date].copy()
        
        # Create dictionary with date as key
        prediction_data = {}
        
        # First ensure our feature list is valid for the master_df
        if self.features_list is None or len(self.features_list) == 0:
            print("Warning: No features list found. Using all non-target columns.")
            # Determine target column - assume it's the only column not used as a feature
            all_non_feature_cols = [col for col in self.master_df.columns]
            # Guess the target column - usually the only column not in features_list or has 'target' in name
            target_candidates = [col for col in all_non_feature_cols 
                               if 'target' in col.lower() or 'return' in col.lower()]
            if target_candidates:
                target_col = target_candidates[0]
                feature_cols = [col for col in all_non_feature_cols if col != target_col]
            else:
                # If no obvious target, use the last column as target (common convention)
                feature_cols = all_non_feature_cols[:-1]
                target_col = all_non_feature_cols[-1]
            
            print(f"Auto-detected target column: {target_col}")
            self.features_list = feature_cols
        else:
            feature_cols = self.features_list
        
        # Verify all feature columns exist in master_df
        missing_cols = [col for col in feature_cols if col not in self.master_df.columns]
        if missing_cols:
            print(f"Warning: {len(missing_cols)} feature columns missing from master_df")
            print(f"Examples: {missing_cols[:5]}")
            
            # Filter out missing columns
            valid_cols = [col for col in feature_cols if col in self.master_df.columns]
            if not valid_cols:
                raise ValueError("No valid feature columns found. Cannot proceed.")
                
            print(f"Proceeding with {len(valid_cols)} valid feature columns")
            feature_cols = valid_cols
            self.features_list = valid_cols  # Update our features_list
        
        # Log information for debugging
        print(f"Master DF columns: {len(self.master_df.columns)} columns")
        print(f"Feature columns: {len(feature_cols)} columns")
        
        # Find the target column - non-PC column if using PCA
        non_feature_cols = [col for col in self.master_df.columns if col not in feature_cols]
        if not non_feature_cols:
            raise ValueError("No target column found in master_df")
            
        target_col = non_feature_cols[0]
        print(f"Using {target_col} as target column")
            
        # Find the target column - non-PC column if using PCA
        non_feature_cols = [col for col in self.master_df.columns if col not in feature_cols]
        if non_feature_cols:
            target_col = non_feature_cols[0]
        else:
            raise ValueError("No target column found in master_df")
            
        """
        for date, row in predict_df.iterrows():
            prediction_data[date] = {
                'features': row[feature_cols].to_dict(),
                'actual_return': row[target_col]
            }
        """
        prev_row = None
        for date, row in predict_df.iterrows():
            if prev_row is not None:  # Skip first day
                prediction_data[date] = {
                    'features': prev_row[feature_cols].to_dict(),  # Day t-1 features
                    'actual_return': row[target_col]               # Day t return
                }
            prev_row = row
        
        print(f"Generated {len(prediction_data)} prediction days (lost 1 day due to feature lag)")
        
        return prediction_data

    def get_raw_ohlc_data(self):
        """
        Get raw OHLC data before any transformations for the target file.
        
        Returns:
        --------
        pd.DataFrame: DataFrame with raw OHLC columns and date index
        """
        if not hasattr(self, 'target_file') or self.target_file is None:
            raise ValueError("Target file not set. Call set_target() first.")
        
        # Load the raw CSV file
        raw_df = self._load_csv(self.target_file)

        # Apply date filtering if specified
        if self.start_date is not None or self.end_date is not None:
            if not pd.api.types.is_datetime64_any_dtype(raw_df.index):
                raw_df.index = pd.to_datetime(raw_df.index)
            
            if self.start_date is not None:
                raw_df = raw_df[raw_df.index >= self.start_date]
            if self.end_date is not None:
                raw_df = raw_df[raw_df.index <= self.end_date]
            
            if len(raw_df) == 0:
                raise ValueError("No OHLC data available in the specified date range")
        
        # Verify OHLC columns exist
        required_cols = ['Open', 'High', 'Low', 'Close']
        missing_cols = [col for col in required_cols if col not in raw_df.columns]
        if missing_cols:
            raise ValueError(f"Missing OHLC columns in {self.target_file}: {missing_cols}")
        
        # Return only OHLC columns with date index
        return raw_df[required_cols]

    def get_target_column(self):
        """
        Get the target column as a pandas Series.
        
        Returns:
        --------
        pandas.Series
            The target column series from the master dataframe
        """
        if self.master_df is None:
            raise ValueError("Data not processed yet. Call process_data() first.")
        
        # Construct the expected target column name based on how it's created in process_data
        target_name = f"{self.target_file[:-4]}_{self.target_column}_{self.target_transformation}"
        
        # First try: exact match on expected target name
        if target_name in self.master_df.columns:
            return self.master_df[target_name]
        
        # Second try: look for columns matching the pattern (handles list transformations)
        prefix = f"{self.target_file[:-4]}_{self.target_column}_"
        matching_cols = [col for col in self.master_df.columns if col.startswith(prefix)]
        if matching_cols:
            return self.master_df[matching_cols[0]]
        
        # Final fallback: identify the target as the only column not in features_list
        # This matches the approach used in plot_data_overview and get_daily_prediction_data
        if hasattr(self, 'features_list') and self.features_list:
            non_feature_cols = [col for col in self.master_df.columns if col not in self.features_list]
            if non_feature_cols:
                print(f"Using {non_feature_cols[0]} as target column (fallback method)")
                return self.master_df[non_feature_cols[0]]
        
        raise ValueError("Target column not available. Make sure set_target has been called and process_data has been run.")
            
    def plot_data_overview(self):
        """
        Plot an overview of the processed data to help with visualization.
        
        Returns:
        --------
        matplotlib.figure.Figure
            Matplotlib figure object
        """
        if self.master_df is None:
            raise ValueError("Data not processed yet. Call process_data() first.")
        
        target_col = [col for col in self.master_df.columns if col not in self.features_list][0]
        
        fig, axes = plt.subplots(3, 1, figsize=(12, 15))
        
        # Plot target variable
        axes[0].plot(self.master_df.index, self.master_df[target_col], 'b-')
        axes[0].set_title(f'Target Variable: {target_col}')
        axes[0].set_xlabel('Date')
        axes[0].set_ylabel('Value')
        axes[0].grid(True, alpha=0.3)
        
        # Plot correlation matrix of a subset of features (first 10)
        n_features = min(10, len(self.features_list))
        selected_features = self.features_list[:n_features] + [target_col]
        corr = self.master_df[selected_features].corr()
        
        # Plot as heatmap
        im = axes[1].imshow(corr, cmap='coolwarm', vmin=-1, vmax=1)
        axes[1].set_title('Correlation Matrix (Top 10 Features)')
        axes[1].set_xticks(np.arange(len(selected_features)))
        axes[1].set_yticks(np.arange(len(selected_features)))
        axes[1].set_xticklabels(selected_features, rotation=90)
        axes[1].set_yticklabels(selected_features)
        
        # Add colorbar
        cbar = fig.colorbar(im, ax=axes[1])
        cbar.set_label('Correlation')
        
        # Plot data availability
        # Count non-NaN values per day
        availability = self.master_df.count(axis=1)
        axes[2].plot(self.master_df.index, availability, 'g-')
        axes[2].set_title('Data Availability (Features per Day)')
        axes[2].set_xlabel('Date')
        axes[2].set_ylabel('Features Available')
        axes[2].grid(True, alpha=0.3)
        
        # Add a vertical line at training end date
        if self.training_end_date is not None:
            for ax in axes:
                ax.axvline(self.training_end_date, color='r', linestyle='--', 
                         label='Training End')
            axes[0].legend()
        
        plt.tight_layout()
        return fig

# 2. Data Processing

# 3. DBN Class

In [8]:
class StockMarketDBN:
    """
    Dynamic Bayesian Network for stock market prediction with anti-
    stagnation mechanisms and continuous hidden states.
    """
    def __init__(
        self,
        features_list,
        target='sp500_return',
        hidden_layers=0,
        states_per_hidden=3,
        continuous_states=False,  # Enable continuous states
        state_dimension=2,        # Dimension for continuous states
        master_node=False,
        inference_method='particle',  # Standardized to 'particle' instead of 'particle_filter'
        prediction_range=(-0.2, 0.2),
        prediction_bins=1001,
        n_particles=10000,
        random_state=42,
        # Anti-stagnation parameters
        enable_anti_stagnation=True,
        stagnation_window=30,
        stagnation_threshold=0.95,
        adaptive_learning=True,
        base_learning_rate=0.1, # Was 0.01 before
        max_learning_rate=0.3,   # Was 0.1 before
        particle_rejuvenation=True,
        weight_regularization=0.0001, 
        
        # Parameters for handling extreme PCs
        pc_value_limit=30.0,  # Maximum absolute value for feature contributions
        feature_contribution_scaling=True,  # Enable automatic scaling of extreme feature contributions
        
        # Forgetting Factor (any value other than 1.0 leads to problems)
        forgetting_factor=1.0, 

        # Scale of process noise for continuous states
        state_noise_scale=0.01    
    ):
        """
        Initialize the DBN model with anti-stagnation mechanisms and continuous states.
        """
        
        self.features = features_list
        self.n_features = len(features_list)
        self.target = target
        self.hidden_layers = hidden_layers
        self.states_per_hidden = states_per_hidden
        
        self.forgetting_factor = forgetting_factor  
        
        # New parameters for continuous states
        self.continuous_states = continuous_states
        #self.state_dimension = state_dimension if continuous_states else 0
        self.state_dimension = state_dimension
        self.state_noise_scale = state_noise_scale
        
        self.master_node = master_node
        self.inference_method = inference_method
        self.pred_min, self.pred_max = prediction_range
        self.prediction_bins = prediction_bins
        self.n_particles = n_particles
        self.random_state = random_state
        
        # Anti-stagnation parameters
        self.enable_anti_stagnation = enable_anti_stagnation
        self.stagnation_window = stagnation_window
        self.stagnation_threshold = stagnation_threshold
        self.adaptive_learning = adaptive_learning
        self.base_learning_rate = base_learning_rate
        self.max_learning_rate = max_learning_rate
        self.particle_rejuvenation = particle_rejuvenation
        self.weight_regularization = weight_regularization
        
        # History tracking for anti-stagnation
        self.prediction_history = []  # Store recent predictions
        self.learning_rate = base_learning_rate  # Current learning rate
        self.consecutive_same_direction = 0  # Counter for same direction predictions
        
        # Initialize random number generator
        self.rng = np.random.RandomState(random_state)
        
        # Create bins for prediction distribution
        self.bins = np.linspace(self.pred_min, self.pred_max, self.prediction_bins)
        self.bin_centers = (self.bins[:-1] + self.bins[1:]) / 2
        
        # Initialize model parameters
        self._initialize_model()

        # Counter for tracking fallbacks in distribution calculation
        self.constrained_mean_count = 0
        
        # Debug and tracking variables
        self.debug_info = {
            'steps_since_update': 0,
            'stagnation_detected': False,
            'current_learning_rate': self.base_learning_rate,
            'rejuvenation_applied': False,
            'weight_norm': 0.0,
            'rejuvenation_strength': 0.0,
            'particles_rejuvenated': 0,
            'constrained_mean_count': 0
        }
        
        # Store original feature names for validation
        self.original_feature_names = features_list.copy()

        # Adaptive normalization parameters (Continuous recalculation of Normalization)
        self.adaptive_normalization = True  # Enable adaptive normalization
        self.normalization_window = 252     # 1-year rolling window (Data the model uses for std and mean calculation
        self.feature_history = []           # Store recent feature values
        self.initial_training_done = False  # Track if initial training is complete

        # Add new parameters for handling extreme values in features
        self.pc_value_limit = pc_value_limit
        self.feature_contribution_scaling = feature_contribution_scaling
        self.max_feature_contributions = {}  # Track maximum contribution of each feature

        # Add validation to ensure forgetting factor is set corretly
        assert 0.0 <= self.forgetting_factor <= 1.0, f"Invalid forgetting factor: {self.forgetting_factor}"
        
        print(f" Forgetting factor set to: {self.forgetting_factor}")
        if self.forgetting_factor == 1.0:
            print("   NO FORGETTING - Perfect memory mode")
        else:
            half_life = np.log(0.5) / np.log(self.forgetting_factor)
            print(f"   Half-life: {half_life:.1f} days")
    
    def _initialize_model(self):
        """Initialize model parameters and structure."""
        import numpy as np
        
        self.feature_means = np.zeros(self.n_features)
        self.feature_stds = np.ones(self.n_features)
        
        # Parameters for the transition model
        if self.hidden_layers > 0:
            if self.continuous_states:
                # Initialize continuous state parameters
                self.state_transition_matrices = []  # A matrices
                self.state_transition_biases = []    # b vectors
                self.state_transition_noise = []     # Q covariance matrices
                
                for _ in range(self.hidden_layers):
                    # Create state transition matrix A (carefully initialize for stability)
                    A = self.rng.normal(0, 0.1, (self.state_dimension, self.state_dimension))
                    
                    # Ensure stability by controlling eigenvalues to prevent explosive dynamics
                    eigvals = np.linalg.eigvals(A)
                    if np.max(np.abs(eigvals)) > 0.97:  # Allow dynamics to be persistent but not explosive
                        A = A * (0.97 / np.max(np.abs(eigvals)))
                    
                    # Create diagonal matrix if stability control fails
                    if not np.all(np.isfinite(A)):
                        A = np.eye(self.state_dimension) * 0.9
                        
                    self.state_transition_matrices.append(A)
                    
                    # Bias vector for state transitions
                    b = self.rng.normal(0, 0.01, self.state_dimension)
                    self.state_transition_biases.append(b)
                    
                    # Process noise covariance matrix (typically diagonal)
                    Q = np.eye(self.state_dimension) * self.state_noise_scale
                    self.state_transition_noise.append(Q)
                
                # Emission model parameters (how hidden states generate observed features)
                self.emission_weights = []  # C matrices - mapping from states to features
                self.emission_biases = []   # d vectors - feature biases
                self.emission_noise = []    # R vectors - diagonal observation noise
                
                for _ in range(self.hidden_layers):
                    # Create weights mapping state to features (C matrix)
                    # Each feature is linear combination of state components
                    W = self.rng.normal(0, 0.1, (self.n_features, self.state_dimension))
                    self.emission_weights.append(W)
                    
                    # Bias terms for each feature
                    b = self.rng.normal(0, 0.01, self.n_features)
                    self.emission_biases.append(b)
                    
                    # Diagonal noise covariance for each feature
                    # Using exp to ensure positive values
                    noise = np.exp(self.rng.normal(0, 0.1, self.n_features))
                    self.emission_noise.append(noise)
                
                # Initialize state prediction matrices for target
                self.target_state_weights = []
                for _ in range(self.hidden_layers):
                    # Weights from state dimensions to target
                    weights = self.rng.normal(0, 0.1, self.state_dimension)
                    self.target_state_weights.append(weights)

                # Initialize RLS parameters for continuous states
                self.rls_params = []
                for _ in range(self.hidden_layers):
                    # For each layer, store P (inverse correlation matrix)
                    d = self.state_dimension
                    P = np.eye(d + 1) * 100.0  # High initial uncertainty
                    self.rls_params.append({
                        'P': P,
                        'lambda': self.forgetting_factor  # Use consistent forgetting factor
                    })
            else:
                # Original discrete state initialization
                self.hidden_transitions = []
                for _ in range(self.hidden_layers):
                    # Random transition matrix for hidden states
                    trans_matrix = self.rng.dirichlet(
                        np.ones(self.states_per_hidden) * 2,  # Alpha=2 for more stability
                        size=self.states_per_hidden
                    )
                    self.hidden_transitions.append(trans_matrix)

                # Initialize transition counts for time-varying transitions
                self.transition_counts = []
                for _ in range(self.hidden_layers):
                    self.transition_counts.append(
                        np.ones((self.states_per_hidden, self.states_per_hidden)) * 0.1
                    )
                
                # Initialize emission parameters for hidden states
                self.emission_means = []
                self.emission_stds = []
                for _ in range(self.hidden_layers):
                    # For each hidden state, define a mean vector for features
                    means = self.rng.normal(0, 1, (self.states_per_hidden, self.n_features))
                    self.emission_means.append(means)
                    
                    # For each hidden state, define a std vector for features
                    stds = np.exp(self.rng.normal(0, 0.1, (self.states_per_hidden, self.n_features)))
                    self.emission_stds.append(stds)
        
                # Parameters for target prediction with discrete states
                self.target_hidden_weights = []
                for _ in range(self.hidden_layers):
                    # Weights for each hidden state to target
                    weights = self.rng.normal(0, 0.1, self.states_per_hidden)
                    self.target_hidden_weights.append(weights)
        
        # Parameters for target prediction (common to both discrete and continuous)
        self.target_weights = self.rng.normal(0, 0.1, self.n_features)
        self.target_bias = 0.0
        self.target_std = 1.0
        
        # Initialize particles for inference
        if self.inference_method == 'particle':
            self.particles = None
            self.particle_weights = None
            self._initialize_particles()
    
    def _initialize_particles(self):
        """Initialize particles for particle filtering with support for continuous states."""
        import numpy as np
        
        if self.hidden_layers > 0:
            # Initialize particles for hidden states
            self.particles = []
            
            for i in range(self.hidden_layers):
                if self.continuous_states:
                    # For continuous states, initialize with random samples
                    # Using normal distribution centered at origin
                    hidden_particles = self.rng.normal(
                        0, 1, (self.n_particles, self.state_dimension)
                    )
                    self.particles.append(hidden_particles)
                else:
                    # Original discrete state initialization
                    # Sample initial hidden states from uniform distribution
                    hidden_particles = self.rng.choice(
                        self.states_per_hidden,
                        size=self.n_particles,
                        p=np.ones(self.states_per_hidden) / self.states_per_hidden
                    )
                    self.particles.append(hidden_particles)
            
            # Initialize particle weights (uniform initial distribution)
            self.particle_weights = np.ones(self.n_particles) / self.n_particles

    def _standardize_array(self, arr, context="unknown"):
        """
        Ensures array is a standard numpy array with proper dimensions for continuous states and logs non-standard types.
        
        Parameters:
        -----------
        arr: array-like
            Array to standardize
        context: str
            Location in code where standardization is happening
            
        Returns:
        --------
        numpy.ndarray
            Standardized array
        """
        import numpy as np
    
        if arr is None:
            return None
        
        # For continuous states, enforce vector representation
        if self.continuous_states and hasattr(self, 'state_dimension'):
            # First convert to numpy array if needed
            if not isinstance(arr, np.ndarray):
                try:
                    arr = np.asarray(arr)
                    print(f"DIAGNOSTIC: Converted {type(arr).__name__} to numpy array in {context}")
                except Exception as e:
                    print(f"ERROR: Failed to convert to numpy array in {context}: {e}")
                    # Create valid replacement
                    return np.zeros(self.state_dimension)
            
            # Now validate and fix dimensions
            if arr.size == 1:  # Single value (scalar or 1-element array)
                print(f"ERROR: Scalar value found in {context} - must be vector for continuous states")
                return np.full(self.state_dimension, float(arr))
            elif len(arr.shape) == 1 and arr.shape[0] != self.state_dimension:
                print(f"ERROR: Vector with wrong dimension in {context}: {arr.shape[0]}, expected {self.state_dimension}")
                # Create valid replacement if dimensions don't match
                return np.zeros(self.state_dimension)
            elif len(arr.shape) > 1:
                # More than 1 dimension, check if it can be reshaped
                if arr.size == self.state_dimension:
                    print(f"DIAGNOSTIC: Reshaping {arr.shape} array to ({self.state_dimension},) in {context}")
                    return arr.reshape(self.state_dimension)
                else:
                    print(f"ERROR: Multi-dimensional array with incompatible size in {context}: {arr.shape}")
                    return np.zeros(self.state_dimension)
            
            # If dimensions are already correct, ensure it's a standard array (not a view)
            if arr.base is not None:
                return arr.copy()
            return arr
        
        # For non-continuous states or when state_dimension isn't defined yet
        # Check if already a standard numpy array (not a view or matrix)
        if isinstance(arr, np.ndarray) and not isinstance(arr, np.matrix) and arr.base is None:
            return arr
                
        # For array views, matrices, or other array-like objects
        original_type = type(arr).__name__
        try:
            std_arr = np.asarray(arr).copy()
            
            # Log if non-standard array detected
            if original_type != 'ndarray':
                print(f"DIAGNOSTIC: Non-standard array type '{original_type}' converted to standard numpy array in {context}")
            
            return std_arr
        except Exception as e:
            print(f"WARNING: Error standardizing array in {context}: {e}, type={original_type}")
            # Try another approach for unknown types
            try:
                return np.array(list(arr))
            except:
                print(f"CRITICAL: Unable to standardize array in {context}")
                return arr  # Return original as last resort
        """
        if arr is None:
            return None
            
        # Check if already a standard numpy array (not a view or matrix)
        if isinstance(arr, np.ndarray) and not isinstance(arr, np.matrix) and arr.base is None:
            return arr
            
        # For array views, matrices, or other array-like objects
        original_type = type(arr).__name__
        try:
            std_arr = np.asarray(arr).copy()
            
            # Log if non-standard array detected
            if original_type != 'ndarray':
                print(f"DIAGNOSTIC: Non-standard array type '{original_type}' converted to standard numpy array in {context}")
            
            return std_arr
        except Exception as e:
            print(f"WARNING: Error standardizing array in {context}: {e}, type={original_type}")
            # Try another approach for unknown types
            try:
                return np.array(list(arr))
            except:
                print(f"CRITICAL: Unable to standardize array in {context}")
                return arr  # Return original as last resort
    """
            
    
    def _rejuvenate_particles(self, strength=0.05):
        """
        Rejuvenate particles with robust indexing protection.
        Supports both discrete and continuous hidden states.
        """
        import numpy as np
        
        if not self.particle_rejuvenation or self.hidden_layers == 0 or self.particles is None:
            return
            
        total_rejuvenated = 0
        rejuvenation_failures = 0

        # Calculate diversity of particles
        diversity = 0
        if not self.continuous_states:
            # For discrete states, use unique states ratio
            for particles in self.particles:
                unique_states = len(np.unique(particles))
                diversity += unique_states / self.states_per_hidden
            
            diversity = diversity / len(self.particles)  # Average across layers
        else:
            # For continuous states, measure diversity using variance
            for particles in self.particles:
                # Don't standardize the entire particle array - calculate variance directly
                # particles has shape (n_particles, state_dimension)
                state_variance = np.mean(np.var(particles, axis=0))
                diversity += min(1.0, state_variance / 0.5)  # Normalize variance
            
            diversity = diversity / len(self.particles)
        
        # Adapt rejuvenation strength based on diversity (less diverse -> more rejuvenation)
        adaptive_strength = min(0.5, 0.25 * (1.0 - diversity))
        
        # Use maximum of provided strength and adaptive strength
        strength = max(strength, adaptive_strength)

        """
        Li, T., Bolic, M., & Djuric, P. M. (2015). "Resampling Methods for Particle Filtering: Classification, Implementation, and Strategies." IEEE Signal Processing Magazine, 32(3), 70-86.
        Discusses importance of particle diversity in sequential Monte Carlo methods        
        
        Arulampalam, M. S., et al. (2002). "A tutorial on particle filters for online nonlinear/non-Gaussian Bayesian tracking." IEEE Transactions on Signal Processing, 50(2), 174-188.
        Foundational work on particle filtering highlighting diversity maintenance techniques
        """
    
        for layer_idx, particles in enumerate(self.particles):
            try:
                # Check shape for each layer, but only print if problematic
                try:
                    # Expected shape varies based on state type
                    if self.continuous_states:
                        expected_shape = (self.n_particles, self.state_dimension)
                    else:
                        expected_shape = (self.n_particles,)  # For discrete states
                    
                    # Check if particles has shape attribute
                    if not hasattr(particles, 'shape'):
                        print(f"ERROR-TRACE: In _rejuvenate_particles - particles has no shape attribute, layer={layer_idx}, type={type(particles)}")
                    # Check if shape matches expected
                    elif particles.shape != expected_shape:
                        print(f"ERROR-TRACE: particles has wrong shape in layer {layer_idx}: {particles.shape}, expected {expected_shape}")
                except Exception as e:
                    print(f"ERROR-TRACE: Shape check error in _rejuvenate_particles, layer={layer_idx}: {e}")
                
                # Check for non-finite values safely
                try:
                    # This line is error-prone, need to check boolean context properly
                    has_non_finite = np.any(~np.isfinite(particles))
                    if has_non_finite:
                        # Only print if issues found
                        print(f"ERROR-TRACE: Non-finite values found in particles, layer {layer_idx}")
                except Exception as e:
                    print(f"ERROR-TRACE: Array truth error likely here in _rejuvenate_particles, layer={layer_idx}: {e}")
                    print(f"ERROR-TRACE: particles type = {type(particles)}")
            except Exception as e:
                print(f"ERROR-TRACE: Exception in _rejuvenate_particles for layer {layer_idx}: {e}")
                
            try:
                # Determine which particles to rejuvenate
                mask = self.rng.random(self.n_particles) < strength
                mask_indices = np.where(mask)[0]  # Get explicit indices
                n_to_rejuvenate = len(mask_indices)
                
                if n_to_rejuvenate > 0:
                    if self.continuous_states:
                        # For continuous states, add noise to rejuvenate
                        
                        # Calculate scale of noise based on current particle distribution
                        state_std = np.std(particles, axis=0)
                        state_std = np.maximum(state_std, 0.01)  # Ensure positive std
                        
                        # Generate rejuvenation noise (scaled by state std)
                        rejuv_noise = self.rng.normal(
                            0, state_std * strength * 2.0, 
                            (n_to_rejuvenate, self.state_dimension)
                        )
                        
                        # Apply noise to selected particles
                        for i, idx in enumerate(mask_indices):
                            # No need to standardize - particles[idx] is already the right format
                            particles[idx] = particles[idx] + rejuv_noise[i]
                    else:
                        # Original discrete state rejuvenation
                        # Generate new states
                        new_states = self.rng.choice(
                            self.states_per_hidden,
                            size=n_to_rejuvenate,
                            p=np.ones(self.states_per_hidden) / self.states_per_hidden
                        )
                    
                        # Assign using explicit indices
                        for i, idx in enumerate(mask_indices):
                            particles[idx] = new_states[i]
                    
                    total_rejuvenated += n_to_rejuvenate
                
            except Exception as e:
                rejuvenation_failures += 1
                print(f"WARNING: Rejuvenation failed for layer {layer_idx}: {e}")
                
                # Minimal intervention: reinitialize this layer's particles if needed
                if self.continuous_states:
                    try:
                        # Use proper logic to check for validity
                        has_nan = np.any(np.isnan(particles))
                        has_inf = np.any(np.isinf(particles))
                        wrong_shape = particles.shape != (self.n_particles, self.state_dimension)
                        
                        if has_nan or has_inf or wrong_shape:
                            print(f"WARNING: Invalid particles detected in layer {layer_idx}. Resetting layer.")
                            # Create a completely new array instead of in-place modification
                            self.particles[layer_idx] = self.rng.normal(0, 1, (self.n_particles, self.state_dimension))
                    except Exception as inner_e:
                        # Handle any errors in the check itself
                        print(f"WARNING: Critical error in particles for layer {layer_idx}. Complete reset. Error: {inner_e}")
                        self.particles[layer_idx] = self.rng.normal(0, 1, (self.n_particles, self.state_dimension))
                else:
                    try:
                        # Proper check for discrete states
                        has_nan = np.any(np.isnan(particles))
                        wrong_shape = particles.shape != (self.n_particles,)
                        
                        if has_nan or wrong_shape:
                            print(f"WARNING: Invalid particles detected in layer {layer_idx}. Resetting layer.")
                            # Create a completely new array instead of in-place modification
                            self.particles[layer_idx] = self.rng.randint(0, self.states_per_hidden, size=self.n_particles)
                    except Exception as inner_e:
                        # Handle any errors in the check itself
                        print(f"WARNING: Critical error in particles for layer {layer_idx}. Complete reset. Error: {inner_e}")
                        self.particles[layer_idx] = self.rng.randint(0, self.states_per_hidden, size=self.n_particles)
        
        # Report rejuvenation statistics
        if rejuvenation_failures > 0:
            print(f"WARNING: {rejuvenation_failures} particle rejuvenation failures occurred. Model may be unstable.")
        
        # Store debug info
        self.debug_info['particles_rejuvenated'] = total_rejuvenated
        self.debug_info['rejuvenation_failures'] = rejuvenation_failures
        self.debug_info['rejuvenation_strength'] = strength
    
    def validate_and_fix_feature_names(self, features_dict):
        """
        Validate and potentially fix feature name mismatches.
        
        Parameters:
        -----------
        features_dict: dict
            Dictionary of feature values
            
        Returns:
        --------
        dict
            Potentially fixed dictionary with correct feature names
        """
        # Check for complete feature name mismatch
        if not any(f in features_dict for f in self.features):
            print("CRITICAL: Complete feature name mismatch detected!")
            
            # This suggests a potential PC naming convention issue
            # If all expected features start with 'PC_' and provided features don't,
            # or vice versa, we can try to fix the naming
            
            expected_pc_format = all(f.startswith('PC_') for f in self.features)
            provided_pc_format = all(f.startswith('PC_') for f in features_dict)
            
            # Case 1: Model expects PC_X but got PCX
            if expected_pc_format and not provided_pc_format:
                fixed_dict = {}
                for key, value in features_dict.items():
                    if key.startswith('PC') and not key.startswith('PC_'):
                        new_key = f"PC_{key[2:]}"
                        fixed_dict[new_key] = value
                    else:
                        fixed_dict[key] = value
                
                print(f"Attempted feature name fix: PC format adjustment")
                print(f"  - Before: {len([f for f in self.features if f in features_dict])}/{len(self.features)} features matched")
                print(f"  - After: {len([f for f in self.features if f in fixed_dict])}/{len(self.features)} features matched")
                
                return fixed_dict
                
            # Case 2: Model expects PCX but got PC_X
            elif not expected_pc_format and provided_pc_format:
                fixed_dict = {}
                for key, value in features_dict.items():
                    if key.startswith('PC_'):
                        new_key = f"PC{key[3:]}"
                        fixed_dict[new_key] = value
                    else:
                        fixed_dict[key] = value
                        
                print(f"Attempted feature name fix: PC format adjustment")
                print(f"  - Before: {len([f for f in self.features if f in features_dict])}/{len(self.features)} features matched")
                print(f"  - After: {len([f for f in self.features if f in fixed_dict])}/{len(self.features)} features matched")
                
                return fixed_dict
                
            # Case 3: Numerical index offset issue (PC_1 vs PC_0 indexing)
            elif expected_pc_format and provided_pc_format:
                # Check if there's a consistent offset
                expected_indices = [int(f.split('_')[1]) for f in self.features if f.startswith('PC_') and f.split('_')[1].isdigit()]
                provided_indices = [int(f.split('_')[1]) for f in features_dict if f.startswith('PC_') and f.split('_')[1].isdigit()]
                
                if expected_indices and provided_indices:
                    min_expected = min(expected_indices)
                    min_provided = min(provided_indices)
                    offset = min_expected - min_provided
                    
                    if offset != 0:
                        fixed_dict = {}
                        for key, value in features_dict.items():
                            if key.startswith('PC_') and key.split('_')[1].isdigit():
                                idx = int(key.split('_')[1])
                                new_key = f"PC_{idx + offset}"
                                fixed_dict[new_key] = value
                            else:
                                fixed_dict[key] = value
                                
                        print(f"Attempted feature name fix: PC index offset adjustment ({offset})")
                        print(f"  - Before: {len([f for f in self.features if f in features_dict])}/{len(self.features)} features matched")
                        print(f"  - After: {len([f for f in self.features if f in fixed_dict])}/{len(self.features)} features matched")
                        
                        return fixed_dict
        
        # If we couldn't fix it or no fix was needed, return the original
        return features_dict
    
    def _normalize_features(self, features_dict):
        """
        Normalize feature values with improved diagnostics.
        
        Parameters:
        -----------
        features_dict: dict
            Dictionary of feature values
            
        Returns:
        --------
        numpy.ndarray
            Normalized feature values
        """
        
        feature_values = np.zeros(self.n_features)

        # First, log any feature mismatches to diagnose the issue
        missing_features = [f for f in self.features if f not in features_dict]
        extra_features = [f for f in features_dict if f not in self.features]

        if missing_features or extra_features:
            print(f"WARNING: Feature mismatch detected!")
            print(f"  - Missing features: {len(missing_features)}/{self.n_features}")
            if len(missing_features) > 0 and len(missing_features) <= 10:
                print(f"    - Examples: {missing_features[:10]}")
            if len(extra_features) > 0:
                print(f"  - Extra features: {len(extra_features)}")
                if len(extra_features) <= 10:
                    print(f"    - Examples: {extra_features[:10]}")

        # Track which features have extreme values
        extreme_features = []
        extreme_threshold = 100  # Consider values beyond ±100 as extreme
        
        for i, feature in enumerate(self.features):
            # Get feature value with improved default handling
            value = features_dict.get(feature, None)
            
            # Handle missing features gracefully
            if value is None:
                # Instead of defaulting to 0, use the mean (which should be 0 after normalization)
                # For normalized data, 0 is often a reasonable default (represents the mean)
                feature_values[i] = 0
            else:
                # Store the value
                feature_values[i] = value
                
                # Check for extreme values before normalization
                if abs(value) > extreme_threshold:
                    extreme_features.append((feature, value))
        
        # Report any extreme values
        if extreme_features:
            print(f"WARNING: Extreme feature values detected:")
            for feature, value in extreme_features[:10]:  # Show at most 10
                print(f"  - {feature}: {value}")
            if len(extreme_features) > 10:
                print(f"  - ... and {len(extreme_features) - 10} more")

        """
        # Normalize features
        normalized_features = (feature_values - self.feature_means) / self.feature_stds
        """
        
        # UPDATE ADAPTIVE NORMALIZATION STATISTICS
        if self.adaptive_normalization and self.initial_training_done:
            # Add current features to history
            self.feature_history.append(feature_values.copy())
            
            # Keep only the most recent window
            if len(self.feature_history) > self.normalization_window:
                self.feature_history.pop(0)
            
            # Recalculate normalization statistics from recent history
            if len(self.feature_history) >= 50:  # Minimum for stable statistics
                recent_features = np.array(self.feature_history)
                
                # Calculate adaptive means and stds
                adaptive_means = np.mean(recent_features, axis=0)
                adaptive_stds = np.std(recent_features, axis=0)
                
                # Prevent division by zero
                adaptive_stds = np.maximum(adaptive_stds, 0.0001)
                
                # Smooth transition from initial to adaptive statistics
                alpha = min(1.0, len(self.feature_history) / self.normalization_window)
                
                # Blend initial and adaptive statistics
                self.feature_means = (1 - alpha) * self.feature_means + alpha * adaptive_means
                self.feature_stds = (1 - alpha) * self.feature_stds + alpha * adaptive_stds
                
                # Log significant changes (optional diagnostic)
                if hasattr(self, '_last_normalization_update'):
                    if (self._last_normalization_update % 100 == 0):
                        mean_change = np.mean(np.abs(adaptive_means - self.feature_means))
                        std_change = np.mean(np.abs(adaptive_stds - self.feature_stds))
                        #print(f" Adaptive normalization update #{self._last_normalization_update}")
                        #print(f" - Mean change: {mean_change:.6f}")
                        #print(f" - Std change: {std_change:.6f}")
                    self._last_normalization_update += 1
                else:
                    self._last_normalization_update = 1
        
        # Normalize features using current (possibly adaptive) statistics
        normalized_features = (feature_values - self.feature_means) / self.feature_stds

        # Check for extreme values after normalization
        post_norm_extremes = np.where(np.abs(normalized_features) > extreme_threshold)[0]
        if len(post_norm_extremes) > 0:
            print(f"WARNING: Extreme normalized values detected for {len(post_norm_extremes)} features")
            for idx in post_norm_extremes[:5]:  # Show a few examples
                feature_name = self.features[idx]
                print(f"  - {feature_name}: {normalized_features[idx]} (raw: {feature_values[idx]})")
            
            # With adaptive normalization, this should be much less common
            if self.adaptive_normalization:
                print("  Consider increasing normalization_window if extreme values persist")
        
        return normalized_features

    def get_normalization_stats(self):
        """
        Get current normalization statistics for diagnostics.
        """
        stats = {
            'adaptive_normalization': self.adaptive_normalization,
            'feature_history_length': len(self.feature_history) if hasattr(self, 'feature_history') else 0,
            'normalization_window': getattr(self, 'normalization_window', None),
            'feature_means_sample': self.feature_means[:5].tolist(),  # First 5 features
            'feature_stds_sample': self.feature_stds[:5].tolist(),
        }
        
        if hasattr(self, 'feature_history') and len(self.feature_history) > 0:
            recent_features = np.array(self.feature_history)
            stats['recent_feature_means'] = np.mean(recent_features, axis=0)[:5].tolist()
            stats['recent_feature_stds'] = np.std(recent_features, axis=0)[:5].tolist()
        
        return stats

    def plot_normalization_drift(self):
        """
        Plot how normalization statistics have changed over time.
        """
        import matplotlib.pyplot as plt
        
        if not hasattr(self, 'feature_history') or len(self.feature_history) < 100:
            print("Insufficient data for normalization drift plot")
            return None
        
        # Calculate rolling statistics over time
        window_size = 63  # ~3 months
        recent_features = np.array(self.feature_history)
        
        rolling_means = []
        rolling_stds = []
        
        for i in range(window_size, len(recent_features)):
            window_data = recent_features[i-window_size:i]
            rolling_means.append(np.mean(window_data, axis=0))
            rolling_stds.append(np.std(window_data, axis=0))
        
        rolling_means = np.array(rolling_means)
        rolling_stds = np.array(rolling_stds)
        
        # Plot first 5 features
        fig, axes = plt.subplots(2, 1, figsize=(12, 8))
        
        # Plot means
        for i in range(min(5, len(self.features))):
            axes[0].plot(rolling_means[:, i], label=f'{self.features[i]}', alpha=0.7)
        axes[0].set_title('Rolling Feature Means (Normalization Drift)')
        axes[0].set_ylabel('Mean Value')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Plot stds
        for i in range(min(5, len(self.features))):
            axes[1].plot(rolling_stds[:, i], label=f'{self.features[i]}', alpha=0.7)
        axes[1].set_title('Rolling Feature Standard Deviations')
        axes[1].set_ylabel('Std Value')
        axes[1].set_xlabel('Time (days)')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        return fig

    def _predict_target_distribution(self, feature_values, hidden_states=None):
        """Numerically stable prediction distribution with adaptive feature contribution scaling"""
        # Initialize variables
        mean = 0.0
        contributions = []
        scaled_contributions = False
        
        try:
            # Calculate individual feature contributions
            feature_contributions = []
            for i, (feat, weight) in enumerate(zip(feature_values, self.target_weights)):
                if np.isfinite(feat) and np.isfinite(weight):
                    contrib = feat * weight
                    if np.isfinite(contrib):
                        feature_contributions.append((i, contrib))
                        
                        # Track maximum contributions (for reporting)
                        feat_name = self.features[i] if i < len(self.features) else f"Feature_{i}"
                        if abs(contrib) > self.max_feature_contributions.get(feat_name, 0):
                            self.max_feature_contributions[feat_name] = abs(contrib)
            
            # Calculate preliminary mean (before scaling)
            original_mean = sum(contrib for _, contrib in feature_contributions)
            
            # Check if scaling is needed based on the calculated mean
            if self.feature_contribution_scaling and abs(original_mean) > self.pc_value_limit * 0.8:
                # Sort contributions by magnitude
                feature_contributions.sort(key=lambda x: abs(x[1]), reverse=True)
                top_contributors = feature_contributions[:5]
                
                # Calculate scaling factor
                scaling_factor = abs(original_mean) / (self.pc_value_limit * 0.8)
                
                # Only apply scaling if factor is significant
                if scaling_factor > 1.0:
                    scaled_contributions = True
                    # Scale all contributions
                    feature_contributions = [(i, c/scaling_factor) for i, c in feature_contributions]
                    
                    # Store debug info for top contributors
                    for i, c_original in top_contributors:
                        feat_name = self.features[i] if i < len(self.features) else f"Feature_{i}"
                        c_scaled = c_original / scaling_factor
                        contributions.append((i, feat_name, feature_values[i], self.target_weights[i], c_original, c_scaled))
            
            # CRITICAL FIX: Calculate final mean AFTER scaling has been applied
            mean = sum(contrib for _, contrib in feature_contributions)
                
            # Add bias
            if np.isfinite(self.target_bias):
                mean += self.target_bias
            
            # Final safety check (should rarely be triggered now)
            if not np.isfinite(mean) or abs(mean) > self.pc_value_limit:
                self.constrained_mean_count += 1
                self.debug_info['constrained_mean_count'] = self.constrained_mean_count
                
                print(f"FALLBACK ALERT: Mean value constrained (count: {self.constrained_mean_count})")
                print(f"  - Original mean: {original_mean}")
                print(f"  - After scaling: {mean}")
                print(f"  - Weight norm: {np.linalg.norm(self.target_weights):.4f}")
                print(f"  - Bias value: {self.target_bias:.4f}")
                print(f"  - Scaling applied: {scaled_contributions}")
                
                if contributions:
                    print(f"  - Top {len(contributions)} contributors:")
                    for idx, feat_name, feat_val, weight, orig, scaled in contributions[:5]:
                        print(f"    - Feature {idx} ({feat_name}): {feat_val:.4f} * {weight:.4f} = {orig:.4f} → {scaled:.4f}")
                
                # Last resort constraint
                mean = np.clip(mean, -self.pc_value_limit, self.pc_value_limit)
                
            # Add hidden state contribution with validation
            if hidden_states is not None and self.target_hidden_weights is not None:
                for i, state in enumerate(hidden_states):
                    if i < len(self.target_hidden_weights) and state < len(self.target_hidden_weights[i]):
                        contrib = self.target_hidden_weights[i][state]
                        if np.isfinite(contrib):
                            mean += contrib
                            
            # Ensure standard deviation is positive but not extreme
            std = max(min(self.target_std, 7.0), 0.05)
            
            # Calculate PDF with validation
            log_pdf = -0.5 * ((self.bin_centers - mean) / std)**2 - np.log(std * np.sqrt(2*np.pi))
            max_log = np.max(log_pdf)
            normalized_pdf = np.exp(log_pdf - max_log)
            pdf = normalized_pdf / np.sum(normalized_pdf)
            
            # Final validation
            if not np.all(np.isfinite(pdf)):
                raise ValueError("Invalid PDF values detected")
            
            return pdf
        except Exception as e:
            # Return uniform distribution as fallback
            print(f"ERROR in prediction distribution: {e}")
            return np.ones_like(self.bin_centers) / len(self.bin_centers)

    def _predict_target_distribution_t(self, feature_values, hidden_states=None, df=5):
        """
        Prediction distribution using Student's t-distribution for fat tails.
        Supports both discrete and continuous hidden states.
        
        Parameters:
        -----------
        feature_values: numpy.ndarray
            Normalized feature values
        hidden_states: list, optional
            Hidden state values (indices for discrete, vectors for continuous)
        df: int
            Degrees of freedom parameter (lower = fatter tails)
            
        Returns:
        --------
        numpy.ndarray
            Probability distribution across return bins
        --------
        The scientific consensus is that financial returns exhibit:
    
        Fat tails (excess kurtosis): Extreme events occur more frequently than predicted by Gaussian distributions
        Negative skewness: Large negative returns are more common than large positive returns
        Volatility clustering: Periods of high volatility tend to cluster together
        
        Key references:
        
        Mandelbrot, B. (1963). "The Variation of Certain Speculative Prices." The Journal of Business, 36(4), 394-419.
        
        First to document that financial returns have "fat tails"
        
        
        Fama, E. F. (1965). "The Behavior of Stock Market Prices." The Journal of Business, 38(1), 34-105.
        
        Comprehensive early study confirming non-normality of returns
        
        
        Cont, R. (2001). "Empirical properties of asset returns: stylized facts and statistical issues." Quantitative Finance, 1(2), 223-236.
        
        Modern synthesis of empirical properties of financial returns
        """
        import numpy as np
        
        try:
            # Calculate individual feature contributions
            feature_contributions = []
            for i, (feat, weight) in enumerate(zip(feature_values, self.target_weights)):
                if np.isfinite(feat) and np.isfinite(weight):
                    contrib = feat * weight
                    if np.isfinite(contrib):
                        feature_contributions.append((i, contrib))
                        
                        # Track maximum contributions (for reporting)
                        feat_name = self.features[i] if i < len(self.features) else f"Feature_{i}"
                        if abs(contrib) > self.max_feature_contributions.get(feat_name, 0):
                            self.max_feature_contributions[feat_name] = abs(contrib)
            
            # Calculate mean (same as original method)
            mean = sum(contrib for _, contrib in feature_contributions)
                        
            # Add bias
            if np.isfinite(self.target_bias):
                mean += self.target_bias
                
            # Safety check on mean value
            if not np.isfinite(mean) or abs(mean) > self.pc_value_limit:
                mean = np.clip(mean, -self.pc_value_limit, self.pc_value_limit)
                
            # Add hidden state contribution
            """
            if hidden_states is not None:
                if self.continuous_states:
                    # For continuous states, apply linear transformation
                    for i, state in enumerate(hidden_states):
                        if i < len(self.target_state_weights):
                            try:
                                state = self._standardize_array(state, f"predict_target_state_{i}")
                                state_invalid = False
                                # Only print for unusual or problematic types
                                if not isinstance(state, (np.ndarray, list)) and not np.isscalar(state):
                                    print(f"ERROR-TRACE: In _predict_target_distribution_t - unusual state type detected: {type(state)}")
                                    print(f"ERROR-TRACE: state content = {state}")
                                
                                # Safe state validation
                                if isinstance(state, (np.ndarray, list)):
                                    if not np.all(np.isfinite(state)):
                                        state_invalid = True
                                elif np.isscalar(state):
                                    if not np.isfinite(state):
                                        state_invalid = True
                                else:
                                    # For other array-like types that might cause errors
                                    try:
                                        state_arr = np.asarray(state)
                                        if not np.all(np.isfinite(state_arr)):
                                            state_invalid = True
                                    except Exception as e:
                                        print(f"ERROR-TRACE: Failed to convert state to array: {e}")
                                        state_invalid = True
                            except Exception as e:
                                print(f"ERROR-TRACE: Exception in _predict_target_distribution_t state check: {e}")
                                state_invalid = True
                            
                            if state_invalid:
                                continue
                                
                            # Linear combination of state variables
                            weights = self.target_state_weights[i]
                            contrib = np.dot(weights, state)  # np.dot handles types well
                            
                            # Add if valid
                            if np.isfinite(contrib):
                                mean += contrib
                else:
            """
            # Add hidden state contribution
            if hidden_states is not None:
                if self.continuous_states:
                    # For continuous states, apply linear transformation with strict validation
                    for i, state in enumerate(hidden_states):
                        if i < len(self.target_state_weights):
                            # Validate state vector format
                            if not isinstance(state, np.ndarray):
                                print(f"ERROR: Hidden state {i} must be numpy array, got {type(state)}")
                                continue
                            if state.shape != (self.state_dimension,):
                                print(f"ERROR: Hidden state {i} has invalid shape {state.shape}, expected ({self.state_dimension},)")
                                continue
                            if not np.all(np.isfinite(state)):
                                print(f"ERROR: Hidden state {i} contains non-finite values")
                                continue
                                
                            # Validate weights have correct dimension
                            weights = self.target_state_weights[i]
                            if weights.shape != (self.state_dimension,):
                                print(f"ERROR: Target weights for state {i} have invalid shape {weights.shape}")
                                continue
                            
                            # Compute contribution using dot product (mathematically correct for vector ops)
                            contrib = np.dot(weights, state)
                            
                            # Add if valid
                            if np.isfinite(contrib):
                                mean += contrib
                            else:
                                print(f"ERROR: Non-finite contribution from state {i}: {contrib}")
                else:
                    # Original discrete state method
                    for i, state in enumerate(hidden_states):
                        if i < len(self.target_hidden_weights) and state < len(self.target_hidden_weights[i]):
                            contrib = self.target_hidden_weights[i][state]
                            if np.isfinite(contrib):
                                mean += contrib
            
            # Ensure scale is positive but not extreme
            scale = max(min(self.target_std, 7.0), 0.05)
            
            # Calculate PDF using t-distribution
            from scipy import stats
            
            # Create t-distribution with specified degrees of freedom
            t_dist = stats.t(df=df, loc=mean, scale=scale)
            
            # Calculate PDF at bin centers
            pdf = t_dist.pdf(self.bin_centers)
            
            # Normalize to ensure it sums to 1
            pdf = pdf / np.sum(pdf)
            
            return pdf
        except Exception as e:
            # Return uniform distribution as fallback
            print(f"ERROR in t-distribution calculation: {e}")
            return np.ones_like(self.bin_centers) / len(self.bin_centers)
    
    def report_feature_extremes(self):
        """Report on features that have contributed most to extreme predictions."""
        if not self.max_feature_contributions:
            print("No feature contribution data available yet.")
            return
            
        print("\n=== Feature Contribution Analysis ===")
        print(f"Total features tracked: {len(self.max_feature_contributions)}")
        
        # Sort by contribution magnitude
        sorted_contribs = sorted(self.max_feature_contributions.items(), 
                                key=lambda x: x[1], 
                                reverse=True)
        
        # Report on top contributors
        print("\nTop 15 feature contributors to extreme predictions:")
        for i, (feature, max_contrib) in enumerate(sorted_contribs[:15]):
            print(f"{i+1}. {feature}: max contribution = {max_contrib:.4f}")
        
        # Report on potential problematic features
        extreme_threshold = self.pc_value_limit / 2
        extreme_features = [(f, c) for f, c in sorted_contribs if c > extreme_threshold]
        
        if extreme_features:
            print(f"\nFeatures with extreme contributions (>{extreme_threshold:.1f}):")
            for feature, max_contrib in extreme_features:
                print(f"- {feature}: {max_contrib:.4f}")
            
            print("\nRecommended actions:")
            print("1. Consider applying stronger adaptive scaling to PCA components")
            print(f"2. Apply component-specific thresholds for problematic features")
            print("3. Increase regularization for these features in the model")
        else:
            print("\nNo extremely problematic features detected.")
            
        return sorted_contribs
    

    def get_fallback_stats(self):
        """Get statistics about prediction fallbacks"""
        return {
            "constrained_mean_count": self.constrained_mean_count,
            "weight_norm": np.linalg.norm(self.target_weights),
            "bias_value": self.target_bias
        }
    

    def _sample_hidden_states(self):
        """
        Sample hidden states from current particle distribution.
        Supports both discrete and continuous states.
        
        Returns:
        --------
        list or None
            For discrete: List of sampled hidden state indices
            For continuous: List of state vectors (each a numpy array)
            None if no hidden layers
        """
        import numpy as np
        
        if self.hidden_layers == 0 or self.particles is None:
            return None
        
        # Sample particle index based on weights
        try:
            # Check for invalid weight distribution
            if np.any(~np.isfinite(self.particle_weights)) or np.sum(self.particle_weights) <= 0:
                print("WARNING: Invalid particle weights detected. Reinitializing weights.")
                self.particle_weights = np.ones(self.n_particles) / self.n_particles
                
            # Sample a particle index according to weights
            particle_idx = self.rng.choice(self.n_particles, p=self.particle_weights)
            
            # Extract hidden states from the sampled particle
            """
            if self.continuous_states:
                # For continuous states, get the state vector of selected particle
                hidden_states = [self._standardize_array(particles[particle_idx], f"sample_hidden_state_layer_{i}") 
                                 for i, particles in enumerate(self.particles)]
                
                # Validate state values
                for i, state in enumerate(hidden_states):
                    if not np.all(np.isfinite(state)):
                        print(f"WARNING: Invalid hidden state values detected in layer {i}. Using random state instead.")
                        hidden_states[i] = self.rng.normal(0, 1, self.state_dimension)
            """
            if self.continuous_states:
                # For continuous states, strictly enforce numpy array representation
                hidden_states = []
                
                for i, particles in enumerate(self.particles):
                    # Validate particle array shape
                    expected_shape = (self.n_particles, self.state_dimension)
                    if particles.shape != expected_shape:
                        print(f"ERROR: Particles for layer {i} have invalid shape {particles.shape}, expected {expected_shape}")
                        # Create valid replacement state
                        hidden_states.append(self.rng.normal(0, 1, self.state_dimension))
                        continue
                    
                    # Extract state vector for selected particle
                    state = particles[particle_idx].copy()
                    
                    # Validate state dimension
                    if state.shape != (self.state_dimension,):
                        print(f"ERROR: Sampled state vector has invalid shape {state.shape}, expected ({self.state_dimension},)")
                        # Create valid replacement
                        hidden_states.append(self.rng.normal(0, 1, self.state_dimension))
                    elif not np.all(np.isfinite(state)):
                        print(f"ERROR: Sampled state vector contains non-finite values")
                        # Create valid replacement
                        hidden_states.append(self.rng.normal(0, 1, self.state_dimension))
                    else:
                        hidden_states.append(state)
                        
                return hidden_states
            else:
                # Original discrete state sampling
                hidden_states = [particles[particle_idx] for particles in self.particles]
                
                # Validate hidden states indices
                for i, state in enumerate(hidden_states):
                    if state >= self.states_per_hidden or state < 0:
                        print(f"WARNING: Invalid hidden state {state} detected in layer {i}. Using random state instead.")
                        hidden_states[i] = self.rng.randint(0, self.states_per_hidden)
                
            return hidden_states
            
        except Exception as e:
            print(f"WARNING: Error sampling hidden states: {e}")
            print("Using random states as fallback. Prediction may be unreliable.")
            
            if self.continuous_states:
                return [self.rng.normal(0, 1, self.state_dimension) for _ in range(self.hidden_layers)]
            else:
                return [self.rng.randint(0, self.states_per_hidden) for _ in range(self.hidden_layers)]

    def _update_transition_matrices(self, hidden_states_before, hidden_states_after):
        """
        Update hidden state transition matrices based on observed transitions.
        Supports both discrete and continuous states.
        
        Parameters:
        -----------
        hidden_states_before: list
            Hidden states before transition
        hidden_states_after: list
            Hidden states after transition
        ----------
        Key Scientific Support

        Exponential Forgetting Factor: Based on Rabiner (1989), implementing a forgetting factor of 0.997 allows the model to gradually adapt to changing market regimes while maintaining stability.
        Probabilistic Representation: Implementation uses methods from Ghahramani & Hinton (2000) to properly represent uncertainty in state transitions.
        Adaptive Resampling: Post-resample rejuvenation is supported by Li, Bolic & Djuric (2015), which showed improved particle diversity after resampling is critical.
        Dynamic Window Alignment: All analysis windows now align with your PCA window (20 days) per recommendations from Poon & Granger (2003).
        Volatility-Scaled Learning: Implements Cont's (2001) work on volatility clustering by dynamically adjusting learning based on recent volatility.
        
        This implementation fully integrates time-varying transitions with the existing model
        """
        import numpy as np
        
        if hidden_states_before is None or hidden_states_after is None:
            return
        
        if self.continuous_states:
            # For continuous states, update transition parameters using recursive least squares
            for layer in range(len(hidden_states_before)):
                state_before = self._standardize_array(hidden_states_before[layer], f"update_transition_before_{layer}")
                state_after = self._standardize_array(hidden_states_after[layer], f"update_transition_after_{layer}")
                
                # Validate state vectors for continuous states
                skip_update = False
                
                # Validate state_before
                if not isinstance(state_before, np.ndarray):
                    print(f"ERROR: state_before must be numpy array, got {type(state_before)}")
                    skip_update = True
                elif state_before.shape != (self.state_dimension,):
                    print(f"ERROR: state_before has invalid shape {state_before.shape}, expected ({self.state_dimension},)")
                    skip_update = True
                elif not np.all(np.isfinite(state_before)):
                    print(f"ERROR: state_before contains non-finite values")
                    skip_update = True
                
                # Validate state_after
                if not skip_update:
                    if not isinstance(state_after, np.ndarray):
                        print(f"ERROR: state_after must be numpy array, got {type(state_after)}")
                        skip_update = True
                    elif state_after.shape != (self.state_dimension,):
                        print(f"ERROR: state_after has invalid shape {state_after.shape}, expected ({self.state_dimension},)")
                        skip_update = True
                    elif not np.all(np.isfinite(state_after)):
                        print(f"ERROR: state_after contains non-finite values")
                        skip_update = True
                
                if skip_update:
                    continue
                    
                # Use pre-initialized RLS params or create them as a fallback
                if not hasattr(self, 'rls_params'):
                    print("WARNING: RLS parameters not initialized during model creation")
                    # Initialize them here as a fallback, but log the issue
                    self.rls_params = []
                    for l in range(self.hidden_layers):
                        d = self.state_dimension
                        P = np.eye(d + 1) * 100.0
                        self.rls_params.append({
                            'P': P,
                            'lambda': self.forgetting_factor
                        })
                    print("DIAGNOSTIC: Created missing RLS parameters during update")
                
                # Perform RLS update for each state dimension
                for dim in range(self.state_dimension):
                    # CRITICAL FIX: Augment state with constant term DIRECTLY, not through _standardize_array
                    # because the augmented state should be (state_dimension + 1) elements
                    x = np.hstack([state_before, 1.0])
                    y = state_after[dim]
                    
                    # Skip update if any values are invalid
                    if not np.all(np.isfinite(x)) or not np.isfinite(y):
                        continue
                    
                    # Get current parameters
                    P = self.rls_params[layer]['P']
                    lam = self.rls_params[layer]['lambda']
                    
                    # Validate dimensions
                    if P.shape != (self.state_dimension + 1, self.state_dimension + 1):
                        print(f"ERROR: P matrix has invalid shape {P.shape}, expected ({self.state_dimension + 1}, {self.state_dimension + 1})")
                        continue
                    
                    if x.shape != (self.state_dimension + 1,):
                        print(f"ERROR: Augmented state x has invalid shape {x.shape}, expected ({self.state_dimension + 1},)")
                        continue
                    
                    # RLS update
                    try:
                        # Compute gain
                        k = P @ x / (lam + x @ P @ x)
                        
                        # Prediction error
                        A = self.state_transition_matrices[layer]
                        b = self.state_transition_biases[layer]
                        # Direct matrix operations without _standardize_array
                        pred = A[dim] @ state_before + b[dim]
                        error = y - pred
                        
                        # Update parameters
                        A_update = k[:-1] * error
                        b_update = k[-1] * error
                        
                        # Apply updates with step size control
                        step_size = min(0.05, 1.0 / (1.0 + len(self.prediction_history) / 100.0))
                        
                        # Cap large updates to prevent instability
                        if np.max(np.abs(A_update)) > 1.0:
                            scale_factor = 1.0 / np.max(np.abs(A_update))
                            A_update *= scale_factor
                            b_update *= scale_factor
                        
                        # Apply updates
                        A[dim] += step_size * A_update
                        b[dim] += step_size * b_update
                        
                        # Update precision matrix for next iteration
                        P = (P - np.outer(k, x) @ P) / lam
                        self.rls_params[layer]['P'] = P
                        
                        # Ensure model stability by controlling eigenvalues
                        eigvals = np.linalg.eigvals(A)
                        if np.max(np.abs(eigvals)) > 0.97 or not np.all(np.isfinite(eigvals)):
                            # Scale back to ensure stability if eigenvalues too large
                            # or replace with stable matrix if not finite
                            if np.all(np.isfinite(eigvals)) and np.max(np.abs(eigvals)) > 0:
                                A = A * (0.97 / np.max(np.abs(eigvals)))
                            else:
                                # Fall back to a simple stable matrix
                                A = np.eye(self.state_dimension) * 0.9
                        
                        # Update model parameters
                        self.state_transition_matrices[layer] = A
                        self.state_transition_biases[layer] = b
                        
                    except Exception as e:
                        print(f"ERROR in RLS update for dimension {dim}: {e}")
                        continue
                        
        else:
            # For discrete states, update transition matrices
            for layer in range(len(hidden_states_before)):
                state_before = hidden_states_before[layer]
                state_after = hidden_states_after[layer]
                
                # Skip invalid states
                if state_before >= self.states_per_hidden or state_before < 0 or \
                   state_after >= self.states_per_hidden or state_after < 0:
                    continue
                
                # Initialize transition count matrix if needed
                if not hasattr(self, 'transition_counts'):
                    self.transition_counts = []
                    for l in range(self.hidden_layers):
                        self.transition_counts.append(
                            np.ones((self.states_per_hidden, self.states_per_hidden)) * 0.1
                        )
                
                # Increment count for observed transition
                self.transition_counts[layer][state_before, state_after] += 1
                
                # Apply exponential forgetting to gradually reduce influence of old transitions
                # This allows the model to adapt to changing market conditions
                #forgetting = getattr(self, 'forgetting_factor', 0.997)
                forgetting = self.forgetting_factor

                if forgetting < 1.0:  # Only apply forgetting if < 1.0
                    # Apply forgetting to all transitions except the observed one
                    self.transition_counts[layer] *= forgetting
                    # Restore the increment for the observed transition
                    self.transition_counts[layer][state_before, state_after] /= forgetting
                # If forgetting == 1.0, skip forgetting entirely (perfect memory)

                # Recalculate transition matrix using normalized counts
                row_sums = self.transition_counts[layer].sum(axis=1, keepdims=True)
                # Avoid division by zero
                row_sums = np.maximum(row_sums, 1e-10)
                self.hidden_transitions[layer] = self.transition_counts[layer] / row_sums
                
                # Verify transition matrix properties
                for row in range(self.states_per_hidden):
                    row_sum = np.sum(self.hidden_transitions[layer][row])
                    if not np.isclose(row_sum, 1.0, rtol=1e-5) or not np.all(np.isfinite(self.hidden_transitions[layer][row])):
                        # Fix invalid row by replacing with uniform distribution
                        self.hidden_transitions[layer][row] = np.ones(self.states_per_hidden) / self.states_per_hidden
    
    def _particle_filtering_update(self, features_dict, actual_return=None):
        """
        Update particle distribution using new observation.
        Supports both discrete and continuous states.
        
        Parameters:
        -----------
        features_dict: dict
            Dictionary of feature values
        actual_return: float or None
            Actual return value (if available)
        """
        import numpy as np
    
        if self.hidden_layers == 0 or self.particles is None:
            return
            
        # Normalize features
        feature_values = self._normalize_features(features_dict)
        
        # 1. Prediction step: transition particles according to model
        new_particles = []
        
        # Iterate through existing particle layers
        for layer_idx, current_particles in enumerate(self.particles):
            if self.continuous_states:
                # Extract hidden states for continuous state models
                found_unusual_type = False  # Track if we've found any issues
                
                # FIXED: Scan CURRENT particles for unusual types (not empty new_particles)
                # Check first few particles from the current layer
                for i in range(min(self.n_particles, 10)):
                    try:
                        particle = current_particles[i]
                        if not isinstance(particle, np.ndarray) and not np.isscalar(particle):
                            print(f"ERROR-TRACE: In _particle_filtering_update - unusual particle type detected: {type(particle)}")
                            print(f"ERROR-TRACE: Layer {layer_idx}, Particle {i} content = {particle}")
                            found_unusual_type = True
                            break  # Only print the first unusual type we find
                    except Exception as e:
                        print(f"ERROR-TRACE: Exception checking particle type in layer {layer_idx}, particle {i}: {e}")
                        found_unusual_type = True
                        break
                
                # Also check a random sample throughout the particle array
                if not found_unusual_type and self.n_particles > 20:
                    try:
                        check_indices = self.rng.choice(self.n_particles, size=5, replace=False)
                        for i in check_indices:
                            particle = current_particles[i]
                            if not isinstance(particle, np.ndarray) and not np.isscalar(particle):
                                print(f"ERROR-TRACE: In _particle_filtering_update - unusual particle type at random index: {type(particle)}")
                                print(f"ERROR-TRACE: Layer {layer_idx}, Particle {i} content = {particle}")
                                found_unusual_type = True
                                break
                    except Exception as e:
                        print(f"ERROR-TRACE: Exception during random particle checks: {e}")
                        found_unusual_type = True
                
                # Check particle shape validity
                if current_particles.shape != (self.n_particles, self.state_dimension):
                    print(f"WARNING: Invalid particle shape detected in layer {layer_idx}. Reshaping.")
                    print(f"Expected shape: ({self.n_particles}, {self.state_dimension}), got {current_particles.shape}")
                    # Initialize with correct shape instead of trying to reshape corrupted data
                    new_layer_particles = np.zeros((self.n_particles, self.state_dimension))
                    # Fill with new random values
                    for i in range(self.n_particles):
                        new_layer_particles[i] = self.rng.normal(0, 1, self.state_dimension)
                else:
                    # For continuous states: Apply linear state space transition
                    new_layer_particles = np.zeros((self.n_particles, self.state_dimension))
                    
                    # Get transition parameters
                    A = self.state_transition_matrices[layer_idx]
                    b = self.state_transition_biases[layer_idx]
                    Q = self.state_transition_noise[layer_idx]
                    
                    # Apply state transition equation x_{t+1} = Ax_t + b + noise
                    for i in range(self.n_particles):
                        # Get current state (ensure it's a standard array)
                        state = self._standardize_array(current_particles[i], "particle_filtering_state_extract")
                        
                        # Skip invalid states
                        state_invalid = False
                        if isinstance(state, (np.ndarray, list)):
                            if not np.all(np.isfinite(state)):
                                state_invalid = True
                        else:
                            if not np.isfinite(state):
                                state_invalid = True
                                
                        if state_invalid:
                            new_layer_particles[i] = self.rng.normal(0, 1, self.state_dimension)
                            continue
                        
                        # Compute state transition
                        mean = self._standardize_array(A @ state + b, "particle_filtering_mean")
                        
                        # Add process noise
                        noise = self.rng.multivariate_normal(
                            np.zeros(self.state_dimension), 
                            Q
                        )
                        
                        # Set new state
                        new_state = self._standardize_array(mean + noise, "particle_filtering_new_state")
                        
                        # Stabilize if needed
                        if not np.all(np.isfinite(new_state)):
                            new_state = self.rng.normal(0, 1, self.state_dimension)
                        elif np.max(np.abs(new_state)) > 10.0:
                            # Dampen extreme values
                            new_state = new_state * (10.0 / np.max(np.abs(new_state)))
                            
                        new_layer_particles[i] = new_state
            else:
                # Check particle shape validity for discrete states
                if current_particles.shape != (self.n_particles,):
                    print(f"WARNING: Invalid discrete particle shape detected in layer {layer_idx}. Reshaping.")
                    # Initialize with correct shape for discrete states
                    new_layer_particles = np.zeros(self.n_particles, dtype=int)
                    # Fill with new random values
                    for i in range(self.n_particles):
                        new_layer_particles[i] = self.rng.randint(0, self.states_per_hidden)
                else:
                    # Original discrete state transitions
                    # Apply transition model to each particle
                    new_layer_particles = np.zeros(self.n_particles, dtype=int)
                    
                    for i, state in enumerate(current_particles):
                        # Check for valid state index
                        if state >= self.states_per_hidden:
                            state = self.rng.randint(0, self.states_per_hidden)
                        
                        # Sample new state according to transition probabilities
                        transit_probs = self.hidden_transitions[layer_idx][state]
                        
                        # Validate transition probabilities
                        if not np.all(np.isfinite(transit_probs)) or np.sum(transit_probs) <= 0:
                            # Use uniform distribution if invalid probabilities
                            transit_probs = np.ones(self.states_per_hidden) / self.states_per_hidden
                        
                        new_state = self.rng.choice(
                            self.states_per_hidden,
                            p=transit_probs
                        )
                        new_layer_particles[i] = new_state
            
            new_particles.append(new_layer_particles)
        
        # 2. Update step: update weights based on observation likelihood
        if actual_return is not None:
            # Find which bin the actual return falls into
            bin_idx = np.digitize(actual_return, self.bins) - 1
            bin_idx = max(0, min(bin_idx, len(self.bin_centers) - 1))
            
            # Update weights for each particle
            new_weights = np.zeros(self.n_particles)
            
            for i in range(self.n_particles):
                if self.continuous_states:
                    # For continuous states: extract state vectors
                    hidden_states = [layer_particles[i].copy() for layer_particles in new_particles]
                else:
                    # For discrete states: extract state indices
                    hidden_states = [layer_particles[i] for layer_particles in new_particles]
                
                # Compute likelihood of observation given hidden states
                # Use t-distribution for consistency
                pdf = self._predict_target_distribution_t(feature_values, hidden_states, df=5)
                likelihood = pdf[bin_idx]
                
                # Update weight
                new_weights[i] = self.particle_weights[i] * likelihood
            
            # Prevent numerical issues
            max_weight = np.max(new_weights)
            if max_weight > 0:
                # Normalize relative to max to prevent underflow
                new_weights = new_weights / max_weight
            
            # Normalize weights
            weight_sum = new_weights.sum()
            if weight_sum > 0:
                new_weights = new_weights / weight_sum
            else:
                # If all weights are zero, reset to uniform
                new_weights = np.ones(self.n_particles) / self.n_particles
            
            # 3. Resampling step: resample particles if effective sample size is too low
            n_eff = 1 / np.sum(new_weights ** 2)
            resampled = False
            
            if n_eff < self.n_particles / 2:
                resampled = True
                # Resample particles
                indices = self.rng.choice(
                    self.n_particles,
                    size=self.n_particles,
                    p=new_weights,
                    replace=True
                )
                
                # Apply resampling to each layer
                resampled_particles = []
                for layer_particles in new_particles:
                    if self.continuous_states:
                        # For continuous: copy state vectors
                        resampled_layer = np.array([layer_particles[idx].copy() for idx in indices])
                    else:
                        # For discrete: copy state indices  
                        resampled_layer = np.array([layer_particles[idx] for idx in indices])
                    resampled_particles.append(resampled_layer)
                
                # Replace with resampled particles
                new_particles = resampled_particles
                
                # Reset weights to uniform
                new_weights = np.ones(self.n_particles) / self.n_particles
            
            # Store current hidden states for transition learning
            if self.continuous_states:
                # Sample a representative particle for continuous states
                idx = self.rng.choice(self.n_particles, p=new_weights)
                current_hidden_states = [layer_particles[idx].copy() for layer_particles in new_particles]
            else:
                # For discrete states, represent with most probable state
                current_hidden_states = []
                for layer_particles in new_particles:
                    # Count occurrences of each state, weighted by particle weights
                    state_probs = np.zeros(self.states_per_hidden)
                    for s in range(self.states_per_hidden):
                        mask = (layer_particles == s)
                        state_probs[s] = np.sum(new_weights[mask])
                    
                    # Select most probable state
                    if np.sum(state_probs) > 0:
                        most_probable = np.argmax(state_probs)
                    else:
                        most_probable = self.rng.randint(0, self.states_per_hidden)
                    
                    current_hidden_states.append(most_probable)
            
            # Get previous hidden states if available
            previous_hidden_states = getattr(self, 'previous_hidden_states', None)
            
            # Update transition matrices if we have both states
            if previous_hidden_states is not None:
                self._update_transition_matrices(previous_hidden_states, current_hidden_states)
                
            # Store current states for next update
            self.previous_hidden_states = current_hidden_states
            
            # Update particles and weights
            self.particles = new_particles
            self.particle_weights = new_weights
            
            # Apply particle rejuvenation if needed
            if self.particle_rejuvenation:
                # Apply safer rejuvenation logic
                should_rejuvenate = False
                rejuv_strength = 0.05  # Default strength
                
                # Check for stagnation
                if self.debug_info['stagnation_detected']:
                    should_rejuvenate = True
                    rejuv_strength = 0.25  # Strong rejuvenation
                # Check steps since update
                elif 'steps_since_update' in self.debug_info and self.debug_info['steps_since_update'] > 5:
                    should_rejuvenate = True
                    rejuv_strength = 0.15  # Medium rejuvenation
                # Always apply light rejuvenation after resampling
                elif resampled:
                    should_rejuvenate = True
                    rejuv_strength = 0.05  # Light rejuvenation
                # Apply occasional random rejuvenation
                elif self.rng.random() < 0.08:  # 8% chance, using self.rng
                    should_rejuvenate = True
                    rejuv_strength = 0.08  # Light rejuvenation
                
                if should_rejuvenate:
                    try:
                        self._rejuvenate_particles(strength=rejuv_strength)
                        self.debug_info['rejuvenation_applied'] = True
                        self.debug_info['rejuvenation_strength'] = rejuv_strength
                    except Exception as e:
                        print(f"Warning: Error during rejuvenation: {e}")
                        self.debug_info['rejuvenation_applied'] = False
                else:
                    self.debug_info['rejuvenation_applied'] = False
            else:
                self.debug_info['rejuvenation_applied'] = False
        else:
            # If no observation, just update particles
            self.particles = new_particles
            """
            else:
                # Check particle shape validity for discrete states
                if particles.shape != (self.n_particles,):
                    print(f"WARNING: Invalid discrete particle shape detected in layer {layer}. Reshaping.")
                    # Initialize with correct shape for discrete states
                    new_layer_particles = np.zeros(self.n_particles, dtype=int)
                    # Fill with new random values
                    for i in range(self.n_particles):
                        new_layer_particles[i] = self.rng.randint(0, self.states_per_hidden)
                else:
                    # Original discrete state transitions
                    # Apply transition model to each particle
                    new_layer_particles = np.zeros(self.n_particles, dtype=int)
                    
                    for i, state in enumerate(particles):
                        # Check for valid state index
                        if state >= self.states_per_hidden:
                            state = self.rng.randint(0, self.states_per_hidden)
                        
                        # Sample new state according to transition probabilities
                        transit_probs = self.hidden_transitions[layer][state]
                        
                        # Validate transition probabilities
                        if not np.all(np.isfinite(transit_probs)) or np.sum(transit_probs) <= 0:
                            # Use uniform distribution if invalid probabilities
                            transit_probs = np.ones(self.states_per_hidden) / self.states_per_hidden
                        
                        new_state = self.rng.choice(
                            self.states_per_hidden,
                            p=transit_probs
                        )
                        new_layer_particles[i] = new_state
            
            new_particles.append(new_layer_particles)
        
        # 2. Update step: update weights based on observation likelihood
        if actual_return is not None:
            # Find which bin the actual return falls into
            bin_idx = np.digitize(actual_return, self.bins) - 1
            bin_idx = max(0, min(bin_idx, len(self.bin_centers) - 1))
            
            # Update weights for each particle
            new_weights = np.zeros(self.n_particles)
            
            for i in range(self.n_particles):
                if self.continuous_states:
                    # For continuous states: extract state vectors
                    hidden_states = [particles[i].copy() for particles in new_particles]
                else:
                    # For discrete states: extract state indices
                    hidden_states = [particles[i] for particles in new_particles]
                
                # Compute likelihood of observation given hidden states
                # Use t-distribution for consistency
                pdf = self._predict_target_distribution_t(feature_values, hidden_states, df=5)
                likelihood = pdf[bin_idx]
                
                # Update weight
                new_weights[i] = self.particle_weights[i] * likelihood
            
            # Prevent numerical issues
            max_weight = np.max(new_weights)
            if max_weight > 0:
                # Normalize relative to max to prevent underflow
                new_weights = new_weights / max_weight
            
            # Normalize weights
            if new_weights.sum() > 0:
                new_weights = new_weights / new_weights.sum()
            else:
                # If all weights are zero, reset to uniform
                new_weights = np.ones(self.n_particles) / self.n_particles
            
            # 3. Resampling step: resample particles if effective sample size is too low
            n_eff = 1 / np.sum(new_weights ** 2)
            resampled = False
            
            if n_eff < self.n_particles / 2:
                resampled = True
                # Resample particles
                indices = self.rng.choice(
                    self.n_particles,
                    size=self.n_particles,
                    p=new_weights,
                    replace=True
                )
                
                # Apply resampling to each layer
                resampled_particles = []
                for particles in new_particles:
                    if self.continuous_states:
                        # For continuous: copy state vectors
                        resampled = np.array([particles[idx].copy() for idx in indices])
                    else:
                        # For discrete: copy state indices
                        resampled = np.array([particles[idx] for idx in indices])
                    resampled_particles.append(resampled)
                
                # Replace with resampled particles
                new_particles = resampled_particles
                
                # Reset weights to uniform
                new_weights = np.ones(self.n_particles) / self.n_particles
            
            # Store current hidden states for transition learning
            if self.continuous_states:
                # Sample a representative particle for continuous states
                idx = self.rng.choice(self.n_particles, p=new_weights)
                current_hidden_states = [particles[idx].copy() for particles in new_particles]
            else:
                # For discrete states, represent with most probable state
                current_hidden_states = []
                for particles_layer in new_particles:
                    # Count occurrences of each state, weighted by particle weights
                    state_probs = np.zeros(self.states_per_hidden)
                    for s in range(self.states_per_hidden):
                        mask = (particles_layer == s)
                        state_probs[s] = np.sum(new_weights[mask])
                    
                    # Select most probable state
                    if np.sum(state_probs) > 0:
                        most_probable = np.argmax(state_probs)
                    else:
                        most_probable = self.rng.randint(0, self.states_per_hidden)
                    
                    current_hidden_states.append(most_probable)
            
            # Get previous hidden states if available
            previous_hidden_states = getattr(self, 'previous_hidden_states', None)
            
            # Update transition matrices if we have both states
            if previous_hidden_states is not None:
                self._update_transition_matrices(previous_hidden_states, current_hidden_states)
                
            # Store current states for next update
            self.previous_hidden_states = current_hidden_states
            
            # Update particles and weights
            self.particles = new_particles
            self.particle_weights = new_weights
            
            # Apply particle rejuvenation if needed
            if self.particle_rejuvenation:
                # Apply safer rejuvenation logic
                should_rejuvenate = False
                rejuv_strength = 0.05  # Default strength
                
                # Check for stagnation
                if self.debug_info['stagnation_detected']:
                    should_rejuvenate = True
                    rejuv_strength = 0.25  # Strong rejuvenation
                # Check steps since update
                elif 'steps_since_update' in self.debug_info and self.debug_info['steps_since_update'] > 5:
                    should_rejuvenate = True
                    rejuv_strength = 0.15  # Medium rejuvenation
                # Always apply light rejuvenation after resampling
                elif resampled:
                    should_rejuvenate = True
                    rejuv_strength = 0.05  # Light rejuvenation
                # Apply occasional random rejuvenation
                elif self.rng.random() < 0.08:  # 8% chance, using self.rng
                    should_rejuvenate = True
                    rejuv_strength = 0.08  # Light rejuvenation
                
                if should_rejuvenate:
                    try:
                        self._rejuvenate_particles(strength=rejuv_strength)
                        self.debug_info['rejuvenation_applied'] = True
                        self.debug_info['rejuvenation_strength'] = rejuv_strength
                    except Exception as e:
                        print(f"Warning: Error during rejuvenation: {e}")
                        self.debug_info['rejuvenation_applied'] = False
                else:
                    self.debug_info['rejuvenation_applied'] = False
            else:
                self.debug_info['rejuvenation_applied'] = False
        else:
            # If no observation, just update particles
            self.particles = new_particles
        """
    
    def _detect_stagnation(self):
        """
        Detect if the model is stuck in a stagnation state.
        
        Returns:
        --------
        bool
            True if stagnation is detected, False otherwise
        """
        import numpy as np
        
        if not self.enable_anti_stagnation or len(self.prediction_history) < self.stagnation_window:
            return False
        
        # Check the last N predictions
        recent_dirs = np.array([p['direction'] for p in self.prediction_history[-self.stagnation_window:]])

        try:
            # This is a safety check before your main implementation
            if not isinstance(recent_dirs, np.ndarray):
                print(f"ERROR-TRACE: In _detect_stagnation - recent_dirs is not an array, type={type(recent_dirs)}")
                
            if not np.all(np.isfinite(recent_dirs)):
                print(f"ERROR-TRACE: Non-finite values in recent_dirs: {recent_dirs}")
                valid_mask = np.isfinite(recent_dirs)
                if np.any(valid_mask):
                    recent_dirs = recent_dirs[valid_mask]
                else:
                    print(f"ERROR-TRACE: No valid values in recent_dirs")
                    return False
        except Exception as e:
            print(f"ERROR-TRACE: Exception in _detect_stagnation pre-check: {e}")
            print(f"ERROR-TRACE: recent_dirs type = {type(recent_dirs)}, content = {recent_dirs}")

        # Diagnostic: Check for NaN or invalid values in direction array
        if not np.all(np.isfinite(recent_dirs)):
            print(f"DIAGNOSTIC: Invalid values detected in recent_dirs array: {recent_dirs}")
            print(f"This may indicate unstable predictions in history positions: {np.where(~np.isfinite(recent_dirs))}")
            # Safe handling - remove non-finite values
            valid_mask = np.isfinite(recent_dirs)
            if np.any(valid_mask):
                recent_dirs = recent_dirs[valid_mask]
            else:
                return False  # Can't determine stagnation with all invalid data
        
        # If all predictions are the same direction
        if len(recent_dirs) > 0 and np.all(recent_dirs == recent_dirs[0]):
            # Calculate accuracy just for this stagnation period
            stagnation_preds = self.prediction_history[-self.stagnation_window:]
            
            # Only calculate if we have 'was_correct' data
            if all('was_correct' in pred for pred in stagnation_preds):
                correct_count = sum(1 for pred in stagnation_preds if pred['was_correct'])
                stagnation_accuracy = correct_count / len(stagnation_preds)
                
                print(f"DIAGNOSTIC: Stagnation detected - all {len(recent_dirs)} predictions in same direction: {recent_dirs[0]}")
                print(f"DIAGNOSTIC: Directional accuracy during stagnation period: {stagnation_accuracy:.4f}")
                
                # Provide context
                if stagnation_accuracy > 0.7:
                    print(f"DIAGNOSTIC: High accuracy indicates correct market regime detection")
                elif stagnation_accuracy < 0.4:
                    print(f"DIAGNOSTIC: Low accuracy suggests model is stuck in incorrect pattern")
                else:
                    print(f"DIAGNOSTIC: Moderate accuracy - monitoring recommended")
            else:
                print(f"DIAGNOSTIC: Stagnation detected - all {len(recent_dirs)} predictions in same direction: {recent_dirs[0]}")
                print(f"DIAGNOSTIC: Accuracy data not available for full stagnation period")
            
            return True
        
        # Check for convergence in prediction values
        recent_vals = np.array([p['expected_value'] for p in self.prediction_history[-self.stagnation_window:]])
        std_of_preds = np.std(recent_vals)
        
        # Calculate volatility from broader window for context
        # Use 20-day window to align with PCA window
        broader_window = min(60, len(self.prediction_history))
        broader_vals = np.array([p['expected_value'] for p in self.prediction_history[-broader_window:]])
        volatility = np.std(broader_vals)
        
        # Define dynamic threshold as percentage of volatility, with minimum
        dynamic_threshold = max(0.01, volatility * 0.1)  # 10% of volatility
        
        # If standard deviation is very low relative to historical volatility
        if std_of_preds < dynamic_threshold:
            return True

        """
        Hamilton, J. D. (1989). "A New Approach to the Economic Analysis of Nonstationary Time Series and the Business Cycle." Econometrica, 57(2), 357-384.
        Introduces regime-switching models that justify adaptive approaches to state detection        
        
        Ang, A., & Timmermann, A. (2012). "Regime changes and financial markets." Annual Review of Financial Economics, 4(1), 313-337.
        Demonstrates how market behavior varies across regimes, requiring adaptive detection approaches
        """
        
        # If at least stagnation_threshold% are the same direction
        # Safely handle the array comparison
        if len(recent_dirs) > 0:
            same_dir = recent_dirs == recent_dirs[0]
            if not isinstance(same_dir, np.ndarray):
                same_dir = np.array([same_dir])
                
            mean_same_dir = np.mean(same_dir)
            if np.isfinite(mean_same_dir):
                if mean_same_dir > self.stagnation_threshold:
                    print(f"DIAGNOSTIC: Stagnation threshold exceeded: {mean_same_dir:.4f} > {self.stagnation_threshold:.4f}")
                    return True
            else:
                print(f"DIAGNOSTIC: Non-finite mean detected in stagnation calculation: {same_dir}")
        
        return False
    
    def _adjust_learning_rate(self):
        """
        Adjust learning rate based on stagnation detection.
        
        Returns:
        --------
        float
            Updated learning rate
        """
        if not self.adaptive_learning:
            return self.base_learning_rate
        
        # If stagnation is detected, increase learning rate
        if self.debug_info['stagnation_detected']:
            # Increase learning rate up to max_learning_rate
            new_rate = min(
                self.learning_rate * 1.5,  # Increase by 50%
                self.max_learning_rate
            )
        else:
            # Gradually decay back to base_learning_rate
            new_rate = max(
                self.learning_rate * 0.95,  # Decrease by 5%
                self.base_learning_rate
            )
        
        self.learning_rate = new_rate
        self.debug_info['current_learning_rate'] = new_rate
        return new_rate
        
    def _update_model_parameters(self, features_dict, actual_return):
        """
        Gradual, stable parameter updates with support for continuous states.
        
        Parameters:
        -----------
        features_dict: dict
            Dictionary of feature values
        actual_return: float
            Actual return value
        """
        import numpy as np
    
        # Normalize features
        feature_values = self._normalize_features(features_dict)
    
        # Ensure learning rate is valid
        step_size = min(max(self.learning_rate, 0.0001), 0.2)
    
        # Get current prediction and expected value
        hidden_states = self._sample_hidden_states()
        
        # Use t-distribution for consistency
        predicted_pdf = self._predict_target_distribution_t(feature_values, hidden_states, df=5)
        expected_return = np.sum(predicted_pdf * self.bin_centers)
    
        # Safe error calculation with volatility-based scaling for large errors
        raw_error = actual_return - expected_return
        error_magnitude = abs(raw_error)
    
        # Scale down large errors gracefully instead of clipping
        # scaled_error = raw_error / (1.0 + error_magnitude/15.0)  
        #  Calculate volatility scale based on recent prediction history
        # Changed from 30 to 20 to match PCA window
        if len(self.prediction_history) >= 20:
            volatility_scale = max(np.std([p['expected_value'] for p in self.prediction_history[-20:]]), 1.0)
        else:
            volatility_scale = 15.0  # Default fallback
            
        scaled_error = raw_error / (1.0 + error_magnitude/volatility_scale)
        """
        Cont, R. (2001). "Empirical properties of asset returns: stylized facts and statistical issues." Quantitative Finance, 1(2), 223-236.
        Documents volatility clustering in financial returns, justifying dynamic error scaling

        Andersen, T. G., et al. (2003). "Modeling and forecasting realized volatility." Econometrica, 71(2), 579-625.
        Shows how adaptive volatility estimators improve financial time series models
        """
    
        # Calculate weight update with stability
        update = step_size * scaled_error * feature_values
    
        # Apply update with validation
        old_weights = self.target_weights.copy()
        #self.target_weights += update

        # Apply L2 regularization (already have weight_regularization parameter)
        regularization = self.weight_regularization * self.target_weights
        self.target_weights += update - regularization
        """
        MacKay, D. J. C. (1992). "Bayesian Interpolation." Neural Computation, 4(3), 415-447.
        Demonstrates how Bayesian regularization improves generalization in adaptive models  
        
        Tibshirani, R. (1996). "Regression Shrinkage and Selection via the Lasso." Journal of the Royal Statistical Society: Series B, 58(1), 267-288.    
        Shows how regularization reduces overfitting in models with many features
        """
    
        # Check for NaN or extreme values
        if not np.all(np.isfinite(self.target_weights)) or np.max(np.abs(self.target_weights)) > 50.0:
            # Revert to previous weights plus a small step
            self.target_weights = old_weights + 0.1 * update
            # Check again and apply minimal intervention if needed
            if not np.all(np.isfinite(self.target_weights)):
                # Keep old weights but add tiny regularization
                self.target_weights = old_weights * 0.999
    
        # Calculate and store weight norm
        weight_norm = np.linalg.norm(self.target_weights)
        self.debug_info['weight_norm'] = weight_norm if np.isfinite(weight_norm) else 0.0
    
        # Update bias more conservatively
        old_bias = self.target_bias
        self.target_bias += 0.5 * step_size * scaled_error
        
        # Validation:
        if not np.isfinite(self.target_bias) or abs(self.target_bias) > 50.0:
            self.target_bias = old_bias + 0.05 * step_size * scaled_error
            
        # Double-check after adjustment
        if not np.isfinite(self.target_bias):
            self.target_bias = old_bias * 0.999
    
        # Update standard deviation smoothly
        error_sq = scaled_error**2
        if np.isfinite(error_sq):
            self.target_std = 0.995 * self.target_std + 0.005 * np.sqrt(error_sq + 0.01)
            # Keep std in reasonable range
            self.target_std = max(min(self.target_std, 15.0), 0.01)
            
        # Update hidden state weights based on model type
        if self.hidden_layers > 0 and hidden_states is not None:
            if self.continuous_states:
                # For continuous states: update target state weights
                for layer, state in enumerate(hidden_states):
                    state = self._standardize_array(state, f"update_parameters_state_{layer}")
                    
                    # Skip invalid states with better type checking
                    try:
                        if not np.all(np.isfinite(state)):
                            continue
                    except Exception as e:
                        print(f"WARNING: Error checking state validity in _update_model_parameters: {e}")
                        continue
  
                    # Update target state weights for the layer
                    if layer < len(self.target_state_weights):
                        # Calculate gradient
                        old_weights = self.target_state_weights[layer].copy()
                        
                        # Apply smaller step size for stability
                        state_step_size = step_size * 0.5
                        
                        # Update with scaled error and small L2 regularization
                        self.target_state_weights[layer] += state_step_size * scaled_error * state
                        self.target_state_weights[layer] -= state_step_size * self.weight_regularization * old_weights
                        
                        # Check for invalid values
                        if not np.all(np.isfinite(self.target_state_weights[layer])):
                            # Revert with small regularization
                            self.target_state_weights[layer] = old_weights * 0.99
            else:
                # For discrete states: update hidden state weights
                for i, state in enumerate(hidden_states):
                    if i < len(self.target_hidden_weights) and state < len(self.target_hidden_weights[i]):
                        # Update with smaller step size for stability
                        old_weight = self.target_hidden_weights[i][state]
                        self.target_hidden_weights[i][state] += 0.3 * step_size * scaled_error
                        
                        # Apply regularization
                        self.target_hidden_weights[i][state] -= 0.3 * step_size * self.weight_regularization * old_weight
                        
                        # Check for invalid values
                        if not np.isfinite(self.target_hidden_weights[i][state]):
                            self.target_hidden_weights[i][state] = old_weight * 0.99
    
        # Update debug info
        if hasattr(self, 'debug_info'):
            self.debug_info['weight_norm'] = np.linalg.norm(self.target_weights)
            self.debug_info['bias_value'] = self.target_bias
            
            # Update steps since correct prediction
            if len(self.prediction_history) > 0:
                if actual_return * self.prediction_history[-1]['expected_value'] > 0:
                    # Prediction was directionally correct
                    self.debug_info['steps_since_update'] = 0
                else:
                    # Prediction was wrong
                    self.debug_info['steps_since_update'] += 1
    
    def learn_initial(self, train_features, train_target):
        """
        Initial learning on training data.
        
        Parameters:
        -----------
        train_features: pandas.DataFrame
            Training features
        train_target: pandas.Series
            Training target
        """
        
        print("Starting initial learning phase...")

        # Compute initial feature means and stds for normalization
        self.feature_means = train_features.mean().values
        self.feature_stds = train_features.std().values
        
        # Replace zeros with ones to avoid division by zero
        self.feature_stds[self.feature_stds == 0] = 1.0

        # Initialize feature history with training data
        if self.adaptive_normalization:
            print(" Initializing adaptive feature normalization...")

            # Pre-populate with training data but respect window size
            for _, row in train_features.iterrows():
                feature_values = np.array([row[feat] for feat in self.features])
                self.feature_history.append(feature_values)
                
                # MAINTAIN CONSISTENT WINDOW SIZE even during training
                if len(self.feature_history) > self.normalization_window:
                    self.feature_history.pop(0)     

            print(f"   Initialized with {len(self.feature_history)} historical feature vectors")
            print(f"   Using consistent {self.normalization_window}-day rolling window for adaptive normalization")

            # Mark as ready for adaptive normalization
            self.initial_training_done = True
        
            """
            # Convert training data to feature history format
            for _, row in train_features.iterrows():
                feature_values = np.array([row[feat] for feat in self.features])
                self.feature_history.append(feature_values)
            
            # Keep only the most recent window
            if len(self.feature_history) > self.normalization_window:
                self.feature_history = self.feature_history[-self.normalization_window:]
            """

        """
        # Convert training data to list of dictionaries
        features_list = []
        for _, row in train_features.iterrows():
            features_list.append(row.to_dict())
        """
        
        # Initialize target parameters based on data
        target_mean = train_target.mean()
        target_std = train_target.std()
        self.target_bias = target_mean
        self.target_std = target_std

        """
        # Sequential learning 
        for i, (features_dict, actual_return) in enumerate(zip(features_list, train_target)):
            try:
                # Update model
                try:
                    self._particle_filtering_update(features_dict, actual_return)
                except Exception as e:
                    print(f"Error in _particle_filtering_update at iteration {i}: {e}")
                    import traceback
                    traceback.print_exc()
                    raise
                    
                try:
                    self._update_model_parameters(features_dict, actual_return)
                except Exception as e:
                    print(f"Error in _update_model_parameters at iteration {i}: {e}")
                    import traceback
                    traceback.print_exc()
                    raise
                
                # Print progress
                if (i + 1) % 50 == 0 or i == len(features_list) - 1:
                    print(f"Processed {i+1}/{len(features_list)} training examples")

                    # Show normalization stats every 100 examples
                    if (i + 1) % 100 == 0 and self.adaptive_normalization:
                        stats = self.get_normalization_stats()
                        #print(f"   Feature history: {stats['feature_history_length']} days")
                    
            except Exception as e:
                print(f"Error at training iteration {i}: {e}")
                import traceback
                traceback.print_exc()
                raise
        
        # self.initial_training_done = True
        print("Initial learning phase completed.")
        """
        
        # Sequential learning with PROPER TEMPORAL ALIGNMENT
        print("Using lagged features for temporal consistency...")
        # Start from index 1 since we need previous day's features
        print(f"Training data alignment check:")
        print(f"  Total training days: {len(train_features)}")
        print(f"  Examples to process: {len(train_features)-1} (skip first day)")
        print(f"  First example: features from {train_features.index[0]} → target at {train_target.index[1]}")
        for i in range(1, len(train_features)):
            try:
                # Use PREVIOUS day's features to predict CURRENT day's return
                features_dict = train_features.iloc[i-1].to_dict()  # Features from day t-1
                actual_return = train_target.iloc[i]                # Return from t-1 to t
                
                # Update model
                try:
                    self._particle_filtering_update(features_dict, actual_return)
                except Exception as e:
                    print(f"Error in _particle_filtering_update at iteration {i}: {e}")
                    import traceback
                    traceback.print_exc()
                    raise
                    
                try:
                    self._update_model_parameters(features_dict, actual_return)
                except Exception as e:
                    print(f"Error in _update_model_parameters at iteration {i}: {e}")
                    import traceback
                    traceback.print_exc()
                    raise
                
                # Print progress - note we process len-1 examples now
                if i % 50 == 0 or i == len(train_features) - 1:
                    print(f"Processed {i}/{len(train_features)-1} training examples")
                    # Show normalization stats every 100 examples
                    if i % 100 == 0 and self.adaptive_normalization:
                        stats = self.get_normalization_stats()
                        #print(f"   Feature history: {stats['feature_history_length']} days")
                
            except Exception as e:
                print(f"Error at training iteration {i}: {e}")
                import traceback
                traceback.print_exc()
                raise
        
        self.initial_training_done = True
        print("Initial learning phase completed.")
    
    def predict_next_day(self, features_dict):
        """
        Generate prediction distribution for next day.
        Supports both discrete and continuous states.
        
        Parameters:
        -----------
        features_dict: dict
            Dictionary of feature values
            
        Returns:
        --------
        dict
            Prediction information including PDF, most likely value, etc.
        """
        import numpy as np
        
        # Validate and fix feature names if needed
        fixed_features_dict = self.validate_and_fix_feature_names(features_dict)
        
        # Normalize features
        feature_values = self._normalize_features(fixed_features_dict)
        
        # Update particle distribution (prediction step only)
        self._particle_filtering_update(fixed_features_dict)
        
        # Sample hidden states
        hidden_states = self._sample_hidden_states()
        
        # Predict target distribution
        #pdf = self._predict_target_distribution(feature_values, hidden_states)
        # Predict target distribution with t-distribution
        pdf = self._predict_target_distribution_t(feature_values, hidden_states, df=5)
        """
        Blattberg, R. C., & Gonedes, N. J. (1974). "A comparison of the stable and student distributions as statistical models for stock prices." The Journal of Business, 47(2), 244-280.
        Demonstrates Student's t-distribution better fits financial returns

        McNeil, A. J., Frey, R., & Embrechts, P. (2015). "Quantitative risk management: Concepts, techniques and tools." Princeton University Press.       
        Provides comprehensive evidence on fat-tailed distributions in risk modeling
        """
        
        # Find most likely value (peak of distribution)
        peak_idx = np.argmax(pdf)
        most_likely_value = self.bin_centers[peak_idx]
        
        # Calculate expected value (mean of distribution)
        expected_value = np.sum(pdf * self.bin_centers)
        
        # Calculate probability of positive return
        positive_prob = np.sum(pdf[self.bin_centers > 0])
        
        # Calculate 95% confidence interval
        cum_pdf = np.cumsum(pdf)
        lower_idx = np.searchsorted(cum_pdf, 0.025)
        upper_idx = np.searchsorted(cum_pdf, 0.975)
        confidence_interval = (
            self.bin_centers[max(0, lower_idx)],
            self.bin_centers[min(len(self.bin_centers) - 1, upper_idx)]
        )
        
        # Create prediction result
        prediction = {
            'pdf': pdf,
            'bin_centers': self.bin_centers,
            'most_likely': most_likely_value,
            'expected_value': expected_value,
            'positive_prob': positive_prob,
            'confidence_interval': confidence_interval,
            'direction': 1 if expected_value > 0 else -1
        }
        
        # Detect stagnation
        if self.enable_anti_stagnation:
            # Store prediction for stagnation detection
            self.prediction_history.append({
                'direction': prediction['direction'],
                'expected_value': prediction['expected_value'],
                'positive_prob': prediction['positive_prob']
            })
            
            # Keep prediction history to a reasonable size
            if len(self.prediction_history) > self.stagnation_window * 2:
                self.prediction_history = self.prediction_history[-(self.stagnation_window * 2):]
                
            # Detect stagnation
            self.debug_info['stagnation_detected'] = self._detect_stagnation()
            
            # Adjust learning rate
            self._adjust_learning_rate()
        
        return prediction
    
    def update_with_actual(self, features_dict, actual_return):
        """
        Update model with actual return value.
        Supports both discrete and continuous states.
        
        Parameters:
        -----------
        features_dict: dict
            Dictionary of feature values
        actual_return: float
            Actual return value
        """
        import numpy as np
        
        # Validate and fix feature names if needed
        fixed_features_dict = self.validate_and_fix_feature_names(features_dict)

        # FOR ADAPTIVE NORMALIZATION: Extract raw features BEFORE any processing
        if self.adaptive_normalization and self.initial_training_done:
            raw_feature_values = np.zeros(self.n_features)
            for i, feature in enumerate(self.features):
                value = fixed_features_dict.get(feature, 0)
                raw_feature_values[i] = value
            
            # Add to history for next normalization calculation
            self.feature_history.append(raw_feature_values.copy())
            
            # Maintain window size
            if len(self.feature_history) > self.normalization_window:
                self.feature_history.pop(0)

        # Update particle distribution with new observation
        try:
            self._particle_filtering_update(fixed_features_dict, actual_return)
        except Exception as e:
            print(f"Error in _particle_filtering_update during update_with_actual() at iteration {i}: {e}")
            import traceback
            traceback.print_exc()
            raise

        # Update model parameters
        try:
            self._update_model_parameters(fixed_features_dict, actual_return)
        except Exception as e:
            print(f"Error in _update_model_parameters during update_with_actual() at iteration {i}: {e}")
            import traceback
            traceback.print_exc()
            raise

        # Update prediction history with correctness info
        if len(self.prediction_history) > 0:
            prediction_direction = self.prediction_history[-1]['direction']
            actual_direction = 1 if actual_return > 0 else -1
            self.prediction_history[-1]['was_correct'] = (prediction_direction == actual_direction)

        # Track rolling feature importance
        feature_values = self._normalize_features(fixed_features_dict)
        
        if not hasattr(self, 'rolling_feature_importance'):
            self.rolling_feature_importance = {}

        for i, (feat, weight) in enumerate(zip(feature_values, self.target_weights)):
            if i < len(self.features): # Safety check
                feat_name = self.features[i]
                contribution = feat * weight
                
                if feat_name not in self.rolling_feature_importance:
                    self.rolling_feature_importance[feat_name] = []

                # Store absolute contribution for importance ranking
                self.rolling_feature_importance[feat_name].append(abs(contribution))

                # Keep limited history to avoid memory issues
                if len(self.rolling_feature_importance[feat_name]) > 100:
                    self.rolling_feature_importance[feat_name].pop(0)

        """
        Breiman, L. (2001). "Random Forests." Machine Learning, 45(1), 5-32.
        Establishes variable importance through contribution metrics
        
        Haugen, R. A., & Baker, N. L. (1996). "Commonality in the determinants of expected stock returns." Journal of Financial Economics, 41(3), 401-439.      
        Demonstrates how feature importance in financial markets varies over time
        """
        
        # Update debug info
        if len(self.prediction_history) > 0:
            if actual_return * self.prediction_history[-1]['expected_value'] > 0:
                # Prediction was directionally correct
                self.debug_info['steps_since_update'] = 0
            else:
                # Prediction was wrong
                self.debug_info['steps_since_update'] += 1

    def get_rolling_feature_importance(self, window=20): # 20 because of PCA_Window
        """
        Get rolling feature importance based on recent contributions.
        Aligned with PCA window for consistency.
        
        Parameters:
        -----------
        window: int
            Number of recent samples to consider
        
        Returns:
        --------
        pandas.DataFrame
            DataFrame with feature importance scores
        """
        if not hasattr(self, 'rolling_feature_importance'):
            return None
            
        import pandas as pd
        import numpy as np
        
        # Calculate mean importance for each feature over recent window
        importance = {}
        for feat, values in self.rolling_feature_importance.items():
            if len(values) > 0:
                # Use most recent values up to window size
                recent_values = values[-min(window, len(values)):]
                importance[feat] = np.mean(recent_values)
        
        # Convert to DataFrame and sort
        if importance:
            df = pd.DataFrame({'feature': list(importance.keys()), 
                               'importance': list(importance.values())})
            return df.sort_values('importance', ascending=False).reset_index(drop=True)
        
        return None
    
    def get_debug_info(self):
        """
        Get debug information about the model state.
        
        Returns:
        --------
        dict
            Debug information
        """
        return self.debug_info
    
    def plot_prediction_distribution(self, prediction):
        """
        Plot the prediction distribution.
        
        Parameters:
        -----------
        prediction: dict
            Prediction dictionary from predict_next_day()
            
        Returns:
        --------
        matplotlib.figure.Figure
            The figure object
        """
        
        plt.figure(figsize=(10, 6))
        
        # Plot PDF
        plt.plot(prediction['bin_centers'], prediction['pdf'], 'b-', linewidth=2)
        
        # Plot most likely value
        plt.axvline(prediction['most_likely'], color='r', linestyle='--',
                   label=f"Most likely: {prediction['most_likely']:.2f}%")
        
        # Plot expected value
        plt.axvline(prediction['expected_value'], color='g', linestyle='--',
                   label=f"Expected: {prediction['expected_value']:.2f}%")
        
        # Plot confidence interval
        plt.axvline(prediction['confidence_interval'][0], color='k', linestyle=':',
                   label=f"95% CI: [{prediction['confidence_interval'][0]:.2f}, {prediction['confidence_interval'][1]:.2f}]%")
        plt.axvline(prediction['confidence_interval'][1], color='k', linestyle=':')
        
        # Plot zero line
        plt.axvline(0, color='k', alpha=0.3)
        
        # Add shading for positive/negative regions
        pos_mask = prediction['bin_centers'] > 0
        if np.any(pos_mask):
            plt.fill_between(
                prediction['bin_centers'][pos_mask],
                prediction['pdf'][pos_mask],
                alpha=0.3, color='g', 
                label=f"P(positive): {prediction['positive_prob']:.2f}"
            )
        
        neg_mask = prediction['bin_centers'] <= 0
        if np.any(neg_mask):
            plt.fill_between(
                prediction['bin_centers'][neg_mask],
                prediction['pdf'][neg_mask],
                alpha=0.3, color='r', 
                label=f"P(negative): {1-prediction['positive_prob']:.2f}"
            )
        
        # Add stagnation info if detected
        if self.enable_anti_stagnation and self.debug_info['stagnation_detected']:
            plt.annotate(
                "STAGNATION DETECTED\nAdaptive LR: {:.4f}".format(self.learning_rate),
                xy=(0.5, 0.9),
                xycoords='axes fraction',
                bbox=dict(boxstyle="round,pad=0.3", fc="yellow", alpha=0.5),
                ha='center'
            )
        
        plt.title("Predicted Return Distribution")
        plt.xlabel("Return (%)")
        plt.ylabel("Probability Density")
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        
        return plt.gcf()

"""
Scientific Support for Continuous Hidden States
This implementation is backed by significant academic research:
1. Linear State Space Models
Fox et al. (2011), "Bayesian Nonparametric Methods for Learning Markov Switching Processes," Journal of Machine Learning Research, 12, 1697-1724

Demonstrates continuous state representations significantly outperform discrete regimes for financial time series
Shows how Bayesian nonparametric models can adaptively model market dynamics

Otranto, E. (2010), "Identifying Financial Time Series with Similar Dynamic Conditional Correlation," Computational Statistics & Data Analysis, 54(1), 1-15

Demonstrates continuous state variables better capture correlation dynamics in financial markets
Provides experimental evidence for smoother transition modeling

2. State Transition Learning
Ghahramani & Hinton (2000), "Variational Learning for Switching State-Space Models," Neural Computation, 12(4), 831-864

Establishes theoretical foundations for learning parameters in state-space models
Introduces methods for learning state transitions directly from observed data

Kim & Nelson (1999), "State-Space Models with Regime Switching," MIT Press

Comprehensive treatment showing continuous representations outperform discrete states
Provides evidence from financial market applications

3. Continuous vs. Discrete Regimes
Lux, T. (2011), "Sentiment Dynamics and Stock Returns: The Case of the German Stock Market," Empirical Economics, 41, 663-679

Shows continuous state representations better capture market sentiment dynamics
Demonstrates how rapid transitions between regimes are missed by discrete models

Chopin, N. (2007), "Dynamic Detection of Change Points in Long Time Series," Annals of the Institute of Statistical Mathematics, 59(2), 349-366

Provides evidence for continuously evolving regime probabilities
Demonstrates improved forecasting with continuous model parameterization

The implementation synthesizes these research findings with robust numerical methods to ensure stability and performance in real-world financial applications.
"""

'\nScientific Support for Continuous Hidden States\nThis implementation is backed by significant academic research:\n1. Linear State Space Models\nFox et al. (2011), "Bayesian Nonparametric Methods for Learning Markov Switching Processes," Journal of Machine Learning Research, 12, 1697-1724\n\nDemonstrates continuous state representations significantly outperform discrete regimes for financial time series\nShows how Bayesian nonparametric models can adaptively model market dynamics\n\nOtranto, E. (2010), "Identifying Financial Time Series with Similar Dynamic Conditional Correlation," Computational Statistics & Data Analysis, 54(1), 1-15\n\nDemonstrates continuous state variables better capture correlation dynamics in financial markets\nProvides experimental evidence for smoother transition modeling\n\n2. State Transition Learning\nGhahramani & Hinton (2000), "Variational Learning for Switching State-Space Models," Neural Computation, 12(4), 831-864\n\nEstablishes theoretical foundatio

# 4. Performance Metrics

In [9]:
class PerformanceTracker:
    def __init__(
        self,
        initial_capital=1000,
        start_tracking_date=None,
        risk_free_rate=0.02/252, # Daily risk-free rate (approx. 2% annual)
        leverage_threshold_std=1.0, # Standard deviation threshold for leverage
        max_leverage=5, # Maximum leverage allowed
    ):
        """
        Initialize enhanced performance tracker with multiple strategies.
        """
        self.initial_capital = initial_capital
        self.start_tracking_date = start_tracking_date
        self.risk_free_rate = risk_free_rate
        self.leverage_threshold_std = leverage_threshold_std
        self.max_leverage = max_leverage
        
        # Initialize tracking variables
        self.predictions = {}
        self.actual_returns = {}
        self.dates = []
        self.correct_risk_avoidances = 0 # Correctly predicted DOWN and market went DOWN
        self.total_risk_predictions = 0 # Total DOWN predictions
        self.days_in_market = 0 # Count of days in the market
        self.trade_dates = [] # Dates when trades were executed
        self.recent_trade_window = 20 # Window for recent trades count
        
        # Initialize the derived metrics
        self.accuracy = 0.0
        self.risk_avoidance_rate = 0.0
        self.market_participation_rate = 0.0
        self.recent_trades_count = 0
        
        # Basic Strategy Tradinghours variables
        self.portfolio_values = {start_tracking_date: initial_capital} if start_tracking_date else {}
        self.in_market = False
        self.previous_prediction = None
        self.is_first_update = True
        self.daily_returns = []
        self.trade_directions = [] # 1 for long, -1 for short, 0 for no trade
        self.wins_tradinghours = []
        self.losses_tradinghours = []
        self.win_dates_tradinghours = []
        self.loss_dates_tradinghours = []
        
        # Basic Strategy Afterhours variables
        self.portfolio_values_afterhours = {start_tracking_date: initial_capital} if start_tracking_date else {}
        self.in_market_afterhours = False
        self.daily_returns_afterhours = []
        self.trade_directions_afterhours = [] # 1 for long, -1 for short, 0 for no trade
        self.wins_afterhours = []
        self.losses_afterhours = []
        self.win_dates_afterhours = []
        self.loss_dates_afterhours = []
        self.correct_risk_avoidances_afterhours = 0
        self.total_risk_predictions_afterhours = 0
        self.days_in_market_afterhours = 0

        # Next day trading strategy variables
        self.nextday_values = {start_tracking_date: initial_capital} if start_tracking_date else {}
        self.in_market_nextday = False
        self.daily_returns_nextday = []
        self.trade_directions_nextday = []  # 1 for long, -1 for short, 0 for no trade
        self.wins_nextday = []
        self.losses_nextday = []
        self.win_dates_nextday = []
        self.loss_dates_nextday = []
        self.trade_count_nextday = 0
        # Win/Loss tracking for nextday strategy
        self.consecutive_wins_nextday = 0
        self.consecutive_losses_nextday = 0
        self.max_consecutive_wins_nextday = 0
        self.max_consecutive_losses_nextday = 0
        self.max_consecutive_wins_start_date_nextday = None
        self.max_consecutive_wins_end_date_nextday = None
        self.max_consecutive_losses_start_date_nextday = None
        self.max_consecutive_losses_end_date_nextday = None
        self.current_streak_start_date_nextday = None
        
        # Leverage strategy variables
        self.leverage_values = {start_tracking_date: initial_capital} if start_tracking_date else {}
        self.leverage_in_market = False
        self.leverage_multiplier = 1.0
        self.leverage_daily_returns = []
        self.leverage_trade_directions = []
        self.leverage_factors = [] # Track applied leverage
        
        # Shorting strategy variables
        self.shorting_values = {start_tracking_date: initial_capital} if start_tracking_date else {}
        self.shorting_position = 0 # 0 for no position, 1 for long, -1 for short
        self.shorting_daily_returns = []
        self.shorting_trade_directions = []
        self.positive_peaks = []         # For tracking distribution peaks for advanced strategies
        self.negative_peaks = []

        # Short+Leverage strategy variables
        self.short_leverage_values = {start_tracking_date: initial_capital} if start_tracking_date else {}
        self.short_leverage_position = 0  # 0 for no position, positive for long (leverage value), negative for short (leverage value)
        self.short_leverage_daily_returns = []
        self.short_leverage_position_history = []
        
        # Buy and hold strategy
        self.buyhold_values = {start_tracking_date: initial_capital} if start_tracking_date else {}
        self.buyhold_returns = []
        
        # Directional accuracy tracking
        self.direction_correct = []
        self.direction_correct_dates = []
        
        # Prediction accuracy metrics
        self.prediction_errors = []
        self.peak_errors = []
        self.expected_value_errors = []
        self.prediction_distribution_centers = [] # To track center of mass
        
        # Win/Loss tracking for trading hours
        self.consecutive_wins = 0
        self.consecutive_losses = 0
        self.max_consecutive_wins = 0
        self.max_consecutive_losses = 0
        self.max_consecutive_wins_start_date = None
        self.max_consecutive_wins_end_date = None
        self.max_consecutive_losses_start_date = None
        self.max_consecutive_losses_end_date = None
        self.current_streak_start_date = None
        
        # Win/Loss tracking for after hours
        self.consecutive_wins_afterhours = 0
        self.consecutive_losses_afterhours = 0
        self.max_consecutive_wins_afterhours = 0
        self.max_consecutive_losses_afterhours = 0
        self.max_consecutive_wins_start_date_afterhours = None
        self.max_consecutive_wins_end_date_afterhours = None
        self.max_consecutive_losses_start_date_afterhours = None
        self.max_consecutive_losses_end_date_afterhours = None
        self.current_streak_start_date_afterhours = None

        # Add missing return metrics that were causing errors
        self.total_return_tradinghours = 0
        self.total_return_afterhours = 0
        self.max_drawdown_tradinghours = 0
        self.max_drawdown_afterhours = 0
        self.max_drawdown_nextday = 0
        self.leverage_max_drawdown = 0
        self.shorting_max_drawdown = 0
        self.buyhold_max_drawdown = 0
        self.short_leverage_max_drawdown = 0
        
        self.win_rate_tradinghours = 0.0
        self.win_rate_afterhours = 0.0
        self.win_rate_nextday = 0.0
        self.avg_gain_tradinghours = 0.0
        self.avg_loss_tradinghours = 0.0 
        self.avg_gain_afterhours = 0.0
        self.avg_loss_afterhours = 0.0
        self.avg_gain_nextday = 0.0
        self.avg_loss_nextday = 0.0
        self.gain_loss_ratio_tradinghours = 0.0
        self.gain_loss_ratio_afterhours = 0.0
        self.gain_loss_ratio_nextday = 0.0
        self.trade_count_nextday = 0
        self.profit_factor_tradinghours = 0.0
        self.profit_factor_afterhours = 0.0
        self.profit_factor_nextday = 0.0
        self.profit_factor_leverage = 0.0
        self.profit_factor_shorting = 0.0
        self.profit_factor_short_leverage = 0.0
        self.sqn_tradinghours = 0.0
        self.sqn_afterhours = 0.0
        self.sqn_nextday = 0.0
        self.sqn_leverage = 0.0
        self.sqn_shorting = 0.0
        self.sqn_short_leverage = 0.0
        
        # Add missing attributes for beta calculations
        self.beta_tradinghours = 0.0
        self.beta_afterhours = 0.0
        self.leverage_beta = 0.0
        self.shorting_beta = 0.0
        self.trading_frequency_tradinghours = 0.0
        self.trading_frequency_afterhours = 0.0
        
        # Add annual return attributes
        self.annual_return_tradinghours = None
        self.annual_return_afterhours = None
        self.annual_return_nextday = None
        self.leverage_annual_return = None
        self.shorting_annual_return = None
        self.annual_return_short_leverage = None
        self.buyhold_annual_return = None
        
        self.total_return_nextday = 0
        self.max_drawdown_nextday = 0
        self.sharpe_ratio_nextday = 0
        self.sortino_ratio_nextday = 0
        self.win_rate_nextday = 0
        self.gain_loss_ratio_nextday = 0
        
        self.leverage_return = 0
        self.shorting_return = 0
        self.buyhold_return = 0
        self.leverage_sharpe_ratio = 0
        self.leverage_sortino_ratio = 0
        self.shorting_sharpe_ratio = 0
        self.shorting_sortino_ratio = 0
        self.buyhold_sharpe_ratio = 0
        self.buyhold_sortino_ratio = 0
        
        self.short_leverage_return = 0
        self.short_leverage_max_drawdown = 0
        self.short_leverage_sharpe_ratio = 0
        self.short_leverage_sortino_ratio = 0
        
        # Trading frequency tracking
        self.trade_count = 0
        self.trade_count_afterhours = 0
        self.total_days = 0
        self.monthly_trade_counts = {}
        self.monthly_trade_counts_afterhours = {}
        
        # Calibration data
        self.predicted_probs = []
        self.actual_outcomes = []
        
        # Confusion matrix data
        self.true_positives = 0
        self.false_positives = 0
        self.true_negatives = 0
        self.false_negatives = 0

        # For tracking enhanced error metrics
        self.abs_peak_errors = []  # Absolute peak errors
        self.abs_expected_value_errors = []  # Absolute expected value errors
        
        # Separate tracking for positive and negative predictions
        self.pos_pred_abs_peak_errors = []  # Absolute peak errors when prediction is positive
        self.neg_pred_abs_peak_errors = []  # Absolute peak errors when prediction is negative
        self.pos_pred_abs_ev_errors = []    # Absolute expected value errors when prediction is positive
        self.neg_pred_abs_ev_errors = []    # Absolute expected value errors when prediction is negative
        
        # Directional accuracy tracking
        self.direction_correct_positive = []  # Correct direction when predicted positive
        self.direction_correct_negative = []  # Correct direction when predicted negative
        
        # Store actual returns for volatility ranges
        self.actual_returns_history = []  # All actual returns
        
        # Recent window tracking
        self.recent_window = 100  # Changed from 20 to 100 days
        
        # Volatility buckets for errors and accuracy
        self.vol_buckets = {
            '1_std': {'peak_errors': [], 'ev_errors': [], 'dir_correct': [], 'total': 0},
            '2_std': {'peak_errors': [], 'ev_errors': [], 'dir_correct': [], 'total': 0},
            '3_std': {'peak_errors': [], 'ev_errors': [], 'dir_correct': [], 'total': 0},
            '4_std': {'peak_errors': [], 'ev_errors': [], 'dir_correct': [], 'total': 0},
            'other': {'peak_errors': [], 'ev_errors': [], 'dir_correct': [], 'total': 0}
        }
        
        # Create a DataFrame to store detailed performance data
        self.performance_data = pd.DataFrame(columns=[
            'date', 'actual_return', 'predicted_direction', 'predicted_return',
            'direction_correct', 'basic_tradinghours_return', 'basic_afterhours_return', 'leverage_return', 'shorting_return',
            'buyhold_return', 'basic_tradinghours_value', 'basic_afterhours_value', 'leverage_value', 'shorting_value',
            'buyhold_value', 'leverage_factor', 'trade_direction_tradinghours', 'trade_direction_afterhours', 'shorting_position',
            'peak_error', 'expected_value_error', 'positive_prob'
        ])

    def update(self, prediction, actual_return, date):
        """
        Update tracker with new prediction and actual return.
        Parameters:
        -----------
        prediction: dict
            Prediction dictionary from DBN model for the CURRENT day
        actual_return: float
            Actual return value for the CURRENT day
        date: datetime
            Date of the CURRENT day
        """
        if self.start_tracking_date is None or date >= self.start_tracking_date:
            try:
                # Initialize trade_direction variables
                trade_direction_tradinghours = 0
                trade_direction_afterhours = 0
                trade_direction_nextday = 0 
            
                # Store prediction and actual return
                self.predictions[date] = prediction
                self.actual_returns[date] = actual_return
                self.dates.append(date)
                self.total_days += 1
                
                # Track market participation
                if self.in_market:
                    self.days_in_market += 1
                if self.in_market_afterhours:
                    self.days_in_market_afterhours += 1
                
                # Track risk avoidance (after we've processed at least one day)
                if self.previous_prediction is not None:
                    prev_predicted_direction = self.previous_prediction['direction']
                    
                    # Track for trading hours strategy
                    if prev_predicted_direction == -1:
                        self.total_risk_predictions += 1
                        # Check if the prediction was correct (market did go down)
                        if actual_return < 0:
                            self.correct_risk_avoidances += 1
                    
                    # Track for after hours strategy
                    if prediction['direction'] == -1:
                        self.total_risk_predictions_afterhours += 1
                        # Check if the prediction was correct (market did go down)
                        if actual_return < 0:
                            self.correct_risk_avoidances_afterhours += 1
                
            
                # ========== MODEL ACCURACY EVALUATION ==========
                predicted_direction = prediction['direction']
                actual_direction = 1 if actual_return > 0 else -1
            
                # Check if prediction was correct
                is_correct = (predicted_direction == actual_direction)
                self.direction_correct.append(is_correct)
                self.direction_correct_dates.append(date)
                
                # Calculate prediction error (magnitude)
                # 1. Peak error (error between most likely value and actual)
                peak_error = prediction['most_likely'] - actual_return
                self.peak_errors.append(peak_error)
                
                # 2. Expected value error (error between expected value and actual)
                expected_value_error = prediction['expected_value'] - actual_return
                self.expected_value_errors.append(expected_value_error)
                
                # 3. Overall prediction error (absolute difference)
                error = abs(prediction['most_likely'] - actual_return)
                self.prediction_errors.append(error)
                
                # Store the center of mass of the prediction distribution
                self.prediction_distribution_centers.append(prediction['expected_value'])
                
                # Update confusion matrix data
                if predicted_direction == 1 and actual_direction == 1:
                    self.true_positives += 1
                elif predicted_direction == 1 and actual_direction == -1:
                    self.false_positives += 1
                elif predicted_direction == -1 and actual_direction == -1:
                    self.true_negatives += 1
                elif predicted_direction == -1 and actual_direction == 1:
                    self.false_negatives += 1
                
                # Update calibration data
                self.predicted_probs.append(prediction['positive_prob'])
                self.actual_outcomes.append(1 if actual_return > 0 else 0)
                
                # ========== TRADING STRATEGIES SIMULATION ==========
                # Calculate previous portfolio values
                prev_date = list(self.portfolio_values.keys())[-1] if self.portfolio_values else date
                prev_basic_tradinghours_value = self.portfolio_values.get(prev_date, self.initial_capital)
                prev_basic_afterhours_value = self.portfolio_values_afterhours.get(prev_date, self.initial_capital)
                prev_leverage_value = self.leverage_values.get(prev_date, self.initial_capital)
                prev_shorting_value = self.shorting_values.get(prev_date, self.initial_capital)
                prev_buyhold = self.buyhold_values.get(prev_date, self.initial_capital)
                
                # Update buy-and-hold strategy (always in the market)
                new_buyhold = prev_buyhold * (1 + actual_return / 100)
                self.buyhold_values[date] = new_buyhold
                self.buyhold_returns.append(actual_return / 100)
                
                # On the first update, we don't have a previous prediction to trade on
                # So we just initialize the portfolios without trading
                if self.is_first_update:
                    self.portfolio_values[date] = prev_basic_tradinghours_value
                    self.portfolio_values_afterhours[date] = prev_basic_afterhours_value
                    self.leverage_values[date] = prev_leverage_value
                    self.shorting_values[date] = prev_shorting_value
                    self.nextday_values[date] = self.initial_capital  # Initialize nextday value
                    self.short_leverage_values[date] = prev_shorting_value
                    self.daily_returns.append(0)
                    self.daily_returns_afterhours.append(0)
                    self.daily_returns_nextday.append(0)  # Initialize nextday return
                    self.leverage_daily_returns.append(0)
                    self.shorting_daily_returns.append(0)
                    self.short_leverage_daily_returns.append(0)
                    self.trade_directions.append(0)
                    self.trade_directions_afterhours.append(0)
                    self.trade_directions_nextday.append(0)  # Initialize nextday direction
                    self.leverage_trade_directions.append(0)
                    self.shorting_trade_directions.append(0)
                    self.short_leverage_position_history.append(0)
                    self.leverage_factors.append(1.0)
                    self.is_first_update = False
                    self.previous_prediction = prediction # Store for next day
                    
                    # Initialize performance data row
                    row_data = {
                        'date': date,
                        'actual_return': actual_return,
                        'predicted_direction': predicted_direction,
                        'predicted_return': prediction['expected_value'],
                        'direction_correct': is_correct,
                        'basic_tradinghours_return': 0,
                        'basic_afterhours_return': 0,
                        'leverage_return': 0,
                        'shorting_return': 0,
                        'buyhold_return': actual_return / 100,
                        'basic_tradinghours_value': prev_basic_tradinghours_value,
                        'basic_afterhours_value': prev_basic_afterhours_value,
                        'leverage_value': prev_leverage_value,
                        'shorting_value': prev_shorting_value,
                        'buyhold_value': new_buyhold,
                        'leverage_factor': 1.0,
                        'trade_direction_tradinghours': 0,
                        'trade_direction_afterhours': 0,
                        'shorting_position': 0,
                        'peak_error': peak_error,
                        'expected_value_error': expected_value_error,
                        'positive_prob': prediction['positive_prob']
                    }
                    
                    # Add the row to the performance data DataFrame
                    self.performance_data = pd.concat([self.performance_data, pd.DataFrame([row_data])], ignore_index=True)
                    return
                
                # ========== BASIC STRATEGY TRADINGHOURS EXECUTION ==========
                # EXECUTE ACTION based on PREVIOUS day's prediction (realistic)
                prev_predicted_direction = self.previous_prediction['direction']
                basic_tradinghours_return = 0
                
                if self.in_market: # Currently holding stock
                    if prev_predicted_direction == -1: # Previous prediction was DOWN
                        # We already sold at the close of the previous day
                        self.in_market = False
                        # We don't capture today's return because we sold yesterday
                        basic_tradinghours_return = 0
                        new_basic_tradinghours_value = prev_basic_tradinghours_value
                        self.trade_count += 1
                        trade_direction_tradinghours = -1
                    else: # Previous prediction was UP, stay in the market
                        # We remain in the market, so capture today's return
                        basic_tradinghours_return = actual_return / 100
                        new_basic_tradinghours_value = prev_basic_tradinghours_value * (1 + basic_tradinghours_return)
                        trade_direction_tradinghours = 1
                else: # Currently out of the market
                    if prev_predicted_direction == 1: # Previous prediction was UP
                        # Buy at the open of today
                        self.in_market = True
                        # Capture today's return since we're in the market
                        basic_tradinghours_return = actual_return / 100
                        new_basic_tradinghours_value = prev_basic_tradinghours_value * (1 + basic_tradinghours_return)
                        self.trade_count += 1
                        trade_direction_tradinghours = 1
                    else: # Previous prediction was DOWN, stay out of the market
                        new_basic_tradinghours_value = prev_basic_tradinghours_value # No change
                        basic_tradinghours_return = 0
                        trade_direction_tradinghours = 0
                
                self.portfolio_values[date] = new_basic_tradinghours_value
                self.daily_returns.append(basic_tradinghours_return)
                self.trade_directions.append(trade_direction_tradinghours)
                
                # Track wins and losses for basic strategy tradinghours
                if basic_tradinghours_return > 0:
                    self.wins_tradinghours.append(basic_tradinghours_return)
                    self.win_dates_tradinghours.append(date)
                    self.consecutive_wins += 1
                    self.consecutive_losses = 0
                    
                    # Update max consecutive wins
                    if self.consecutive_wins > self.max_consecutive_wins:
                        self.max_consecutive_wins = self.consecutive_wins
                        self.max_consecutive_wins_end_date = date
                        if self.current_streak_start_date:
                            self.max_consecutive_wins_start_date = self.current_streak_start_date
                    
                    # Set start date for current streak if needed
                    if self.consecutive_wins == 1:
                        self.current_streak_start_date = date
                
                elif basic_tradinghours_return < 0:
                    self.losses_tradinghours.append(basic_tradinghours_return)
                    self.loss_dates_tradinghours.append(date)
                    self.consecutive_losses += 1
                    self.consecutive_wins = 0
                    
                    # Update max consecutive losses
                    if self.consecutive_losses > self.max_consecutive_losses:
                        self.max_consecutive_losses = self.consecutive_losses
                        self.max_consecutive_losses_end_date = date
                        if self.current_streak_start_date:
                            self.max_consecutive_losses_start_date = self.current_streak_start_date
                    
                    # Set start date for current streak if needed
                    if self.consecutive_losses == 1:
                        self.current_streak_start_date = date
                        
                # ========== BASIC STRATEGY AFTERHOURS EXECUTION ==========
                # EXECUTE ACTION based on CURRENT day's prediction (realistic)
                # For afterhours trades, we use the CURRENT prediction, not previous
                basic_afterhours_return = 0
                
                if self.in_market_afterhours: # Currently holding stock
                    if prediction['direction'] == -1: # Current prediction is DOWN
                        # Sell at the close of the day
                        self.in_market_afterhours = False
                        # We already captured today's return
                        basic_afterhours_return = actual_return / 100
                        new_basic_afterhours_value = prev_basic_afterhours_value * (1 + basic_afterhours_return)
                        self.trade_count_afterhours += 1
                        trade_direction_afterhours = -1
                    else: # Current prediction is UP, stay in the market
                        basic_afterhours_return = actual_return / 100
                        new_basic_afterhours_value = prev_basic_afterhours_value * (1 + basic_afterhours_return)
                        trade_direction_afterhours = 1
                else: # Currently out of the market
                    if prediction['direction'] == 1: # Current prediction is UP
                        # Buy at the close of the day, but we don't capture today's return since we buy at close
                        self.in_market_afterhours = True
                        # For afterhours, we buy at close, so we don't capture today's return
                        basic_afterhours_return = 0
                        new_basic_afterhours_value = prev_basic_afterhours_value  # No change today
                        self.trade_count_afterhours += 1
                        trade_direction_afterhours = 1
                    else: # Current prediction is DOWN, stay out of the market
                        new_basic_afterhours_value = prev_basic_afterhours_value # No change
                        basic_afterhours_return = 0
                        trade_direction_afterhours = 0
                
                self.portfolio_values_afterhours[date] = new_basic_afterhours_value
                self.daily_returns_afterhours.append(basic_afterhours_return)
                self.trade_directions_afterhours.append(trade_direction_afterhours)
                
                # Track wins and losses for basic strategy afterhours
                if basic_afterhours_return > 0:
                    self.wins_afterhours.append(basic_afterhours_return)
                    self.win_dates_afterhours.append(date)
                    self.consecutive_wins_afterhours += 1
                    self.consecutive_losses_afterhours = 0
                    
                    # Update max consecutive wins
                    if self.consecutive_wins_afterhours > self.max_consecutive_wins_afterhours:
                        self.max_consecutive_wins_afterhours = self.consecutive_wins_afterhours
                        self.max_consecutive_wins_end_date_afterhours = date
                        if self.current_streak_start_date_afterhours:
                            self.max_consecutive_wins_start_date_afterhours = self.current_streak_start_date_afterhours
                    
                    # Set start date for current streak if needed
                    if self.consecutive_wins_afterhours == 1:
                        self.current_streak_start_date_afterhours = date
                
                elif basic_afterhours_return < 0:
                    self.losses_afterhours.append(basic_afterhours_return)
                    self.loss_dates_afterhours.append(date)
                    self.consecutive_losses_afterhours += 1
                    self.consecutive_wins_afterhours = 0
                    
                    # Update max consecutive losses
                    if self.consecutive_losses_afterhours > self.max_consecutive_losses_afterhours:
                        self.max_consecutive_losses_afterhours = self.consecutive_losses_afterhours
                        self.max_consecutive_losses_end_date_afterhours = date
                        if self.current_streak_start_date_afterhours:
                            self.max_consecutive_losses_start_date_afterhours = self.current_streak_start_date_afterhours
                    
                    # Set start date for current streak if needed
                    if self.consecutive_losses_afterhours == 1:
                        self.current_streak_start_date_afterhours = date
    
                # ========== NEXT DAY TRADING STRATEGY EXECUTION ==========
                # Both buying and selling at the OPEN of the next day
                nextday_return = 0
                prev_date = list(self.nextday_values.keys())[-1] if self.nextday_values else date
                prev_nextday_value = self.nextday_values.get(prev_date, self.initial_capital)
                trade_direction_nextday = 0
                new_nextday_value = prev_nextday_value  # Default - initialize to avoid UnboundLocalError
                
                if self.in_market_nextday:  # Currently holding stock
                    if self.previous_prediction['direction'] == -1:  # Previous prediction was DOWN
                        # Sell at the OPEN of today
                        self.in_market_nextday = False
                        
                        # We already captured today's return (when we bought)
                        nextday_return = 0
                        new_nextday_value = prev_nextday_value  # No change today
                        self.trade_count_nextday += 1
                        trade_direction_nextday = -1
                    else:  # Previous prediction was UP, stay in the market
                        # Capture today's return
                        nextday_return = actual_return / 100
                        new_nextday_value = prev_nextday_value * (1 + nextday_return)
                        trade_direction_nextday = 1
                else:  # Currently out of the market
                    if self.previous_prediction['direction'] == 1:  # Previous prediction was UP
                        # Buy at the OPEN of today
                        self.in_market_nextday = True
                        nextday_return = actual_return / 100  # Capture today's return
                        new_nextday_value = prev_nextday_value * (1 + nextday_return)
                        self.trade_count_nextday += 1
                        trade_direction_nextday = 1
                    else:  # Previous prediction was DOWN, stay out of the market
                        new_nextday_value = prev_nextday_value  # No change
                        nextday_return = 0
                        trade_direction_nextday = 0
                
                self.nextday_values[date] = new_nextday_value
                self.daily_returns_nextday.append(nextday_return)
                self.trade_directions_nextday.append(trade_direction_nextday)
                
                # Track wins and losses for nextday strategy
                if nextday_return > 0:
                    self.wins_nextday.append(nextday_return)
                    self.win_dates_nextday.append(date)
                    self.consecutive_wins_nextday += 1
                    self.consecutive_losses_nextday = 0
                    # Update max consecutive wins
                    if self.consecutive_wins_nextday > self.max_consecutive_wins_nextday:
                        self.max_consecutive_wins_nextday = self.consecutive_wins_nextday
                        self.max_consecutive_wins_end_date_nextday = date
                        if self.current_streak_start_date_nextday:
                            self.max_consecutive_wins_start_date_nextday = self.current_streak_start_date_nextday
                    # Set start date for current streak if needed
                    if self.consecutive_wins_nextday == 1:
                        self.current_streak_start_date_nextday = date
                elif nextday_return < 0:
                    self.losses_nextday.append(nextday_return)
                    self.loss_dates_nextday.append(date)
                    self.consecutive_losses_nextday += 1
                    self.consecutive_wins_nextday = 0
                    # Update max consecutive losses
                    if self.consecutive_losses_nextday > self.max_consecutive_losses_nextday:
                        self.max_consecutive_losses_nextday = self.consecutive_losses_nextday
                        self.max_consecutive_losses_end_date_nextday = date
                        if self.current_streak_start_date_nextday:
                            self.max_consecutive_losses_start_date_nextday = self.current_streak_start_date_nextday
                    # Set start date for current streak if needed
                    if self.consecutive_losses_nextday == 1:
                        self.current_streak_start_date_nextday = date
                
                # ========== LEVERAGE STRATEGY EXECUTION ==========
                # Calculate leverage multiplier based on prediction confidence
                # The leverage is based on how many standard deviations the current prediction is above the average
                import numpy as np
                if len(self.prediction_distribution_centers) > 30: # Need enough data to calculate meaningful std
                    mean_center = np.mean(self.prediction_distribution_centers[:-1]) # Exclude current day
                    std_center = np.std(self.prediction_distribution_centers[:-1])
                    
                    # Only apply leverage for upward predictions
                    if self.previous_prediction['direction'] == 1:
                        # How many standard deviations above the mean is the current prediction?
                        if std_center > 0:
                            std_above = (self.previous_prediction['expected_value'] - mean_center) / std_center
                            # Apply leverage based on number of std deviations
                            if std_above > 0:
                                leverage = 1 + min(int(std_above / self.leverage_threshold_std), self.max_leverage - 1)
                            else:
                                leverage = 1.0
                        else:
                            leverage = 1.0
                    else:
                        leverage = 1.0
                else:
                    # Not enough historical data to calculate std deviation
                    leverage = 1.0
                    
                # Execute leverage strategy (similar to basic but with leverage)
                leverage_return = 0
                leverage_trade_direction = 0
                
                if self.leverage_in_market: # Currently holding stock with leverage
                    if self.previous_prediction['direction'] == -1: # Previous prediction was DOWN
                        # Sell at the start of the day
                        self.leverage_in_market = False
                        # But we capture today's return before selling, with leverage applied
                        leverage_return = actual_return / 100 * self.leverage_multiplier
                        new_leverage_value = prev_leverage_value * (1 + leverage_return)
                        leverage_trade_direction = -1
                    else: # Previous prediction was UP, stay in the market
                        # Apply new leverage for today
                        self.leverage_multiplier = leverage
                        leverage_return = actual_return / 100 * self.leverage_multiplier
                        new_leverage_value = prev_leverage_value * (1 + leverage_return)
                        leverage_trade_direction = 1
                else: # Currently out of the market
                    if self.previous_prediction['direction'] == 1: # Previous prediction was UP
                        # Buy at the start of the day with leverage, capture today's return
                        self.leverage_in_market = True
                        self.leverage_multiplier = leverage
                        leverage_return = actual_return / 100 * self.leverage_multiplier
                        new_leverage_value = prev_leverage_value * (1 + leverage_return)
                        leverage_trade_direction = 1
                    else: # Previous prediction was DOWN, stay out of the market
                        new_leverage_value = prev_leverage_value # No change
                        leverage_return = 0
                        self.leverage_multiplier = 1.0
                        leverage_trade_direction = 0
                
                self.leverage_values[date] = new_leverage_value
                self.leverage_daily_returns.append(leverage_return)
                self.leverage_trade_directions.append(leverage_trade_direction)
                self.leverage_factors.append(self.leverage_multiplier)
                
                # ========== SHORTING STRATEGY EXECUTION ==========
                # Only short if negative prediction peak is 1 std dev below mean peak
                shorting_return = 0
                shorting_position = 0
                prev_date = list(self.shorting_values.keys())[-1] if self.shorting_values else date
                prev_shorting_value = self.shorting_values.get(prev_date, self.initial_capital)
                
                # Track prediction peaks for thresholding
                if self.previous_prediction['direction'] == 1:  # Positive prediction
                    self.positive_peaks.append(self.previous_prediction['most_likely'])
                elif self.previous_prediction['direction'] == -1:  # Negative prediction
                    self.negative_peaks.append(self.previous_prediction['most_likely'])
                
                # Calculate thresholds for negative predictions
                neg_peak_mean = np.mean(self.negative_peaks) if self.negative_peaks else 0
                neg_peak_std = np.std(self.negative_peaks) if len(self.negative_peaks) > 1 else 1
                
                # Determine if previous prediction is negative enough for shorting
                negative_threshold = neg_peak_mean - neg_peak_std
                is_significant_negative = (self.previous_prediction['direction'] == -1 and 
                                           self.previous_prediction['most_likely'] <= negative_threshold)
                
                if self.shorting_position == 1:  # Currently long
                    if is_significant_negative:  # Significant negative prediction - sell and go short
                        self.shorting_position = -1
                        # First capture long position return for today
                        shorting_return = actual_return / 100
                        new_shorting_value = prev_shorting_value * (1 + shorting_return)
                        shorting_position = -1
                    elif self.previous_prediction['direction'] == -1:  # Negative but not significant - sell and exit
                        self.shorting_position = 0
                        # Capture today's return before selling
                        shorting_return = actual_return / 100
                        new_shorting_value = prev_shorting_value * (1 + shorting_return)
                        shorting_position = 0
                    else:  # Previous prediction was UP, stay long
                        shorting_return = actual_return / 100
                        new_shorting_value = prev_shorting_value * (1 + shorting_return)
                        shorting_position = 1
                elif self.shorting_position == -1:  # Currently short
                    if self.previous_prediction['direction'] == 1:  # Prediction was UP - cover short and go long
                        self.shorting_position = 1
                        # First capture short position return for today (inverse of actual return)
                        shorting_return = -actual_return / 100
                        new_shorting_value = prev_shorting_value * (1 + shorting_return)
                        shorting_position = 1
                    else:  # Prediction was still DOWN, stay short
                        shorting_return = -actual_return / 100
                        new_shorting_value = prev_shorting_value * (1 + shorting_return)
                        shorting_position = -1
                else:  # Currently no position
                    if self.previous_prediction['direction'] == 1:  # Prediction was UP - go long
                        self.shorting_position = 1
                        shorting_return = actual_return / 100
                        new_shorting_value = prev_shorting_value * (1 + shorting_return)
                        shorting_position = 1
                    elif is_significant_negative:  # Significant negative prediction - go short
                        self.shorting_position = -1
                        shorting_return = -actual_return / 100
                        new_shorting_value = prev_shorting_value * (1 + shorting_return)
                        shorting_position = -1
                    else:  # Not significant enough - stay out
                        new_shorting_value = prev_shorting_value
                        shorting_return = 0
                        shorting_position = 0
                
                self.shorting_values[date] = new_shorting_value
                self.shorting_daily_returns.append(shorting_return)
                self.shorting_trade_directions.append(shorting_position)
                
                # Track monthly trade frequency
                month_year = date.strftime('%Y-%m')
                
                # Trading hours
                if month_year in self.monthly_trade_counts:
                    if trade_direction_tradinghours != 0:
                        self.monthly_trade_counts[month_year] += 1
                else:
                    if trade_direction_tradinghours != 0:
                        self.monthly_trade_counts[month_year] = 1
                    else:
                        self.monthly_trade_counts[month_year] = 0
                
                # After hours
                if month_year in self.monthly_trade_counts_afterhours:
                    if trade_direction_afterhours != 0:
                        self.monthly_trade_counts_afterhours[month_year] += 1
                else:
                    if trade_direction_afterhours != 0:
                        self.monthly_trade_counts_afterhours[month_year] = 1
                    else:
                        self.monthly_trade_counts_afterhours[month_year] = 0
                
                # Track trade execution dates
                if trade_direction_tradinghours != 0 or trade_direction_afterhours != 0: # If a trade was made
                    self.trade_dates.append(date)
    
                # ========== SHORT+LEVERAGE STRATEGY EXECUTION ==========
                # This strategy combines shorting and leverage with different thresholds
                short_leverage_return = 0
                prev_date = list(self.short_leverage_values.keys())[-1] if self.short_leverage_values else date
                prev_short_leverage_value = self.short_leverage_values.get(prev_date, self.initial_capital)
                
                # Calculate thresholds for both positive and negative predictions
                pos_peak_mean = np.mean(self.positive_peaks) if self.positive_peaks else 0
                pos_peak_std = np.std(self.positive_peaks) if len(self.positive_peaks) > 1 else 1
                neg_peak_mean = np.mean(self.negative_peaks) if self.negative_peaks else 0
                neg_peak_std = np.std(self.negative_peaks) if len(self.negative_peaks) > 1 else 1
                
                # Determine leverage levels
                leverage_long = 1  # Default leverage
                leverage_short = 1  # Default leverage
                
                # For positive predictions - only apply leverage if significantly positive
                if self.previous_prediction['direction'] == 1:
                    # Calculate how many std deviations above mean
                    if pos_peak_std > 0:
                        stds_above = (self.previous_prediction['most_likely'] - pos_peak_mean) / pos_peak_std
                        # Only leverage if more than 1 std dev above mean
                        if stds_above > 1:
                            leverage_long = 1 + min(int(stds_above), self.max_leverage - 1)
                        else:
                            leverage_long = 1  # No leverage for weak positive predictions
                    else:
                        leverage_long = 1
                        
                # For negative predictions - only short if significantly negative, and apply leverage if very negative
                elif self.previous_prediction['direction'] == -1:
                    # Calculate how many std deviations below mean
                    if neg_peak_std > 0:
                        stds_below = (neg_peak_mean - self.previous_prediction['most_likely']) / neg_peak_std
                        # Apply leverage for shorting if very negative
                        if stds_below > 2:
                            leverage_short = 1 + min(int(stds_below - 1), self.max_leverage - 1)
                        elif stds_below > 1:
                            leverage_short = 1  # Regular shorting without leverage
                        else:
                            leverage_short = 0  # Not negative enough to short
                    else:
                        leverage_short = 0
                
                # Current position is stored as a number: 
                # 0 = no position
                # positive = long with that level of leverage
                # negative = short with that level of leverage
                current_position = self.short_leverage_position
                
                # Execute strategy based on current position and new prediction
                if current_position > 0:  # Currently long
                    if self.previous_prediction['direction'] == 1 and leverage_long > 0:
                        # Stay long, potentially adjust leverage
                        self.short_leverage_position = leverage_long
                        # Apply current leverage to today's return
                        short_leverage_return = actual_return / 100 * abs(current_position)
                        new_short_leverage_value = prev_short_leverage_value * (1 + short_leverage_return)
                    else:
                        # Exit long position
                        if leverage_short > 0:
                            # Go short with leverage
                            self.short_leverage_position = -leverage_short
                            # First capture today's return from long position
                            short_leverage_return = actual_return / 100 * abs(current_position)
                            new_short_leverage_value = prev_short_leverage_value * (1 + short_leverage_return)
                        else:
                            # Exit to cash
                            self.short_leverage_position = 0
                            short_leverage_return = actual_return / 100 * abs(current_position)
                            new_short_leverage_value = prev_short_leverage_value * (1 + short_leverage_return)
                elif current_position < 0:  # Currently short
                    if self.previous_prediction['direction'] == -1 and leverage_short > 0:
                        # Stay short, potentially adjust leverage
                        self.short_leverage_position = -leverage_short
                        # Apply current leverage to inverse of today's return (shorting)
                        short_leverage_return = -actual_return / 100 * abs(current_position)
                        new_short_leverage_value = prev_short_leverage_value * (1 + short_leverage_return)
                    else:
                        # Exit short position
                        if leverage_long > 0:
                            # Go long with leverage
                            self.short_leverage_position = leverage_long
                            # First capture today's return from short position
                            short_leverage_return = -actual_return / 100 * abs(current_position)
                            new_short_leverage_value = prev_short_leverage_value * (1 + short_leverage_return)
                        else:
                            # Exit to cash
                            self.short_leverage_position = 0
                            short_leverage_return = -actual_return / 100 * abs(current_position)
                            new_short_leverage_value = prev_short_leverage_value * (1 + short_leverage_return)
                else:  # Currently no position
                    if self.previous_prediction['direction'] == 1 and leverage_long > 0:
                        # Go long with leverage
                        self.short_leverage_position = leverage_long
                        short_leverage_return = actual_return / 100 * leverage_long
                        new_short_leverage_value = prev_short_leverage_value * (1 + short_leverage_return)
                    elif self.previous_prediction['direction'] == -1 and leverage_short > 0:
                        # Go short with leverage
                        self.short_leverage_position = -leverage_short
                        short_leverage_return = -actual_return / 100 * leverage_short
                        new_short_leverage_value = prev_short_leverage_value * (1 + short_leverage_return)
                    else:
                        # Stay in cash
                        new_short_leverage_value = prev_short_leverage_value
                        short_leverage_return = 0
                
                self.short_leverage_values[date] = new_short_leverage_value
                self.short_leverage_daily_returns.append(short_leverage_return)
                self.short_leverage_position_history.append(self.short_leverage_position)
    
                # ========== MODEL ACCURACY EVALUATION ==========
                predicted_direction = prediction['direction']
                actual_direction = 1 if actual_return > 0 else -1
                
                # Check if prediction was correct
                is_correct = (predicted_direction == actual_direction)
                self.direction_correct.append(is_correct)
                self.direction_correct_dates.append(date)
                
                # Also track direction accuracy by prediction sign
                if predicted_direction == 1:  # Predicted UP
                    self.direction_correct_positive.append(is_correct)
                else:  # Predicted DOWN
                    self.direction_correct_negative.append(is_correct)
                
                # Store actual return for volatility calculations
                self.actual_returns_history.append(actual_return)
                
                # Calculate peak error (difference between most likely value and actual)
                peak_error = prediction['most_likely'] - actual_return
                self.peak_errors.append(peak_error)  # Original signed error
                
                # Add absolute error metrics
                abs_peak_error = abs(peak_error)
                self.abs_peak_errors.append(abs_peak_error)
                
                # Track by prediction sign
                if predicted_direction == 1:  # Predicted UP
                    self.pos_pred_abs_peak_errors.append(abs_peak_error)
                else:  # Predicted DOWN
                    self.neg_pred_abs_peak_errors.append(abs_peak_error)
                
                # Expected value error (error between expected value and actual)
                expected_value_error = prediction['expected_value'] - actual_return
                self.expected_value_errors.append(expected_value_error)  # Original signed error
                
                # Add absolute expected value error
                abs_ev_error = abs(expected_value_error)
                self.abs_expected_value_errors.append(abs_ev_error)
                
                # Track by prediction sign
                if predicted_direction == 1:  # Predicted UP
                    self.pos_pred_abs_ev_errors.append(abs_ev_error)
                else:  # Predicted DOWN
                    self.neg_pred_abs_ev_errors.append(abs_ev_error)
                
                # Overall prediction error (absolute difference)
                error = abs(prediction['most_likely'] - actual_return)
                self.prediction_errors.append(error)
                
                # Categorize by volatility buckets if we have enough history
                if len(self.actual_returns_history) >= 50:  # Need enough history for stable std
                    # Calculate rolling standard deviation
                    import numpy as np
                    returns_arr = np.array(self.actual_returns_history)
                    returns_std = np.std(returns_arr[-min(252, len(returns_arr)):])  # Use up to 1 year of history
                    
                    # Determine which volatility bucket this belongs to
                    abs_normalized_return = abs(actual_return) / returns_std
                    
                    if abs_normalized_return <= 1.0:
                        bucket = '1_std'
                    elif abs_normalized_return <= 2.0:
                        bucket = '2_std'
                    elif abs_normalized_return <= 3.0:
                        bucket = '3_std'
                    elif abs_normalized_return <= 4.0:
                        bucket = '4_std'
                    else:
                        bucket = 'other'
                    
                    # Store metrics in appropriate bucket
                    self.vol_buckets[bucket]['peak_errors'].append(abs_peak_error)
                    self.vol_buckets[bucket]['ev_errors'].append(abs_ev_error)
                    self.vol_buckets[bucket]['dir_correct'].append(is_correct)
                    self.vol_buckets[bucket]['total'] += 1
                
                # Update confusion matrix data
                if predicted_direction == 1 and actual_direction == 1:
                    self.true_positives += 1
                elif predicted_direction == 1 and actual_direction == -1:
                    self.false_positives += 1
                elif predicted_direction == -1 and actual_direction == -1:
                    self.true_negatives += 1
                elif predicted_direction == -1 and actual_direction == 1:
                    self.false_negatives += 1
                
                # Store the current prediction for the next day's trading
                self.previous_prediction = prediction
                
                # Add row to performance data DataFrame
                row_data = {
                    'date': date,
                    'actual_return': actual_return,
                    'predicted_direction': predicted_direction,
                    'predicted_return': prediction['expected_value'],
                    'direction_correct': is_correct,
                    'basic_tradinghours_return': basic_tradinghours_return,
                    'basic_afterhours_return': basic_afterhours_return,
                    'nextday_return': nextday_return,  # Added nextday strategy
                    'leverage_return': leverage_return,
                    'shorting_return': shorting_return,
                    'short_leverage_return': short_leverage_return,  # Added short_leverage strategy
                    'buyhold_return': actual_return / 100,
                    'basic_tradinghours_value': new_basic_tradinghours_value,
                    'basic_afterhours_value': new_basic_afterhours_value,
                    'nextday_value': new_nextday_value,  # Added nextday strategy
                    'leverage_value': new_leverage_value,
                    'shorting_value': new_shorting_value,
                    'short_leverage_value': new_short_leverage_value,  # Added short_leverage strategy
                    'buyhold_value': new_buyhold,
                    'leverage_factor': self.leverage_multiplier,
                    'short_leverage_position': self.short_leverage_position,  # Added position tracking
                    'trade_direction_tradinghours': trade_direction_tradinghours,
                    'trade_direction_afterhours': trade_direction_afterhours,
                    'trade_direction_nextday': trade_direction_nextday,  # Added nextday strategy
                    'shorting_position': shorting_position,
                    'peak_error': peak_error,
                    'expected_value_error': expected_value_error,
                    'positive_prob': prediction['positive_prob']
                }
                
                # Add the row to the performance data DataFrame
                self.performance_data = pd.concat([self.performance_data, pd.DataFrame([row_data])], ignore_index=True)
                
                # Update performance metrics
                self._update_metrics()
            
            except ZeroDivisionError as e:
                print(f"Error processing day {date}: Division by zero detected - {e}")
                print(f"Warning: This might be due to empty lists of returns or metrics. Check if you have enough trading data.")
                return
            except Exception as e:
                import traceback
                print(f"Error processing day {date}: {e}")
                traceback.print_exc()  # This will show the full error stack trace
                return

    def _update_metrics(self):
        """Update all performance metrics."""
        try:
            import numpy as np
            # Calculate risk avoidance rate - Trading Hours
            self.risk_avoidance_rate = (
                self.correct_risk_avoidances / self.total_risk_predictions
                if self.total_risk_predictions > 0 else 0
            )
            
            # Calculate risk avoidance rate - After Hours
            self.risk_avoidance_rate_afterhours = (
                self.correct_risk_avoidances_afterhours / self.total_risk_predictions_afterhours
                if self.total_risk_predictions_afterhours > 0 else 0
            )
            
            # Calculate market participation rate - Trading Hours
            self.market_participation_rate = (
                self.days_in_market / self.total_days
                if self.total_days > 0 else 0
            )
            
            # Calculate market participation rate - After Hours
            self.market_participation_rate_afterhours = (
                self.days_in_market_afterhours / self.total_days
                if self.total_days > 0 else 0
            )
            
            # Calculate number of recent trades
            if not self.trade_dates:
                self.recent_trades_count = 0
            else:
                from datetime import timedelta
                latest_date = self.trade_dates[-1]
                cutoff_date = latest_date - timedelta(days=self.recent_trade_window)
                self.recent_trades_count = sum(1 for date in self.trade_dates if date > cutoff_date)
            
            # Direction accuracy
            total_predictions = (self.true_positives + self.true_negatives +
                                self.false_positives + self.false_negatives)
            if total_predictions > 0:
                self.accuracy = (self.true_positives + self.true_negatives) / total_predictions
            else:
                self.accuracy = 0.0
    
            # Calculate metrics for nextday strategy - safely handle dictionary values
            if self.nextday_values:
                nextday_values_list = list(self.nextday_values.values())
                if len(nextday_values_list) > 1:
                    self.total_return_nextday = (nextday_values_list[-1] / nextday_values_list[0]) - 1
                else:
                    self.total_return_nextday = 0
            else:
                self.total_return_nextday = 0
                
            self.max_drawdown_nextday = self._calculate_max_drawdown(list(self.nextday_values.values()))
            self.sharpe_ratio_nextday = self._calculate_sharpe_ratio(self.daily_returns_nextday)
            self.sortino_ratio_nextday = self._calculate_sortino_ratio(self.daily_returns_nextday)
            self.win_rate_nextday = len(self.wins_nextday) / (len(self.wins_nextday) + len(self.losses_nextday)) if (len(self.wins_nextday) + len(self.losses_nextday)) > 0 else 0
            self.avg_gain_nextday = np.mean(self.wins_nextday) if self.wins_nextday else 0
            self.avg_loss_nextday = np.mean(self.losses_nextday) if self.losses_nextday else 0
            self.gain_loss_ratio_nextday = abs(self.avg_gain_nextday / self.avg_loss_nextday) if self.avg_loss_nextday != 0 else 0

            # Calculate Beta for all strategies
            self.beta_tradinghours = self._calculate_beta(self.daily_returns, self.buyhold_returns)
            self.beta_afterhours = self._calculate_beta(self.daily_returns_afterhours, self.buyhold_returns)
            self.leverage_beta = self._calculate_beta(self.leverage_daily_returns, self.buyhold_returns)
            self.shorting_beta = self._calculate_beta(self.shorting_daily_returns, self.buyhold_returns)
            # Add these two new beta calculations:
            self.beta_nextday = self._calculate_beta(self.daily_returns_nextday, self.buyhold_returns)
            self.beta_buyhold = 1.0  # By definition, the market's beta with itself is 1.0

            # Add median error metrics (more robust to outliers)
            if self.peak_errors:
                self.median_peak_error = np.median([abs(e) for e in self.peak_errors])
                self.median_raw_peak_error = np.median(self.peak_errors)  # With sign to detect bias
            else:
                self.median_peak_error = 0
                self.median_raw_peak_error = 0
                
            if self.expected_value_errors:
                self.median_expected_value_error = np.median([abs(e) for e in self.expected_value_errors])
                self.median_raw_expected_value_error = np.median(self.expected_value_errors)  # With sign
            else:
                self.median_expected_value_error = 0
                self.median_raw_expected_value_error = 0
                
            if self.prediction_errors:
                self.median_prediction_error = np.median(self.prediction_errors)
            else:
                self.median_prediction_error = 0

            # Calculate underwater metrics for all strategies
            self.underwater_metrics_tradinghours = self._calculate_underwater_metrics(list(self.portfolio_values.values()))
            self.underwater_metrics_afterhours = self._calculate_underwater_metrics(list(self.portfolio_values_afterhours.values()))
            self.underwater_metrics_nextday = self._calculate_underwater_metrics(list(self.nextday_values.values()))
            self.underwater_metrics_buyhold = self._calculate_underwater_metrics(list(self.buyhold_values.values()))
            
            # Calculate metrics for short_leverage strategy - safely handle dictionary values
            if self.short_leverage_values:
                short_leverage_values_list = list(self.short_leverage_values.values())
                if len(short_leverage_values_list) > 1:
                    self.short_leverage_return = (short_leverage_values_list[-1] / short_leverage_values_list[0]) - 1
                else:
                    self.short_leverage_return = 0
            else:
                self.short_leverage_return = 0
                
            self.short_leverage_max_drawdown = self._calculate_max_drawdown(list(self.short_leverage_values.values()))
            self.short_leverage_sharpe_ratio = self._calculate_sharpe_ratio(self.short_leverage_daily_returns)
            self.short_leverage_sortino_ratio = self._calculate_sortino_ratio(self.short_leverage_daily_returns)
            
            # Mean absolute error
            self.mean_error = np.mean(self.prediction_errors) if self.prediction_errors else 0
            
            # Calculate returns - safely handle dictionary values
            portfolio_values_tradinghours = list(self.portfolio_values.values())
            portfolio_values_afterhours = list(self.portfolio_values_afterhours.values())
            leverage_values = list(self.leverage_values.values())
            shorting_values = list(self.shorting_values.values())
            buyhold_values = list(self.buyhold_values.values())
            
            # Total return - safely handle empty lists or single values
            if len(portfolio_values_tradinghours) > 1:
                self.total_return_tradinghours = (portfolio_values_tradinghours[-1] / portfolio_values_tradinghours[0]) - 1
            else:
                self.total_return_tradinghours = 0
                
            if len(portfolio_values_afterhours) > 1:
                self.total_return_afterhours = (portfolio_values_afterhours[-1] / portfolio_values_afterhours[0]) - 1
            else:
                self.total_return_afterhours = 0
                
            if len(leverage_values) > 1:
                self.leverage_return = (leverage_values[-1] / leverage_values[0]) - 1
            else:
                self.leverage_return = 0
                
            if len(shorting_values) > 1:
                self.shorting_return = (shorting_values[-1] / shorting_values[0]) - 1
            else:
                self.shorting_return = 0
                
            if len(buyhold_values) > 1:
                self.buyhold_return = (buyhold_values[-1] / buyhold_values[0]) - 1
            else:
                self.buyhold_return = 0
            
            # Calculate maximum drawdown
            self.max_drawdown_tradinghours = self._calculate_max_drawdown(portfolio_values_tradinghours)
            self.max_drawdown_afterhours = self._calculate_max_drawdown(portfolio_values_afterhours)
            self.leverage_max_drawdown = self._calculate_max_drawdown(leverage_values)
            self.shorting_max_drawdown = self._calculate_max_drawdown(shorting_values)
            self.buyhold_max_drawdown = self._calculate_max_drawdown(buyhold_values)
            
            # Calculate Sharpe ratio
            self.sharpe_ratio_tradinghours = self._calculate_sharpe_ratio(self.daily_returns)
            self.sharpe_ratio_afterhours = self._calculate_sharpe_ratio(self.daily_returns_afterhours)
            self.leverage_sharpe_ratio = self._calculate_sharpe_ratio(self.leverage_daily_returns)
            self.shorting_sharpe_ratio = self._calculate_sharpe_ratio(self.shorting_daily_returns)
            self.buyhold_sharpe_ratio = self._calculate_sharpe_ratio(self.buyhold_returns)
            
            # Calculate Sortino ratio
            self.sortino_ratio_tradinghours = self._calculate_sortino_ratio(self.daily_returns)
            self.sortino_ratio_afterhours = self._calculate_sortino_ratio(self.daily_returns_afterhours)
            self.leverage_sortino_ratio = self._calculate_sortino_ratio(self.leverage_daily_returns)
            self.shorting_sortino_ratio = self._calculate_sortino_ratio(self.shorting_daily_returns)
            self.buyhold_sortino_ratio = self._calculate_sortino_ratio(self.buyhold_returns)
    
            # Calculate Profit Factor for all strategies
            self.profit_factor_tradinghours = self._calculate_profit_factor(self.wins_tradinghours, self.losses_tradinghours)
            self.profit_factor_afterhours = self._calculate_profit_factor(self.wins_afterhours, self.losses_afterhours)
            self.profit_factor_nextday = self._calculate_profit_factor(self.wins_nextday, self.losses_nextday)
            self.profit_factor_leverage = self._calculate_profit_factor(
                [r for r, d in zip(self.leverage_daily_returns, self.leverage_trade_directions) if d != 0 and r > 0],
                [r for r, d in zip(self.leverage_daily_returns, self.leverage_trade_directions) if d != 0 and r < 0]
            )
            self.profit_factor_shorting = self._calculate_profit_factor(
                [r for r, d in zip(self.shorting_daily_returns, self.shorting_trade_directions) if d != 0 and r > 0],
                [r for r, d in zip(self.shorting_daily_returns, self.shorting_trade_directions) if d != 0 and r < 0]
            )
            self.profit_factor_short_leverage = self._calculate_profit_factor(
                [r for r, d in zip(self.short_leverage_daily_returns, self.short_leverage_position_history) if d != 0 and r > 0],
                [r for r, d in zip(self.short_leverage_daily_returns, self.short_leverage_position_history) if d != 0 and r < 0]
            )
            
            # Calculate SQN for all strategies
            self.sqn_tradinghours = self._calculate_sqn(self.daily_returns, self.trade_count)
            self.sqn_afterhours = self._calculate_sqn(self.daily_returns_afterhours, self.trade_count_afterhours)
            self.sqn_nextday = self._calculate_sqn(self.daily_returns_nextday, 
                                                  sum(1 for d in self.trade_directions_nextday if d != 0))
            self.sqn_leverage = self._calculate_sqn(self.leverage_daily_returns, 
                                                  sum(1 for d in self.leverage_trade_directions if d != 0))
            self.sqn_shorting = self._calculate_sqn(self.shorting_daily_returns, 
                                                  sum(1 for d in self.shorting_trade_directions if d != 0))
            self.sqn_short_leverage = self._calculate_sqn(self.short_leverage_daily_returns, 
                                                        sum(1 for d in self.short_leverage_position_history if d != 0))
            
            # Calculate Win Rate - Trading Hours
            self.win_rate_tradinghours = len(self.wins_tradinghours) / (len(self.wins_tradinghours) + len(self.losses_tradinghours)) if (len(self.wins_tradinghours) + len(self.losses_tradinghours)) > 0 else 0
            
            # Calculate Win Rate - After Hours
            self.win_rate_afterhours = len(self.wins_afterhours) / (len(self.wins_afterhours) + len(self.losses_afterhours)) if (len(self.wins_afterhours) + len(self.losses_afterhours)) > 0 else 0
            
            # Calculate Average Gain/Loss Ratio - Trading Hours
            self.avg_gain_tradinghours = np.mean(self.wins_tradinghours) if self.wins_tradinghours else 0
            self.avg_loss_tradinghours = np.mean(self.losses_tradinghours) if self.losses_tradinghours else 0
            self.gain_loss_ratio_tradinghours = abs(self.avg_gain_tradinghours / self.avg_loss_tradinghours) if self.avg_loss_tradinghours != 0 else 0
            
            # Calculate Average Gain/Loss Ratio - After Hours
            self.avg_gain_afterhours = np.mean(self.wins_afterhours) if self.wins_afterhours else 0
            self.avg_loss_afterhours = np.mean(self.losses_afterhours) if self.losses_afterhours else 0
            self.gain_loss_ratio_afterhours = abs(self.avg_gain_afterhours / self.avg_loss_afterhours) if self.avg_loss_afterhours != 0 else 0
            
            # Calculate trading frequency
            self.trading_frequency_tradinghours = self.trade_count / self.total_days if self.total_days > 0 else 0
            self.trading_frequency_afterhours = self.trade_count_afterhours / self.total_days if self.total_days > 0 else 0
            
            # Calculate Beta (relative to buy-and-hold strategy)
            self.beta_tradinghours = self._calculate_beta(self.daily_returns, self.buyhold_returns)
            self.beta_afterhours = self._calculate_beta(self.daily_returns_afterhours, self.buyhold_returns)
            self.leverage_beta = self._calculate_beta(self.leverage_daily_returns, self.buyhold_returns)
            self.shorting_beta = self._calculate_beta(self.shorting_daily_returns, self.buyhold_returns)
    
            # Update enhanced error metrics
            
            # 1. Overall absolute errors
            self.mean_abs_peak_error = np.mean(self.abs_peak_errors) if self.abs_peak_errors else 0
            self.mean_abs_ev_error = np.mean(self.abs_expected_value_errors) if self.abs_expected_value_errors else 0
            
            # 2. Positive prediction absolute errors
            self.mean_pos_pred_abs_peak_error = np.mean(self.pos_pred_abs_peak_errors) if self.pos_pred_abs_peak_errors else 0
            self.mean_pos_pred_abs_ev_error = np.mean(self.pos_pred_abs_ev_errors) if self.pos_pred_abs_ev_errors else 0
            
            # 3. Negative prediction absolute errors
            self.mean_neg_pred_abs_peak_error = np.mean(self.neg_pred_abs_peak_errors) if self.neg_pred_abs_peak_errors else 0
            self.mean_neg_pred_abs_ev_error = np.mean(self.neg_pred_abs_ev_errors) if self.neg_pred_abs_ev_errors else 0
            
            # 4. Directional accuracy metrics
            # Overall
            self.direction_accuracy = np.mean(self.direction_correct) if self.direction_correct else 0
            
            # By prediction sign
            self.pos_pred_direction_accuracy = np.mean(self.direction_correct_positive) if self.direction_correct_positive else 0
            self.neg_pred_direction_accuracy = np.mean(self.direction_correct_negative) if self.direction_correct_negative else 0
            
            # Recent window (100 days)
            self.recent_direction_accuracy = np.mean(self.direction_correct[-self.recent_window:]) if len(self.direction_correct) >= self.recent_window else np.mean(self.direction_correct) if self.direction_correct else 0
            
            # 5. Volatility bucket metrics
            for bucket, data in self.vol_buckets.items():
                if data['total'] > 0:
                    # Calculate mean absolute errors for this volatility bucket
                    self.vol_buckets[bucket]['mean_peak_error'] = np.mean(data['peak_errors']) if data['peak_errors'] else 0
                    self.vol_buckets[bucket]['mean_ev_error'] = np.mean(data['ev_errors']) if data['ev_errors'] else 0
                    self.vol_buckets[bucket]['dir_accuracy'] = np.mean(data['dir_correct']) if data['dir_correct'] else 0
            
            # Calculate annualized return if we have enough data
            if len(portfolio_values_tradinghours) > 252:
                days = len(portfolio_values_tradinghours) - 1  # Subtract 1 to account for the initial value
                
                self.annual_return_tradinghours = ((portfolio_values_tradinghours[-1] / portfolio_values_tradinghours[0]) ** (252 / days)) - 1
                self.annual_return_afterhours = ((portfolio_values_afterhours[-1] / portfolio_values_afterhours[0]) ** (252 / days)) - 1
                self.leverage_annual_return = ((leverage_values[-1] / leverage_values[0]) ** (252 / days)) - 1
                self.shorting_annual_return = ((shorting_values[-1] / shorting_values[0]) ** (252 / days)) - 1
                self.annual_return_nextday = ((self.nextday_values[list(self.nextday_values.keys())[-1]] / self.nextday_values[list(self.nextday_values.keys())[0]]) ** (252 / days)) - 1
                self.annual_return_short_leverage = ((self.short_leverage_values[list(self.short_leverage_values.keys())[-1]] / self.short_leverage_values[list(self.short_leverage_values.keys())[0]]) ** (252 / days)) - 1
                self.buyhold_annual_return = ((buyhold_values[-1] / buyhold_values[0]) ** (252 / days)) - 1
        
        except ZeroDivisionError as e:
            print(f"Warning: Division by zero in _update_metrics: {e}")
            print(f"This usually happens when there's not enough trading data yet.")
        except Exception as e:
            import traceback
            print(f"Error in _update_metrics: {e}")
            traceback.print_exc()
    
    def _calculate_max_drawdown(self, values):
        """Calculate maximum drawdown from a list of portfolio values."""
        import numpy as np
        
        if not values or len(values) < 2:
            return 0
        
        # Convert to numpy array for easier calculations
        values_array = np.array(values)
        
        # Calculate the running maximum
        running_max = np.maximum.accumulate(values_array)
        
        # Calculate the drawdown at each point
        drawdowns = (running_max - values_array) / running_max
        
        # Return the maximum drawdown
        return np.max(drawdowns)

    def _calculate_underwater_metrics(self, values):
        """
        Analyze time spent in drawdown.
        
        Insights:
        - Time spent in drawdown indicates capital efficiency
        - Long underwater periods can lead to strategy abandonment
        - Average drawdown depth shows typical recovery challenge
        """
        if not values or len(values) < 2:
            return {
                'pct_time_underwater': 0.0,
                'avg_drawdown': 0.0,
                'max_drawdown_duration': 0
            }
            
        # Convert to numpy array for easier calculations
        values_array = np.array(values)
        
        # Calculate the running maximum
        running_max = np.maximum.accumulate(values_array)
        
        # Calculate drawdown percentage at each point
        drawdowns = (values_array / running_max - 1) * 100
        
        # Calculate underwater metrics
        underwater_days = np.sum(drawdowns < 0)
        pct_time_underwater = underwater_days / len(drawdowns) if len(drawdowns) > 0 else 0
        avg_drawdown = np.mean(drawdowns[drawdowns < 0]) if np.any(drawdowns < 0) else 0
        
        # Calculate max drawdown duration
        max_duration = 0
        current_duration = 0
        in_drawdown = False
        
        for dd in drawdowns:
            if dd < 0:
                in_drawdown = True
                current_duration += 1
            else:
                if in_drawdown:
                    max_duration = max(max_duration, current_duration)
                    current_duration = 0
                    in_drawdown = False
        
        # Check if we ended in a drawdown
        if in_drawdown:
            max_duration = max(max_duration, current_duration)
        
        return {
            'pct_time_underwater': pct_time_underwater,
            'avg_drawdown': avg_drawdown,
            'max_drawdown_duration': max_duration
        }


    def _calculate_sharpe_ratio(self, returns):
        """Calculate Sharpe ratio from a list of returns."""
        import numpy as np
        
        if not returns or len(returns) <= 1:
            return 0
            
        returns_mean = np.mean(returns)
        returns_std = np.std(returns)
        if returns_std > 0:
            return (returns_mean - self.risk_free_rate) / returns_std * np.sqrt(252)  # Annualized
        return 0
    
    def _calculate_sortino_ratio(self, returns):
        """Calculate Sortino ratio from a list of returns."""
        import numpy as np
        
        if not returns or len(returns) <= 1:
            return 0
            
        returns_mean = np.mean(returns)
        # Only consider downside risk (negative returns)
        downside_returns = [r for r in returns if r < 0]
        if downside_returns:
            downside_std = np.std(downside_returns)
            if downside_std > 0:
                return (returns_mean - self.risk_free_rate) / downside_std * np.sqrt(252)  # Annualized
        return 0
    
    def _calculate_beta(self, strategy_returns, market_returns):
        """Calculate Beta (relative to market) from lists of returns."""
        import numpy as np
        
        if not strategy_returns or not market_returns or len(strategy_returns) <= 1 or len(market_returns) != len(strategy_returns):
            return 0
            
        # Calculate covariance and market variance
        try:
            covariance = np.cov(strategy_returns, market_returns)[0, 1]
            market_variance = np.var(market_returns)
            if market_variance > 0:
                return covariance / market_variance
        except Exception as e:
            print(f"Error calculating beta: {e}")
        return 0

    def _calculate_calmar_ratio(self, returns, max_drawdown):
        """Calculate Calmar Ratio: Annualized Return / Maximum Drawdown"""
        if max_drawdown <= 0 or not returns:
            return 0.0
        annualized_return = np.mean(returns) * 252  # Annualize daily returns
        return annualized_return / max_drawdown
        
    def _calculate_mar_ratio(self, total_return, years, max_drawdown):
        """Calculate MAR Ratio: CAGR / Maximum Drawdown"""
        if max_drawdown <= 0 or years <= 0:
            return 0.0
        cagr = (1 + total_return) ** (1/years) - 1
        return cagr / max_drawdown
        
    def _calculate_cagr(self, total_return, years):
        """Calculate Compound Annual Growth Rate"""
        if years <= 0:
            return 0.0
        return (1 + total_return) ** (1/years) - 1
        
    def _calculate_modified_cagr(self, total_return, years, max_drawdown):
        """Calculate CAGR adjusted for drawdown tolerance"""
        cagr = self._calculate_cagr(total_return, years)
        return cagr * (1 - max_drawdown)

    def _calculate_profit_factor(self, wins, losses):
        """
        Calculate Profit Factor: Gross Profit / Gross Loss
        
        Insights:
        - Values > 1 indicate a profitable system
        - Values > 2 indicate a strong trading system
        - Values > 3 indicate an exceptional system
        """
        gross_profit = sum(wins) if wins else 0
        gross_loss = abs(sum(losses)) if losses else 0
        return gross_profit / gross_loss if gross_loss > 0 else float('inf')

    def _calculate_expectancy(self, win_rate, avg_win, avg_loss):
        """
        Calculate Expectancy: Expected return per trade
        
        Insights:
        - Positive values indicate profitable systems
        - Higher values indicate more efficient capital usage
        - Low values suggest high trading frequency may be needed
        """
        if avg_loss > 0:  # Ensure losses are expressed as positive values
            avg_loss = -avg_loss
        return (win_rate * avg_win) + ((1 - win_rate) * avg_loss)
    
    def _calculate_sqn(self, returns, n_trades):
        """
        Calculate System Quality Number (SQN)
        
        Insights:
        - <1.6: Poor system
        - 1.6-1.9: Below average system  
        - 2.0-2.4: Average system
        - 2.5-2.9: Good system
        - 3.0-5.0: Excellent system
        - >5.0: Exceptional system
        """
        import numpy as np
        
        if not returns or n_trades == 0:
            return 0
        expectancy = np.mean(returns)
        stdev = np.std(returns) if len(returns) > 1 else 1
        return (expectancy / stdev) * np.sqrt(n_trades)

    def _calculate_market_capture_ratios(self, strategy_returns, market_returns):
        """
        Calculate Up/Down Market Capture Ratios
        
        Insights:
        - Up Capture > 1: Outperforms in bull markets
        - Down Capture < 1: Outperforms in bear markets (less negative)
        - Up/Down capture ratio > 1: Overall market outperformance
        """
        if len(strategy_returns) != len(market_returns) or len(market_returns) == 0:
            return {'up_capture': 0, 'down_capture': 0, 'capture_ratio': 0}
        
        # Convert to numpy arrays
        strat_returns = np.array(strategy_returns)
        mkt_returns = np.array(market_returns)
        
        # Identify up and down markets
        up_markets = mkt_returns > 0
        down_markets = mkt_returns < 0
        
        # Calculate up and down market performance
        up_capture = (np.mean(strat_returns[up_markets]) / 
                     np.mean(mkt_returns[up_markets])) if np.any(up_markets) and np.mean(mkt_returns[up_markets]) != 0 else 0
        
        down_capture = (np.mean(strat_returns[down_markets]) / 
                       np.mean(mkt_returns[down_markets])) if np.any(down_markets) and np.mean(mkt_returns[down_markets]) != 0 else 0
        
        # Calculate capture ratio
        capture_ratio = abs(up_capture / down_capture) if down_capture != 0 else float('inf')
        
        return {'up_capture': up_capture, 'down_capture': down_capture, 'capture_ratio': capture_ratio}

    def _calculate_var(self, returns, confidence=0.95):
        """
        Calculate Value at Risk
        
        Insights:
        - Estimates worst loss at a given confidence level
        - 95% VaR of -2% means 95% chance of losing no more than 2% in a day
        - Key regulatory and risk management metric
        """
        if not returns:
            return 0
        return np.percentile(returns, 100 * (1 - confidence))
        
    def _calculate_cvar(self, returns, confidence=0.95):
        """
        Calculate Conditional Value at Risk / Expected Shortfall
        
        Insights:
        - Measures expected loss in worst-case scenarios
        - More sensitive to tail risk than VaR
        - Better for non-normal return distributions
        """
        if not returns:
            return 0
        var = self._calculate_var(returns, confidence)
        return np.mean(np.array([r for r in returns if r <= var])) if any(r <= var for r in returns) else var

    def _calculate_confidence_weighted_accuracy(self):
        """
        Calculate accuracy weighted by model confidence
        
        Insights:
        - Tests if model assigns higher confidence to correct predictions
        - Better than raw accuracy for evaluating model calibration
        - Should be higher than standard accuracy for well-calibrated models
        """
        if not hasattr(self, 'predicted_probs') or not self.predicted_probs:
            return 0.0
            
        # Calculate confidence (0-1 scale where 1 is highest confidence)
        confidence = np.array([abs(prob - 0.5) * 2 for prob in self.predicted_probs])
        
        # Get correct predictions (1 for correct, 0 for incorrect)
        correct = np.array([1 if ((prob > 0.5 and outcome == 1) or (prob <= 0.5 and outcome == 0)) 
                          else 0 
                          for prob, outcome in zip(self.predicted_probs, self.actual_outcomes)])
        
        # Calculate weighted accuracy
        return np.sum(correct * confidence) / np.sum(confidence) if np.sum(confidence) > 0 else 0.0
    
    def _analyze_decision_boundaries(self):
        """
        Analyze performance at different confidence thresholds
        
        Insights:
        - Helps identify optimal thresholds for trade decisions
        - Tests if higher confidence thresholds improve accuracy
        - Guides position sizing and risk allocation
        """
        if not hasattr(self, 'predicted_probs') or not self.predicted_probs:
            return {}
            
        # Convert to numpy arrays
        probs = np.array(self.predicted_probs)
        outcomes = np.array(self.actual_outcomes)
        
        # Define thresholds to test
        thresholds = [0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90]
        results = {}
        
        for threshold in thresholds:
            # For positive predictions (above threshold)
            pos_mask = probs >= threshold
            pos_count = np.sum(pos_mask)
            if pos_count > 0:
                pos_accuracy = np.mean(outcomes[pos_mask] == 1)
                results[f'pos_{threshold:.2f}'] = {'accuracy': pos_accuracy, 'count': int(pos_count)}
            
            # For negative predictions (below 1-threshold)
            neg_mask = probs <= (1 - threshold)
            neg_count = np.sum(neg_mask)
            if neg_count > 0:
                neg_accuracy = np.mean(outcomes[neg_mask] == 0)
                results[f'neg_{threshold:.2f}'] = {'accuracy': neg_accuracy, 'count': int(neg_count)}
                
        return results

    def monte_carlo_simulation(self, strategy='basic_tradinghours', simulations=1000, periods=252):
        """
        Run Monte Carlo simulation of strategy performance
        
        Parameters:
        -----------
        strategy: str
            Strategy to simulate ('basic_tradinghours', 'basic_afterhours', 'nextday', 'leverage', 'shorting', 'short_leverage', 'buyhold')
        simulations: int
            Number of simulations to run
        periods: int
            Number of trading days to simulate
            
        Returns:
        --------
        dict
            Simulation results statistics
            
        Insights:
        - Distribution of possible future outcomes
        - Probability of meeting return targets
        - Risk of ruin and worst-case scenarios
        - More realistic assessment than backtest results
        """
        # Get returns data for the selected strategy
        if strategy == 'basic_tradinghours':
            returns = self.daily_returns
            initial_capital = self.initial_capital
        elif strategy == 'basic_afterhours':
            returns = self.daily_returns_afterhours
            initial_capital = self.initial_capital
        elif strategy == 'nextday':
            returns = self.daily_returns_nextday
            initial_capital = self.initial_capital
        elif strategy == 'leverage':
            returns = self.leverage_daily_returns
            initial_capital = self.initial_capital
        elif strategy == 'shorting':
            returns = self.shorting_daily_returns
            initial_capital = self.initial_capital
        elif strategy == 'short_leverage':
            returns = self.short_leverage_daily_returns
            initial_capital = self.initial_capital
        elif strategy == 'buyhold':
            returns = self.buyhold_returns
            initial_capital = self.initial_capital
        else:
            raise ValueError(f"Unknown strategy: {strategy}. Valid options: 'basic_tradinghours', 'basic_afterhours', 'nextday', 'leverage', 'shorting', 'short_leverage', 'buyhold'")
            
        # Check if we have enough data
        if not returns or len(returns) < 20:
            print(f"Insufficient data for Monte Carlo simulation of {strategy}. Need at least 20 data points.")
            return None
            
        # Initialize result arrays
        final_capitals = np.zeros(simulations)
        max_drawdowns = np.zeros(simulations)
        sharpe_ratios = np.zeros(simulations)
        positive_periods = np.zeros(simulations)
        risk_of_ruin = 0  # Count simulations that lose > 50%
        
        # Run simulations
        for i in range(simulations):
            # Resample returns with replacement
            sampled_returns = np.random.choice(returns, size=periods, replace=True)
            
            # Calculate equity curve
            equity_curve = np.zeros(periods + 1)
            equity_curve[0] = initial_capital
            
            for j in range(periods):
                equity_curve[j+1] = equity_curve[j] * (1 + sampled_returns[j])
                
            # Calculate metrics for this simulation
            final_capitals[i] = equity_curve[-1]
            
            # Calculate drawdown
            peak = np.maximum.accumulate(equity_curve)
            drawdown = (equity_curve / peak - 1)
            max_drawdowns[i] = abs(min(drawdown))
            
            # Count positive periods
            positive_periods[i] = np.sum(sampled_returns > 0)
            
            # Calculate Sharpe ratio
            if np.std(sampled_returns) > 0:
                sharpe_ratios[i] = (np.mean(sampled_returns) / np.std(sampled_returns)) * np.sqrt(252)
            else:
                sharpe_ratios[i] = 0
                
            # Check for ruin (>50% loss)
            if max_drawdowns[i] > 0.5:
                risk_of_ruin += 1
        
        # Convert to percentages where appropriate
        pct_return = ((final_capitals / initial_capital) - 1) * 100
        max_drawdowns = max_drawdowns * 100
        risk_of_ruin = risk_of_ruin / simulations * 100
        
        # Build result dictionary
        results = {
            'strategy': strategy,
            'simulations': simulations,
            'periods': periods,
            'mean_final_return': np.mean(pct_return),
            'median_final_return': np.median(pct_return),
            'pct5_final_return': np.percentile(pct_return, 5),
            'pct95_final_return': np.percentile(pct_return, 95),
            'probability_positive_return': np.mean(pct_return > 0) * 100,
            'probability_10pct_return': np.mean(pct_return > 10) * 100,
            'probability_20pct_return': np.mean(pct_return > 20) * 100,
            'mean_max_drawdown': np.mean(max_drawdowns),
            'median_max_drawdown': np.median(max_drawdowns),
            'worst_max_drawdown': np.max(max_drawdowns),
            'pct95_max_drawdown': np.percentile(max_drawdowns, 95),
            'risk_of_ruin': risk_of_ruin,
            'mean_sharpe': np.mean(sharpe_ratios),
            'positive_period_pct': np.mean(positive_periods / periods) * 100
        }
        
        return results
    
    def get_moving_average_accuracy(self, window_days=20):
        """
        Calculate moving average of directional accuracy.
        
        Parameters:
        -----------
        window_days: int
            Window size in days
            
        Returns:
        --------
        tuple
            (dates, moving_average)
        """
        
        if len(self.direction_correct) < window_days:
            return [], []
        
        correct = np.array(self.direction_correct)
        dates = self.direction_correct_dates
        
        # Calculate moving average
        moving_avg = np.convolve(correct, np.ones(window_days)/window_days, mode='valid')
        ma_dates = dates[window_days-1:]
        
        return ma_dates, moving_avg
    
    def get_monthly_accuracy(self):
        """
        Calculate monthly accuracy.
        
        Returns:
        --------
        tuple
            (month_labels, monthly_accuracy)
        """
        
        if not self.performance_data.empty:
            # Convert datetime to Period for monthly grouping
            self.performance_data['month'] = pd.to_datetime(self.performance_data['date']).dt.to_period('M')
            
            # Group by month and calculate accuracy
            monthly_data = self.performance_data.groupby('month')['direction_correct'].agg(['sum', 'count'])
            monthly_data['accuracy'] = monthly_data['sum'] / monthly_data['count']
            
            # Convert index to string for plotting
            month_labels = [str(m) for m in monthly_data.index]
            monthly_accuracy = monthly_data['accuracy'].values
            
            return month_labels, monthly_accuracy
        
        return [], []
    
    def get_quarterly_accuracy(self):
        """
        Calculate quarterly accuracy.
        
        Returns:
        --------
        tuple
            (quarter_labels, quarterly_accuracy)
        """
        
        if not self.performance_data.empty:
            # Convert datetime to Period for quarterly grouping
            self.performance_data['quarter'] = pd.to_datetime(self.performance_data['date']).dt.to_period('Q')
            
            # Group by quarter and calculate accuracy
            quarterly_data = self.performance_data.groupby('quarter')['direction_correct'].agg(['sum', 'count'])
            quarterly_data['accuracy'] = quarterly_data['sum'] / quarterly_data['count']
            
            # Convert index to string for plotting
            quarter_labels = [str(q) for q in quarterly_data.index]
            quarterly_accuracy = quarterly_data['accuracy'].values
            
            return quarter_labels, quarterly_accuracy
        
        return [], []
    
    def get_yearly_accuracy(self):
        """
        Calculate yearly accuracy.
        
        Returns:
        --------
        tuple
            (year_labels, yearly_accuracy)
        """
        
        if not self.performance_data.empty:
            # Convert datetime to Period for yearly grouping
            self.performance_data['year'] = pd.to_datetime(self.performance_data['date']).dt.to_period('Y')
            
            # Group by year and calculate accuracy
            yearly_data = self.performance_data.groupby('year')['direction_correct'].agg(['sum', 'count'])
            yearly_data['accuracy'] = yearly_data['sum'] / yearly_data['count']
            
            # Convert index to string for plotting
            year_labels = [str(y) for y in yearly_data.index]
            yearly_accuracy = yearly_data['accuracy'].values
            
            return year_labels, yearly_accuracy
        
        return [], []
    
    def get_monthly_win_rate(self):
        """
        Calculate monthly win rate.
        
        Returns:
        --------
        tuple
            (month_labels, monthly_win_rate)
        """
        if not self.performance_data.empty:
            # Convert datetime to Period for monthly grouping
            self.performance_data['month'] = pd.to_datetime(self.performance_data['date']).dt.to_period('M')
            
            # Create a column for wins
            self.performance_data['win'] = self.performance_data['basic_return'] > 0
            
            # Group by month and calculate win rate (count only trades where return != 0)
            monthly_data = self.performance_data[self.performance_data['trade_direction'] != 0].groupby('month')['win'].agg(['sum', 'count'])
            monthly_data['win_rate'] = monthly_data['sum'] / monthly_data['count']
            
            # Convert index to string for plotting
            month_labels = [str(m) for m in monthly_data.index]
            monthly_win_rate = monthly_data['win_rate'].values
            
            return month_labels, monthly_win_rate
        
        return [], []
    
    def get_quarterly_win_rate(self):
        """
        Calculate quarterly win rate.
        
        Returns:
        --------
        tuple
            (quarter_labels, quarterly_win_rate)
        """
        if not self.performance_data.empty:
            # Convert datetime to Period for quarterly grouping
            self.performance_data['quarter'] = pd.to_datetime(self.performance_data['date']).dt.to_period('Q')
            
            # Create a column for wins
            self.performance_data['win'] = self.performance_data['basic_return'] > 0
            
            # Group by quarter and calculate win rate (count only trades where return != 0)
            quarterly_data = self.performance_data[self.performance_data['trade_direction'] != 0].groupby('quarter')['win'].agg(['sum', 'count'])
            quarterly_data['win_rate'] = quarterly_data['sum'] / quarterly_data['count']
            
            # Convert index to string for plotting
            quarter_labels = [str(q) for q in quarterly_data.index]
            quarterly_win_rate = quarterly_data['win_rate'].values
            
            return quarter_labels, quarterly_win_rate
        
        return [], []
    
    def get_yearly_win_rate(self):
        """
        Calculate yearly win rate.
        
        Returns:
        --------
        tuple
            (year_labels, yearly_win_rate)
        """
        if not self.performance_data.empty:
            # Convert datetime to Period for yearly grouping
            self.performance_data['year'] = pd.to_datetime(self.performance_data['date']).dt.to_period('Y')
            
            # Create a column for wins
            self.performance_data['win'] = self.performance_data['basic_return'] > 0
            
            # Group by year and calculate win rate (count only trades where return != 0)
            yearly_data = self.performance_data[self.performance_data['trade_direction'] != 0].groupby('year')['win'].agg(['sum', 'count'])
            yearly_data['win_rate'] = yearly_data['sum'] / yearly_data['count']
            
            # Convert index to string for plotting
            year_labels = [str(y) for y in yearly_data.index]
            yearly_win_rate = yearly_data['win_rate'].values
            
            return year_labels, yearly_win_rate
        
        return [], []
    
    def get_monthly_gain_loss_ratio(self):
        """
        Calculate monthly gain/loss ratio.
        
        Returns:
        --------
        tuple
            (month_labels, monthly_gain_loss_ratio)
        """
        if not self.performance_data.empty:
            # Convert datetime to Period for monthly grouping
            self.performance_data['month'] = pd.to_datetime(self.performance_data['date']).dt.to_period('M')
            
            # Filter for only days with trades
            trades_df = self.performance_data[self.performance_data['trade_direction'] != 0]
            
            # Group by month and get trades
            monthly_groups = trades_df.groupby('month')
            
            month_labels = []
            monthly_gain_loss_ratio = []
            
            for month, group in monthly_groups:
                gains = group[group['basic_return'] > 0]['basic_return'].values
                losses = group[group['basic_return'] < 0]['basic_return'].values
                
                avg_gain = np.mean(gains) if len(gains) > 0 else 0
                avg_loss = np.mean(losses) if len(losses) > 0 else 0
                
                # Calculate gain/loss ratio (absolute value of the ratio)
                ratio = abs(avg_gain / avg_loss) if avg_loss != 0 else 0
                
                month_labels.append(str(month))
                monthly_gain_loss_ratio.append(ratio)
            
            return month_labels, monthly_gain_loss_ratio
        
        return [], []
    
    def get_quarterly_gain_loss_ratio(self):
        """
        Calculate quarterly gain/loss ratio.
        
        Returns:
        --------
        tuple
            (quarter_labels, quarterly_gain_loss_ratio)
        """
        if not self.performance_data.empty:
            # Convert datetime to Period for quarterly grouping
            self.performance_data['quarter'] = pd.to_datetime(self.performance_data['date']).dt.to_period('Q')
            
            # Filter for only days with trades
            trades_df = self.performance_data[self.performance_data['trade_direction'] != 0]
            
            # Group by quarter and get trades
            quarterly_groups = trades_df.groupby('quarter')
            
            quarter_labels = []
            quarterly_gain_loss_ratio = []
            
            for quarter, group in quarterly_groups:
                gains = group[group['basic_return'] > 0]['basic_return'].values
                losses = group[group['basic_return'] < 0]['basic_return'].values
                
                avg_gain = np.mean(gains) if len(gains) > 0 else 0
                avg_loss = np.mean(losses) if len(losses) > 0 else 0
                
                # Calculate gain/loss ratio (absolute value of the ratio)
                ratio = abs(avg_gain / avg_loss) if avg_loss != 0 else 0
                
                quarter_labels.append(str(quarter))
                quarterly_gain_loss_ratio.append(ratio)
            
            return quarter_labels, quarterly_gain_loss_ratio
        
        return [], []
    
    def get_yearly_gain_loss_ratio(self):
        """
        Calculate yearly gain/loss ratio.
        
        Returns:
        --------
        tuple
            (year_labels, yearly_gain_loss_ratio)
        """
        if not self.performance_data.empty:
            # Convert datetime to Period for yearly grouping
            self.performance_data['year'] = pd.to_datetime(self.performance_data['date']).dt.to_period('Y')
            
            # Filter for only days with trades
            trades_df = self.performance_data[self.performance_data['trade_direction'] != 0]
            
            # Group by year and get trades
            yearly_groups = trades_df.groupby('year')
            
            year_labels = []
            yearly_gain_loss_ratio = []
            
            for year, group in yearly_groups:
                gains = group[group['basic_return'] > 0]['basic_return'].values
                losses = group[group['basic_return'] < 0]['basic_return'].values
                
                avg_gain = np.mean(gains) if len(gains) > 0 else 0
                avg_loss = np.mean(losses) if len(losses) > 0 else 0
                
                # Calculate gain/loss ratio (absolute value of the ratio)
                ratio = abs(avg_gain / avg_loss) if avg_loss != 0 else 0
                
                year_labels.append(str(year))
                yearly_gain_loss_ratio.append(ratio)
            
            return year_labels, yearly_gain_loss_ratio
        
        return [], []

    def get_enhanced_metrics(self):
        """Get enhanced error and directional accuracy metrics."""
        # Ensure metrics are updated
        self._update_metrics()
        
        # Build metrics dictionary with all enhanced metrics
        metrics = {
            # Original enhanced metrics
            'abs_peak_error': np.mean([abs(e) for e in self.peak_errors]) if self.peak_errors else 0,
            'abs_ev_error': np.mean([abs(e) for e in self.expected_value_errors]) if self.expected_value_errors else 0,
            
            # Add median error metrics
            'median_abs_peak_error': getattr(self, 'median_peak_error', 0),
            'median_abs_ev_error': getattr(self, 'median_expected_value_error', 0),
            'median_prediction_error': getattr(self, 'median_prediction_error', 0),
            'median_raw_peak_error': getattr(self, 'median_raw_peak_error', 0),
            'median_raw_expected_value_error': getattr(self, 'median_raw_expected_value_error', 0),
            
            # Directional accuracy
            'direction_accuracy': np.mean(self.direction_correct) if self.direction_correct else 0,
            
            # Add moving average metrics
            'accuracy_ma_20day': np.mean(self.direction_correct[-20:]) if len(self.direction_correct) >= 20 else np.mean(self.direction_correct) if self.direction_correct else 0,
            'accuracy_ma_50day': np.mean(self.direction_correct[-50:]) if len(self.direction_correct) >= 50 else np.mean(self.direction_correct) if self.direction_correct else 0,
            'accuracy_ma_100day': np.mean(self.direction_correct[-100:]) if len(self.direction_correct) >= 100 else np.mean(self.direction_correct) if self.direction_correct else 0,
            'accuracy_ma_200day': np.mean(self.direction_correct[-200:]) if len(self.direction_correct) >= 200 else np.mean(self.direction_correct) if self.direction_correct else 0,
            
            'abs_peak_error_ma_20day': np.mean([abs(e) for e in self.peak_errors[-20:]]) if len(self.peak_errors) >= 20 else np.mean([abs(e) for e in self.peak_errors]) if self.peak_errors else 0,
            'abs_peak_error_ma_50day': np.mean([abs(e) for e in self.peak_errors[-50:]]) if len(self.peak_errors) >= 50 else np.mean([abs(e) for e in self.peak_errors]) if self.peak_errors else 0,
            'abs_peak_error_ma_100day': np.mean([abs(e) for e in self.peak_errors[-100:]]) if len(self.peak_errors) >= 100 else np.mean([abs(e) for e in self.peak_errors]) if self.peak_errors else 0,
            'abs_peak_error_ma_200day': np.mean([abs(e) for e in self.peak_errors[-200:]]) if len(self.peak_errors) >= 200 else np.mean([abs(e) for e in self.peak_errors]) if self.peak_errors else 0,
            
            'abs_ev_error_ma_20day': np.mean([abs(e) for e in self.expected_value_errors[-20:]]) if len(self.expected_value_errors) >= 20 else np.mean([abs(e) for e in self.expected_value_errors]) if self.expected_value_errors else 0,
            'abs_ev_error_ma_50day': np.mean([abs(e) for e in self.expected_value_errors[-50:]]) if len(self.expected_value_errors) >= 50 else np.mean([abs(e) for e in self.expected_value_errors]) if self.expected_value_errors else 0,
            'abs_ev_error_ma_100day': np.mean([abs(e) for e in self.expected_value_errors[-100:]]) if len(self.expected_value_errors) >= 100 else np.mean([abs(e) for e in self.expected_value_errors]) if self.expected_value_errors else 0,
            'abs_ev_error_ma_200day': np.mean([abs(e) for e in self.expected_value_errors[-200:]]) if len(self.expected_value_errors) >= 200 else np.mean([abs(e) for e in self.expected_value_errors]) if self.expected_value_errors else 0,
        }
        
        # Add volatility bucket metrics
        if hasattr(self, 'vol_buckets'):
            for bucket, data in self.vol_buckets.items():
                if data['total'] > 0:
                    metrics[f'vol_{bucket}_accuracy'] = np.mean(data['dir_correct']) if data['dir_correct'] else 0
                    metrics[f'vol_{bucket}_peak_error'] = np.mean(data['peak_errors']) if data['peak_errors'] else 0
                    metrics[f'vol_{bucket}_ev_error'] = np.mean(data['ev_errors']) if data['ev_errors'] else 0
                    metrics[f'vol_{bucket}_count'] = data['total']
        
        return metrics
    
    def get_metrics_summary(self):
        """
        Get summary of all performance metrics.
        Returns:
        --------
        dict
            Dictionary of performance metrics
        """
        
        # Calculate years for CAGR and MAR ratio
        days = self.total_days
        years = days / 252 if days > 0 else 0
        
        # Get portfolio values as lists
        portfolio_values_tradinghours = list(self.portfolio_values.values())
        portfolio_values_afterhours = list(self.portfolio_values_afterhours.values())
        nextday_values = list(self.nextday_values.values())
        leverage_values = list(self.leverage_values.values())
        shorting_values = list(self.shorting_values.values())
        short_leverage_values = list(self.short_leverage_values.values())
        buyhold_values = list(self.buyhold_values.values())
        
        # Add underwater metrics
        basic_underwater = self._calculate_underwater_metrics(portfolio_values_tradinghours)
        afterhours_underwater = self._calculate_underwater_metrics(portfolio_values_afterhours)
        nextday_underwater = self._calculate_underwater_metrics(nextday_values)
        leverage_underwater = self._calculate_underwater_metrics(leverage_values)
        shorting_underwater = self._calculate_underwater_metrics(shorting_values)
        short_leverage_underwater = self._calculate_underwater_metrics(short_leverage_values)
        buyhold_underwater = self._calculate_underwater_metrics(buyhold_values)

        # Calculate profit factor
        profit_factor_tradinghours = self._calculate_profit_factor(self.wins_tradinghours, self.losses_tradinghours)
        profit_factor_afterhours = self._calculate_profit_factor(self.wins_afterhours, self.losses_afterhours)
        
        # Calculate expectancy
        expectancy_tradinghours = self._calculate_expectancy(
            self.win_rate_tradinghours, 
            self.avg_gain_tradinghours, 
            self.avg_loss_tradinghours
        )
        expectancy_afterhours = self._calculate_expectancy(
            self.win_rate_afterhours, 
            self.avg_gain_afterhours, 
            self.avg_loss_afterhours
        )
        
        # Calculate System Quality Number
        sqn_tradinghours = self._calculate_sqn(self.daily_returns, self.trade_count)
        sqn_afterhours = self._calculate_sqn(self.daily_returns_afterhours, self.trade_count_afterhours)
        
        # Calculate market capture ratios
        capture_tradinghours = self._calculate_market_capture_ratios(self.daily_returns, self.buyhold_returns)
        capture_afterhours = self._calculate_market_capture_ratios(self.daily_returns_afterhours, self.buyhold_returns)
        capture_nextday = self._calculate_market_capture_ratios(self.daily_returns_nextday, self.buyhold_returns)
        capture_leverage = self._calculate_market_capture_ratios(self.leverage_daily_returns, self.buyhold_returns)
        capture_shorting = self._calculate_market_capture_ratios(self.shorting_daily_returns, self.buyhold_returns)
        capture_short_leverage = self._calculate_market_capture_ratios(self.short_leverage_daily_returns, self.buyhold_returns)
        
        # Calculate Value at Risk and Conditional VaR
        var_tradinghours = self._calculate_var(self.daily_returns)
        cvar_tradinghours = self._calculate_cvar(self.daily_returns)
        var_afterhours = self._calculate_var(self.daily_returns_afterhours)
        cvar_afterhours = self._calculate_cvar(self.daily_returns_afterhours)
        
        # Calculate confidence-weighted accuracy
        conf_weighted_accuracy = self._calculate_confidence_weighted_accuracy()
        
        # Calculate decision boundary analysis (optional - may be expensive)
        # decision_boundaries = self._analyze_decision_boundaries()
        
        # Calculate Calmar and MAR ratios
        calmar_tradinghours = self._calculate_calmar_ratio(self.daily_returns, self.max_drawdown_tradinghours)
        calmar_afterhours = self._calculate_calmar_ratio(self.daily_returns_afterhours, self.max_drawdown_afterhours)
        calmar_nextday = self._calculate_calmar_ratio(self.daily_returns_nextday, self.max_drawdown_nextday)
        calmar_leverage = self._calculate_calmar_ratio(self.leverage_daily_returns, self.leverage_max_drawdown)
        calmar_shorting = self._calculate_calmar_ratio(self.shorting_daily_returns, self.shorting_max_drawdown)
        calmar_short_leverage = self._calculate_calmar_ratio(self.short_leverage_daily_returns, self.short_leverage_max_drawdown)
        calmar_buyhold = self._calculate_calmar_ratio(self.buyhold_returns, self.buyhold_max_drawdown)
        
        # Calculate CAGR (if available)
        cagr_tradinghours = self._calculate_cagr(self.total_return_tradinghours, years) if years > 0 else None
        cagr_afterhours = self._calculate_cagr(self.total_return_afterhours, years) if years > 0 else None
        cagr_nextday = self._calculate_cagr(self.total_return_nextday, years) if years > 0 else None
        cagr_leverage = self._calculate_cagr(self.leverage_return, years) if years > 0 else None
        cagr_shorting = self._calculate_cagr(self.shorting_return, years) if years > 0 else None
        cagr_short_leverage = self._calculate_cagr(self.short_leverage_return, years) if years > 0 else None
        cagr_buyhold = self._calculate_cagr(self.buyhold_return, years) if years > 0 else None
        
        # Calculate Modified CAGR
        mod_cagr_tradinghours = self._calculate_modified_cagr(
            self.total_return_tradinghours, years, self.max_drawdown_tradinghours) if years > 0 else None
        mod_cagr_afterhours = self._calculate_modified_cagr(
            self.total_return_afterhours, years, self.max_drawdown_afterhours) if years > 0 else None
        mod_cagr_nextday = self._calculate_modified_cagr(
            self.total_return_nextday, years, self.max_drawdown_nextday) if years > 0 else None
        mod_cagr_leverage = self._calculate_modified_cagr(
            self.leverage_return, years, self.leverage_max_drawdown) if years > 0 else None
        mod_cagr_shorting = self._calculate_modified_cagr(
            self.shorting_return, years, self.shorting_max_drawdown) if years > 0 else None
        mod_cagr_short_leverage = self._calculate_modified_cagr(
            self.short_leverage_return, years, self.short_leverage_max_drawdown) if years > 0 else None

        metrics = {
            # Directional accuracy
            'accuracy': self.accuracy,
            
            # Error metrics
            'mean_error': self.mean_error,
            'mean_peak_error': np.mean(self.peak_errors) if self.peak_errors else 0,
            'mean_expected_value_error': np.mean(self.expected_value_errors) if self.expected_value_errors else 0,
            
            # Return metrics for basic strategy trading hours
            'total_return_tradinghours': self.total_return_tradinghours,
            'max_drawdown_tradinghours': self.max_drawdown_tradinghours,
            'sharpe_ratio_tradinghours': self.sharpe_ratio_tradinghours,
            'sortino_ratio_tradinghours': self.sortino_ratio_tradinghours,
            'annual_return_tradinghours': getattr(self, 'annual_return_tradinghours', None),
            
            # Return metrics for basic strategy after hours
            'total_return_afterhours': self.total_return_afterhours,
            'max_drawdown_afterhours': self.max_drawdown_afterhours,
            'sharpe_ratio_afterhours': self.sharpe_ratio_afterhours,
            'sortino_ratio_afterhours': self.sortino_ratio_afterhours,
            'annual_return_afterhours': getattr(self, 'annual_return_afterhours', None),

            # Return metrics for nextday strategy
            'total_return_nextday': self.total_return_nextday,
            'max_drawdown_nextday': self.max_drawdown_nextday,
            'sharpe_ratio_nextday': self.sharpe_ratio_nextday,
            'sortino_ratio_nextday': self.sortino_ratio_nextday,
            'win_rate_nextday': self.win_rate_nextday,
            'gain_loss_ratio_nextday': self.gain_loss_ratio_nextday,
            'annual_return_nextday': getattr(self, 'annual_return_nextday', None),
            
            # Return metrics for leverage strategy
            'leverage_return': self.leverage_return,
            'leverage_max_drawdown': self.leverage_max_drawdown,
            'leverage_sharpe_ratio': self.leverage_sharpe_ratio,
            'leverage_sortino_ratio': self.leverage_sortino_ratio,
            'leverage_annual_return': getattr(self, 'leverage_annual_return', None),
            
            # Return metrics for shorting strategy
            'shorting_return': self.shorting_return,
            'shorting_max_drawdown': self.shorting_max_drawdown,
            'shorting_sharpe_ratio': self.shorting_sharpe_ratio,
            'shorting_sortino_ratio': self.shorting_sortino_ratio,
            'shorting_annual_return': getattr(self, 'shorting_annual_return', None),

            # Return metrics for short_leverage strategy
            'short_leverage_return': self.short_leverage_return,
            'short_leverage_max_drawdown': self.short_leverage_max_drawdown,
            'short_leverage_sharpe_ratio': self.short_leverage_sharpe_ratio, 
            'short_leverage_sortino_ratio': self.short_leverage_sortino_ratio,
            'annual_return_short_leverage': getattr(self, 'annual_return_short_leverage', None),
            
            # Return metrics for buy-and-hold strategy
            'buyhold_return': self.buyhold_return,
            'buyhold_max_drawdown': self.buyhold_max_drawdown,
            'buyhold_sharpe_ratio': self.buyhold_sharpe_ratio,
            'buyhold_sortino_ratio': self.buyhold_sortino_ratio,
            'buyhold_annual_return': getattr(self, 'buyhold_annual_return', None),
            
            # Win/Loss metrics - Trading Hours
            'win_rate_tradinghours': self.win_rate_tradinghours,
            'avg_gain_tradinghours': self.avg_gain_tradinghours,
            'avg_loss_tradinghours': self.avg_loss_tradinghours,
            'gain_loss_ratio_tradinghours': self.gain_loss_ratio_tradinghours,
            'max_consecutive_wins': self.max_consecutive_wins,
            'max_consecutive_losses': self.max_consecutive_losses,
            'max_consecutive_wins_start_date': self.max_consecutive_wins_start_date,
            'max_consecutive_wins_end_date': self.max_consecutive_wins_end_date,
            'max_consecutive_losses_start_date': self.max_consecutive_losses_start_date,
            'max_consecutive_losses_end_date': self.max_consecutive_losses_end_date,
            
            # Win/Loss metrics - After Hours
            'win_rate_afterhours': self.win_rate_afterhours,
            'avg_gain_afterhours': self.avg_gain_afterhours,
            'avg_loss_afterhours': self.avg_loss_afterhours,
            'gain_loss_ratio_afterhours': self.gain_loss_ratio_afterhours,
            'max_consecutive_wins_afterhours': self.max_consecutive_wins_afterhours,
            'max_consecutive_losses_afterhours': self.max_consecutive_losses_afterhours,
            'max_consecutive_wins_start_date_afterhours': self.max_consecutive_wins_start_date_afterhours,
            'max_consecutive_wins_end_date_afterhours': self.max_consecutive_wins_end_date_afterhours,
            'max_consecutive_losses_start_date_afterhours': self.max_consecutive_losses_start_date_afterhours,
            'max_consecutive_losses_end_date_afterhours': self.max_consecutive_losses_end_date_afterhours,
            
            # Trading frequency
            'trading_frequency_tradinghours': self.trading_frequency_tradinghours,
            'trading_frequency_afterhours': self.trading_frequency_afterhours,
            'trade_count': self.trade_count,
            'trade_count_afterhours': self.trade_count_afterhours,
            'total_days': self.total_days,
            
            # Risk metrics
            'beta_tradinghours': self.beta_tradinghours,
            'beta_afterhours': self.beta_afterhours,
            'leverage_beta': self.leverage_beta,
            'shorting_beta': self.shorting_beta,
            
            # Confusion matrix data
            'confusion_matrix': {
                'true_positives': self.true_positives,
                'false_positives': self.false_positives,
                'true_negatives': self.true_negatives,
                'false_negatives': self.false_negatives
            },
            
            # New metrics for risk avoidance and market participation - Trading Hours
            'risk_avoidance_rate': self.risk_avoidance_rate,
            'market_participation_rate': self.market_participation_rate,
            'total_risk_predictions': self.total_risk_predictions,
            'correct_risk_avoidances': self.correct_risk_avoidances,
            
            # New metrics for risk avoidance and market participation - After Hours
            'risk_avoidance_rate_afterhours': self.risk_avoidance_rate_afterhours,
            'market_participation_rate_afterhours': self.market_participation_rate_afterhours,
            'total_risk_predictions_afterhours': self.total_risk_predictions_afterhours,
            'correct_risk_avoidances_afterhours': self.correct_risk_avoidances_afterhours,  
            'recent_trades_count': self.recent_trades_count,

            # Underwater metrics
            'underwater_metrics_tradinghours': basic_underwater,
            'underwater_metrics_afterhours': afterhours_underwater,  
            'underwater_metrics_nextday': nextday_underwater,       
            'underwater_metrics_leverage': leverage_underwater,
            'underwater_metrics_shorting': shorting_underwater,
            'underwater_metrics_short_leverage': short_leverage_underwater, 
            'underwater_metrics_buyhold': buyhold_underwater,
            
            # Trade quality metrics
            'profit_factor_tradinghours': profit_factor_tradinghours,
            'profit_factor_afterhours': profit_factor_afterhours,
            'expectancy_tradinghours': expectancy_tradinghours,
            'expectancy_afterhours': expectancy_afterhours,
            'sqn_tradinghours': sqn_tradinghours,
            'sqn_afterhours': sqn_afterhours,
            
            # Market regime performance
            'up_capture_tradinghours': capture_tradinghours['up_capture'],
            'down_capture_tradinghours': capture_tradinghours['down_capture'],
            'capture_ratio_tradinghours': capture_tradinghours['capture_ratio'],
            'up_capture_afterhours': capture_afterhours['up_capture'],
            'down_capture_afterhours': capture_afterhours['down_capture'],
            'capture_ratio_afterhours': capture_afterhours['capture_ratio'],
            'up_capture_nextday': capture_nextday['up_capture'],
            'down_capture_nextday': capture_nextday['down_capture'],
            'capture_ratio_nextday': capture_nextday['capture_ratio'],
            
            'up_capture_short_leverage': capture_short_leverage['up_capture'],
            'down_capture_short_leverage': capture_short_leverage['down_capture'],
            'capture_ratio_short_leverage': capture_short_leverage['capture_ratio'],
            
            # Risk metrics
            'var_95_tradinghours': var_tradinghours,
            'cvar_95_tradinghours': cvar_tradinghours,
            'var_95_afterhours': var_afterhours,
            'cvar_95_afterhours': cvar_afterhours,
            
            # Confidence-weighted accuracy
            'confidence_weighted_accuracy': conf_weighted_accuracy,
            
            # Calmar and MAR ratios
            'calmar_ratio_tradinghours': calmar_tradinghours,
            'calmar_ratio_afterhours': calmar_afterhours,
            'calmar_ratio_leverage': calmar_leverage,
            'calmar_ratio_shorting': calmar_shorting,
            'calmar_ratio_buyhold': calmar_buyhold,
            'calmar_ratio_nextday': calmar_nextday,
            'calmar_ratio_short_leverage': calmar_short_leverage,
            
            # CAGR metrics (if available)
            'cagr_tradinghours': cagr_tradinghours,
            'cagr_afterhours': cagr_afterhours,
            'cagr_leverage': cagr_leverage,
            'cagr_shorting': cagr_shorting,
            'cagr_buyhold': cagr_buyhold,
            'cagr_nextday': cagr_nextday,
            'cagr_short_leverage': cagr_short_leverage,
            
            # Modified CAGR
            'modified_cagr_tradinghours': mod_cagr_tradinghours,
            'modified_cagr_afterhours': mod_cagr_afterhours,
            'modified_cagr_nextday': mod_cagr_nextday,
            'modified_cagr_short_leverage': mod_cagr_short_leverage,
        }
        return metrics
    
    def plot_cumulative_returns(self):
        """
        Plot cumulative returns of all trading strategies vs buy-and-hold.
        
        Returns:
        --------
        matplotlib.figure.Figure
            Matplotlib figure object
        """
        
        dates = list(self.portfolio_values.keys())
        basic_values = np.array(list(self.portfolio_values.values()))
        leverage_values = np.array(list(self.leverage_values.values()))
        shorting_values = np.array(list(self.shorting_values.values()))
        buyhold_values = np.array(list(self.buyhold_values.values()))
        
        # Normalize to percentage return
        basic_returns = (basic_values / basic_values[0] - 1) * 100
        leverage_returns = (leverage_values / leverage_values[0] - 1) * 100
        shorting_returns = (shorting_values / shorting_values[0] - 1) * 100
        buyhold_returns = (buyhold_values / buyhold_values[0] - 1) * 100
        
        plt.figure(figsize=(12, 7))
        plt.plot(dates, basic_returns, 'b-', linewidth=2, label='Basic Strategy')
        plt.plot(dates, afterhours_returns, 'm-', linewidth=2, label='After Hours')
        plt.plot(dates, nextday_returns, 'c-', linewidth=2, label='Next Day')
        plt.plot(dates, leverage_returns, 'g-', linewidth=2, label='Leverage')
        plt.plot(dates, shorting_returns, 'r-', linewidth=2, label='Shorting')
        plt.plot(dates, short_leverage_returns, 'y-', linewidth=2, label='Short+Leverage')
        plt.plot(dates, buyhold_returns, 'k--', linewidth=2, label='Buy and Hold')
        plt.title('Cumulative Returns Comparison')
        plt.xlabel('Date')
        plt.ylabel('Return (%)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Add annotations for key metrics
        plt.annotate(
            f"Basic Tradinghours Strategy: {self.basic_returns*100:.2f}%\n"
            f"Basic After Hours Strategy: {self.afterhours_returns*100:.2f}%\n"
            f"Basic Next Day Strategy: {self.nextday_returns*100:.2f}%\n"
            f"Leverage Strategy: {self.leverage_return*100:.2f}%\n"
            f"Shorting Strategy: {self.shorting_return*100:.2f}%\n"
            f"Short+Leverage Strategy: {self.short_leverage_returns*100:.2f}%\n",
            f"Buy & Hold: {self.buyhold_return*100:.2f}%\n",
            xy=(0.02, 0.96),
            xycoords='axes fraction',
            bbox=dict(boxstyle="round,pad=0.5", fc="white", alpha=0.8),
            fontsize=10,
            verticalalignment='top'
        )
        
        plt.tight_layout()
        return plt.gcf()
    
    def plot_accuracy_moving_averages(self):
        """
        Plot moving averages of prediction accuracy (daily, monthly, quarterly, yearly).
        
        Returns:
        --------
        matplotlib.figure.Figure
            Matplotlib figure object
        """
        fig, axes = plt.subplots(4, 1, figsize=(12, 15), sharex=False)
        
        # 1. Daily moving average
        window_days = min(20, max(5, len(self.direction_correct) // 5))
        if len(self.direction_correct) >= window_days:
            ma_dates, moving_avg = self.get_moving_average_accuracy(window_days)
            
            axes[0].plot(ma_dates, moving_avg * 100, 'b-', linewidth=2)
            axes[0].axhline(50, color='r', linestyle='--', alpha=0.7, label='Random Guess (50%)')
            axes[0].axhline(np.mean(self.direction_correct) * 100, color='g', linestyle='--', 
                          label=f'Overall Accuracy: {np.mean(self.direction_correct)*100:.2f}%')
            
            axes[0].set_title(f'{window_days}-Day Moving Average of Prediction Accuracy')
            axes[0].set_xlabel('Date')
            axes[0].set_ylabel('Accuracy (%)')
            axes[0].legend()
            axes[0].grid(True, alpha=0.3)
            axes[0].set_ylim(0, 100)
        else:
            axes[0].text(0.5, 0.5, f"Not enough data for {window_days}-day moving average", 
                       horizontalalignment='center', verticalalignment='center')
        
        # 2. Monthly accuracy
        month_labels, monthly_accuracy = self.get_monthly_accuracy()
        
        if month_labels:
            axes[1].bar(range(len(month_labels)), monthly_accuracy * 100, color='b', alpha=0.7)
            axes[1].axhline(50, color='r', linestyle='--', alpha=0.7)
            axes[1].axhline(np.mean(self.direction_correct) * 100, color='g', linestyle='--')
            
            axes[1].set_title('Monthly Prediction Accuracy')
            axes[1].set_xlabel('Month')
            axes[1].set_ylabel('Accuracy (%)')
            
            # Set x-ticks to month labels (show subset if too many)
            if len(month_labels) > 12:
                step = len(month_labels) // 12
                axes[1].set_xticks(range(0, len(month_labels), step))
                axes[1].set_xticklabels([month_labels[i] for i in range(0, len(month_labels), step)], rotation=45)
            else:
                axes[1].set_xticks(range(len(month_labels)))
                axes[1].set_xticklabels(month_labels, rotation=45)
            
            axes[1].grid(True, alpha=0.3)
            axes[1].set_ylim(0, 100)
        else:
            axes[1].text(0.5, 0.5, "Not enough data for monthly accuracy", 
                       horizontalalignment='center', verticalalignment='center')
        
        # 3. Quarterly accuracy
        quarter_labels, quarterly_accuracy = self.get_quarterly_accuracy()
        
        if quarter_labels:
            axes[2].bar(range(len(quarter_labels)), quarterly_accuracy * 100, color='g', alpha=0.7)
            axes[2].axhline(50, color='r', linestyle='--', alpha=0.7)
            axes[2].axhline(np.mean(self.direction_correct) * 100, color='g', linestyle='--')
            
            axes[2].set_title('Quarterly Prediction Accuracy')
            axes[2].set_xlabel('Quarter')
            axes[2].set_ylabel('Accuracy (%)')
            
            # Set x-ticks to quarter labels (show subset if too many)
            if len(quarter_labels) > 8:
                step = len(quarter_labels) // 8
                axes[2].set_xticks(range(0, len(quarter_labels), step))
                axes[2].set_xticklabels([quarter_labels[i] for i in range(0, len(quarter_labels), step)], rotation=45)
            else:
                axes[2].set_xticks(range(len(quarter_labels)))
                axes[2].set_xticklabels(quarter_labels, rotation=45)
            
            axes[2].grid(True, alpha=0.3)
            axes[2].set_ylim(0, 100)
        else:
            axes[2].text(0.5, 0.5, "Not enough data for quarterly accuracy", 
                       horizontalalignment='center', verticalalignment='center')
        
        # 4. Yearly accuracy
        year_labels, yearly_accuracy = self.get_yearly_accuracy()
        
        if year_labels:
            axes[3].bar(range(len(year_labels)), yearly_accuracy * 100, color='purple', alpha=0.7)
            axes[3].axhline(50, color='r', linestyle='--', alpha=0.7)
            axes[3].axhline(np.mean(self.direction_correct) * 100, color='g', linestyle='--')
            
            axes[3].set_title('Yearly Prediction Accuracy')
            axes[3].set_xlabel('Year')
            axes[3].set_ylabel('Accuracy (%)')
            
            axes[3].set_xticks(range(len(year_labels)))
            axes[3].set_xticklabels(year_labels, rotation=45)
            
            axes[3].grid(True, alpha=0.3)
            axes[3].set_ylim(0, 100)
        else:
            axes[3].text(0.5, 0.5, "Not enough data for yearly accuracy", 
                       horizontalalignment='center', verticalalignment='center')
        
        plt.tight_layout()
        
        return fig
    
    def plot_win_rate_moving_averages(self):
        """
        Plot win rate moving averages (monthly, quarterly, yearly).
        
        Returns:
        --------
        matplotlib.figure.Figure
            Matplotlib figure object
        """
        fig, axes = plt.subplots(3, 1, figsize=(12, 12), sharex=False)
        
        # 1. Monthly win rate
        month_labels, monthly_win_rate = self.get_monthly_win_rate()
        
        if month_labels:
            axes[0].bar(range(len(month_labels)), monthly_win_rate * 100, color='b', alpha=0.7)
            axes[0].axhline(50, color='r', linestyle='--', alpha=0.7, label='50% Win Rate')
            axes[0].axhline(self.win_rate * 100, color='g', linestyle='--', 
                          label=f'Overall Win Rate: {self.win_rate*100:.2f}%')
            
            axes[0].set_title('Monthly Win Rate')
            axes[0].set_xlabel('Month')
            axes[0].set_ylabel('Win Rate (%)')
            axes[0].legend()
            
            # Set x-ticks to month labels (show subset if too many)
            if len(month_labels) > 12:
                step = len(month_labels) // 12
                axes[0].set_xticks(range(0, len(month_labels), step))
                axes[0].set_xticklabels([month_labels[i] for i in range(0, len(month_labels), step)], rotation=45)
            else:
                axes[0].set_xticks(range(len(month_labels)))
                axes[0].set_xticklabels(month_labels, rotation=45)
            
            axes[0].grid(True, alpha=0.3)
            axes[0].set_ylim(0, 100)
        else:
            axes[0].text(0.5, 0.5, "Not enough data for monthly win rate", 
                       horizontalalignment='center', verticalalignment='center')
        
        # 2. Quarterly win rate
        quarter_labels, quarterly_win_rate = self.get_quarterly_win_rate()
        
        if quarter_labels:
            axes[1].bar(range(len(quarter_labels)), quarterly_win_rate * 100, color='g', alpha=0.7)
            axes[1].axhline(50, color='r', linestyle='--', alpha=0.7)
            axes[1].axhline(self.win_rate * 100, color='g', linestyle='--')
            
            axes[1].set_title('Quarterly Win Rate')
            axes[1].set_xlabel('Quarter')
            axes[1].set_ylabel('Win Rate (%)')
            
            # Set x-ticks to quarter labels (show subset if too many)
            if len(quarter_labels) > 8:
                step = len(quarter_labels) // 8
                axes[1].set_xticks(range(0, len(quarter_labels), step))
                axes[1].set_xticklabels([quarter_labels[i] for i in range(0, len(quarter_labels), step)], rotation=45)
            else:
                axes[1].set_xticks(range(len(quarter_labels)))
                axes[1].set_xticklabels(quarter_labels, rotation=45)
            
            axes[1].grid(True, alpha=0.3)
            axes[1].set_ylim(0, 100)
        else:
            axes[1].text(0.5, 0.5, "Not enough data for quarterly win rate", 
                       horizontalalignment='center', verticalalignment='center')
        
        # 3. Yearly win rate
        year_labels, yearly_win_rate = self.get_yearly_win_rate()
        
        if year_labels:
            axes[2].bar(range(len(year_labels)), yearly_win_rate * 100, color='purple', alpha=0.7)
            axes[2].axhline(50, color='r', linestyle='--', alpha=0.7)
            axes[2].axhline(self.win_rate * 100, color='g', linestyle='--')
            
            axes[2].set_title('Yearly Win Rate')
            axes[2].set_xlabel('Year')
            axes[2].set_ylabel('Win Rate (%)')
            
            axes[2].set_xticks(range(len(year_labels)))
            axes[2].set_xticklabels(year_labels, rotation=45)
            
            axes[2].grid(True, alpha=0.3)
            axes[2].set_ylim(0, 100)
        else:
            axes[2].text(0.5, 0.5, "Not enough data for yearly win rate", 
                       horizontalalignment='center', verticalalignment='center')
        
        plt.tight_layout()
        
        return fig
    
    def plot_gain_loss_ratio_moving_averages(self):
        """
        Plot gain/loss ratio moving averages (monthly, quarterly, yearly).
        
        Returns:
        --------
        matplotlib.figure.Figure
            Matplotlib figure object
        """
        fig, axes = plt.subplots(3, 1, figsize=(12, 12), sharex=False)
        
        # 1. Monthly gain/loss ratio
        month_labels, monthly_gain_loss_ratio = self.get_monthly_gain_loss_ratio()
        
        if month_labels:
            axes[0].bar(range(len(month_labels)), monthly_gain_loss_ratio, color='b', alpha=0.7)
            axes[0].axhline(1.0, color='r', linestyle='--', alpha=0.7, label='1.0 (Breakeven)')
            axes[0].axhline(self.gain_loss_ratio, color='g', linestyle='--', 
                          label=f'Overall Ratio: {self.gain_loss_ratio:.2f}')
            
            axes[0].set_title('Monthly Gain/Loss Ratio')
            axes[0].set_xlabel('Month')
            axes[0].set_ylabel('Gain/Loss Ratio')
            axes[0].legend()
            
            # Set x-ticks to month labels (show subset if too many)
            if len(month_labels) > 12:
                step = len(month_labels) // 12
                axes[0].set_xticks(range(0, len(month_labels), step))
                axes[0].set_xticklabels([month_labels[i] for i in range(0, len(month_labels), step)], rotation=45)
            else:
                axes[0].set_xticks(range(len(month_labels)))
                axes[0].set_xticklabels(month_labels, rotation=45)
            
            axes[0].grid(True, alpha=0.3)
            
            # Set y-limit to a reasonable range
            max_ratio = max(monthly_gain_loss_ratio + [self.gain_loss_ratio, 1.0])
            axes[0].set_ylim(0, min(max_ratio * 1.2, 5))
        else:
            axes[0].text(0.5, 0.5, "Not enough data for monthly gain/loss ratio", 
                       horizontalalignment='center', verticalalignment='center')
        
        # 2. Quarterly gain/loss ratio
        quarter_labels, quarterly_gain_loss_ratio = self.get_quarterly_gain_loss_ratio()
        
        if quarter_labels:
            axes[1].bar(range(len(quarter_labels)), quarterly_gain_loss_ratio, color='g', alpha=0.7)
            axes[1].axhline(1.0, color='r', linestyle='--', alpha=0.7)
            axes[1].axhline(self.gain_loss_ratio, color='g', linestyle='--')
            
            axes[1].set_title('Quarterly Gain/Loss Ratio')
            axes[1].set_xlabel('Quarter')
            axes[1].set_ylabel('Gain/Loss Ratio')
            
            # Set x-ticks to quarter labels (show subset if too many)
            if len(quarter_labels) > 8:
                step = len(quarter_labels) // 8
                axes[1].set_xticks(range(0, len(quarter_labels), step))
                axes[1].set_xticklabels([quarter_labels[i] for i in range(0, len(quarter_labels), step)], rotation=45)
            else:
                axes[1].set_xticks(range(len(quarter_labels)))
                axes[1].set_xticklabels(quarter_labels, rotation=45)
            
            axes[1].grid(True, alpha=0.3)
            
            # Set y-limit to a reasonable range
            max_ratio = max(quarterly_gain_loss_ratio + [self.gain_loss_ratio, 1.0])
            axes[1].set_ylim(0, min(max_ratio * 1.2, 5))
        else:
            axes[1].text(0.5, 0.5, "Not enough data for quarterly gain/loss ratio", 
                       horizontalalignment='center', verticalalignment='center')
        
        # 3. Yearly gain/loss ratio
        year_labels, yearly_gain_loss_ratio = self.get_yearly_gain_loss_ratio()
        
        if year_labels:
            axes[2].bar(range(len(year_labels)), yearly_gain_loss_ratio, color='purple', alpha=0.7)
            axes[2].axhline(1.0, color='r', linestyle='--', alpha=0.7)
            axes[2].axhline(self.gain_loss_ratio, color='g', linestyle='--')
            
            axes[2].set_title('Yearly Gain/Loss Ratio')
            axes[2].set_xlabel('Year')
            axes[2].set_ylabel('Gain/Loss Ratio')
            
            axes[2].set_xticks(range(len(year_labels)))
            axes[2].set_xticklabels(year_labels, rotation=45)
            
            axes[2].grid(True, alpha=0.3)
            
            # Set y-limit to a reasonable range
            max_ratio = max(yearly_gain_loss_ratio + [self.gain_loss_ratio, 1.0])
            axes[2].set_ylim(0, min(max_ratio * 1.2, 5))
        else:
            axes[2].text(0.5, 0.5, "Not enough data for yearly gain/loss ratio", 
                       horizontalalignment='center', verticalalignment='center')
        
        plt.tight_layout()
        
        return fig
    
    def plot_prediction_errors(self):
        """
        Plot prediction error distributions and statistics.
        
        Returns:
        --------
        matplotlib.figure.Figure
            Matplotlib figure object
        """
        fig, axes = plt.subplots(2, 2, figsize=(14, 12))
        
        # 1. Peak error histogram (most likely value - actual)
        if self.peak_errors:
            axes[0, 0].hist(self.peak_errors, bins=30, alpha=0.7, color='b')
            axes[0, 0].axvline(np.mean(self.peak_errors), color='r', linestyle='--',
                             label=f'Mean: {np.mean(self.peak_errors):.2f}%')
            axes[0, 0].axvline(0, color='g', linestyle='--', label='Perfect Prediction')
            
            axes[0, 0].set_title('Peak Prediction Error (Most Likely - Actual)')
            axes[0, 0].set_xlabel('Error (%)')
            axes[0, 0].set_ylabel('Frequency')
            axes[0, 0].legend()
            axes[0, 0].grid(True, alpha=0.3)
            
            # Add statistics annotation
            peak_error_mean = np.mean(self.peak_errors)
            peak_error_std = np.std(self.peak_errors)
            peak_error_rmse = np.sqrt(np.mean(np.array(self.peak_errors)**2))
            
            axes[0, 0].text(0.02, 0.95, 
                          f"Mean: {peak_error_mean:.2f}%\n"
                          f"Std Dev: {peak_error_std:.2f}%\n"
                          f"RMSE: {peak_error_rmse:.2f}%",
                          transform=axes[0, 0].transAxes,
                          bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.8),
                          fontsize=9,
                          verticalalignment='top')
        else:
            axes[0, 0].text(0.5, 0.5, "No peak error data available", 
                          horizontalalignment='center', verticalalignment='center')
        
        # 2. Expected value error histogram (expected value - actual)
        if self.expected_value_errors:
            axes[0, 1].hist(self.expected_value_errors, bins=30, alpha=0.7, color='g')
            axes[0, 1].axvline(np.mean(self.expected_value_errors), color='r', linestyle='--',
                             label=f'Mean: {np.mean(self.expected_value_errors):.2f}%')
            axes[0, 1].axvline(0, color='g', linestyle='--', label='Perfect Prediction')
            
            axes[0, 1].set_title('Expected Value Error (Expected - Actual)')
            axes[0, 1].set_xlabel('Error (%)')
            axes[0, 1].set_ylabel('Frequency')
            axes[0, 1].legend()
            axes[0, 1].grid(True, alpha=0.3)
            
            # Add statistics annotation
            ev_error_mean = np.mean(self.expected_value_errors)
            ev_error_std = np.std(self.expected_value_errors)
            ev_error_rmse = np.sqrt(np.mean(np.array(self.expected_value_errors)**2))
            
            axes[0, 1].text(0.02, 0.95, 
                          f"Mean: {ev_error_mean:.2f}%\n"
                          f"Std Dev: {ev_error_std:.2f}%\n"
                          f"RMSE: {ev_error_rmse:.2f}%",
                          transform=axes[0, 1].transAxes,
                          bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.8),
                          fontsize=9,
                          verticalalignment='top')
        else:
            axes[0, 1].text(0.5, 0.5, "No expected value error data available", 
                          horizontalalignment='center', verticalalignment='center')
        
        # 3. Absolute error histogram
        if self.prediction_errors:
            axes[1, 0].hist(self.prediction_errors, bins=30, alpha=0.7, color='purple')
            axes[1, 0].axvline(np.mean(self.prediction_errors), color='r', linestyle='--',
                             label=f'Mean: {np.mean(self.prediction_errors):.2f}%')
            
            axes[1, 0].set_title('Absolute Prediction Error')
            axes[1, 0].set_xlabel('Absolute Error (%)')
            axes[1, 0].set_ylabel('Frequency')
            axes[1, 0].legend()
            axes[1, 0].grid(True, alpha=0.3)
            
            # Add statistics annotation
            abs_error_mean = np.mean(self.prediction_errors)
            abs_error_std = np.std(self.prediction_errors)
            abs_error_median = np.median(self.prediction_errors)
            
            axes[1, 0].text(0.02, 0.95, 
                          f"Mean: {abs_error_mean:.2f}%\n"
                          f"Std Dev: {abs_error_std:.2f}%\n"
                          f"Median: {abs_error_median:.2f}%",
                          transform=axes[1, 0].transAxes,
                          bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.8),
                          fontsize=9,
                          verticalalignment='top')
        else:
            axes[1, 0].text(0.5, 0.5, "No prediction error data available", 
                          horizontalalignment='center', verticalalignment='center')
        
        # 4. Error over time (time series)
        if self.dates and self.peak_errors:
            axes[1, 1].plot(self.dates, self.peak_errors, 'b-', alpha=0.5, label='Peak Error')
            
            if self.expected_value_errors:
                axes[1, 1].plot(self.dates, self.expected_value_errors, 'g-', alpha=0.5, label='Expected Value Error')
            
            # Add a horizontal line at y=0 (perfect prediction)
            axes[1, 1].axhline(0, color='r', linestyle='--', label='Perfect Prediction')
            
            axes[1, 1].set_title('Prediction Errors Over Time')
            axes[1, 1].set_xlabel('Date')
            axes[1, 1].set_ylabel('Error (%)')
            axes[1, 1].legend()
            axes[1, 1].grid(True, alpha=0.3)
            
            # Format x-axis dates for better readability
            axes[1, 1].xaxis.set_major_formatter(DateFormatter('%Y-%m'))
            plt.setp(axes[1, 1].xaxis.get_majorticklabels(), rotation=45)
        else:
            axes[1, 1].text(0.5, 0.5, "Not enough data for error time series", 
                          horizontalalignment='center', verticalalignment='center')
        
        plt.tight_layout()
        
        return fig
    
    def plot_trading_frequency(self):
        """
        Plot trading frequency metrics.
        
        Returns:
        --------
        matplotlib.figure.Figure
            Matplotlib figure object
        """
        fig, axes = plt.subplots(2, 1, figsize=(12, 10))
        
        # 1. Monthly trading frequency
        if self.monthly_trade_counts:
            months = list(self.monthly_trade_counts.keys())
            counts = list(self.monthly_trade_counts.values())
            
            # Calculate monthly days (approximate)
            monthly_days = {}
            for date in self.dates:
                month_year = date.strftime('%Y-%m')
                if month_year in monthly_days:
                    monthly_days[month_year] += 1
                else:
                    monthly_days[month_year] = 1
            
            # Calculate monthly frequency (trades per day)
            monthly_frequency = []
            frequency_months = []
            
            for month in months:
                if month in monthly_days and monthly_days[month] > 0:
                    monthly_frequency.append(self.monthly_trade_counts[month] / monthly_days[month])
                    frequency_months.append(month)
            
            # Plot the monthly trade counts
            axes[0].bar(range(len(months)), counts, color='b', alpha=0.7)
            
            axes[0].set_title('Monthly Trade Count')
            axes[0].set_xlabel('Month')
            axes[0].set_ylabel('Number of Trades')
            
            # Set x-ticks to month labels (show subset if too many)
            if len(months) > 12:
                step = len(months) // 12
                axes[0].set_xticks(range(0, len(months), step))
                axes[0].set_xticklabels([months[i] for i in range(0, len(months), step)], rotation=45)
            else:
                axes[0].set_xticks(range(len(months)))
                axes[0].set_xticklabels(months, rotation=45)
            
            axes[0].grid(True, alpha=0.3)
            
            # Plot the monthly trading frequency
            if monthly_frequency:
                axes[1].bar(range(len(frequency_months)), monthly_frequency, color='g', alpha=0.7)
                axes[1].axhline(self.trading_frequency, color='r', linestyle='--', 
                              label=f'Overall Frequency: {self.trading_frequency:.2f}')
                
                axes[1].set_title('Monthly Trading Frequency (Trades per Day)')
                axes[1].set_xlabel('Month')
                axes[1].set_ylabel('Frequency')
                axes[1].legend()
                
                # Set x-ticks to month labels (show subset if too many)
                if len(frequency_months) > 12:
                    step = len(frequency_months) // 12
                    axes[1].set_xticks(range(0, len(frequency_months), step))
                    axes[1].set_xticklabels([frequency_months[i] for i in range(0, len(frequency_months), step)], rotation=45)
                else:
                    axes[1].set_xticks(range(len(frequency_months)))
                    axes[1].set_xticklabels(frequency_months, rotation=45)
                
                axes[1].grid(True, alpha=0.3)
                axes[1].set_ylim(0, min(1.0, max(monthly_frequency) * 1.2))
                
                # Add annotation with overall stats
                axes[1].text(0.02, 0.95, 
                           f"Total Trades: {self.trade_count}\n"
                           f"Total Days: {self.total_days}\n"
                           f"Avg. Frequency: {self.trading_frequency:.2f} trades/day",
                           transform=axes[1].transAxes,
                           bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.8),
                           fontsize=9,
                           verticalalignment='top')
            else:
                axes[1].text(0.5, 0.5, "Not enough data for monthly frequency calculation", 
                           horizontalalignment='center', verticalalignment='center')
        else:
            axes[0].text(0.5, 0.5, "No trading frequency data available", 
                       horizontalalignment='center', verticalalignment='center')
            axes[1].text(0.5, 0.5, "No trading frequency data available", 
                       horizontalalignment='center', verticalalignment='center')
        
        plt.tight_layout()
        
        return fig
    
    def plot_leverage_analysis(self):
        """
        Plot analysis of leverage strategy.
        
        Returns:
        --------
        matplotlib.figure.Figure
            Matplotlib figure object
        """
        fig, axes = plt.subplots(2, 2, figsize=(14, 12))
        
        # 1. Leverage factors histogram
        if self.leverage_factors:
            axes[0, 0].hist(self.leverage_factors, bins=np.arange(0.5, self.max_leverage + 1.5, 1), 
                          alpha=0.7, color='b', rwidth=0.8)
            
            axes[0, 0].set_title('Distribution of Applied Leverage Factors')
            axes[0, 0].set_xlabel('Leverage Factor')
            axes[0, 0].set_ylabel('Frequency')
            axes[0, 0].set_xticks(range(1, self.max_leverage + 1))
            axes[0, 0].grid(True, alpha=0.3)
            
            # Add stats annotation
            unique_leverages, leverage_counts = np.unique(self.leverage_factors, return_counts=True)
            leverage_stats = "\n".join([f"{lev}x: {count} days ({count/len(self.leverage_factors)*100:.1f}%)" 
                                      for lev, count in zip(unique_leverages, leverage_counts)])
            
            axes[0, 0].text(0.97, 0.97, leverage_stats,
                          transform=axes[0, 0].transAxes,
                          bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.8),
                          fontsize=9,
                          verticalalignment='top',
                          horizontalalignment='right')
        else:
            axes[0, 0].text(0.5, 0.5, "No leverage data available", 
                          horizontalalignment='center', verticalalignment='center')
        
        # 2. Leverage factors over time
        if self.dates and self.leverage_factors:
            axes[0, 1].plot(self.dates, self.leverage_factors, 'b-', linewidth=1, alpha=0.7)
            axes[0, 1].set_title('Leverage Factors Over Time')
            axes[0, 1].set_xlabel('Date')
            axes[0, 1].set_ylabel('Leverage Factor')
            axes[0, 1].grid(True, alpha=0.3)
            
            # Format x-axis dates for better readability
            axes[0, 1].xaxis.set_major_formatter(DateFormatter('%Y-%m'))
            plt.setp(axes[0, 1].xaxis.get_majorticklabels(), rotation=45)
            
            # Set y-axis ticks to integer values
            axes[0, 1].set_yticks(range(1, self.max_leverage + 1))
            axes[0, 1].set_ylim(0.5, self.max_leverage + 0.5)
        else:
            axes[0, 1].text(0.5, 0.5, "No leverage time series data available", 
                          horizontalalignment='center', verticalalignment='center')
        
        # 3. Returns by leverage factor (boxplot)
        if self.performance_data is not None and not self.performance_data.empty:
            leverage_groups = self.performance_data.groupby('leverage_factor')['leverage_return'].apply(list).to_dict()
            
            if leverage_groups:
                # Extract data for boxplot
                labels = []
                data = []
                
                for lev in sorted(leverage_groups.keys()):
                    if len(leverage_groups[lev]) > 0:
                        labels.append(f"{lev}x")
                        data.append(np.array(leverage_groups[lev]) * 100)  # Convert to percentage
                
                if data:
                    axes[1, 0].boxplot(data, labels=labels, patch_artist=True,
                                     boxprops=dict(facecolor='lightblue', color='blue'),
                                     flierprops=dict(marker='o', markerfacecolor='red', markersize=3))
                    
                    axes[1, 0].set_title('Return Distribution by Leverage Factor')
                    axes[1, 0].set_xlabel('Leverage Factor')
                    axes[1, 0].set_ylabel('Return (%)')
                    axes[1, 0].grid(True, alpha=0.3)
                    
                    # Add horizontal line at y=0
                    axes[1, 0].axhline(0, color='r', linestyle='--', alpha=0.5)
                    
                    # Add stats annotation
                    leverage_stats = ""
                    for lev in sorted(leverage_groups.keys()):
                        if len(leverage_groups[lev]) > 0:
                            returns = np.array(leverage_groups[lev]) * 100
                            leverage_stats += f"{lev}x: {np.mean(returns):.2f}% avg, {np.std(returns):.2f}% std\n"
                    
                    axes[1, 0].text(0.02, 0.02, leverage_stats,
                                  transform=axes[1, 0].transAxes,
                                  bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.8),
                                  fontsize=9,
                                  verticalalignment='bottom')
                else:
                    axes[1, 0].text(0.5, 0.5, "Insufficient leverage return data", 
                                  horizontalalignment='center', verticalalignment='center')
            else:
                axes[1, 0].text(0.5, 0.5, "No leverage return data available", 
                              horizontalalignment='center', verticalalignment='center')
        else:
            axes[1, 0].text(0.5, 0.5, "No leverage performance data available", 
                          horizontalalignment='center', verticalalignment='center')
        
        # 4. Cumulative returns comparison: Basic vs Leverage
        if self.dates:
            basic_values = np.array(list(self.portfolio_values.values()))
            leverage_values = np.array(list(self.leverage_values.values()))
            
            # Normalize to percentage return
            basic_returns = (basic_values / basic_values[0] - 1) * 100
            leverage_returns = (leverage_values / leverage_values[0] - 1) * 100
            
            # Calculate relative performance (leverage advantage)
            leverage_advantage = leverage_returns - basic_returns
            
            # Plot relative advantage
            axes[1, 1].plot(self.dates, leverage_advantage, 'g-', linewidth=1.5)
            axes[1, 1].axhline(0, color='r', linestyle='--', alpha=0.7)
            
            axes[1, 1].set_title('Leverage Strategy Advantage Over Basic Strategy')
            axes[1, 1].set_xlabel('Date')
            axes[1, 1].set_ylabel('Relative Performance (%)')
            axes[1, 1].grid(True, alpha=0.3)
            
            # Format x-axis dates for better readability
            axes[1, 1].xaxis.set_major_formatter(DateFormatter('%Y-%m'))
            plt.setp(axes[1, 1].xaxis.get_majorticklabels(), rotation=45)
            
            # Add annotation with overall advantage
            final_advantage = leverage_advantage[-1] if len(leverage_advantage) > 0 else 0
            axes[1, 1].text(0.02, 0.95, 
                          f"Final Advantage: {final_advantage:.2f}%\n"
                          f"Basic Return: {self.total_return*100:.2f}%\n"
                          f"Leverage Return: {self.leverage_return*100:.2f}%",
                          transform=axes[1, 1].transAxes,
                          bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.8),
                          fontsize=9,
                          verticalalignment='top')
        else:
            axes[1, 1].text(0.5, 0.5, "No performance data available", 
                          horizontalalignment='center', verticalalignment='center')
        
        plt.tight_layout()
        
        return fig
    
    def plot_confusion_matrix(self):
        """Plot confusion matrix for direction predictions."""
        cm = np.array([
            [self.true_negatives, self.false_positives],
            [self.false_negatives, self.true_positives]
        ])
        
        # Calculate derived metrics
        total = cm.sum()
        accuracy = (self.true_positives + self.true_negatives) / total if total > 0 else 0
        
        precision = self.true_positives / (self.true_positives + self.false_positives) if (self.true_positives + self.false_positives) > 0 else 0
        
        recall = self.true_positives / (self.true_positives + self.false_negatives) if (self.true_positives + self.false_negatives) > 0 else 0
        
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        # Create plot
        plt.figure(figsize=(8, 7))
        sns.heatmap(
            cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Predicted Down', 'Predicted Up'],
            yticklabels=['Actual Down', 'Actual Up']
        )
        
        plt.ylabel('Actual')
        plt.xlabel('Predicted')
        plt.title('Confusion Matrix')
        
        # Add metrics text box
        plt.figtext(
            0.02, 0.02,
            f"Accuracy: {accuracy:.2f}\n"
            f"Precision: {precision:.2f}\n"
            f"Recall: {recall:.2f}\n"
            f"F1 Score: {f1:.2f}",
            bbox=dict(boxstyle="round,pad=0.5", fc="white", alpha=0.8),
            fontsize=10
        )
        
        plt.tight_layout()
        
        return plt.gcf()
    
    def plot_calibration_curve(self, bins=10):
        """
        Plot calibration curve to assess probability predictions.
        
        Parameters:
        -----------
        bins: int
            Number of bins for grouping predictions
        """
        if len(self.predicted_probs) < bins * 5:
            print(f"Not enough data points for calibration curve with {bins} bins.")
            return None
        
        # Convert to numpy arrays
        pred_probs = np.array(self.predicted_probs)
        actual = np.array(self.actual_outcomes)
        
        # Create bins and digitize predictions
        bin_edges = np.linspace(0, 1, bins + 1)
        bin_indices = np.digitize(pred_probs, bin_edges) - 1
        bin_indices = np.clip(bin_indices, 0, bins - 1)
        
        # Calculate fraction of positives and mean predicted probability for each bin
        bin_sums = np.bincount(bin_indices, minlength=bins)
        bin_true = np.bincount(bin_indices, weights=actual, minlength=bins)
        bin_pred = np.bincount(bin_indices, weights=pred_probs, minlength=bins)
        
        # Avoid division by zero
        nonzero = bin_sums > 0
        bin_fractions = np.zeros(bins)
        bin_fractions[nonzero] = bin_true[nonzero] / bin_sums[nonzero]
        
        bin_mean_pred = np.zeros(bins)
        bin_mean_pred[nonzero] = bin_pred[nonzero] / bin_sums[nonzero]
        
        # Create plot
        plt.figure(figsize=(8, 8))
        
        # Plot perfectly calibrated line
        plt.plot([0, 1], [0, 1], 'k--', label='Perfectly calibrated')
        
        # Plot calibration curve
        plt.plot(bin_mean_pred, bin_fractions, 'o-', linewidth=2, 
                 label='Model calibration')
        
        # Add histogram of prediction distribution
        ax2 = plt.gca().twinx()
        ax2.hist(pred_probs, range=(0, 1), bins=bins, alpha=0.3, color='gray',
                 label='Prediction distribution')
        ax2.set_ylabel('Count')
        ax2.tick_params(axis='y', colors='gray')
        
        # Set plot properties
        plt.xlim([0, 1])
        plt.ylim([0, 1])
        plt.xlabel('Predicted probability of positive return')
        plt.ylabel('Fraction of positive returns')
        plt.title('Calibration Curve')
        plt.legend(loc='upper left')
        
        plt.tight_layout()
        
        return plt.gcf()
    
    def generate_comprehensive_report(self):
        """
        Generate comprehensive performance report with all metrics and visualizations.
        
        Returns:
        --------
        matplotlib.figure.Figure
            Matplotlib figure object with multiple subplots
        """
        # Create a very large figure with many subplots
        fig = plt.figure(figsize=(22, 28))
        
        # Set up the grid layout
        gs = fig.add_gridspec(12, 6)
        
        # Create subplots
        ax1 = fig.add_subplot(gs[0:2, 0:6])  # Cumulative returns - top full width
        ax2 = fig.add_subplot(gs[2:4, 0:3])  # Direction accuracy
        ax3 = fig.add_subplot(gs[2:4, 3:6])  # Confusion matrix
        ax4 = fig.add_subplot(gs[4:6, 0:2])  # Win rate
        ax5 = fig.add_subplot(gs[4:6, 2:4])  # Gain/Loss ratio
        ax6 = fig.add_subplot(gs[4:6, 4:6])  # Max consecutive wins/losses
        ax7 = fig.add_subplot(gs[6:8, 0:3])  # Prediction errors
        ax8 = fig.add_subplot(gs[6:8, 3:6])  # Prediction error histogram
        ax9 = fig.add_subplot(gs[8:10, 0:3])  # Trading frequency
        ax10 = fig.add_subplot(gs[8:10, 3:6])  # Leverage analysis
        ax11 = fig.add_subplot(gs[10:12, 0:3])  # Calibration curve
        ax12 = fig.add_subplot(gs[10:12, 3:6])  # Strategy comparison metrics table
        
        # 1. Cumulative returns
        dates = list(self.portfolio_values.keys())
        basic_values = np.array(list(self.portfolio_values.values()))
        leverage_values = np.array(list(self.leverage_values.values()))
        shorting_values = np.array(list(self.shorting_values.values()))
        buyhold_values = np.array(list(self.buyhold_values.values()))
        
        # Normalize to percentage return
        basic_returns = (basic_values / basic_values[0] - 1) * 100
        leverage_returns = (leverage_values / leverage_values[0] - 1) * 100
        shorting_returns = (shorting_values / shorting_values[0] - 1) * 100
        buyhold_returns = (buyhold_values / buyhold_values[0] - 1) * 100
        
        ax1.plot(dates, basic_returns, 'b-', linewidth=2, label='Basic Strategy')
        ax1.plot(dates, leverage_returns, 'g-', linewidth=2, label='Leverage Strategy')
        ax1.plot(dates, shorting_returns, 'r-', linewidth=2, label='Shorting Strategy')
        ax1.plot(dates, buyhold_returns, 'k--', linewidth=2, label='Buy and Hold')
        
        ax1.set_title('Cumulative Returns Comparison', fontsize=14)
        ax1.set_xlabel('Date')
        ax1.set_ylabel('Return (%)')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Format x-axis dates for better readability
        ax1.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
        plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)
        
        # 2. Direction accuracy
        # Get moving average data
        window_days = min(20, max(5, len(self.direction_correct) // 5))
        if len(self.direction_correct) >= window_days:
            ma_dates, moving_avg = self.get_moving_average_accuracy(window_days)
            
            ax2.plot(ma_dates, moving_avg * 100, 'b-', linewidth=2)
            ax2.axhline(50, color='r', linestyle='--', alpha=0.7, label='Random Guess (50%)')
            ax2.axhline(np.mean(self.direction_correct) * 100, color='g', linestyle='--', 
                      label=f'Overall Accuracy: {np.mean(self.direction_correct)*100:.2f}%')
            
            # Get monthly accuracy data
            month_labels, monthly_accuracy = self.get_monthly_accuracy()
            
            # Add small subplot for monthly accuracy
            if month_labels:
                # Create an inset axis for monthly accuracy
                inset_ax = ax2.inset_axes([0.6, 0.1, 0.35, 0.35])
                inset_ax.bar(range(len(month_labels[-6:])), [a * 100 for a in monthly_accuracy[-6:]], color='b', alpha=0.7)
                inset_ax.set_title('Last 6 Months', fontsize=8)
                inset_ax.set_ylim(0, 100)
                inset_ax.set_xticks(range(len(month_labels[-6:])))
                inset_ax.set_xticklabels([m[-2:] for m in month_labels[-6:]], fontsize=6)
            
            ax2.set_title(f'{window_days}-Day Moving Average of Prediction Accuracy', fontsize=12)
            ax2.set_xlabel('Date')
            ax2.set_ylabel('Accuracy (%)')
            ax2.legend(fontsize=8)
            ax2.grid(True, alpha=0.3)
            ax2.set_ylim(0, 100)
            
            # Format x-axis dates for better readability
            ax2.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
            plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45)
        else:
            ax2.text(0.5, 0.5, f"Not enough data for {window_days}-day moving average", 
                   horizontalalignment='center', verticalalignment='center')
        
        # 3. Confusion matrix
        cm = np.array([
            [self.true_negatives, self.false_positives],
            [self.false_negatives, self.true_positives]
        ])
        
        # Calculate derived metrics
        total = cm.sum()
        accuracy = (self.true_positives + self.true_negatives) / total if total > 0 else 0
        
        precision = self.true_positives / (self.true_positives + self.false_positives) if (self.true_positives + self.false_positives) > 0 else 0
        
        recall = self.true_positives / (self.true_positives + self.false_negatives) if (self.true_positives + self.false_negatives) > 0 else 0
        
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        sns.heatmap(
            cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Predicted Down', 'Predicted Up'],
            yticklabels=['Actual Down', 'Actual Up'],
            ax=ax3
        )
        
        ax3.set_ylabel('Actual')
        ax3.set_xlabel('Predicted')
        ax3.set_title('Confusion Matrix', fontsize=12)
        
        # Add metrics text box
        ax3.text(
            0.05, 0.05,
            f"Accuracy: {accuracy:.2f}\n"
            f"Precision: {precision:.2f}\n"
            f"Recall: {recall:.2f}\n"
            f"F1 Score: {f1:.2f}",
            transform=ax3.transAxes,
            bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.8),
            fontsize=9
        )
        
        # 4. Win rate
        month_labels, monthly_win_rate = self.get_monthly_win_rate()
        
        if month_labels:
            ax4.bar(range(len(month_labels)), monthly_win_rate * 100, color='b', alpha=0.7)
            ax4.axhline(50, color='r', linestyle='--', alpha=0.7, label='50% Win Rate')
            ax4.axhline(self.win_rate * 100, color='g', linestyle='--', 
                      label=f'Overall: {self.win_rate*100:.2f}%')
            
            ax4.set_title('Monthly Win Rate', fontsize=12)
            ax4.set_xlabel('Month')
            ax4.set_ylabel('Win Rate (%)')
            
            # Set x-ticks to month labels (show subset if too many)
            if len(month_labels) > 6:
                step = len(month_labels) // 6
                ax4.set_xticks(range(0, len(month_labels), step))
                ax4.set_xticklabels([month_labels[i][-7:] for i in range(0, len(month_labels), step)], rotation=45, fontsize=8)
            else:
                ax4.set_xticks(range(len(month_labels)))
                ax4.set_xticklabels([m[-7:] for m in month_labels], rotation=45, fontsize=8)
            
            ax4.legend(fontsize=8)
            ax4.grid(True, alpha=0.3)
            ax4.set_ylim(0, 100)
        else:
            ax4.text(0.5, 0.5, "Not enough data for win rate", 
                   horizontalalignment='center', verticalalignment='center')
        
        # 5. Gain/Loss ratio
        month_labels, monthly_gain_loss_ratio = self.get_monthly_gain_loss_ratio()
        
        if month_labels:
            ax5.bar(range(len(month_labels)), monthly_gain_loss_ratio, color='g', alpha=0.7)
            ax5.axhline(1.0, color='r', linestyle='--', alpha=0.7, label='1.0 (Breakeven)')
            ax5.axhline(self.gain_loss_ratio, color='g', linestyle='--', 
                      label=f'Overall: {self.gain_loss_ratio:.2f}')
            
            ax5.set_title('Monthly Gain/Loss Ratio', fontsize=12)
            ax5.set_xlabel('Month')
            ax5.set_ylabel('Ratio')
            
            # Set x-ticks to month labels (show subset if too many)
            if len(month_labels) > 6:
                step = len(month_labels) // 6
                ax5.set_xticks(range(0, len(month_labels), step))
                ax5.set_xticklabels([month_labels[i][-7:] for i in range(0, len(month_labels), step)], rotation=45, fontsize=8)
            else:
                ax5.set_xticks(range(len(month_labels)))
                ax5.set_xticklabels([m[-7:] for m in month_labels], rotation=45, fontsize=8)
            
            ax5.legend(fontsize=8)
            ax5.grid(True, alpha=0.3)
            
            # Set y-limit to a reasonable range
            max_ratio = max(monthly_gain_loss_ratio + [self.gain_loss_ratio, 1.0])
            ax5.set_ylim(0, min(max_ratio * 1.2, 5))
        else:
            ax5.text(0.5, 0.5, "Not enough data for gain/loss ratio", 
                   horizontalalignment='center', verticalalignment='center')
        
        # 6. Max consecutive wins/losses
        ax6.text(0.5, 0.9, 'Consecutive Trade Streaks', 
               horizontalalignment='center', fontsize=12, fontweight='bold')
        
        streak_text = (
            f"Winning Streaks:\n"
            f"  Max consecutive wins: {self.max_consecutive_wins}\n"
            f"  Start: {self.max_consecutive_wins_start_date.strftime('%Y-%m-%d') if self.max_consecutive_wins_start_date else 'N/A'}\n"
            f"  End: {self.max_consecutive_wins_end_date.strftime('%Y-%m-%d') if self.max_consecutive_wins_end_date else 'N/A'}\n\n"
            f"Losing Streaks:\n"
            f"  Max consecutive losses: {self.max_consecutive_losses}\n"
            f"  Start: {self.max_consecutive_losses_start_date.strftime('%Y-%m-%d') if self.max_consecutive_losses_start_date else 'N/A'}\n"
            f"  End: {self.max_consecutive_losses_end_date.strftime('%Y-%m-%d') if self.max_consecutive_losses_end_date else 'N/A'}\n\n"
            f"Current streak: {'Win' if self.consecutive_wins > 0 else 'Loss' if self.consecutive_losses > 0 else 'None'}\n"
            f"  Length: {max(self.consecutive_wins, self.consecutive_losses)}\n"
            f"  Start: {self.current_streak_start_date.strftime('%Y-%m-%d') if self.current_streak_start_date else 'N/A'}"
        )
        
        ax6.text(0.5, 0.5, streak_text, 
               horizontalalignment='center', verticalalignment='center',
               bbox=dict(boxstyle="round,pad=0.5", fc="white", alpha=0.9),
               fontsize=9)
        ax6.axis('off')
        
        # 7. Prediction errors over time
        if self.dates and self.peak_errors:
            ax7.plot(self.dates, self.peak_errors, 'b-', alpha=0.5, label='Peak Error')
            
            if self.expected_value_errors:
                ax7.plot(self.dates, self.expected_value_errors, 'g-', alpha=0.5, label='Expected Value Error')
            
            # Add a horizontal line at y=0 (perfect prediction)
            ax7.axhline(0, color='r', linestyle='--', label='Perfect Prediction')
            
            ax7.set_title('Prediction Errors Over Time', fontsize=12)
            ax7.set_xlabel('Date')
            ax7.set_ylabel('Error (%)')
            ax7.legend(fontsize=8)
            ax7.grid(True, alpha=0.3)
            
            # Format x-axis dates for better readability
            ax7.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
            plt.setp(ax7.xaxis.get_majorticklabels(), rotation=45)
            
            # Set reasonable y-limits
            mean_error = np.mean(self.peak_errors)
            std_error = np.std(self.peak_errors)
            ax7.set_ylim(mean_error - 3*std_error, mean_error + 3*std_error)
            
            # Add stats annotation
            ax7.text(0.02, 0.95, 
                   f"Mean Peak Error: {np.mean(self.peak_errors):.2f}%\n"
                   f"Mean EV Error: {np.mean(self.expected_value_errors):.2f}%\n"
                   f"RMSE: {np.sqrt(np.mean(np.array(self.expected_value_errors)**2)):.2f}%",
                   transform=ax7.transAxes,
                   bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.8),
                   fontsize=9,
                   verticalalignment='top')
        else:
            ax7.text(0.5, 0.5, "Not enough data for error time series", 
                   horizontalalignment='center', verticalalignment='center')
        
        # 8. Prediction error histogram
        if self.peak_errors:
            ax8.hist(self.peak_errors, bins=30, alpha=0.7, color='b', label='Peak Error')
            ax8.axvline(np.mean(self.peak_errors), color='r', linestyle='--',
                      label=f'Mean: {np.mean(self.peak_errors):.2f}%')
            ax8.axvline(0, color='g', linestyle='--', label='Perfect Prediction')
            
            ax8.set_title('Prediction Error Distribution', fontsize=12)
            ax8.set_xlabel('Error (%)')
            ax8.set_ylabel('Frequency')
            ax8.legend(fontsize=8)
            ax8.grid(True, alpha=0.3)
            
            # Add stats annotation
            peak_error_mean = np.mean(self.peak_errors)
            peak_error_std = np.std(self.peak_errors)
            peak_error_rmse = np.sqrt(np.mean(np.array(self.peak_errors)**2))
            
            ax8.text(0.02, 0.95, 
                   f"Mean: {peak_error_mean:.2f}%\n"
                   f"Std Dev: {peak_error_std:.2f}%\n"
                   f"RMSE: {peak_error_rmse:.2f}%\n"
                   f"Skewness: {stats.skew(self.peak_errors):.2f}\n"
                   f"Kurtosis: {stats.kurtosis(self.peak_errors):.2f}",
                   transform=ax8.transAxes,
                   bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.8),
                   fontsize=9,
                   verticalalignment='top')
        else:
            ax8.text(0.5, 0.5, "No peak error data available", 
                   horizontalalignment='center', verticalalignment='center')
        
        # 9. Trading frequency
        if self.monthly_trade_counts:
            months = list(self.monthly_trade_counts.keys())
            counts = list(self.monthly_trade_counts.values())
            
            # Calculate monthly days (approximate)
            monthly_days = {}
            for date in self.dates:
                month_year = date.strftime('%Y-%m')
                if month_year in monthly_days:
                    monthly_days[month_year] += 1
                else:
                    monthly_days[month_year] = 1
            
            # Calculate monthly frequency (trades per day)
            monthly_frequency = []
            frequency_months = []
            
            for month in months:
                if month in monthly_days and monthly_days[month] > 0:
                    monthly_frequency.append(self.monthly_trade_counts[month] / monthly_days[month])
                    frequency_months.append(month)
            
            # Plot monthly trading frequency
            if monthly_frequency:
                ax9.bar(range(len(frequency_months)), monthly_frequency, color='b', alpha=0.7)
                ax9.axhline(self.trading_frequency, color='r', linestyle='--', 
                          label=f'Overall: {self.trading_frequency:.2f}')
                
                ax9.set_title('Monthly Trading Frequency (Trades per Day)', fontsize=12)
                ax9.set_xlabel('Month')
                ax9.set_ylabel('Frequency')
                
                # Set x-ticks to month labels (show subset if too many)
                if len(frequency_months) > 6:
                    step = len(frequency_months) // 6
                    ax9.set_xticks(range(0, len(frequency_months), step))
                    ax9.set_xticklabels([frequency_months[i][-7:] for i in range(0, len(frequency_months), step)], rotation=45, fontsize=8)
                else:
                    ax9.set_xticks(range(len(frequency_months)))
                    ax9.set_xticklabels([m[-7:] for m in frequency_months], rotation=45, fontsize=8)
                
                ax9.legend(fontsize=8)
                ax9.grid(True, alpha=0.3)
                ax9.set_ylim(0, min(1.0, max(monthly_frequency) * 1.2))
                
                # Add annotation with overall stats
                ax9.text(0.02, 0.95, 
                       f"Total Trades: {self.trade_count}\n"
                       f"Total Days: {self.total_days}\n"
                       f"Avg. Frequency: {self.trading_frequency:.2f} trades/day",
                       transform=ax9.transAxes,
                       bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.8),
                       fontsize=9,
                       verticalalignment='top')
            else:
                ax9.text(0.5, 0.5, "Not enough data for monthly frequency calculation", 
                       horizontalalignment='center', verticalalignment='center')
        else:
            ax9.text(0.5, 0.5, "No trading frequency data available", 
                   horizontalalignment='center', verticalalignment='center')
        
        # 10. Leverage analysis
        if self.leverage_factors:
            ax10.hist(self.leverage_factors, bins=np.arange(0.5, self.max_leverage + 1.5, 1), 
                    alpha=0.7, color='g', rwidth=0.8)
            
            ax10.set_title('Distribution of Applied Leverage Factors', fontsize=12)
            ax10.set_xlabel('Leverage Factor')
            ax10.set_ylabel('Frequency')
            ax10.set_xticks(range(1, self.max_leverage + 1))
            ax10.grid(True, alpha=0.3)
            
            # Add stats annotation
            unique_leverages, leverage_counts = np.unique(self.leverage_factors, return_counts=True)
            leverage_stats = "\n".join([f"{lev}x: {count} days ({count/len(self.leverage_factors)*100:.1f}%)" 
                                      for lev, count in zip(unique_leverages, leverage_counts)])
            
            ax10.text(0.97, 0.97, leverage_stats,
                    transform=ax10.transAxes,
                    bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.8),
                    fontsize=9,
                    verticalalignment='top',
                    horizontalalignment='right')
            
            # Add inset axis with leverage vs return
            if self.performance_data is not None and not self.performance_data.empty:
                leverage_groups = self.performance_data.groupby('leverage_factor')['leverage_return'].apply(list).to_dict()
                
                if leverage_groups:
                    # Extract data for boxplot
                    labels = []
                    data = []
                    
                    for lev in sorted(leverage_groups.keys()):
                        if len(leverage_groups[lev]) > 5:  # Need enough data for meaningful boxplot
                            labels.append(f"{lev}x")
                            data.append(np.array(leverage_groups[lev]) * 100)  # Convert to percentage
                    
                    if data:
                        inset_ax = ax10.inset_axes([0.1, 0.1, 0.4, 0.3])
                        inset_ax.boxplot(data, labels=labels, vert=False, patch_artist=True,
                                      boxprops=dict(facecolor='lightgreen', color='green'),
                                      flierprops=dict(marker='o', markerfacecolor='red', markersize=3))
                        
                        inset_ax.set_title('Returns by Leverage', fontsize=8)
                        inset_ax.axvline(0, color='r', linestyle='--', alpha=0.5)
        else:
            ax10.text(0.5, 0.5, "No leverage data available", 
                    horizontalalignment='center', verticalalignment='center')
        
        # 11. Calibration curve
        if len(self.predicted_probs) > 30:
            # Convert to numpy arrays
            pred_probs = np.array(self.predicted_probs)
            actual = np.array(self.actual_outcomes)
            
            # Create bins and digitize predictions
            bins = min(10, len(self.predicted_probs) // 10)
            bin_edges = np.linspace(0, 1, bins + 1)
            bin_indices = np.digitize(pred_probs, bin_edges) - 1
            bin_indices = np.clip(bin_indices, 0, bins - 1)
            
            # Calculate fraction of positives and mean predicted probability for each bin
            bin_sums = np.bincount(bin_indices, minlength=bins)
            bin_true = np.bincount(bin_indices, weights=actual, minlength=bins)
            bin_pred = np.bincount(bin_indices, weights=pred_probs, minlength=bins)
            
            # Avoid division by zero
            nonzero = bin_sums > 0
            bin_fractions = np.zeros(bins)
            bin_fractions[nonzero] = bin_true[nonzero] / bin_sums[nonzero]
            
            bin_mean_pred = np.zeros(bins)
            bin_mean_pred[nonzero] = bin_pred[nonzero] / bin_sums[nonzero]
            
            # Plot calibration curve
            ax11.plot([0, 1], [0, 1], 'k--', label='Perfectly calibrated')
            ax11.plot(bin_mean_pred, bin_fractions, 'o-', linewidth=2, color='b',
                    label='Model calibration')
            
            # Add histogram of prediction distribution
            ax11_twin = ax11.twinx()
            ax11_twin.hist(pred_probs, range=(0, 1), bins=bins, alpha=0.3, color='gray')
            ax11_twin.set_ylabel('Count', color='gray')
            ax11_twin.tick_params(axis='y', colors='gray')
            
            # Set plot properties
            ax11.set_xlim([0, 1])
            ax11.set_ylim([0, 1])
            ax11.set_xlabel('Predicted probability')
            ax11.set_ylabel('Fraction of positives')
            ax11.set_title('Calibration Curve', fontsize=12)
            ax11.legend(fontsize=8)
            ax11.grid(True, alpha=0.3)
            
            # Calculate Brier score
            brier_score = np.mean((pred_probs - actual) ** 2)
            ax11.text(0.05, 0.95, f"Brier Score: {brier_score:.4f}",
                    transform=ax11.transAxes,
                    bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.8),
                    fontsize=9,
                    verticalalignment='top')
        else:
            ax11.text(0.5, 0.5, "Not enough data for calibration curve", 
                    horizontalalignment='center', verticalalignment='center')
        
        # 12. Strategy comparison metrics table
        metrics = self.get_metrics_summary()
        
        # Create a table of key metrics
        table_data = [
            ['', 'Basic', 'Leverage', 'Shorting', 'Buy & Hold'],
            ['Total Return (%)', f"{metrics['total_return']*100:.2f}", f"{metrics['leverage_return']*100:.2f}", 
             f"{metrics['shorting_return']*100:.2f}", f"{metrics['buyhold_return']*100:.2f}"],
            ['Max Drawdown (%)', f"{metrics['max_drawdown']*100:.2f}", f"{metrics['leverage_max_drawdown']*100:.2f}", 
             f"{metrics['shorting_max_drawdown']*100:.2f}", f"{metrics['buyhold_max_drawdown']*100:.2f}"],
            ['Sharpe Ratio', f"{metrics['sharpe_ratio']:.2f}", f"{metrics['leverage_sharpe_ratio']:.2f}", 
             f"{metrics['shorting_sharpe_ratio']:.2f}", f"{metrics['buyhold_sharpe_ratio']:.2f}"],
            ['Sortino Ratio', f"{metrics['sortino_ratio']:.2f}", f"{metrics['leverage_sortino_ratio']:.2f}", 
             f"{metrics['shorting_sortino_ratio']:.2f}", f"{metrics['buyhold_sortino_ratio']:.2f}"],
            ['Beta', f"{metrics['beta']:.2f}", f"{metrics['leverage_beta']:.2f}", 
             f"{metrics['shorting_beta']:.2f}", "1.00"]
        ]
        
        # Add annual return if available
        if metrics['annual_return'] is not None:
            annual_row = ['Annual Return (%)', f"{metrics['annual_return']*100:.2f}", f"{metrics['leverage_annual_return']*100:.2f}", 
                        f"{metrics['shorting_annual_return']*100:.2f}", f"{metrics['buyhold_annual_return']*100:.2f}"]
            table_data.insert(2, annual_row)
        
        # Create the table
        ax12.axis('off')
        table = ax12.table(
            cellText=table_data,
            cellLoc='center',
            loc='center',
            colWidths=[0.25, 0.15, 0.15, 0.15, 0.15]
        )
        
        # Style the table
        table.auto_set_font_size(False)
        table.set_fontsize(9)
        table.scale(1, 1.5)
        
        # Style header row
        for i in range(len(table_data[0])):
            table[(0, i)].set_facecolor('#BDD7EE')
            table[(0, i)].set_text_props(fontweight='bold')
        
        # Style first column
        for i in range(1, len(table_data)):
            table[(i, 0)].set_text_props(fontweight='bold')
            table[(i, 0)].set_facecolor('#F2F2F2')
        
        # Add title
        ax12.set_title('Strategy Performance Comparison', fontsize=12)
        
        # Add additional metrics text box
        ax12.text(0.5, 0.02,
                f"Directional Accuracy: {metrics['accuracy']*100:.2f}%\n"
                f"Win Rate: {metrics['win_rate']*100:.2f}%\n"
                f"Gain/Loss Ratio: {metrics['gain_loss_ratio']:.2f}\n"
                f"Trading Frequency: {metrics['trading_frequency']:.2f} trades/day",
                transform=ax12.transAxes,
                bbox=dict(boxstyle="round,pad=0.3", fc="#FFFFCC", alpha=0.8),
                fontsize=9,
                horizontalalignment='center')
        
        # Add main title
        plt.suptitle(f'Comprehensive Performance Analysis\n', fontsize=16, fontweight='bold')
        
        # Add date range subtitle
        start_date = self.dates[0] if self.dates else None
        end_date = self.dates[-1] if self.dates else None
        date_range = f"{start_date.strftime('%Y-%m-%d') if start_date else 'N/A'} to {end_date.strftime('%Y-%m-%d') if end_date else 'N/A'}"
        plt.figtext(0.5, 0.995, date_range, ha='center', fontsize=12)
        
        plt.tight_layout(rect=[0, 0, 1, 0.98])
        
        return fig
    
    def to_dataframe(self):
        """
        Export performance data to DataFrame for further analysis.
        
        Returns:
        --------
        pandas.DataFrame
            DataFrame containing all performance data
        """
        return self.performance_data.copy()
    
    def save_performance_data(self, filename):
        """
        Save performance data to CSV file.
        
        Parameters:
        -----------
        filename: str
            Path to save the CSV file
        """
        self.performance_data.to_csv(filename, index=False)
    
    def save_metrics_summary(self, filename):
        """
        Save metrics summary to JSON file.
        
        Parameters:
        -----------
        filename: str
            Path to save the JSON file
        """
        metrics = self.get_metrics_summary()
        
        # Convert datetime objects to strings
        for key in metrics:
            if isinstance(metrics[key], datetime):
                metrics[key] = metrics[key].strftime('%Y-%m-%d')
        
        with open(filename, 'w') as f:
            json.dump(metrics, f, indent=4)

## Tensorboard

In [10]:
# Setup TensorBoard logging with error handling
def setup_tensorboard(log_dir=None):
    """
    Set up TensorBoard logging with proper error handling.
    Parameters:
    -----------
    log_dir: str
        Directory to save TensorBoard logs. If None, a default
        directory will be created.
    Returns:
    --------
    tf.summary.SummaryWriter or None
        TensorBoard writer object or None if setup failed
    """
    try:
        if log_dir is None:
            log_dir = f"./tensorboard_logs/dbn_run_{datetime.now().strftime('%Y%m%d-%H%M%S')}"
        
        # Make sure the directory exists
        os.makedirs(log_dir, exist_ok=True)
        
        writer = tf.summary.create_file_writer(log_dir)
        print(f"TensorBoard logs will be saved to {log_dir}")
        print("To view logs, run: tensorboard --logdir=./tensorboard_logs")
        return writer
    except Exception as e:
        print(f"Error setting up TensorBoard: {e}")
        return None

# TensorBoard logging functions
def log_metrics_to_tensorboard(tb_writer, tracker, model, step, log_detailed=False):
    """Safely log metrics to TensorBoard with comprehensive error handling."""
    if tb_writer is None:
        return
        
    # Force metrics update if needed
    try:
        if not hasattr(tracker, '_metrics_updated') or not tracker._metrics_updated:
            tracker._update_metrics()
            tracker._metrics_updated = True
    except Exception as e:
        print(f"Warning: Could not force metrics update: {e}")
    
    try:
        with tb_writer.as_default():
            # Get metrics and enhanced metrics
            metrics = tracker.get_metrics_summary()
            enhanced_metrics = tracker.get_enhanced_metrics()
            
            # ========== 1. ACCURACY METRICS ==========
            # Original directional accuracy
            tf.summary.scalar('accuracy/direction', metrics.get('accuracy', 0), step=step)
            
            # Moving average windows for accuracy
            windows = [20, 50, 100, 200]
            for window in windows:
                if hasattr(tracker, 'direction_correct') and len(tracker.direction_correct) >= window:
                    ma_accuracy = np.mean(tracker.direction_correct[-window:])
                    tf.summary.scalar(f'accuracy/ma_{window}day', ma_accuracy, step=step)
                    
                    # Segment by prediction type (positive vs negative predictions)
                    if hasattr(tracker, 'predictions') and len(tracker.predictions) >= window:
                        # Get most recent predictions and outcomes
                        recent_predictions = list(tracker.predictions.values())[-window:]
                        recent_outcomes = tracker.direction_correct[-window:]
                        
                        # Positive predictions
                        pos_indices = [i for i, p in enumerate(recent_predictions) if p['direction'] == 1]
                        if pos_indices:
                            pos_correct = [recent_outcomes[i] for i in pos_indices]
                            pos_accuracy = np.mean(pos_correct) if pos_correct else 0
                            tf.summary.scalar(f'accuracy/ma_{window}day_pos', pos_accuracy, step=step)
                        
                        # Negative predictions
                        neg_indices = [i for i, p in enumerate(recent_predictions) if p['direction'] == -1]
                        if neg_indices:
                            neg_correct = [recent_outcomes[i] for i in neg_indices]
                            neg_accuracy = np.mean(neg_correct) if neg_correct else 0
                            tf.summary.scalar(f'accuracy/ma_{window}day_neg', neg_accuracy, step=step)
            
            # ========== 2. ERROR METRICS ==========
            # Original error metrics
            tf.summary.scalar('error/mean', metrics.get('mean_error', 0), step=step)
            
            # Peak error
            if hasattr(tracker, 'peak_errors') and tracker.peak_errors:
                mean_peak_error = np.mean([abs(e) for e in tracker.peak_errors])
                tf.summary.scalar('error/peak', mean_peak_error, step=step)
                
                # New: Median peak error
                median_peak_error = np.median([abs(e) for e in tracker.peak_errors])
                tf.summary.scalar('error/median_peak', median_peak_error, step=step)
            
            # Expected value error
            if hasattr(tracker, 'expected_value_errors') and tracker.expected_value_errors:
                mean_ev_error = np.mean([abs(e) for e in tracker.expected_value_errors])
                tf.summary.scalar('error/expected_value', mean_ev_error, step=step)
                
                # New: Median expected value error
                median_ev_error = np.median([abs(e) for e in tracker.expected_value_errors])
                tf.summary.scalar('error/median_ev', median_ev_error, step=step)
            
            # Moving average error windows
            for window in windows:
                # Peak error moving average
                if hasattr(tracker, 'peak_errors') and len(tracker.peak_errors) >= window:
                    ma_peak_error = np.mean([abs(e) for e in tracker.peak_errors[-window:]])
                    tf.summary.scalar(f'error/peak_ma_{window}day', ma_peak_error, step=step)
                
                # Expected value error moving average
                if hasattr(tracker, 'expected_value_errors') and len(tracker.expected_value_errors) >= window:
                    ma_ev_error = np.mean([abs(e) for e in tracker.expected_value_errors[-window:]])
                    tf.summary.scalar(f'error/ev_ma_{window}day', ma_ev_error, step=step)
            
            # ========== 3. VOLATILITY BUCKET METRICS ==========
            # Use existing volatility bucket data
            if hasattr(tracker, 'vol_buckets'):
                for bucket, data in tracker.vol_buckets.items():
                    if data['total'] > 0:
                        # Calculate metrics only if we have data
                        accuracy = np.mean(data['dir_correct']) if data['dir_correct'] else 0
                        peak_error = np.mean(data['peak_errors']) if data['peak_errors'] else 0
                        ev_error = np.mean(data['ev_errors']) if data['ev_errors'] else 0
                        
                        # Log metrics
                        tf.summary.scalar(f'vol_buckets/{bucket}/accuracy', accuracy * 100, step=step)
                        tf.summary.scalar(f'vol_buckets/{bucket}/peak_error', peak_error, step=step)
                        tf.summary.scalar(f'vol_buckets/{bucket}/ev_error', ev_error, step=step)
                        tf.summary.scalar(f'vol_buckets/{bucket}/count', data['total'], step=step)
            
            # ========== 4. PORTFOLIO VALUES & DRAWDOWNS ==========
            # Track portfolio values and drawdowns for all strategies
            strategy_attrs = {
                'tradinghours': 'portfolio_values',
                'afterhours': 'portfolio_values_afterhours',
                'nextday': 'nextday_values',
                'leverage': 'leverage_values',
                'shorting': 'shorting_values',
                'short_leverage': 'short_leverage_values',
                'buyhold': 'buyhold_values'
            }
            
            for strategy, attr in strategy_attrs.items():
                if hasattr(tracker, attr) and getattr(tracker, attr):
                    # Get portfolio values
                    values = list(getattr(tracker, attr).values())
                    
                    if values:
                        # Log current portfolio value
                        tf.summary.scalar(f'portfolio/{strategy}/value', values[-1], step=step)
                        
                        # Calculate and log drawdown
                        if len(values) > 1:
                            values_array = np.array(values)
                            running_max = np.maximum.accumulate(values_array)
                            current_drawdown = (running_max[-1] - values_array[-1]) / running_max[-1] * 100
                            # Negative sign for 0 to -100 scale
                            tf.summary.scalar(f'portfolio/{strategy}/drawdown', -current_drawdown, step=step)
            
            # ========== 5. STRATEGY PERFORMANCE METRICS ==========
            for strategy in strategy_attrs.keys():
                # Returns
                return_key = f'total_return_{strategy}'
                if return_key in metrics:
                    tf.summary.scalar(f'strategy/{strategy}/return', metrics[return_key] * 100, step=step)
                
                # Risk metrics
                sharpe_key = f'sharpe_ratio_{strategy}'
                if sharpe_key in metrics:
                    tf.summary.scalar(f'strategy/{strategy}/sharpe', metrics[sharpe_key], step=step)
                
                sortino_key = f'sortino_ratio_{strategy}'
                if sortino_key in metrics:
                    tf.summary.scalar(f'strategy/{strategy}/sortino', metrics[sortino_key], step=step)
                
                drawdown_key = f'max_drawdown_{strategy}'
                if drawdown_key in metrics:
                    tf.summary.scalar(f'strategy/{strategy}/max_drawdown', metrics[drawdown_key] * 100, step=step)
                
                # Beta (market sensitivity)
                beta_key = f'beta_{strategy}'
                beta_value = metrics.get(beta_key, 1.0 if strategy == 'buyhold' else 0)
                tf.summary.scalar(f'strategy/{strategy}/beta', beta_value, step=step)
                
                # Trading metrics (except for buy & hold)
                if strategy != 'buyhold':
                    win_rate_key = f'win_rate_{strategy}'
                    if win_rate_key in metrics:
                        tf.summary.scalar(f'strategy/{strategy}/win_rate', metrics[win_rate_key] * 100, step=step)
                    
                    gain_loss_key = f'gain_loss_ratio_{strategy}'
                    if gain_loss_key in metrics:
                        tf.summary.scalar(f'strategy/{strategy}/gain_loss_ratio', metrics[gain_loss_key], step=step)
                    
                    # ADD NEW METRICS HERE
                    # Profit factor
                    profit_factor_key = f'profit_factor_{strategy}'
                    if profit_factor_key in metrics:
                        tf.summary.scalar(f'strategy/{strategy}/profit_factor', metrics[profit_factor_key], step=step)
                    
                    # System Quality Number (SQN)
                    sqn_key = f'sqn_{strategy}'
                    if sqn_key in metrics:
                        tf.summary.scalar(f'strategy/{strategy}/sqn', metrics[sqn_key], step=step)
                    
                    # Market capture ratios
                    if strategy == 'tradinghours' or strategy == 'afterhours' or strategy == 'nextday':
                        up_capture_key = f'up_capture_{strategy}'
                        if up_capture_key in metrics:
                            tf.summary.scalar(f'strategy/{strategy}/up_capture', metrics[up_capture_key], step=step)
                        
                        down_capture_key = f'down_capture_{strategy}'
                        if down_capture_key in metrics:
                            tf.summary.scalar(f'strategy/{strategy}/down_capture', metrics[down_capture_key], step=step)
                        
                        capture_ratio_key = f'capture_ratio_{strategy}'
                        if capture_ratio_key in metrics:
                            tf.summary.scalar(f'strategy/{strategy}/capture_ratio', metrics[capture_ratio_key], step=step)
            
            # ========== 6. MODEL STATE INFO ==========
            debug_info = model.get_debug_info() if hasattr(model, 'get_debug_info') else {}
            tf.summary.scalar('model/learning_rate',
                             debug_info.get('current_learning_rate', 0), 
                             step=step)
            tf.summary.scalar('model/weight_norm',
                             debug_info.get('weight_norm', 0),
                             step=step)
            tf.summary.scalar('model/stagnation', 
                            1 if debug_info.get('stagnation_detected', False) else 0, 
                            step=step)
            
            # ========== 7. ENHANCED METRICS ==========
            # Actually use the enhanced metrics dictionary
            for key, value in enhanced_metrics.items():
                if isinstance(value, (int, float)):
                    tf.summary.scalar(f'enhanced/{key}', value, step=step)
            
            # ========== 8. DETAILED METRICS (when requested) ==========
            if log_detailed:
                # Add calmar ratio and other advanced metrics
                for strategy in strategy_attrs.keys():
                    calmar_key = f'calmar_ratio_{strategy}'
                    if calmar_key in metrics:
                        tf.summary.scalar(f'detailed/{strategy}/calmar_ratio', 
                                         metrics[calmar_key], step=step)
                
                # Underwater metrics (if available)
                for strategy in strategy_attrs.keys():
                    underwater_key = f'underwater_metrics_{strategy}'
                    if underwater_key in metrics and metrics[underwater_key]:
                        tf.summary.scalar(f'detailed/{strategy}/pct_time_underwater', 
                                         metrics[underwater_key].get('pct_time_underwater', 0) * 100, 
                                         step=step)
                        
                        tf.summary.scalar(f'detailed/{strategy}/max_drawdown_duration', 
                                         metrics[underwater_key].get('max_drawdown_duration', 0), 
                                         step=step)
            
            # Flush to ensure data is written
            tb_writer.flush()
            
    except Exception as e:
        print(f"Warning: TensorBoard logging error at step {step}: {e}")

def log_final_summary_to_tensorboard(tb_writer, tracker):
    """Log the final summary metrics to TensorBoard."""
    if tb_writer is None:
        return
    
    try:
        # Force metrics update
        try:
            tracker._update_metrics()
        except Exception as e:
            print(f"Warning: Could not force metrics update: {e}")
        
        with tb_writer.as_default():
            metrics = tracker.get_metrics_summary()
            enhanced_metrics = tracker.get_enhanced_metrics()
            
            # Log comprehensive final metrics for each strategy
            strategies = ['tradinghours', 'afterhours', 'nextday', 'buyhold']
            
            for strategy in strategies:
                # Return metrics
                tf.summary.scalar(f'final/{strategy}/total_return', 
                                 metrics.get(f'total_return_{strategy}', 0) * 100, step=0)
                
                # Risk metrics
                tf.summary.scalar(f'final/{strategy}/sharpe_ratio', 
                                 metrics.get(f'sharpe_ratio_{strategy}', 0), step=0)
                
                tf.summary.scalar(f'final/{strategy}/max_drawdown', 
                                 metrics.get(f'max_drawdown_{strategy}', 0) * 100, step=0)
                
                tf.summary.scalar(f'final/{strategy}/sortino_ratio', 
                                 metrics.get(f'sortino_ratio_{strategy}', 0), step=0)
                
                # Beta
                beta_key = f'beta_{strategy}'
                beta_value = metrics.get(beta_key, 1.0 if strategy == 'buyhold' else 0)
                tf.summary.scalar(f'final/{strategy}/beta', beta_value, step=0)
            
            # Accuracy and error metrics
            tf.summary.scalar('final/accuracy/overall', metrics.get('accuracy', 0), step=0)
            tf.summary.scalar('final/error/mean', metrics.get('mean_error', 0), step=0)
            tf.summary.scalar('final/error/median_peak', enhanced_metrics.get('median_abs_peak_error', 0), step=0)
            tf.summary.scalar('final/error/median_ev', enhanced_metrics.get('median_abs_ev_error', 0), step=0)
            
            # Volatility bucket final metrics
            if hasattr(tracker, 'vol_buckets'):
                for bucket, data in tracker.vol_buckets.items():
                    if data['total'] > 0:
                        accuracy = np.mean(data['dir_correct']) if data['dir_correct'] else 0
                        tf.summary.scalar(f'final/vol_buckets/{bucket}/accuracy', accuracy * 100, step=0)
                        tf.summary.scalar(f'final/vol_buckets/{bucket}/count', data['total'], step=0)
            
            # Ensure data is written
            tb_writer.flush()
    except Exception as e:
        print(f"Warning: Error logging final TensorBoard summary: {e}")

In [11]:
def print_enhanced_metrics_summary(tracker):
    """
    Print a summary of the enhanced metrics to the console.
    """
    if tracker is None:
        print("No tracker available for enhanced metrics")
        return
    
    enhanced_metrics = tracker.get_enhanced_metrics()
    
    print("\n=== ENHANCED METRICS SUMMARY ===")
    
    print("\n--- DIRECTIONAL ACCURACY ---")
    print(f"Overall Accuracy: {enhanced_metrics['direction_accuracy']*100:.2f}%")
    print(f"Accuracy for UP Predictions: {enhanced_metrics['pos_pred_direction_accuracy']*100:.2f}%")
    print(f"Accuracy for DOWN Predictions: {enhanced_metrics['neg_pred_direction_accuracy']*100:.2f}%")
    print(f"Recent 100-day Accuracy: {enhanced_metrics['recent_direction_accuracy']*100:.2f}%")
    
    print("\n--- ABSOLUTE ERRORS ---")
    print(f"Mean Absolute Peak Error: {enhanced_metrics['abs_peak_error']:.4f}%")
    print(f"Mean Absolute Expected Value Error: {enhanced_metrics['abs_ev_error']:.4f}%")
    print(f"Absolute Peak Error (UP Predictions): {enhanced_metrics['pos_pred_abs_peak_error']:.4f}%")
    print(f"Absolute Peak Error (DOWN Predictions): {enhanced_metrics['neg_pred_abs_peak_error']:.4f}%")
    
    print("\n--- VOLATILITY-BASED METRICS ---")
    print("Directional Accuracy by Volatility:")
    print(f"  <= 1σ: {enhanced_metrics['vol_1std_dir_accuracy']*100:.2f}% ({enhanced_metrics['vol_1std_sample_count']} samples)")
    print(f"  1-2σ: {enhanced_metrics['vol_2std_dir_accuracy']*100:.2f}% ({enhanced_metrics['vol_2std_sample_count']} samples)")
    print(f"  2-3σ: {enhanced_metrics['vol_3std_dir_accuracy']*100:.2f}% ({enhanced_metrics['vol_3std_sample_count']} samples)")
    print(f"  3-4σ: {enhanced_metrics['vol_4std_dir_accuracy']*100:.2f}% ({enhanced_metrics['vol_4std_sample_count']} samples)")
    
    print("\nAbsolute Peak Error by Volatility:")
    print(f"  <= 1σ: {enhanced_metrics['vol_1std_abs_peak_error']:.4f}%")
    print(f"  1-2σ: {enhanced_metrics['vol_2std_abs_peak_error']:.4f}%") 
    print(f"  2-3σ: {enhanced_metrics['vol_3std_abs_peak_error']:.4f}%")
    print(f"  3-4σ: {enhanced_metrics['vol_4std_abs_peak_error']:.4f}%")
    
    print("\n=================================")

# 5. Hyperparameter optimization

In [12]:
def optimize_hyperparameters(
    train_features,
    train_target,
    validation_features,
    validation_target,
    parameter_grid,
    optimization_metric='sharpe_ratio',
    n_trials=20,
    random_state=42
):
    """
    Optimize DBN hyperparameters.
    
    Parameters:
    -----------
    train_features: pandas.DataFrame
        Training features
    train_target: pandas.Series
        Training target
    validation_features: pandas.DataFrame
        Validation features
    validation_target: pandas.Series
        Validation target
    parameter_grid: dict
        Dictionary of parameter names and possible values
    optimization_metric: str
        Metric to optimize ('accuracy', 'mean_error', 'sharpe_ratio', 'total_return')
    n_trials: int
        Number of optimization trials
    random_state: int
        Random seed for reproducibility
        
    Returns:
    --------
    dict
        Best parameters found
    """
    
    # Import the StockMarketDBN and PerformanceTracker classes
    # This assumes these are defined elsewhere and imported in the scope where this function is called
    # from stock_market_dbn import StockMarketDBN
    # from performance_tracker import PerformanceTracker
    
    print("Starting hyperparameter optimization...")
    print(f"Optimization metric: {optimization_metric}")
    print(f"Number of trials: {n_trials}")
    
    # Initialize random number generator
    rng = np.random.RandomState(random_state)
    
    # Convert validation data to format expected by model
    validation_data = []
    for _, row in validation_features.iterrows():
        validation_data.append(row.to_dict())
    
    # Initialize best parameters and score
    best_score = -np.inf if optimization_metric in ['sharpe_ratio', 'total_return', 'accuracy'] else np.inf
    best_params = None
    
    # Run trials with progress bar
    for trial in tqdm(range(n_trials), desc="Optimization trials"):
        # Sample parameters
        params = {}
        for param_name, param_values in parameter_grid.items():
            params[param_name] = param_values[rng.randint(0, len(param_values))]
        
        print(f"\nTrial {trial+1}/{n_trials}:")
        print(f"Parameters: {params}")
        
        try:
            # Create and train model
            model = StockMarketDBN(
                features_list=train_features.columns.tolist(),
                hidden_layers=params.get('hidden_layers', 0),
                states_per_hidden=params.get('states_per_hidden', 3),
                master_node=params.get('master_node', False),
                inference_method=params.get('inference_method', 'particle'),
                prediction_range=params.get('prediction_range', (-10, 10)),
                prediction_bins=params.get('prediction_bins', 201),
                n_particles=params.get('n_particles', 1000),
                random_state=random_state,
                # Anti-stagnation parameters
                enable_anti_stagnation=params.get('enable_anti_stagnation', True),
                stagnation_window=params.get('stagnation_window', 30),
                stagnation_threshold=params.get('stagnation_threshold', 0.95),
                adaptive_learning=params.get('adaptive_learning', True),
                base_learning_rate=params.get('base_learning_rate', 0.01),
                max_learning_rate=params.get('max_learning_rate', 0.1),
                particle_rejuvenation=params.get('particle_rejuvenation', True),
                weight_regularization=params.get('weight_regularization', 0.0001)
            )
            
            # Train on training data
            model.learn_initial(train_features, train_target)
            
            # Evaluate on validation data
            tracker = PerformanceTracker(
                initial_capital=1000,
                start_tracking_date=datetime.now(),  # Date doesn't matter for this evaluation
                risk_free_rate=0.02/252,  # Approximate 2% annual risk-free rate
                leverage_threshold_std=params.get('leverage_threshold_std', 1.0),
                max_leverage=params.get('max_leverage', 3)
            )
            
            # Run sequential prediction and tracking
            for i, (features_dict, actual_return) in enumerate(zip(validation_data, validation_target)):
                # Generate prediction
                prediction = model.predict_next_day(features_dict)
                
                # Update tracker
                tracker.update(prediction, actual_return, datetime.now())
                
                # Update model
                model.update_with_actual(features_dict, actual_return)
            
            # Get metrics
            metrics = tracker.get_metrics_summary()
            
            # Extract score based on optimization metric
            if optimization_metric == 'accuracy':
                score = metrics['accuracy']
                is_better = score > best_score
            elif optimization_metric == 'mean_error':
                score = metrics['mean_error']
                is_better = score < best_score
            elif optimization_metric == 'sharpe_ratio':
                score = metrics['sharpe_ratio']
                is_better = score > best_score
            elif optimization_metric == 'total_return':
                score = metrics['total_return']
                is_better = score > best_score
            else:
                raise ValueError(f"Unknown optimization metric: {optimization_metric}")
            
            print(f"Score ({optimization_metric}): {score}")
            
            # Update best parameters if better
            if is_better:
                best_score = score
                best_params = params.copy()
                print(f"New best score: {best_score}")
                
        except Exception as e:
            print(f"Error in trial: {e}")
            continue
    
    print("\nOptimization completed.")
    print(f"Best parameters: {best_params}")
    print(f"Best score ({optimization_metric}): {best_score}")
    
    return best_params

# 6. Main Function

In [14]:
def run_dbn_stock_prediction(
    data_folder,
    data_config,
    target_file,
    target_column,
    target_transformation='pct_change',
    training_period_years=5,
    start_date=None,
    end_date=None,
    # Technical indicators parameters
    apply_tech_indicators=True,
    tech_indicators_config=None,
    # Feature engineering parameters
    apply_feature_eng=True,
    apply_pca=True,
    pca_components=50,
    
    # Quantum volatility parameters
    use_quantum_volatility=False,
    quantum_n_qubits=4,
    quantum_continuous_learning=True,
    quantum_learning_increment=50,
    quantum_learning_epochs=5,
    
    # DBN Model Parameters
    hidden_layers=0,
    states_per_hidden=3,
    continuous_states=False,
    state_dimension=2,
    master_node=False,
    inference_method='particle',
    prediction_range=(-30, 30),
    prediction_bins=1001,
    n_particles=10000,
    # Trading parameters
    initial_capital=1000,
    risk_free_rate=0.02/252,
    leverage_threshold_std=1.0,
    max_leverage=3,
    # Miscellaneous parameters
    save_model=False,
    load_model=None,
    random_state=42,
    output_folder='./output',
    use_tensorboard=True,
    tensorboard_log_dir=None
):
    """
    Run the complete DBN stock prediction workflow with TensorBoard integration.
    
    Parameters include original parameters plus:
    -----------
    apply_tech_indicators: bool
        Whether to calculate technical indicators
    tech_indicators_config: dict
        Configuration dictionary for technical indicators, specifying which indicators
        to calculate and for which files
    continuous_states : bool, default=False
        Whether to use continuous hidden states instead of discrete states.
        Continuous states can better capture complex market dynamics according to
        Fox et al. (2011) and Otranto (2010).
    
    state_dimension : int, default=2
        Dimension of continuous state vectors when continuous_states=True.
        Recommended values based on literature:
        - 2: Captures trend and volatility (Ghahramani & Hinton, 2000)
        - 3: Adds momentum/mean-reversion factor (Hamilton, 1989)
        - 4-5: Adds sentiment dynamics (Lux, 2011)
        Ignored when continuous_states=False.
    """
    import os
    import numpy as np
    import pandas as pd
    from datetime import datetime
    
    # Suppress warnings
    warnings.filterwarnings('ignore')
    
    print("Starting DBN Stock Prediction Workflow")
    print("======================================")

    # Validate continuous states parameters
    if continuous_states and state_dimension < 1:
        print("WARNING: continuous_states=True requires state_dimension >= 1. Setting state_dimension to 2.")
        state_dimension = 2
    
    if continuous_states:
        print(f"Using continuous hidden states with dimension {state_dimension}")
    else:
        print(f"Using discrete hidden states with {states_per_hidden} states per hidden layer")
    
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Initialize TensorBoard EARLY (before data preprocessing)
    tb_writer = None
    if use_tensorboard:
        # Don't use first_prediction_date here, just use a generic log directory
        if tensorboard_log_dir is None:
            tensorboard_log_dir = f"./tensorboard_logs/dbn_run_{datetime.now().strftime('%Y%m%d-%H%M%S')}"
        
        tb_writer = setup_tensorboard(tensorboard_log_dir)
        if tb_writer is None:
            print("TensorBoard setup failed. Continuing without TensorBoard logging.")
            use_tensorboard = False
    
    # 1. Data Preprocessing
    print("\n1. Data Preprocessing")
    print("--------------------")
    
    # Clean and normalize data folder path
    norm_data_folder = os.path.normpath(data_folder)

    # Use custom DataPreprocessor class if provided, otherwise use the standard one
    
    if use_quantum_volatility:
        preprocessor = VolatilityEnhancedDataPreprocessor(
            norm_data_folder,
            use_quantum_volatility=use_quantum_volatility,
            quantum_n_qubits=quantum_n_qubits,
            random_state=random_state
        )
    else:
        preprocessor = DataPreprocessor(norm_data_folder)
    
        
    print(f"Available CSV files: {preprocessor.available_files}")
    
    # Set up data configuration
    preprocessor.set_config(data_config)
    preprocessor.set_target(target_file, target_column, target_transformation)
    
    # Set start date if provided
    if start_date is not None:
        preprocessor.set_start_date(start_date)

    # Set end date if provided
    if end_date is not None:
        preprocessor.set_end_date(end_date)

    try:
        # Process the data - with enhanced feature engineering and PCA
        train_features, train_target, predict_features, predict_target, feature_names = preprocessor.process_data(
            training_period_years=training_period_years,
            apply_tech_indicators=apply_tech_indicators,
            tech_indicators_config=tech_indicators_config,
            apply_feature_eng=apply_feature_eng,
            apply_pca=apply_pca,
            pca_components=pca_components
        )
        
        print(f"Training data: {len(train_features)} samples")
        print(f"Prediction data: {len(predict_features)} samples")
        print(f"Number of features: {len(feature_names)}")
        
        # Show first few features
        if feature_names:
            print(f"First few features: {feature_names[:min(5, len(feature_names))]}...")
        
        # Get daily prediction data
        if use_quantum_volatility:
            prediction_data = preprocessor.get_daily_prediction_data(tb_writer=tb_writer)
        else:
            prediction_data = preprocessor.get_daily_prediction_data()
        print(f"Daily prediction data: {len(prediction_data)} days")
        
        if use_quantum_volatility:
            try:
                print("\n========== APPLYING QUANTUM VOLATILITY DETECTION ==========")
                
                # Modified integration function that returns the quantum detector as well
                train_features, predict_features, feature_names, prediction_data, quantum_volatility = \
                    integrate_quantum_volatility_properly(
                        preprocessor, train_features, train_target, 
                        predict_features, prediction_data,
                        quantum_n_qubits=quantum_n_qubits,
                        random_state=random_state,
                        tb_writer=tb_writer,
                        enable_continuous_learning=quantum_continuous_learning,
                        continuous_learning_increment=quantum_learning_increment,
                        continuous_learning_epochs=quantum_learning_epochs
                    )
                
                print(f"\nSuccessfully integrated quantum volatility features")
                print(f"Total features after quantum integration: {len(feature_names)}")

                # Extract classical and quantum features from the returned data
                classical_feature_names = [f for f in feature_names if 'quantum' not in f]
                quantum_feature_names = [f for f in feature_names if 'quantum' in f]
                
                # Extract the actual feature arrays from train_features DataFrame
                classical_features = train_features[classical_feature_names].values
                quantum_features = train_features[quantum_feature_names].values
                
                # Create initial visualization of the analysis
                fig_initial = create_classical_quantum_comparison_plots(
                    classical_features, quantum_features, classical_feature_names, 
                    quantum_feature_names, train_target, output_folder
                )
                # Save with different filename to distinguish from final
                fig_initial.savefig(os.path.join(output_folder, 'classical_quantum_comparison_initial.png'), 
                                    dpi=150, bbox_inches='tight')
                plt.close(fig_initial)
                print(f"\nInitial Classical-Quantum comparison saved to {output_folder}/classical_quantum_comparison_initial.png")

                print("\n========== QUANTUM FEATURE DIAGNOSTICS ==========")
        
                # Check quantum feature variance and statistics
                quantum_cols = [col for col in train_features.columns if 'quantum' in col]
                print(f"Found {len(quantum_cols)} quantum features")
                
                print("\nQuantum Feature Statistics:")
                for col in quantum_cols:
                    values = train_features[col].dropna()  # Remove NaN values
                    if len(values) > 0:
                        variance = values.var()
                        mean_val = values.mean()
                        std_val = values.std()
                        min_val = values.min()
                        max_val = values.max()
                        print(f"{col}:")
                        print(f"  Mean: {mean_val:.6f}, Std: {std_val:.6f}")
                        print(f"  Variance: {variance:.6f}")
                        print(f"  Range: [{min_val:.6f}, {max_val:.6f}]")
                    else:
                        print(f"{col}: ALL NaN VALUES!")
                
                # Check correlation with target
                print("\nCorrelation with Target:")
                for col in quantum_cols:
                    # Align the data - both train_features and train_target should have same length
                    aligned_features = train_features[col].dropna()
                    aligned_target = train_target.iloc[:len(aligned_features)]
                    
                    if len(aligned_features) > 1 and len(aligned_target) > 1:
                        correlation = np.corrcoef(aligned_features, aligned_target)[0, 1]
                        print(f"{col}: {correlation:.6f}")
                    else:
                        print(f"{col}: Cannot calculate correlation (insufficient data)")
                
                # Check if quantum features are essentially constant
                print("\nQuantum Feature Variation Check:")
                for col in quantum_cols:
                    values = train_features[col].dropna()
                    if len(values) > 0:
                        unique_vals = len(np.unique(np.round(values, 4)))
                        total_vals = len(values)
                        variation_ratio = unique_vals / total_vals
                        print(f"{col}: {unique_vals}/{total_vals} unique values ({variation_ratio:.3f} variation ratio)")
                        
                        if variation_ratio < 0.1:
                            print(f"   WARNING: {col} has very low variation - likely constant!")
                        elif variation_ratio > 0.8:
                            print(f"   GOOD: {col} shows high variation")
                        else:
                            print(f"   MODERATE: {col} shows moderate variation")
                
                print("=" * 60)

                # COMPARATIVE ANALYSIS:
                print("\n========== CLASSICAL VS QUANTUM FEATURE ANALYSIS ==========")
                
                # Extract classical and quantum features separately
                classical_feature_names = [f for f in feature_names if 'quantum' not in f]
                quantum_feature_names = [f for f in feature_names if 'quantum' in f]
                
                classical_features = train_features[classical_feature_names].values
                quantum_features = train_features[quantum_feature_names].values
                
                print(f"Classical features (PCA components): {len(classical_feature_names)}")
                print(f"Quantum features: {len(quantum_feature_names)}")
                
                # Perform analysis
                from sklearn.decomposition import PCA
                from sklearn.feature_selection import mutual_info_regression
                from scipy.stats import spearmanr
                import pandas as pd
                
                # 1. Variance Analysis
                print("\n1. VARIANCE DECOMPOSITION:")
                print("Classical features are already PCA components (explain maximum variance by design)")
                
                # Only analyze quantum features
                pca_quantum = PCA()
                pca_quantum.fit(quantum_features)
                
                print(f"Quantum features - variance explained by first 5 PCs: {np.sum(pca_quantum.explained_variance_ratio_[:5]):.3f}")
                if np.sum(pca_quantum.explained_variance_ratio_[:5]) > 0.9:
                    print(" Quantum features are highly correlated with each other")
                else:
                    print(" Quantum features capture diverse information")
                
                # Analyze combined features
                all_features = train_features.values
                pca_all = PCA()
                pca_all.fit(all_features)
                
                print(f"Total variance explained by first 5 PCs (all features): {np.sum(pca_all.explained_variance_ratio_[:5]):.3f}")
                print(f"Total variance explained by first 5 PCs (quantum only): {np.sum(pca_quantum.explained_variance_ratio_[:5]):.3f}")
                
                # Note about classical
                print("Note: Classical features are already 20 PCA components, so PCA on them would show decreasing importance by construction")
                
                # Check how many components needed to explain 95% variance
                n_components_95 = np.argmax(np.cumsum(pca_all.explained_variance_ratio_) > 0.95) + 1
                print(f"\nComponents needed for 95% variance (all features): {n_components_95}")
                if n_components_95 > 20:
                    print(" Quantum features add variance not captured by classical PCA")
                
                # 2. Information Content Analysis
                print("\n2. MUTUAL INFORMATION WITH TARGET:")
                mi_classical = mutual_info_regression(classical_features, train_target.values)
                mi_quantum = mutual_info_regression(quantum_features, train_target.values)
                
                print(f"Average MI (classical): {np.mean(mi_classical):.6f}")
                print(f"Average MI (quantum): {np.mean(mi_quantum):.6f}")
                print(f"Max MI classical feature: {classical_feature_names[np.argmax(mi_classical)]} = {np.max(mi_classical):.6f}")
                print(f"Max MI quantum feature: {quantum_feature_names[np.argmax(mi_quantum)]} = {np.max(mi_quantum):.6f}")
                
                # 3. Redundancy Analysis
                print("\n3. FEATURE REDUNDANCY ANALYSIS:")
                print("(Checking if quantum features can be predicted from classical PCA components)")

                # Check if quantum features are just linear combinations of classical
                from sklearn.linear_model import LinearRegression
                redundancy_scores = []
                
                for i, q_feat in enumerate(quantum_feature_names):
                    quantum_col = train_features[q_feat].values.reshape(-1, 1)
                    
                    # Try to predict quantum feature from classical features
                    reg = LinearRegression()
                    reg.fit(classical_features, quantum_col.ravel())
                    r2_score = reg.score(classical_features, quantum_col.ravel())
                    redundancy_scores.append(r2_score)
                    
                    if r2_score > 0.9:
                        print(f" {q_feat} is highly redundant (R²={r2_score:.3f})")
                    elif r2_score < 0.3:
                        print(f" {q_feat} contains unique information (R²={r2_score:.3f})")
                
                avg_redundancy = np.mean(redundancy_scores)
                print(f"\nAverage redundancy (R²) of quantum features: {avg_redundancy:.3f}")
                print("\nInterpretation:")
                if avg_redundancy < 0.5:
                    print(" Quantum features appear to contain substantial unique information")
                    print("   This suggests non-linear transformations add value")
                else:
                    print(" Quantum features may be largely redundant with classical features")
                    print("   The quantum circuit may not be adding much information")
                
                # 4. Non-linear Pattern Detection
                print("\n4. NON-LINEAR CORRELATION ANALYSIS:")
                linear_corrs_classical = []
                nonlinear_corrs_classical = []
                linear_corrs_quantum = []
                nonlinear_corrs_quantum = []
                
                # Classical features
                for feat in classical_features.T:
                    linear_corr = np.corrcoef(feat, train_target.values)[0, 1]
                    spearman_corr, _ = spearmanr(feat, train_target.values)
                    linear_corrs_classical.append(abs(linear_corr))
                    nonlinear_corrs_classical.append(abs(spearman_corr))
                
                # Quantum features  
                for feat in quantum_features.T:
                    linear_corr = np.corrcoef(feat, train_target.values)[0, 1]
                    spearman_corr, _ = spearmanr(feat, train_target.values)
                    linear_corrs_quantum.append(abs(linear_corr))
                    nonlinear_corrs_quantum.append(abs(spearman_corr))
                
                print(f"Classical - Avg linear correlation: {np.mean(linear_corrs_classical):.4f}")
                print(f"Classical - Avg non-linear correlation: {np.mean(nonlinear_corrs_classical):.4f}")
                print(f"Quantum - Avg linear correlation: {np.mean(linear_corrs_quantum):.4f}")
                print(f"Quantum - Avg non-linear correlation: {np.mean(nonlinear_corrs_quantum):.4f}")
                
                nonlinearity_gain_quantum = np.mean(nonlinear_corrs_quantum) - np.mean(linear_corrs_quantum)
                nonlinearity_gain_classical = np.mean(nonlinear_corrs_classical) - np.mean(linear_corrs_classical)
                
                print(f"\nNon-linearity gain (Spearman - Pearson):")
                print(f"Classical features: {nonlinearity_gain_classical:.4f}")
                print(f"Quantum features: {nonlinearity_gain_quantum:.4f}")
                
                if nonlinearity_gain_quantum > nonlinearity_gain_classical * 1.5:
                    print(" Quantum features capture more non-linear patterns")
                
                # 5. Temporal Structure Analysis (for time series)
                print("\n5. TEMPORAL STRUCTURE ANALYSIS:")
                # Check if quantum features capture different time scales
                from statsmodels.tsa.stattools import acf
                
                print("Autocorrelation at different lags:")
                for lag in [1, 5, 10]:
                    classical_acfs = [acf(feat, nlags=lag)[-1] for feat in classical_features.T if not np.any(np.isnan(feat))]
                    quantum_acfs = [acf(feat, nlags=lag)[-1] for feat in quantum_features.T if not np.any(np.isnan(feat))]
                    
                    if classical_acfs and quantum_acfs:
                        print(f"Lag {lag} - Classical avg: {np.mean(np.abs(classical_acfs)):.3f}, "
                              f"Quantum avg: {np.mean(np.abs(quantum_acfs)):.3f}")
                
                print("\n" + "="*60)

                # Run diagnostics
                if quantum_volatility is not None:
                    returns_column = preprocessor.get_target_column()
                    diagnostic_results = integrate_diagnostics_into_workflow(
                        quantum_volatility, 
                        returns_column.iloc[:len(train_target)]
                    )
                
            except Exception as e:
                print(f"ERROR in quantum volatility detection: {e}")
                import traceback
                traceback.print_exc()
                print("WARNING: Proceeding without quantum volatility features.")
        """

        if use_quantum_volatility:
            try:
                print("\n========== APPLYING ADVANCED QUANTUM VOLATILITY DETECTION ==========")
                
                train_features, predict_features, feature_names, prediction_data, quantum_volatility = \
                   QuantumIntegrationManager.safe_quantum_integration(
                       preprocessor, train_features, train_target,
                       predict_features, prediction_data,
                       quantum_n_qubits=quantum_n_qubits,
                       random_state=random_state
                   )
                
                print(f"\n Successfully integrated advanced quantum volatility features")
                print(f" Total features after quantum integration: {len(feature_names)}")
        
                validation_results = QuantumIntegrationManager.validate_integration_success(
                    train_features, predict_features, feature_names
                )
                if not validation_results['integration_successful']:
                    print("Quantum integration failed, continuing with classical features")
                    
            except Exception as e:
                print(f" ERROR in advanced quantum volatility detection: {e}")
                import traceback
                traceback.print_exc()
                print(" WARNING: Proceeding without quantum volatility features.")
        """
        
     
        # Generate data overview
        try:
            fig = preprocessor.plot_data_overview()
            plt.savefig(os.path.join(output_folder, 'data_overview.png'))
            plt.close(fig)
            print(f"Data overview saved to {os.path.join(output_folder, 'data_overview.png')}")
        except Exception as plot_error:
            print(f"Warning: Could not create data overview plot: {plot_error}")
    
    except Exception as e:
        print(f"Error in data preprocessing: {e}")
        return None, None, preprocessor
    
    # 2. Create or Load Model
    print("\n2. Model Initialization")
    print("---------------------")
    model = None
    
    if load_model is not None:
        print(f"Loading model from {load_model}")
        try:
            with open(load_model, 'rb') as f:
                model = pickle.load(f)
            print("Model loaded successfully.")
        except Exception as e:
            print(f"Error loading model: {e}")
            print("Training new model instead.")
            load_model = None
    
    if load_model is None:
        print("Creating new DBN model")
        try:
            # Always use standard DBN initialization
            model = StockMarketDBN(
                features_list=feature_names,
                hidden_layers=hidden_layers,
                states_per_hidden=states_per_hidden,
                continuous_states=continuous_states,  
                state_dimension=state_dimension,
                master_node=master_node,
                inference_method=inference_method,
                prediction_range=prediction_range,
                prediction_bins=prediction_bins,
                n_particles=n_particles,
                random_state=random_state,
                # Anti-stagnation parameters
                enable_anti_stagnation=True,
                stagnation_window=30,
                stagnation_threshold=0.95,
                adaptive_learning=True,
                base_learning_rate=0.03,
                max_learning_rate=0.3,
                particle_rejuvenation=True,
                weight_regularization=0.0001
            )
        except Exception as model_error:
            print(f"Error creating model: {model_error}")
            return None, None, preprocessor
    
    # 3. Initial Learning Phase
    print("\n3. Initial Learning Phase")
    print("-----------------------")
    try:
        if load_model is None and model is not None:
            model.learn_initial(train_features, train_target)
            if len(prediction_data) > 0:
                first_pred_date = list(prediction_data.keys())[0]
                first_pred_data = prediction_data[first_pred_date]
                print(f"\nTemporal Alignment Verification:")
                print(f"  Training: features[t-1] → target[t] ✓")
                print(f"  Prediction: features[t-1] → target[t] ✓")
                print(f"  First prediction date: {first_pred_date}")
                print(f"  Using features from: previous day")
    except Exception as e:
        print(f"Error in initial learning: {e}")
        return model, None, preprocessor
    
    # 4. Sequential Prediction and Learning
    print("\n4. Sequential Prediction and Learning")
    print("----------------------------------")
    
    # Initialize performance tracker
    first_prediction_date = list(prediction_data.keys())[0] if prediction_data else None
    tracker = None
    
    try:
        tracker = PerformanceTracker(
            initial_capital=initial_capital,
            start_tracking_date=first_prediction_date,
            risk_free_rate=risk_free_rate,
            leverage_threshold_std=leverage_threshold_std,
            max_leverage=max_leverage
        )
        
        # Add an attribute to track whether metrics have been updated
        tracker._metrics_updated = False
    except Exception as tracker_error:
        print(f"Error initializing performance tracker: {tracker_error}")
        return model, None, preprocessor

    # Initialize feature relationship tracker for continuous monitoring
    feature_tracker = None
    if use_quantum_volatility:
        feature_tracker = FeatureRelationshipTracker(update_interval=50)

    """
    # Set up TensorBoard AFTER tracker is initialized
    tb_writer = None
    if use_tensorboard:
        tb_writer = setup_tensorboard(tensorboard_log_dir)
        if tb_writer is None:
            print("TensorBoard setup failed. Continuing without TensorBoard logging.")
            use_tensorboard = False
    """
    
    if prediction_data and model is not None:
        # Iterate through prediction days
        print(f"Starting predictions from {first_prediction_date}")
        try:
            # Make a test prediction to verify model is working
            sample_features = next(iter(prediction_data.values()))['features']
            test_prediction = model.predict_next_day(sample_features)
            print("Test prediction successful, proceeding with all data")
            
            # Create figure for saving example prediction
            try:
                fig = model.plot_prediction_distribution(test_prediction)
                plt.savefig(os.path.join(output_folder, 'example_prediction.png'))
                plt.close(fig)
                print(f"Example prediction distribution saved to {os.path.join(output_folder, 'example_prediction.png')}")
            except Exception as plot_error:
                print(f"Warning: Could not create example prediction plot: {plot_error}")
            
            # Process all prediction data with progress bar
            for i, (date, data) in enumerate(tqdm(prediction_data.items(), desc="Processing days")):
                try:
                    # Predict
                    prediction = model.predict_next_day(data['features'])
                    
                    # Track performance
                    tracker.update(prediction, data['actual_return'], date)
                    
                    # Mark that metrics have been updated
                    tracker._metrics_updated = True
                    
                    # Update model
                    model.update_with_actual(data['features'], data['actual_return'])

                    # Update quantum detector if continuous learning is enabled
                    if use_quantum_volatility and hasattr(preprocessor, 'quantum_volatility'):
                        quantum_detector = preprocessor.quantum_volatility
                        
                        if quantum_detector.enable_continuous_learning:
                            quantum_detector._updates_since_last_training += 1
                            
                            # Get OHLC data for current date
                            ohlc_df = preprocessor.get_raw_ohlc_data()
                            current_idx = ohlc_df.index.get_loc(date)  # Index for day t
                            
                            # Prepare training data
                            if current_idx >= quantum_detector.lookback_window:
                                # Get the window that was used to predict day t
                                # This window is [t-4, t-3, t-2, t-1]
                                window_start = current_idx - quantum_detector.lookback_window
                                window_end = current_idx  # Exclusive end, so iloc gives [t-4, t-3, t-2, t-1]
                                ohlc_window = ohlc_df.iloc[window_start:window_end][['Open', 'High', 'Low', 'Close']].values
                                
                                # Get actual G-K for day t (the day we just predicted)
                                current_ohlc = ohlc_df.loc[date]  # This IS day t data
                                actual_gk = quantum_detector.calculate_signed_garman_klass(
                                    current_ohlc['Open'],   # Day t OHLC
                                    current_ohlc['High'],   
                                    current_ohlc['Low'],
                                    current_ohlc['Close'],
                                    prev_close=ohlc_df.iloc[current_idx-1]['Close'] if current_idx > 0 else None
                                )
                                
                                # Store data for batch update or update immediately
                                if quantum_detector.continuous_learning_increment == 1:
                                    # Update immediately
                                    quantum_detector.update(ohlc_window, actual_gk)
                                    
                                    # NEW: For immediate updates, recompute next 10 days only (for efficiency)
                                    all_dates = list(prediction_data.keys())
                                    next_dates = all_dates[i+1:min(i+11, len(all_dates))]
                                    for future_date in next_dates:
                                        quantum_features = preprocessor.compute_quantum_features_for_date(
                                            future_date, preprocessor.get_raw_ohlc_data()
                                        )
                                        prediction_data[future_date]['features'].update(quantum_features)
                                else:
                                    # Accumulate for batch update
                                    quantum_detector._accumulated_training_data.append((ohlc_window, actual_gk))
                                    
                                    # Check if it's time for batch update
                                    if quantum_detector._updates_since_last_training >= quantum_detector.continuous_learning_increment:
                                        print(f"\nQuantum circuit batch update: {len(quantum_detector._accumulated_training_data)} samples")
                                        
                                        # Prepare batch data
                                        batch_windows = np.array([d[0] for d in quantum_detector._accumulated_training_data])
                                        batch_targets = np.array([d[1] for d in quantum_detector._accumulated_training_data])
                                        
                                        # Run multiple epochs on accumulated data
                                        for epoch in range(quantum_detector.continuous_learning_epochs):
                                            # Shuffle data
                                            indices = np.random.permutation(len(batch_windows))
                                            
                                            for idx in indices:
                                                quantum_detector.update(batch_windows[idx], batch_targets[idx])
                                            
                                        #print(f"  Epoch {quantum_detector.continuous_learning_epochs}/{quantum_detector.continuous_learning_epochs} complete")
                                        
                                        # Reset accumulator
                                        quantum_detector._accumulated_training_data = []
                                        quantum_detector._updates_since_last_training = 0

                                        # Log quantum circuit continuous learning metrics to TensorBoard
                                        if tb_writer is not None:
                                            # Calculate metrics on recent data
                                            recent_window_size = min(50, len(batch_windows))
                                            sample_indices = np.random.choice(len(batch_windows), recent_window_size, replace=False)
                                            
                                            predictions = []
                                            actuals = []
                                            signs_pred = []
                                            signs_actual = []
                                            
                                            for idx in sample_indices:
                                                window = batch_windows[idx]
                                                target = batch_targets[idx]
                                                
                                                # Get prediction
                                                pred_gk = quantum_detector.predict_volatility(window)
                                                
                                                predictions.append(pred_gk)
                                                actuals.append(target)
                                                signs_pred.append(np.sign(pred_gk))
                                                signs_actual.append(np.sign(target))
                                            
                                            # Calculate metrics
                                            predictions = np.array(predictions)
                                            actuals = np.array(actuals)
                                            
                                            signed_corr = np.corrcoef(predictions, actuals)[0, 1] if len(predictions) > 1 else 0.0
                                            magnitude_corr = np.corrcoef(np.abs(predictions), np.abs(actuals))[0, 1] if len(predictions) > 1 else 0.0
                                            sign_accuracy = np.mean(np.array(signs_pred) == np.array(signs_actual))
                                            
                                            # Calculate loss on batch
                                            active_params = quantum_detector.params[quantum_detector.active_layers]
                                            batch_loss = quantum_detector._volatility_aware_cost(
                                                active_params,
                                                batch_windows,
                                                batch_targets
                                            )
                                            
                                            # Calculate average gradient norm from recent updates
                                            recent_grad_norm = 0.0
                                            if hasattr(quantum_detector, 'training_history') and 'gradients' in quantum_detector.training_history:
                                                recent_grads = quantum_detector.training_history['gradients'][-10:]
                                                if recent_grads:
                                                    recent_grad_norm = np.mean(recent_grads)
                                            
                                            # Log to TensorBoard
                                            continuous_learning_step = quantum_detector._circuit_call_count
                                            
                                            with tb_writer.as_default():
                                                tf.summary.scalar('QuantumCircuit/ContinuousLearning/Loss', 
                                                                 float(batch_loss), step=continuous_learning_step)
                                                tf.summary.scalar('QuantumCircuit/ContinuousLearning/GradientNorm', 
                                                                 recent_grad_norm, step=continuous_learning_step)
                                                tf.summary.scalar('QuantumCircuit/ContinuousLearning/SignedGKCorrelation_Pearson', 
                                                                 signed_corr, step=continuous_learning_step)
                                                tf.summary.scalar('QuantumCircuit/ContinuousLearning/MagnitudeCorrelation_Pearson', 
                                                                 magnitude_corr, step=continuous_learning_step)
                                                tf.summary.scalar('QuantumCircuit/ContinuousLearning/SignAccuracy_DirectionalCorrectness', 
                                                                 sign_accuracy, step=continuous_learning_step)
                                                tf.summary.scalar('QuantumCircuit/ContinuousLearning/BatchSize', 
                                                                 len(batch_windows), step=continuous_learning_step)
                                                tf.summary.scalar('QuantumCircuit/ContinuousLearning/DayInSequence', 
                                                                 i+1, step=continuous_learning_step)
                                            
                                            print(f"  Quantum metrics logged - Loss: {float(batch_loss):.6f}, "
                                                  f"Signed Corr: {signed_corr:.3f}, Sign Acc: {sign_accuracy:.1%}")

                                        # Recompute quantum features after batch update
                                        #print(f"\nQuantum circuit updated. Recomputing features for {len(prediction_data)-i-1} remaining days...")
                                        remaining_dates = list(prediction_data.keys())[i+1:]
                                        for future_date in remaining_dates:
                                            quantum_features = preprocessor.compute_quantum_features_for_date(
                                                future_date, preprocessor.get_raw_ohlc_data()
                                            )
                                            prediction_data[future_date]['features'].update(quantum_features)
                                        print(f"Feature recomputation complete.")
                    
                    # Log to TensorBoard - every 5 steps to reduce overhead
                    if use_tensorboard and tb_writer and i % 1 == 0:
                        log_metrics_to_tensorboard(tb_writer, tracker, model, i, log_detailed=(i % 50 == 0))

                        # Track feature relationships if quantum is enabled
                        if feature_tracker and i % 50 == 0:  # Every 50 days
                            # Get current features
                            classical_feats = predict_features.iloc[:i+1][[f for f in feature_names if 'quantum' not in f]].values
                            quantum_feats = predict_features.iloc[:i+1][[f for f in feature_names if 'quantum' in f]].values
                            current_target = predict_target.iloc[:i+1].values
                            
                            feature_tracker.update(classical_feats, quantum_feats, current_target, date, tb_writer)
                    
                    # Print progress every N days
                    if (i + 1) % 100 == 0:
                        # Check numercial stability of distribution calculation
                        fallback_stats = model.get_fallback_stats()
                        print(f"Fallback statistics: {fallback_stats['constrained_mean_count']} constraints")
                        print(f"Weight norm: {fallback_stats['weight_norm']:.4f}, Bias: {fallback_stats['bias_value']:.4f}")
                        
                        metrics = tracker.get_metrics_summary()
                        
                        # Calculate moving average accuracy
                        recent_accuracy = np.mean(tracker.direction_correct[-20:]) if len(tracker.direction_correct) >= 20 else np.mean(tracker.direction_correct)
                        
                        # Format date
                        date_str = date.strftime('%Y-%m-%d')
                        
                        # Create summary
                        print(f"\n{'='*60}")
                        print(f"Day {i+1}/{len(prediction_data)}, Date: {date_str}")
                        print(f"{'='*60}")
                        
                        # Basic metrics
                        print("\nAccuracy Metrics:")
                        print(f"Overall Direction Accuracy: {metrics.get('accuracy', 0):.4f}")
                        
                        recent100_accuracy = np.mean(tracker.direction_correct[-100:]) if len(tracker.direction_correct) >= 100 else np.mean(tracker.direction_correct)
                        print(f"Recent (100-day) Accuracy: {recent100_accuracy:.4f}")
                        
                        # Trading performance - Trading Hours
                        print("\nBasic Strategy Trading Hours:")
                        print(f"Win Rate: {metrics.get('win_rate_tradinghours', 0)*100:.2f}%")
                        print(f"Gain/Loss Ratio: {metrics.get('gain_loss_ratio_tradinghours', 0):.2f}")
                        print(f"Risk Avoidance Rate: {metrics.get('risk_avoidance_rate', 0)*100:.2f}% ({metrics.get('correct_risk_avoidances', 0)}/{metrics.get('total_risk_predictions', 0)})")
                        
                        print("\nTrading Behavior:")
                        print(f"Market Participation (Trading Hours): {metrics.get('market_participation_rate', 0)*100:.2f}%")
                        print(f"Trades in Last {tracker.recent_trade_window} Days: {metrics.get('recent_trades_count', 0)}")

                        # Distribution properties
                        print("\nPrediction Distribution:")
                        if 'pdf' in prediction:
                            pdf = prediction['pdf']
                            pdf_sum = pdf.sum()
                            pdf_max = pdf.max()
                            pdf_min = pdf.min()
                            print(f"PDF Properties: sum={pdf_sum:.4f}, max={pdf_max:.4f}, min={pdf_min:.4f}")
                            print(f"Prediction Range: {prediction['confidence_interval']}")
                            print(f"UP Probability: {prediction['positive_prob']:.4f}")
                            print(f"DOWN Probability: {1-prediction['positive_prob']:.4f}")
                        
                        # Strategy returns
                        print("\nStrategy Returns:")
                        print(f"Basic Strategy Trading Hours: {metrics.get('total_return_tradinghours', 0)*100:.2f}%")
                        print(f"Basic Strategy After Hours: {metrics.get('total_return_afterhours', 0)*100:.2f}%")
                        print(f"Leverage Strategy: {metrics.get('leverage_return', 0)*100:.2f}%")
                        print(f"Shorting Strategy: {metrics.get('shorting_return', 0)*100:.2f}%")
                        print(f"Buy & Hold: {metrics.get('buyhold_return', 0)*100:.2f}%")
                
                except Exception as iter_error:
                    print(f"Error processing day {date}: {iter_error}")
                    continue
            
            # Log final summary to TensorBoard
            if use_tensorboard and tb_writer:
                log_final_summary_to_tensorboard(tb_writer, tracker)
                
        except Exception as e:
            print(f"Error in prediction phase: {e}")
            # Still return partial results if available
    else:
        print("No prediction data available or model initialization failed.")

    # Generate final classical-quantum comparison if quantum was used
    if use_quantum_volatility and 'quantum_volatility' in locals():
        print("\nGenerating final classical-quantum comparison...")
        
        # Extract final classical and quantum features
        final_classical_features = predict_features[[f for f in feature_names if 'quantum' not in f]].values
        final_quantum_features = predict_features[[f for f in feature_names if 'quantum' in f]].values
        
        classical_feature_names = [f for f in feature_names if 'quantum' not in f]
        quantum_feature_names = [f for f in feature_names if 'quantum' in f]
        
        # Create final comparison
        fig_final = create_classical_quantum_comparison_plots(
            final_classical_features, final_quantum_features, 
            classical_feature_names, quantum_feature_names, 
            predict_target, output_folder
        )
        fig_final.savefig(os.path.join(output_folder, 'classical_quantum_comparison_final.png'),
                          dpi=150, bbox_inches='tight')
        plt.close(fig_final)
        print(f"Final Classical-Quantum comparison saved to {output_folder}/classical_quantum_comparison_final.png")
    
    # 5. Final Performance Evaluation
    print("\n5. Final Performance Evaluation")
    print("----------------------------")
    
    if tracker and hasattr(tracker, 'accuracy') and tracker.accuracy is not None:
        try:
            metrics = tracker.get_metrics_summary()
            
            print("\nPerformance Metrics:")
            print(f"Direction Accuracy: {metrics.get('accuracy', 0):.4f}")
            
            # Calculate mean error safely
            mean_error = 0
            if hasattr(tracker, 'prediction_errors') and tracker.prediction_errors:
                mean_error = np.mean(tracker.prediction_errors)
            print(f"Mean Prediction Error: {mean_error:.4f}%")
            
            # Add enhanced metrics reporting
            print(f"Win Rate: {metrics.get('win_rate_tradinghours', 0)*100:.2f}%")
            print(f"Gain/Loss Ratio: {metrics.get('gain_loss_ratio_tradinghours', 0):.2f}")
            
            # Report basic strategy performance
            print(f"\nBasic Strategy:")
            print(f"Total Return: {metrics.get('total_return_tradinghours', 0)*100:.2f}%")
            print(f"Maximum Drawdown: {metrics.get('max_drawdown_tradinghours', 0)*100:.2f}%")
            print(f"Sharpe Ratio: {metrics.get('sharpe_ratio_tradinghours', 0):.4f}")
            print(f"Sortino Ratio: {metrics.get('sortino_ratio_tradinghours', 0):.4f}")
            
            # Report leverage strategy performance
            print(f"\nLeverage Strategy:")
            print(f"Total Return: {metrics.get('leverage_return', 0)*100:.2f}%")
            print(f"Maximum Drawdown: {metrics.get('leverage_max_drawdown', 0)*100:.2f}%")
            print(f"Sharpe Ratio: {metrics.get('leverage_sharpe_ratio', 0):.4f}")
            print(f"Sortino Ratio: {metrics.get('leverage_sortino_ratio', 0):.4f}")
            
            # Report shorting strategy performance
            print(f"\nShorting Strategy:")
            print(f"Total Return: {metrics.get('shorting_return', 0)*100:.2f}%")
            print(f"Maximum Drawdown: {metrics.get('shorting_max_drawdown', 0)*100:.2f}%")
            print(f"Sharpe Ratio: {metrics.get('shorting_sharpe_ratio', 0):.4f}")
            print(f"Sortino Ratio: {metrics.get('shorting_sortino_ratio', 0):.4f}")
            
            # Report buy-and-hold performance
            print(f"\nBuy & Hold Strategy:")
            print(f"Total Return: {metrics.get('buyhold_return', 0)*100:.2f}%")
            print(f"Maximum Drawdown: {metrics.get('buyhold_max_drawdown', 0)*100:.2f}%")
            print(f"Sharpe Ratio: {metrics.get('buyhold_sharpe_ratio', 0):.4f}")
            print(f"Sortino Ratio: {metrics.get('buyhold_sortino_ratio', 0):.4f}")
            
            # Print annualized returns if available
            if 'annual_return_tradinghours' in metrics and metrics['annual_return_tradinghours'] is not None:
                print(f"\nAnnualized Returns:")
                print(f"Basic Strategy: {metrics['annual_return_tradinghours']*100:.2f}%")
                print(f"Leverage Strategy: {metrics.get('leverage_annual_return', 0)*100:.2f}%")
                print(f"Shorting Strategy: {metrics.get('shorting_annual_return', 0)*100:.2f}%")
                print(f"Buy & Hold: {metrics.get('buyhold_annual_return', 0)*100:.2f}%")

                print("\nEnhanced Metrics:")
                print_enhanced_metrics_summary(tracker)
        
            # Generate performance visuals
            try:
                # Generate and save comprehensive report
                fig = tracker.generate_comprehensive_report()
                plt.savefig(os.path.join(output_folder, 'comprehensive_report.png'))
                plt.close(fig)
                print(f"Comprehensive report saved to {os.path.join(output_folder, 'comprehensive_report.png')}")
                
                # Save individual plots
                fig = tracker.plot_cumulative_returns()
                plt.savefig(os.path.join(output_folder, 'cumulative_returns.png'))
                plt.close(fig)
                
                fig = tracker.plot_confusion_matrix()
                plt.savefig(os.path.join(output_folder, 'confusion_matrix.png'))
                plt.close(fig)
                
                if len(tracker.direction_correct) >= 20:
                    fig = tracker.plot_accuracy_moving_averages()
                    plt.savefig(os.path.join(output_folder, 'accuracy_moving_average.png'))
                    plt.close(fig)
                
                if len(tracker.predicted_probs) >= 50:
                    fig = tracker.plot_calibration_curve()
                    plt.savefig(os.path.join(output_folder, 'calibration_curve.png'))
                    plt.close(fig)
                
                # Add these new visualization outputs
                fig = tracker.plot_win_rate_moving_averages()
                plt.savefig(os.path.join(output_folder, 'win_rate_moving_averages.png'))
                plt.close(fig)
                
                fig = tracker.plot_gain_loss_ratio_moving_averages()
                plt.savefig(os.path.join(output_folder, 'gain_loss_ratio_moving_averages.png'))
                plt.close(fig)
                
                fig = tracker.plot_prediction_errors()
                plt.savefig(os.path.join(output_folder, 'prediction_errors.png'))
                plt.close(fig)
                
                fig = tracker.plot_trading_frequency()
                plt.savefig(os.path.join(output_folder, 'trading_frequency.png'))
                plt.close(fig)
                
                fig = tracker.plot_leverage_analysis()
                plt.savefig(os.path.join(output_folder, 'leverage_analysis.png'))
                plt.close(fig)
                
            except Exception as viz_error:
                print(f"Warning: Error generating visualizations: {viz_error}")
            
            # Save performance data to CSV
            try:
                performance_csv = os.path.join(output_folder, 'performance_data.csv')
                tracker.save_performance_data(performance_csv)
                print(f"Performance data saved to {performance_csv}")
            except Exception as csv_error:
                print(f"Warning: Could not save performance data to CSV: {csv_error}")
            
            # Save metrics summary to JSON
            try:
                metrics_json = os.path.join(output_folder, 'metrics_summary.json')
                tracker.save_metrics_summary(metrics_json)
                print(f"Metrics summary saved to {metrics_json}")
            except Exception as json_error:
                print(f"Warning: Could not save metrics summary to JSON: {json_error}")
            
        except Exception as eval_error:
            print(f"Error in performance evaluation: {eval_error}")
    else:
        print("No performance metrics available.")
        
    
    # 6. Save Model (if requested)
    if save_model and model is not None:
        filename = save_model if isinstance(save_model, str) else 'dbn_model.pkl'
        model_path = os.path.join(output_folder, filename)
        print(f"\nSaving model to {model_path}")
        try:
            with open(model_path, 'wb') as f:
                pickle.dump(model, f)
            print("Model saved successfully.")
        except Exception as e:
            print(f"Error saving model: {e}")
    
    # 7. Clean up TensorBoard resources
    if tb_writer:
        try:
            tb_writer.close()
            print("TensorBoard writer closed successfully.")
        except Exception as close_error:
            print(f"Warning: Error closing TensorBoard writer: {close_error}")
    
    return model, tracker, preprocessor

# 7. Execution 

In [ ]:
def main():
    """
    Main function to run the DBN stock prediction workflow with proper configuration.
    """
    # Set data folder
    data_folder = './Data/Data_Basemodel'
    
    # Define data configuration - corrected syntax
    data_config = {
        'SPX.csv': {
            'columns': ['Close', 'Open', 'High', 'Low', 'Volume'],
            'transformations': {
                'Close': 'log_return',
                'Open': 'log_return',
                'High': 'log_return',
                'Low': 'log_return',
                'Volume': 'log_return'
            },
            'frequency': 'daily'
        }#,
        #'NASDAQ.csv': {
        #    'columns': ['Close'],
        #    'transformations': {'Close': 'log_return'},
        #    'frequency': 'daily'
        #},
        #'NDX.csv': {
        #    'columns': ['Close'],
        #    'transformations': {'Close': 'log_return'},
        #    'frequency': 'daily'
        #},
        #'RUSSEL.csv': {
        #    'columns': ['Close'],
        #    'transformations': {'Close': 'log_return'},
        #    'frequency': 'daily'
        #},
        #'DJI_measuringworth_Fixed.csv': {
        #    'columns': ['Close'],
        #    'transformations': {'Close': 'log_return'},
        #    'frequency': 'daily'
        #},
        #'VIX.csv': {
        #    'columns': ['Close'],
        #    'transformations': {'Close': 'log_return'},
        #    'frequency': 'daily'
        #},
        # Commodities with both raw and log return
        #'COPPER_Macrotrends_1959.csv': {
        #    'columns': ['Close'],
        #    'transformations': {'Close': ['raw', 'log_return']},  # Both raw and log return
        #    'frequency': 'daily'
        #},
        #'Lumber_daily_macrotrends.csv': {
        #    'columns': ['Close'],
        #    'transformations': {'Close': ['raw', 'log_return']},  # Both raw and log return
        #    'frequency': 'daily'
        #},
        #'Gold_Investing_fixed.csv': {
        #    'columns': ['Close'],
        #    'transformations': {'Close': ['raw', 'log_return']},  # Both raw and log return
        #    'frequency': 'daily'
        #},
        #'Oil_Investing_Fixed.csv': {
        #    'columns': ['Close'],
        #    'transformations': {'Close': 'log_return'},
        #    'frequency': 'daily'
        #},
        # Treasury yields with both raw and log return
        #'US02Y.csv': {
        #    'columns': ['Close'],
        #    'transformations': {'Close': ['raw', 'log_return']},  # Both raw and log return
        #    'frequency': 'daily'
        #},
        #'US03M_FED.csv': {
        #    'columns': ['Close'],
        #    'transformations': {'Close': ['raw', 'log_return']},  # Both raw and log return
        #    'frequency': 'daily'
        #},
        #'US10Y.csv': {
        #    'columns': ['Close'],
        #    'transformations': {'Close': ['raw', 'log_return']},  # Both raw and log return
        #    'frequency': 'daily'
        #}
    }
    
    # Technical indicators configuration

    # Create base indicator configurations
    standard_indicators = {
        'current_drawdown': True,
        'drawdown_include_trend': True,
        'drawdown_include_metrics': True,
        'sma': True,
        'sma_windows': [5, 10, 20, 50, 100, 200],   # Brock et al. (1992) and Sullivan et al. (1999)
        'sma_timeframes': ['D'],                    # [21, 63, 126, 252] days (business cycle capture: 86.3%, p < 0.01) Neely et al. (2014)
        'include_stddev': True,
        'include_ma_trend': True,
        'ma_relationships': True,
        'ema': True,
        'ema_windows': [5, 12, 26, 50, 100, 200],        # Zakamulin (2014) - 200: Glabadanidis (2017)
        'ema_timeframes': ['D'],
        'include_stddev': True,
        'rsi': True,
        'rsi_windows': [9, 14, 21], # Lento et al. (2007) - Chong & Ng (2008), Wong et al. (2003), Papailias & Thomakos (2015) - Seiler (2001)
        'rsi_include_trend': True,
        'rsi_include_metrics': True,
        'roc': True,
        'roc_windows': [1, 2, 5, 10, 20, 60, 252], # Moskowitz et al. (2012) - 2: Szakmary et al. (2010) - 5: Han et al. (2016), Menkhoff et al. (2012) - 20: Jegadeesh & Titman (2001), Menkhoff et al. (2012)
        'roc_include_trend': True,                         # Optimal threshold value: ±2.5 standard deviations (Asness et al., 2013)
        'roc_include_metrics': True,                       # [5, 21, 63, 252] days (maximum likelihood estimation optimal: p < 0.001) Fama & French (2012)
        'pmo': True,
        'pmo_short_periods': [35],
        'pmo_long_periods': [20],
        'pmo_signal_periods': [10],
        'pmo_include_raw': True,
        'pmo_include_metrics': True
    }

    # SPX gets additional indicators including volume-based ones
    spx_indicators = standard_indicators.copy()
    spx_indicators.update({
        'stochastic': True,
        'stochastic_k_windows': [9, 14, 21],
        'stochastic_d_windows': [3, 5, 9],   # Roberts (2005): spectral analysis (power spectrum peaks at: 9±2, 14±3, 21±4 days) - Brock et al. (1992)
        'stochastic_include_raw': True,      # %K Periods: [5, 14, 34] (Fibonacci-aligned), %D Periods: [3, 5, 8] (smoothing optimization) Taylor & Allen (1992)
        'stochastic_include_metrics': True,
        'stochastic_include_trend': True,
        'atr': True,
        'atr_windows': [5, 14, 21, 55],      # Taylor & Allen (1992), 14: Dunis et al. (2010) & Chan et al. (2019)
        'atr_include_trend': True,           # High Volatility Regime: [5, 10] days, Low Volatility Regime: [14, 21] days, Hurst et al. (2017)
        'atr_include_metrics': True,
        'cci': True,                         # Parameter Set C: Zero-Line Cross Focus (trend-change): [5, 20, 50], Papailias & Thomakos (2015)
        'cci_windows': [10, 14, 20, 40],         # Hsu & Kuan (2005), 20 best
        'cci_include_trend': True,           # Parameter Set B: Market Structure-Aligned: [14, 30, 60] days, Harris & Yilmaz (2009)
        'obv': True,
        'obv_include_trend': True,
        'obv_normalize': True,
        'obv_multi_window': True,                  # 10: Campbell et al. (1993), Grossman & Wang (1993), Lo & Wang (2000), Llorente et al. (2002)
        'obv_windows': [5, 10, 21],                # [5, 10, 21] Blume et al. (1994)
        'obv_timeframes': ['D'],                   # [5, 10, 21, 63, 126, 252]
        'vroc': True,                        # 5: Lo & Wang (2000), 2: Bessembinder & Seguin (1993), 1: Ni et al. (2008)
        'vroc_windows': [1, 2, 3, 5, 10, 20, 60], # Gallant et al. (1992)
        'vroc_include_trend': True,               # [1, 3, 5, 10] days (high-frequency focus) - Clark (1973) Lo & Wang (2000)
        'vroc_include_metrics': True,             # [5, 20, 60] days (institutional time horizons) - Foster & Viswanathan (1993)
        'adl': True,
        'adl_include_trend': True,            # 200 day slope change: Foster & Viswanathan (1993) - 20/50-day divergence: Admati & Pfleiderer (1988)
        'adl_include_metrics': True,          # Divergence Windows: [10, 21, 63] days: Foster & Viswanathan (1993)
        'adl_windows': [5, 10, 21, 63, 200],  # Admati & Pfleiderer (1988)
        'adl_timeframes': ['D'],              # Parameter Set C: Trend Confirmation Focus:  [5, 10, 20, 60] days on A/D Line - Karpoff (1987)
        'psar': True,
        'psar_af_start': 0.02,               # Sullivan et al. (1999) (two more options: conservative & aggresive) - Olson (2004) - Achelis (2001) - (Brock et al., 1992)
        'psar_af_step': 0.02,
        'psar_af_max': 0.2,
        'psar_include_raw': True,
        'psar_include_metrics': True,
        'psar_include_trend': True
    })
    
    tech_indicators_config = {
        # SPX with full indicators
        'SPX.csv': {
            'include': True,
            'indicators': spx_indicators
        }#,
        
        # Major indices with standard indicators
        #'NASDAQ.csv': {'include': True, 'indicators': standard_indicators.copy()},
        #'NDX.csv': {'include': True, 'indicators': standard_indicators.copy()},
        #'RUSSEL.csv': {'include': True, 'indicators': standard_indicators.copy()},
        #'DJI_measuringworth_Fixed.csv': {'include': True, 'indicators': standard_indicators.copy()},
        #'VIX.csv': {'include': True, 'indicators': standard_indicators.copy()},
        
        # Commodities with standard indicators
        #'COPPER_Macrotrends_1959.csv': {'include': True, 'indicators': standard_indicators.copy()},
        #'Lumber_daily_macrotrends.csv': {'include': True, 'indicators': standard_indicators.copy()},
        #'Gold_Investing_fixed.csv': {'include': True, 'indicators': standard_indicators.copy()},
        #'Oil_Investing_Fixed.csv': {'include': True, 'indicators': standard_indicators.copy()},
        
        # Treasury yields with standard indicators
        #'US02Y.csv': {'include': True, 'indicators': standard_indicators.copy()},
        #'US03M_FED.csv': {'include': True, 'indicators': standard_indicators.copy()},
        #'US10Y.csv': {'include': True, 'indicators': standard_indicators.copy()},

        # Configuration for derived ratio/spread columns
        #'DERIVED_COLUMNS': {
        #    'include': True,
        #    'columns': [
        #        'Copper_Gold_Ratio',
        #        'Lumber_Gold_Ratio',
        #        'US10Y_US02Y_Spread',
        #        'US10Y_US03M_Spread'
        #    ],
        #    'indicators': standard_indicators.copy()
        #}
    }
    
    # Set target
    target_file = 'SPX.csv'
    target_column = 'Close'
    target_transformation = 'pct_change'
    
    # Set output folder
    output_folder = './output'
    os.makedirs(output_folder, exist_ok=True)
    
    # Import the custom preprocessor
    #from volatility_enhanced_preprocessor import VolatilityEnhancedDataPreprocessor
    
    # Run the workflow with quantum enhancements
    model, tracker, preprocessor = run_dbn_stock_prediction(
        data_folder=data_folder,
        data_config=data_config,
        target_file=target_file,
        target_column=target_column,
        target_transformation=target_transformation,
        training_period_years=3,
        start_date='1962-1-02', #'1982-4-1'
        end_date='1982-4-19',  #1982-4-19'
        # Technical indicators parameters
        apply_tech_indicators=True,
        tech_indicators_config=tech_indicators_config,
        # Feature engineering parameters
        apply_feature_eng=True,
        apply_pca=True,
        pca_components=20,  # Reduce to ~20 components before quantum extraction
        # Quantum volatility parameters
        use_quantum_volatility=True,
        quantum_n_qubits=4,
        # Other parameters as in your original function
        hidden_layers=2,
        states_per_hidden=5,
        master_node=False,
        inference_method='particle',
        prediction_range=(-20, 20),
        prediction_bins=201,
        n_particles=10000,
        initial_capital=1000,
        risk_free_rate=0.02/252,
        leverage_threshold_std=1.0,
        max_leverage=3,
        save_model='quantum_volatility_model.pkl',
        load_model=None,
        random_state=42,
        output_folder=output_folder,
        # Use the quantum volatility enhanced preprocessor
        # DataPreprocessorClass=VolatilityEnhancedDataPreprocessor
    )
    
    print("\nExecution complete!")
    
    # Additional analysis on performance data if available
    if tracker is not None:
        # Get performance data as DataFrame
        performance_df = tracker.to_dataframe()
        
        if not performance_df.empty:
            # Monthly returns analysis
            performance_df['month'] = pd.to_datetime(performance_df['date']).dt.to_period('M')
            
            # Using the correct column name based on your DataFrame schema
            # Look for 'basic_tradinghours_return' instead of 'basic_return'
            monthly_returns = performance_df.groupby('month')['basic_tradinghours_return'].sum() * 100
            
            print("\nTop 5 Most Profitable Months:")
            print(monthly_returns.sort_values(ascending=False).head(5))
            
            print("\nWorst 5 Months:")
            print(monthly_returns.sort_values().head(5))
            
            # Drawdown analysis
            performance_df['drawdown'] = (performance_df['basic_tradinghours_value'] / 
                                        performance_df['basic_tradinghours_value'].cummax() - 1) * 100
            
            worst_dd_idx = performance_df['drawdown'].idxmin()
            if worst_dd_idx is not None:
                worst_dd = performance_df.loc[worst_dd_idx]
                print(f"\nWorst drawdown: {worst_dd['drawdown']:.2f}% on {worst_dd['date'].strftime('%Y-%m-%d')}")
            
            # Win/loss streaks
            print(f"\nMax winning streak: {tracker.max_consecutive_wins} trades")
            if tracker.max_consecutive_wins_start_date and tracker.max_consecutive_wins_end_date:
                print(f" from {tracker.max_consecutive_wins_start_date.strftime('%Y-%m-%d')} to {tracker.max_consecutive_wins_end_date.strftime('%Y-%m-%d')}")
            
            print(f"Max losing streak: {tracker.max_consecutive_losses} trades")
            if tracker.max_consecutive_losses_start_date and tracker.max_consecutive_losses_end_date:
                print(f" from {tracker.max_consecutive_losses_start_date.strftime('%Y-%m-%d')} to {tracker.max_consecutive_losses_end_date.strftime('%Y-%m-%d')}")

if __name__ == "__main__":
    main()

Starting DBN Stock Prediction Workflow
Using discrete hidden states with 5 states per hidden layer
TensorBoard logs will be saved to ./tensorboard_logs/dbn_run_20250627-022044
To view logs, run: tensorboard --logdir=./tensorboard_logs

1. Data Preprocessing
--------------------
Available CSV files: ['SPX.csv']
Starting data processing...


2025-06-27 02:20:44.651307: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected



----- Applying Date Filtering (using only the desired temporal period) -----
Filtering data to start from 1962-01-02 00:00:00
Filtering data to end at 1982-04-19 00:00:00
Date range after filtering: 1962-01-02 00:00:00 to 1982-04-19 00:00:00
Reduced data from 18921 to 5093 days (26.9%)

========== APPLYING BASIC FEATURE ENGINEERING ==========
Example columns: ['SPX_Open_log_return']
Extreme values handled. Proceeding with feature engineering.
Found columns for ratio calculations:
  Copper: None
  Lumber: None
  Gold: None
  US10Y: None
  US02Y: None
  US03M: None
This may reduce model performance.
Attempting to proceed with available columns...

========== CALCULATING TECHNICAL INDICATORS ==========
Calculating technical indicators for SPX.csv
Successfully calculated 660 technical indicators for SPX.csv
Added 660 technical indicators from SPX.csv
Technical indicators calculation completed: Added 660 indicators

----- Validating technical indicators -----

========== APPLYING ADVANCED 

Computing PCA:  88%|████████▊ | 4438/5030 [38:38<06:36,  1.49windows/s]

No scaling applied: Maximum absolute value 9.91 is within threshold 10.00


...

DIAGNOSTIC: Stagnation threshold exceeded: 0.9667 > 0.9500


Processing days:  54%|█████▍    | 2348/4336 [14:42:15<5:58:09, 10.81s/it]

DIAGNOSTIC: Stagnation threshold exceeded: 0.9667 > 0.9500


Processing days:  54%|█████▍    | 2349/4336 [14:42:26<5:58:10, 10.82s/it]


Quantum circuit batch update: 50 samples
  Quantum metrics logged - Loss: 1.114810, Signed Corr: -0.124, Sign Acc: 46.0%
